In [23]:
import datetime as dt
import re
import textwrap
from io import StringIO
from pathlib import Path

import requests
import pytz
import pandas as pd
from IPython.display import display, Markdown
from lxml import etree

## Settings

In [24]:
# Create league result files if True
WRITE_MARKDOWN_FILES = False

# Take the best results from this many races as the total score.
# The team league is scored by top 6 races, but a rider might 
# have a different top 6 than the team's top 6 races.
TOP_N_RESULTS = 6

In [25]:
league_ids = {
    'Americas': '102',
    'Americas Women': '125',
    'Europe': '100',
    'Europe Women': '124',
    'Pacific-Asia': '101'
}

# 24 riders were mistakenly DQ'd in the 2nd KCL Europe race ("conegate"), so points
# are calculated with 25 onwards - i.e. 1st position is ranked like 25th.
position_offsets = pd.DataFrame.from_records([
    {'category': 'A', 'event_id': '155628', 'position_offset': 24}
], index=['event_id', 'category'])

In [26]:
# Fetch all the data from ZP.
# Note that ZP does provide individual rankings data via its API at:
# https://zwiftpower.com/api3.php?do=league_standings&id=XXX
# But where's the fun in that?

raw_events = {
    name: requests.get(f'https://zwiftpower.com/api3.php?do=league_event_results', params={'id': league_id}).json()['data']
    for name, league_id in league_ids.items()
}

In [27]:
raw_event_results = [
    requests.get('https://zwiftpower.com/api3.php?do=event_results', 
                             params={'zid': event['zid']}).json()['data']
    for events in raw_events.values() for event in events
]

In [28]:
events = pd.DataFrame.from_records(
    {'league': name,
     'event_id': event['zid'],
     'date': dt.datetime.fromtimestamp(event['tm'], pytz.utc)}
    for name, data in raw_events.items() for event in data
)
events = events.sort_values(by=['league', 'date']).set_index(['league', 'event_id'])
events['ordinal'] = events['date'].groupby('league').rank(method='dense').astype(int)
events.head()

date  ordinal
league   event_id                                   
Americas 152781   2019-02-15 01:00:00+00:00        1
         155997   2019-02-22 01:00:00+00:00        2
         159644   2019-03-01 01:00:00+00:00        3
         163028   2019-03-08 01:00:00+00:00        4
         167034   2019-03-15 00:00:00+00:00        5

In [29]:
raw_placings = (pd.DataFrame.from_records(row for event in raw_event_results for row in event)
                .rename(columns={'zid': 'event_id', 'zwid': 'rider_id', 'tid': 'team_id', 'tname': 'team_name'}))

# Get the league each result is from
raw_placings = pd.merge(raw_placings, events.index.to_frame()[['league']], left_on='event_id', right_on='event_id')

raw_placings.head()

,DT_RowId,act_id,act_name,age,aid,avg_hr,avg_power,avg_wkg,category,cp,...,wkg300,wkg5,wkg60,wkg_ftp,wkg_guess,zada,zeff,event_id,rider_id,league
0,,0,,35,50308,"[166, 0]","[354, 1]","[4.7, 1]",A,1,...,"[6.3, 1]","[14.9, 4]","[7.6, 1]","[4.6, 1]",0,0,1,176476,9044,Americas
1,,0,,41,406161,"[165, 0]","[314, 0]","[4.5, 0]",A,1,...,"[5.5, 0]","[12.7, 0]","[7.4, 0]","[4.4, 0]",0,1,1,176476,1412,Americas
2,,0,,34,2721607,"[156, 0]","[345, 0]","[4.5, 0]",A,1,...,"[5.4, 0]","[10.3, 0]","[7.3, 0]","[4.3, 0]",0,1,0,176476,15522,Americas
3,,0,,33,5562374,"[168, 0]","[347, 0]","[4.6, 0]",A,1,...,"[5.6, 0]","[12.8, 0]","[7.4, 0]","[4.4, 0]",0,0,1,176476,322602,Americas
4,,0,,40,187245,"[154, 0]","[283, 0]","[4.3, 0]",A,1,...,"[5.2, 0]","[12.6, 0]","[7.5, 0]","[4.2, 0]",0,0,1,176476,4641,Americas


In [30]:
def get_html_text(html_string):
    if not html_string:
        return ''
    tree = etree.parse(StringIO(html_string), etree.HTMLParser())
    return tree.xpath('normalize-space()')

def score_for_position(placings):
    effective_placing = placings['position_in_cat'] + placings['position_offset']
    return 1001 - (effective_placing * 2)

In [31]:
position_offsets

,,position_offset
event_id,category,
155628,A,24


In [32]:
placings = (raw_placings[['event_id', 'rider_id', 'pos', 'position_in_cat', 
                          'name', 'category', 'team_name', 'team_id', 'league']]
            .copy()
            .set_index(['event_id', 'category']))

# position_offset adjusts actual finishing positions for scoring purposes (see conegate above)
placings['position_offset'] = position_offsets
placings['position_offset'] = placings['position_offset'].fillna(0).astype(int)

placings = placings.reset_index().set_index(['league', 'category', 'rider_id', 'event_id'])

# names are HTML encoded, (special chars are HTML entities), let's fix that
placings['name'] = placings['name'].map(get_html_text)
placings['team_name'] = placings['team_name'].map(get_html_text)

# People without teams get '0' as a team ID - drop such IDs
placings['team_id'] = placings.apply(lambda row: '' if row['team_name'] == '' and row['team_id'] == '0' else row['team_id'], axis=1)

placings['position_offset'] = placings['position_offset'].fillna(0).astype(int)

# Calculate the points earned by the rider's result
placings['points'] = score_for_position(placings)

placings = placings.join(events['ordinal']).rename(columns={'ordinal': 'event_ordinal'})

# The rank position (1..n) of each event placing by score. We can take the top N ranked 
# points to get the top n results.
placings['point_rank'] = (
    placings.sort_values(['league', 'category', 'rider_id', 'points', 'event_ordinal'],
                         ascending=[True, True, True, False, True])['points']
    .groupby(level=['league', 'category', 'rider_id'], sort=False)
    .rank(method='first', ascending=False, axis=1).astype(int))

placings.head()

pos  position_in_cat             name  \
league   event_id category rider_id                                          
Americas 152781   A        745         1                1    SCOTTIE WEISS   
                           81778       2                2   Eric Loiselle🔰   
                           9044        3                3  Tyler Waldron 🔰   
                           463284      4                4      Seth G ☁️ 🔰   
                           471022      5                5            键盘仁波切   

                                            team_name team_id  \
league   event_id category rider_id                             
Americas 152781   A        745                                  
                           81778     Cannondale/4iiii    2105   
                           9044           ⚠️ Dropouts    3711   
                           463284                 AFC    1405   
                           471022            Team +1S    1414   

                                     position_offset  points  event_ordinal  \
league   event_id category rider_id                                           
Americas 152781   A        745                     0     999              1   
                           81778                   0     997              1   
                           9044                    0     995              1   
                           463284                  0     993              1   
                           471022                  0     991              1   

                                     point_rank  
league   event_id category rider_id              
Americas 152781   A        745                1  
                           81778              1  
                           9044               7  
                           463284             1  
                           471022             2

In [33]:
top_n_placings = placings[placings['point_rank'] <= TOP_N_RESULTS]

rider_ranks = top_n_placings[['name', 'team_name', 'team_id']].groupby(['league', 'category', 'rider_id']).first()

# Calculate each rider's total points
rider_ranks['total_points'] = top_n_placings['points'].groupby(['league', 'category', 'rider_id']).sum()

# Rank the riders (within their category) by score. Riders with the same score are given the same rank position
rider_ranks['rank'] = rider_ranks['total_points'].groupby(['league', 'category']).rank(method='dense', ascending=False).astype(int)

rider_ranks = rider_ranks.sort_values(['league', 'category', 'total_points'], ascending=[True, True, False])

rider_ranks.head()

name         team_name team_id  \
league   category rider_id                                               
Americas A        745          SCOTTIE WEISS                             
                  9044       Tyler Waldron 🔰       ⚠️ Dropouts    3711   
                  92891          Alex Dion 🔰  Cannondale/4iiii    2105   
                  198103    Frédéric Meunier  Cannondale/4iiii    2105   
                  140937       Eugene Chan 🍌               HPP    1528   

                            total_points  rank  
league   category rider_id                      
Americas A        745               5988     1  
                  9044              5986     2  
                  92891             5900     3  
                  198103            5830     4  
                  140937            5824     5

Create markdown tables for publication

In [34]:
def build_markdown_escaper():
    reserved = '\`*_{}[]()#+-.!|'
    reserved_pattern = re.compile(f'[{re.escape(reserved)}]')

    def markdown_escape(text):
        return reserved_pattern.sub(lambda m: '\\' + m.group(0), text)
    return markdown_escape
markdown_escape = build_markdown_escaper()

def markdown_link(text, url):
    return f'[{markdown_escape(text)}]({markdown_escape(url)})'

def rider_name_link(row):
    return markdown_link(row["name"], f'https://zwiftpower.com/profile.php?z={row["rider_id"]}')

def rider_team_name_link(row):
    if not row["team_id"]:
        return None
    return markdown_link(row["team_name"], f'https://zwiftpower.com/team.php?id={row["team_id"]}')

# https://codegolf.stackexchange.com/questions/4707/outputting-ordinal-numbers-1st-2nd-3rd#answer-4712
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

def position_label(position):
    return {1: '🥇', 2: '🥈', 3: '🥉'}.get(position) or ordinal(position)
                         
def position_summary(placings_row):
    pos_label = position_label(placings_row['position_in_cat'])
    race_ord = placings_row['event_ordinal']
    label = f'{pos_label} #{race_ord}'
    
    # Label conegate results with effective position for scoring purposes
    if placings_row['position_offset'] > 0:
        effective_position = position_label(placings_row['position_in_cat'] + placings_row['position_offset'])
        # This is a bit like a cone 🤷
        label = f'{pos_label} (📣{effective_position}) #{race_ord}'
    return f'[{label}](https://zwiftpower.com/events.php?zid={placings_row["event_id"]})'

In [35]:
display_positions = top_n_placings.reset_index().set_index(['league', 'category', 'rider_id', 'point_rank'])[['position_in_cat', 'event_id', 'event_ordinal', 'position_offset']].apply(position_summary, axis=1)
display_positions = display_positions.unstack()
display_positions.head()

point_rank                                                                  1  \
league   category rider_id                                                      
Americas A        250       [17th #1](https://zwiftpower.com/events.php?zi...   
                  745       [🥇 #1](https://zwiftpower.com/events.php?zid=1...   
                  1412      [🥈 #8](https://zwiftpower.com/events.php?zid=1...   
                  3228      [30th #1](https://zwiftpower.com/events.php?zi...   
                  3313      [10th #7](https://zwiftpower.com/events.php?zi...   

point_rank                                                                  2  \
league   category rider_id                                                      
Americas A        250       [20th #8](https://zwiftpower.com/events.php?zi...   
                  745       [🥇 #2](https://zwiftpower.com/events.php?zid=1...   
                  1412      [5th #5](https://zwiftpower.com/events.php?zid...   
                  3228      [30th #2](https://zwiftpower.com/events.php?zi...   
                  3313      [13th #5](https://zwiftpower.com/events.php?zi...   

point_rank                                                                  3  \
league   category rider_id                                                      
Americas A        250       [22nd #7](https://zwiftpower.com/events.php?zi...   
                  745       [🥇 #3](https://zwiftpower.com/events.php?zid=1...   
                  1412      [7th #6](https://zwiftpower.com/events.php?zid...   
                  3228      [34th #5](https://zwiftpower.com/events.php?zi...   
                  3313      [30th #6](https://zwiftpower.com/events.php?zi...   

point_rank                                                                  4  \
league   category rider_id                                                      
Americas A        250       [23rd #6](https://zwiftpower.com/events.php?zi...   
                  745       [🥇 #5](https://zwiftpower.com/events.php?zid=1...   
                  1412                                                    NaN   
                  3228      [38th #3](https://zwiftpower.com/events.php?zi...   
                  3313                                                    NaN   

point_rank                                                                  5  \
league   category rider_id                                                      
Americas A        250       [25th #2](https://zwiftpower.com/events.php?zi...   
                  745       [🥇 #7](https://zwiftpower.com/events.php?zid=1...   
                  1412                                                    NaN   
                  3228      [39th #4](https://zwiftpower.com/events.php?zi...   
                  3313                                                    NaN   

point_rank                                                                  6  
league   category rider_id                                                     
Americas A        250       [34th #3](https://zwiftpower.com/events.php?zi...  
                  745       [4th #4](https://zwiftpower.com/events.php?zid...  
                  1412                                                    NaN  
                  3228                                                    NaN  
                  3313                                                    NaN

In [36]:
display_rider_ranks = rider_ranks[['rank', 'total_points']].copy()
display_rider_ranks['name'] = pd.merge(rider_ranks.index.to_frame(), rider_ranks, 
                                        left_index=True, right_index=True).apply(rider_name_link, axis=1)
display_rider_ranks['team'] = rider_ranks.apply(rider_team_name_link, axis=1)
display_rider_ranks.head()

rank  total_points  \
league   category rider_id                       
Americas A        745          1          5988   
                  9044         2          5986   
                  92891        3          5900   
                  198103       4          5830   
                  140937       5          5824   

                                                                         name  \
league   category rider_id                                                      
Americas A        745       [SCOTTIE WEISS](https://zwiftpower\.com/profil...   
                  9044      [Tyler Waldron 🔰](https://zwiftpower\.com/prof...   
                  92891     [Alex Dion 🔰](https://zwiftpower\.com/profile\...   
                  198103    [Frédéric Meunier](https://zwiftpower\.com/pro...   
                  140937    [Eugene Chan 🍌](https://zwiftpower\.com/profil...   

                                                                         team  
league   category rider_id                                                     
Americas A        745                                                    None  
                  9044      [⚠️ Dropouts](https://zwiftpower\.com/team\.ph...  
                  92891     [Cannondale/4iiii](https://zwiftpower\.com/tea...  
                  198103    [Cannondale/4iiii](https://zwiftpower\.com/tea...  
                  140937     [HPP](https://zwiftpower\.com/team\.php?id=1528)

In [37]:
full_display = pd.merge(display_rider_ranks, display_positions, left_index=True, right_index=True, how='left')
full_display = full_display.fillna('—')

In [38]:
def markdown_table(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)

In [39]:
def get_league_rank_table(league, full_display):
    league_table = full_display.loc[league, :, :]
    league_table = league_table.reset_index().drop(columns=['league', 'rider_id'])
    columns = dict(zip(
        list(league_table.columns),
        ['Category', 'Rank', 'Total Points', 'Name', 'Team']
    ))
    return league_table.rename(columns=columns)

def render_league_table(league_table):
    return markdown_table(league_table)

def league_filename(league):
    return league.lower().replace(' ', '-')

In [40]:
league_names = list(full_display.index.levels[0])

league_tables = {
    league: render_league_table(get_league_rank_table(league, full_display))
    for league in league_names
}

In [41]:
league_markdown = {
    league: (f'''\
## KISS Community League — {league}

Individual rider rankings by top {TOP_N_RESULTS} results.

{league_tables[league]}
''')
    for league in league_names
}

all_tables_markdown = '# KISS Community League Rider Rank Tables\n\n' + '\n\n'.join(league_markdown.values())

In [42]:
if WRITE_MARKDOWN_FILES:
    # Write the top 6 tables to the files we initially published
    if TOP_N_RESULTS == 6:
        dest_dir = Path('table-markdown')
    else:
        dest_dir = Path('table-markdown') / f'top-{TOP_N_RESULTS}'
    dest_dir.mkdir(exist_ok=True, parents=True)

    for league in league_names:
        with open(dest_dir / f'league-table-{league_filename(league)}.md', 'w') as f:
            f.write(league_markdown[league])

    with open(dest_dir / 'all-tables.md', 'w') as f:
        f.write(all_tables_markdown)

In [43]:
# Render the results inline
display(Markdown(all_tables_markdown))

# KISS Community League Rider Rank Tables

## KISS Community League — Americas

Individual rider rankings by top 6 results.

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6
---|---|---|---|---|---|---|---|---|---|---
A|1|5988|[SCOTTIE WEISS](https://zwiftpower\.com/profile\.php?z=745)|—|[🥇 #1](https://zwiftpower.com/events.php?zid=152781)|[🥇 #2](https://zwiftpower.com/events.php?zid=155997)|[🥇 #3](https://zwiftpower.com/events.php?zid=159644)|[🥇 #5](https://zwiftpower.com/events.php?zid=167034)|[🥇 #7](https://zwiftpower.com/events.php?zid=173965)|[4th #4](https://zwiftpower.com/events.php?zid=163028)
A|2|5986|[Tyler Waldron 🔰](https://zwiftpower\.com/profile\.php?z=9044)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥇 #6](https://zwiftpower.com/events.php?zid=170592)|[🥇 #8](https://zwiftpower.com/events.php?zid=176476)|[🥈 #2](https://zwiftpower.com/events.php?zid=155997)|[🥈 #4](https://zwiftpower.com/events.php?zid=163028)|[🥈 #5](https://zwiftpower.com/events.php?zid=167034)|[🥈 #7](https://zwiftpower.com/events.php?zid=173965)
A|3|5900|[Alex Dion 🔰](https://zwiftpower\.com/profile\.php?z=92891)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[6th #8](https://zwiftpower.com/events.php?zid=176476)|[7th #1](https://zwiftpower.com/events.php?zid=152781)|[8th #2](https://zwiftpower.com/events.php?zid=155997)|[8th #4](https://zwiftpower.com/events.php?zid=163028)|[9th #3](https://zwiftpower.com/events.php?zid=159644)|[15th #6](https://zwiftpower.com/events.php?zid=170592)
A|4|5830|[Frédéric Meunier](https://zwiftpower\.com/profile\.php?z=198103)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[8th #8](https://zwiftpower.com/events.php?zid=176476)|[12th #6](https://zwiftpower.com/events.php?zid=170592)|[15th #5](https://zwiftpower.com/events.php?zid=167034)|[16th #2](https://zwiftpower.com/events.php?zid=155997)|[17th #7](https://zwiftpower.com/events.php?zid=173965)|[20th #1](https://zwiftpower.com/events.php?zid=152781)
A|5|5824|[Eugene Chan 🍌](https://zwiftpower\.com/profile\.php?z=140937)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[10th #8](https://zwiftpower.com/events.php?zid=176476)|[13th #2](https://zwiftpower.com/events.php?zid=155997)|[15th #7](https://zwiftpower.com/events.php?zid=173965)|[16th #5](https://zwiftpower.com/events.php?zid=167034)|[17th #3](https://zwiftpower.com/events.php?zid=159644)|[20th #4](https://zwiftpower.com/events.php?zid=163028)
A|6|5774|[David Galati](https://zwiftpower\.com/profile\.php?z=210781)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|[14th #7](https://zwiftpower.com/events.php?zid=173965)|[14th #8](https://zwiftpower.com/events.php?zid=176476)|[20th #3](https://zwiftpower.com/events.php?zid=159644)|[22nd #2](https://zwiftpower.com/events.php?zid=155997)|[22nd #6](https://zwiftpower.com/events.php?zid=170592)|[24th #1](https://zwiftpower.com/events.php?zid=152781)
A|7|5764|[Brian Gefroh](https://zwiftpower\.com/profile\.php?z=414438)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[11th #2](https://zwiftpower.com/events.php?zid=155997)|[17th #6](https://zwiftpower.com/events.php?zid=170592)|[18th #4](https://zwiftpower.com/events.php?zid=163028)|[18th #8](https://zwiftpower.com/events.php?zid=176476)|[27th #7](https://zwiftpower.com/events.php?zid=173965)|[30th #3](https://zwiftpower.com/events.php?zid=159644)
A|8|5726|[Eric Urff](https://zwiftpower\.com/profile\.php?z=514852)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[15th #4](https://zwiftpower.com/events.php?zid=163028)|[20th #7](https://zwiftpower.com/events.php?zid=173965)|[21st #6](https://zwiftpower.com/events.php?zid=170592)|[22nd #8](https://zwiftpower.com/events.php?zid=176476)|[29th #5](https://zwiftpower.com/events.php?zid=167034)|[33rd #2](https://zwiftpower.com/events.php?zid=155997)
A|9|5724|[Mark Minn](https://zwiftpower\.com/profile\.php?z=250)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[17th #1](https://zwiftpower.com/events.php?zid=152781)|[20th #8](https://zwiftpower.com/events.php?zid=176476)|[22nd #7](https://zwiftpower.com/events.php?zid=173965)|[23rd #6](https://zwiftpower.com/events.php?zid=170592)|[25th #2](https://zwiftpower.com/events.php?zid=155997)|[34th #3](https://zwiftpower.com/events.php?zid=159644)
A|10|5716|[Frank Deal](https://zwiftpower\.com/profile\.php?z=90541)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[12th #7](https://zwiftpower.com/events.php?zid=173965)|[15th #8](https://zwiftpower.com/events.php?zid=176476)|[25th #4](https://zwiftpower.com/events.php?zid=163028)|[29th #1](https://zwiftpower.com/events.php?zid=152781)|[29th #2](https://zwiftpower.com/events.php?zid=155997)|[35th #3](https://zwiftpower.com/events.php?zid=159644)
A|11|5714|[Daniel Riddle](https://zwiftpower\.com/profile\.php?z=341672)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[18th #7](https://zwiftpower.com/events.php?zid=173965)|[19th #8](https://zwiftpower.com/events.php?zid=176476)|[23rd #2](https://zwiftpower.com/events.php?zid=155997)|[25th #1](https://zwiftpower.com/events.php?zid=152781)|[28th #3](https://zwiftpower.com/events.php?zid=159644)|[33rd #4](https://zwiftpower.com/events.php?zid=163028)
A|12|5664|[Marc Pensyl \(RCC\)](https://zwiftpower\.com/profile\.php?z=218740)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[18th #5](https://zwiftpower.com/events.php?zid=167034)|[27th #6](https://zwiftpower.com/events.php?zid=170592)|[28th #1](https://zwiftpower.com/events.php?zid=152781)|[28th #7](https://zwiftpower.com/events.php?zid=173965)|[31st #2](https://zwiftpower.com/events.php?zid=155997)|[39th #3](https://zwiftpower.com/events.php?zid=159644)
A|13|4959|[Eric Loiselle🔰](https://zwiftpower\.com/profile\.php?z=81778)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[🥈 #1](https://zwiftpower.com/events.php?zid=152781)|[🥉 #3](https://zwiftpower.com/events.php?zid=159644)|[5th #6](https://zwiftpower.com/events.php?zid=170592)|[6th #4](https://zwiftpower.com/events.php?zid=163028)|[7th #5](https://zwiftpower.com/events.php?zid=167034)|—
A|14|4905|[Mat Wilson](https://zwiftpower\.com/profile\.php?z=322602)|[POPTARTZ](https://zwiftpower\.com/team\.php?id=2880)|[4th #8](https://zwiftpower.com/events.php?zid=176476)|[7th #7](https://zwiftpower.com/events.php?zid=173965)|[9th #5](https://zwiftpower.com/events.php?zid=167034)|[11th #6](https://zwiftpower.com/events.php?zid=170592)|[19th #3](https://zwiftpower.com/events.php?zid=159644)|—
A|15|4845|[Peter Mogg 🍻🚲](https://zwiftpower\.com/profile\.php?z=467127)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[12th #5](https://zwiftpower.com/events.php?zid=167034)|[14th #4](https://zwiftpower.com/events.php?zid=163028)|[16th #6](https://zwiftpower.com/events.php?zid=170592)|[18th #3](https://zwiftpower.com/events.php?zid=159644)|[20th #2](https://zwiftpower.com/events.php?zid=155997)|—
A|16|4837|[David Charles](https://zwiftpower\.com/profile\.php?z=4568)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[7th #8](https://zwiftpower.com/events.php?zid=176476)|[11th #7](https://zwiftpower.com/events.php?zid=173965)|[19th #4](https://zwiftpower.com/events.php?zid=163028)|[23rd #1](https://zwiftpower.com/events.php?zid=152781)|[24th #5](https://zwiftpower.com/events.php?zid=167034)|—
A|17|4821|[Max Sibum](https://zwiftpower\.com/profile\.php?z=156551)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[9th #7](https://zwiftpower.com/events.php?zid=173965)|[12th #2](https://zwiftpower.com/events.php?zid=155997)|[12th #8](https://zwiftpower.com/events.php?zid=176476)|[26th #4](https://zwiftpower.com/events.php?zid=163028)|[33rd #6](https://zwiftpower.com/events.php?zid=170592)|—
A|17|4821|[Alex Side](https://zwiftpower\.com/profile\.php?z=214679)|—|[9th #8](https://zwiftpower.com/events.php?zid=176476)|[11th #4](https://zwiftpower.com/events.php?zid=163028)|[20th #5](https://zwiftpower.com/events.php?zid=167034)|[24th #3](https://zwiftpower.com/events.php?zid=159644)|[28th #2](https://zwiftpower.com/events.php?zid=155997)|—
A|18|4773|[Adrian Amos](https://zwiftpower\.com/profile\.php?z=73357)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[16th #3](https://zwiftpower.com/events.php?zid=159644)|[19th #2](https://zwiftpower.com/events.php?zid=155997)|[19th #7](https://zwiftpower.com/events.php?zid=173965)|[27th #4](https://zwiftpower.com/events.php?zid=163028)|[35th #5](https://zwiftpower.com/events.php?zid=167034)|—
A|19|4763|[Jean\-Yan Perron](https://zwiftpower\.com/profile\.php?z=390308)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|[19th #5](https://zwiftpower.com/events.php?zid=167034)|[23rd #3](https://zwiftpower.com/events.php?zid=159644)|[25th #6](https://zwiftpower.com/events.php?zid=170592)|[26th #2](https://zwiftpower.com/events.php?zid=155997)|[28th #4](https://zwiftpower.com/events.php?zid=163028)|—
A|20|4663|[John Cservek \[AFC\] 🔰](https://zwiftpower\.com/profile\.php?z=3228)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[30th #1](https://zwiftpower.com/events.php?zid=152781)|[30th #2](https://zwiftpower.com/events.php?zid=155997)|[34th #5](https://zwiftpower.com/events.php?zid=167034)|[38th #3](https://zwiftpower.com/events.php?zid=159644)|[39th #4](https://zwiftpower.com/events.php?zid=163028)|—
A|21|3962|[Maverick Millar](https://zwiftpower\.com/profile\.php?z=165166)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥈 #6](https://zwiftpower.com/events.php?zid=170592)|[🥉 #7](https://zwiftpower.com/events.php?zid=173965)|[4th #5](https://zwiftpower.com/events.php?zid=167034)|[12th #1](https://zwiftpower.com/events.php?zid=152781)|—|—
A|22|3952|[M O 🔰](https://zwiftpower\.com/profile\.php?z=45460)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[🥉 #4](https://zwiftpower.com/events.php?zid=163028)|[5th #3](https://zwiftpower.com/events.php?zid=159644)|[7th #2](https://zwiftpower.com/events.php?zid=155997)|[11th #1](https://zwiftpower.com/events.php?zid=152781)|—|—
A|22|3952|[Warren Muir 🔰](https://zwiftpower\.com/profile\.php?z=68432)|—|[🥉 #2](https://zwiftpower.com/events.php?zid=155997)|[6th #5](https://zwiftpower.com/events.php?zid=167034)|[7th #4](https://zwiftpower.com/events.php?zid=163028)|[10th #3](https://zwiftpower.com/events.php?zid=159644)|—|—
A|23|3816|[Stéphane Sévigny](https://zwiftpower\.com/profile\.php?z=161752)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[17th #5](https://zwiftpower.com/events.php?zid=167034)|[21st #1](https://zwiftpower.com/events.php?zid=152781)|[27th #2](https://zwiftpower.com/events.php?zid=155997)|[29th #4](https://zwiftpower.com/events.php?zid=163028)|—|—
A|24|3808|[Tofu Anarchy](https://zwiftpower\.com/profile\.php?z=140251)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[16th #8](https://zwiftpower.com/events.php?zid=176476)|[24th #7](https://zwiftpower.com/events.php?zid=173965)|[27th #1](https://zwiftpower.com/events.php?zid=152781)|[31st #5](https://zwiftpower.com/events.php?zid=167034)|—|—
A|25|3774|[Guy Bergeron \(JZQc\)](https://zwiftpower\.com/profile\.php?z=433249)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[22nd #4](https://zwiftpower.com/events.php?zid=163028)|[26th #7](https://zwiftpower.com/events.php?zid=173965)|[33rd #1](https://zwiftpower.com/events.php?zid=152781)|[34th #2](https://zwiftpower.com/events.php?zid=155997)|—|—
A|26|3742|[Sébastien Desgagné](https://zwiftpower\.com/profile\.php?z=189612)|—|[21st #8](https://zwiftpower.com/events.php?zid=176476)|[29th #7](https://zwiftpower.com/events.php?zid=173965)|[40th #6](https://zwiftpower.com/events.php?zid=170592)|[41st #3](https://zwiftpower.com/events.php?zid=159644)|—|—
A|27|2981|[Ryan Larson VR2209 🔰](https://zwiftpower\.com/profile\.php?z=263018)|[VR2209](https://zwiftpower\.com/team\.php?id=3121)|[🥉 #6](https://zwiftpower.com/events.php?zid=170592)|[4th #2](https://zwiftpower.com/events.php?zid=155997)|[4th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|28|2975|[Casey Schumm 🍕🔰](https://zwiftpower\.com/profile\.php?z=1412)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[🥈 #8](https://zwiftpower.com/events.php?zid=176476)|[5th #5](https://zwiftpower.com/events.php?zid=167034)|[7th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|29|2959|[Brian Hodges](https://zwiftpower\.com/profile\.php?z=399270)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=163028)|[8th #6](https://zwiftpower.com/events.php?zid=170592)|[13th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|29|2959|[Danny Mossman](https://zwiftpower\.com/profile\.php?z=468401)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th #2](https://zwiftpower.com/events.php?zid=155997)|[7th #3](https://zwiftpower.com/events.php?zid=159644)|[10th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—
A|30|2951|[Frederic Beaulieu](https://zwiftpower\.com/profile\.php?z=189463)|—|[8th #3](https://zwiftpower.com/events.php?zid=159644)|[9th #1](https://zwiftpower.com/events.php?zid=152781)|[9th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
A|31|2949|[Eric Sorenson 🦔🔰](https://zwiftpower\.com/profile\.php?z=50641)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[4th #7](https://zwiftpower.com/events.php?zid=173965)|[10th #6](https://zwiftpower.com/events.php?zid=170592)|[13th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—
A|32|2931|[JP Venne](https://zwiftpower\.com/profile\.php?z=50835)|[CLAN KNOX](https://zwiftpower\.com/team\.php?id=3073)|[🥉 #5](https://zwiftpower.com/events.php?zid=167034)|[5th #4](https://zwiftpower.com/events.php?zid=163028)|[28th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|33|2913|[Israël Mercier](https://zwiftpower\.com/profile\.php?z=258075)|—|[6th #7](https://zwiftpower.com/events.php?zid=173965)|[14th #6](https://zwiftpower.com/events.php?zid=170592)|[25th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|34|2909|[Benedict Mercier](https://zwiftpower\.com/profile\.php?z=429891)|—|[5th #7](https://zwiftpower.com/events.php?zid=173965)|[13th #6](https://zwiftpower.com/events.php?zid=170592)|[29th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|35|2897|[Mike Baird \- 1R](https://zwiftpower\.com/profile\.php?z=3313)|[POPTARTZ](https://zwiftpower\.com/team\.php?id=2880)|[10th #7](https://zwiftpower.com/events.php?zid=173965)|[13th #5](https://zwiftpower.com/events.php?zid=167034)|[30th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|36|2891|[Edmund Leduc ⓥ](https://zwiftpower\.com/profile\.php?z=611056)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[16th #7](https://zwiftpower.com/events.php?zid=173965)|[19th #6](https://zwiftpower.com/events.php?zid=170592)|[21st #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|37|2879|[Mathieu Hould](https://zwiftpower\.com/profile\.php?z=71850)|—|[18th #2](https://zwiftpower.com/events.php?zid=155997)|[18th #6](https://zwiftpower.com/events.php?zid=170592)|[26th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|38|2877|[Andrew Walter 🐿️🔰](https://zwiftpower\.com/profile\.php?z=478707)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[18th #1](https://zwiftpower.com/events.php?zid=152781)|[21st #2](https://zwiftpower.com/events.php?zid=155997)|[24th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|39|2873|[Daniel Lucas](https://zwiftpower\.com/profile\.php?z=41577)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[14th #2](https://zwiftpower.com/events.php?zid=155997)|[24th #4](https://zwiftpower.com/events.php?zid=163028)|[27th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—
A|40|2867|[Andrew Sargent](https://zwiftpower\.com/profile\.php?z=46394)|—|[13th #4](https://zwiftpower.com/events.php?zid=163028)|[22nd #1](https://zwiftpower.com/events.php?zid=152781)|[33rd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
A|41|2859|[Adam Schwengler](https://zwiftpower\.com/profile\.php?z=177457)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[17th #2](https://zwiftpower.com/events.php?zid=155997)|[26th #1](https://zwiftpower.com/events.php?zid=152781)|[29th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|42|2841|[Guidel'Esse\#JZQC](https://zwiftpower\.com/profile\.php?z=455587)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #2](https://zwiftpower.com/events.php?zid=155997)|[28th #5](https://zwiftpower.com/events.php?zid=167034)|[38th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
A|43|2827|[Yvan Bérubé](https://zwiftpower\.com/profile\.php?z=182836)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[25th #5](https://zwiftpower.com/events.php?zid=167034)|[31st #4](https://zwiftpower.com/events.php?zid=163028)|[32nd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
A|44|1988|[键盘仁波切](https://zwiftpower\.com/profile\.php?z=471022)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[🥈 #3](https://zwiftpower.com/events.php?zid=159644)|[5th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—
A|45|1974|[Nick Proulx](https://zwiftpower\.com/profile\.php?z=226916)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[6th #2](https://zwiftpower.com/events.php?zid=155997)|[8th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—
A|46|1968|[jesse riley1488](https://zwiftpower\.com/profile\.php?z=149529)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[6th #1](https://zwiftpower.com/events.php?zid=152781)|[11th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
A|47|1960|[Luc Guilbeault](https://zwiftpower\.com/profile\.php?z=212990)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|[9th #4](https://zwiftpower.com/events.php?zid=163028)|[12th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|48|1932|[Fred Goulet](https://zwiftpower\.com/profile\.php?z=350021)|[VelociT](https://zwiftpower\.com/team\.php?id=1071)|[11th #8](https://zwiftpower.com/events.php?zid=176476)|[24th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
A|49|1918|[Marc\-Antoine Pouliot](https://zwiftpower\.com/profile\.php?z=171939)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|[11th #3](https://zwiftpower.com/events.php?zid=159644)|[31st #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
A|50|1916|[Jake Campbell](https://zwiftpower\.com/profile\.php?z=444969)|[BUB](https://zwiftpower\.com/team\.php?id=2059)|[16th #4](https://zwiftpower.com/events.php?zid=163028)|[27th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|51|1914|[Matt Wheeler 🦊 \[VFox\] 🔰](https://zwiftpower\.com/profile\.php?z=452777)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[21st #7](https://zwiftpower.com/events.php?zid=173965)|[23rd #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—
A|52|1902|[Olivier Hebert Cannondale 4IIII](https://zwiftpower\.com/profile\.php?z=939073)|—|[14th #3](https://zwiftpower.com/events.php?zid=159644)|[36th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
A|53|1898|[Blair Drader ZHR B TeamWKG](https://zwiftpower\.com/profile\.php?z=520895)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[21st #4](https://zwiftpower.com/events.php?zid=163028)|[31st #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|54|1894|[Marc Fourcaudot](https://zwiftpower\.com/profile\.php?z=330044)|—|[19th #1](https://zwiftpower.com/events.php?zid=152781)|[35th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
A|55|1890|[Chris Hardee](https://zwiftpower\.com/profile\.php?z=569380)|[DCVELO](https://zwiftpower\.com/team\.php?id=3894)|[24th #8](https://zwiftpower.com/events.php?zid=176476)|[32nd #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—
A|56|1882|[Serge Bisson](https://zwiftpower\.com/profile\.php?z=66189)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[23rd #5](https://zwiftpower.com/events.php?zid=167034)|[37th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|57|1876|[Robert Moore](https://zwiftpower\.com/profile\.php?z=98365)|—|[31st #7](https://zwiftpower.com/events.php?zid=173965)|[32nd #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
A|58|1868|[Tom21 Turner3998](https://zwiftpower\.com/profile\.php?z=453469)|—|[25th #7](https://zwiftpower.com/events.php?zid=173965)|[42nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|59|1866|[kevin thompson](https://zwiftpower\.com/profile\.php?z=92113)|—|[32nd #5](https://zwiftpower.com/events.php?zid=167034)|[36th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
A|59|1866|[L\.Blueberry](https://zwiftpower\.com/profile\.php?z=102968)|—|[32nd #7](https://zwiftpower.com/events.php?zid=173965)|[36th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
A|60|1862|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[32nd #4](https://zwiftpower.com/events.php?zid=163028)|[38th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
A|61|1860|[Paul Nederveld](https://zwiftpower\.com/profile\.php?z=151748)|—|[31st #1](https://zwiftpower.com/events.php?zid=152781)|[40th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
A|62|1848|[John Morris](https://zwiftpower\.com/profile\.php?z=116770)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[37th #6](https://zwiftpower.com/events.php?zid=170592)|[40th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
A|63|995|[Michael Podzaline](https://zwiftpower\.com/profile\.php?z=15522)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[🥉 #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
A|64|993|[Seth G ☁️ 🔰](https://zwiftpower\.com/profile\.php?z=463284)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[4th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|64|993|[K KONNO](https://zwiftpower\.com/profile\.php?z=563135)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[4th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|65|991|[Tom Burke](https://zwiftpower\.com/profile\.php?z=4641)|[Team AM](https://zwiftpower\.com/team\.php?id=2065)|[5th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
A|66|989|[Julian White](https://zwiftpower\.com/profile\.php?z=563359)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[6th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|67|985|[Jero Nefkens](https://zwiftpower\.com/profile\.php?z=78992)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[8th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
A|67|985|[Ryan Mckay](https://zwiftpower\.com/profile\.php?z=245426)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[8th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|68|983|[PJ Kings](https://zwiftpower\.com/profile\.php?z=13546)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|[9th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|69|981|[Eric Gauvin](https://zwiftpower\.com/profile\.php?z=160112)|—|[10th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|69|981|[Max TT 🔰](https://zwiftpower\.com/profile\.php?z=559553)|—|[10th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
A|69|981|[Changho Lee\-TDM](https://zwiftpower\.com/profile\.php?z=1016795)|—|[10th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|70|977|[John Moore](https://zwiftpower\.com/profile\.php?z=306865)|[BTX](https://zwiftpower\.com/team\.php?id=2131)|[12th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|71|975|[Mathieu Thibault](https://zwiftpower\.com/profile\.php?z=318656)|—|[13th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
A|71|975|[Glenn Kempton 🙈](https://zwiftpower\.com/profile\.php?z=725699)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[13th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
A|72|973|[Paul Lavoie](https://zwiftpower\.com/profile\.php?z=13944)|—|[14th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|72|973|[Scott McLean](https://zwiftpower\.com/profile\.php?z=180616)|[Cannondale\-USA](https://zwiftpower\.com/team\.php?id=4103)|[14th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|73|971|[蔡 蔡](https://zwiftpower\.com/profile\.php?z=49402)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[15th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
A|73|971|[David Kim Amado 🥐 🍰 🥖](https://zwiftpower\.com/profile\.php?z=690464)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[15th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|74|969|"[Mark ""Dallas"" Henry](https://zwiftpower\.com/profile\.php?z=332807)"|—|[16th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|75|967|[Brayden Halvorson](https://zwiftpower\.com/profile\.php?z=196936)|—|[17th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|75|967|[N 8](https://zwiftpower\.com/profile\.php?z=1158798)|—|[17th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
A|76|961|[Eric Ellis](https://zwiftpower\.com/profile\.php?z=59459)|—|[20th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|77|959|[Andrés Boza](https://zwiftpower\.com/profile\.php?z=460661)|—|[21st #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|78|957|[Dean Schives](https://zwiftpower\.com/profile\.php?z=65698)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|[22nd #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|79|955|[Chris Gregory](https://zwiftpower\.com/profile\.php?z=149534)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[23rd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|79|955|[Tomo Ishii \(ISHITOMO\)](https://zwiftpower\.com/profile\.php?z=707432)|—|[23rd #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
A|80|941|[Mark Craig](https://zwiftpower\.com/profile\.php?z=266006)|—|[30th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
A|80|941|[Pierre\-Luc Samson](https://zwiftpower\.com/profile\.php?z=652828)|—|[30th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|80|941|[Jeffrey Werner](https://zwiftpower\.com/profile\.php?z=1037294)|—|[30th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|81|937|[Dylan Ogle](https://zwiftpower\.com/profile\.php?z=348233)|[Storage Squad](https://zwiftpower\.com/team\.php?id=4061)|[32nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
A|82|933|[Justin Mayfield](https://zwiftpower\.com/profile\.php?z=948550)|—|[34th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|83|931|[Simon Beaulieu](https://zwiftpower\.com/profile\.php?z=530995)|—|[35th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
A|84|927|[Joseph Bank 🔰](https://zwiftpower\.com/profile\.php?z=11837)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[37th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
A|85|923|[Christian Clarkson \(Tower Cycling\)](https://zwiftpower\.com/profile\.php?z=74394)|—|[39th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|85|923|[Mustafa Branche](https://zwiftpower\.com/profile\.php?z=561065)|[DUCE](https://zwiftpower\.com/team\.php?id=3830)|[39th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|86|919|[Brent Krause](https://zwiftpower\.com/profile\.php?z=475280)|—|[41st #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|86|919|[Nicolas Fortin](https://zwiftpower\.com/profile\.php?z=490825)|—|[41st #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
A|87|917|[Tyson Wieland](https://zwiftpower\.com/profile\.php?z=430793)|[SAKARI](https://zwiftpower\.com/team\.php?id=1187)|[42nd #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
A|87|917|[Lucas Versluys](https://zwiftpower\.com/profile\.php?z=815941)|—|[42nd #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|1|5972|[Matthew Martin](https://zwiftpower\.com/profile\.php?z=7515)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[🥇 #7](https://zwiftpower.com/events.php?zid=173965)|[🥇 #8](https://zwiftpower.com/events.php?zid=176476)|[🥈 #1](https://zwiftpower.com/events.php?zid=152781)|[🥈 #5](https://zwiftpower.com/events.php?zid=167034)|[4th #6](https://zwiftpower.com/events.php?zid=170592)|[7th #3](https://zwiftpower.com/events.php?zid=159644)
B|2|5952|[scott carroll oregon](https://zwiftpower\.com/profile\.php?z=17356)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥉 #3](https://zwiftpower.com/events.php?zid=159644)|[🥉 #5](https://zwiftpower.com/events.php?zid=167034)|[🥉 #7](https://zwiftpower.com/events.php?zid=173965)|[5th #8](https://zwiftpower.com/events.php?zid=176476)|[6th #4](https://zwiftpower.com/events.php?zid=163028)|[7th #1](https://zwiftpower.com/events.php?zid=152781)
B|3|5906|[Brent Dickie](https://zwiftpower\.com/profile\.php?z=210021)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[5th #6](https://zwiftpower.com/events.php?zid=170592)|[5th #7](https://zwiftpower.com/events.php?zid=173965)|[6th #8](https://zwiftpower.com/events.php?zid=176476)|[9th #4](https://zwiftpower.com/events.php?zid=163028)|[12th #1](https://zwiftpower.com/events.php?zid=152781)|[13th #3](https://zwiftpower.com/events.php?zid=159644)
B|4|5894|[tom wilson \[X\]](https://zwiftpower\.com/profile\.php?z=380248)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[6th #1](https://zwiftpower.com/events.php?zid=152781)|[9th #2](https://zwiftpower.com/events.php?zid=155997)|[9th #6](https://zwiftpower.com/events.php?zid=170592)|[9th #7](https://zwiftpower.com/events.php?zid=173965)|[9th #8](https://zwiftpower.com/events.php?zid=176476)|[14th #5](https://zwiftpower.com/events.php?zid=167034)
B|5|5874|[Tommy Belanger](https://zwiftpower\.com/profile\.php?z=444759)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #2](https://zwiftpower.com/events.php?zid=155997)|[4th #5](https://zwiftpower.com/events.php?zid=167034)|[5th #1](https://zwiftpower.com/events.php?zid=152781)|[7th #7](https://zwiftpower.com/events.php?zid=173965)|[13th #8](https://zwiftpower.com/events.php?zid=176476)|[33rd #4](https://zwiftpower.com/events.php?zid=163028)
B|6|5830|[Silas Swaim \[CRYO\-GEN\] B](https://zwiftpower\.com/profile\.php?z=95092)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[4th #7](https://zwiftpower.com/events.php?zid=173965)|[7th #8](https://zwiftpower.com/events.php?zid=176476)|[14th #3](https://zwiftpower.com/events.php?zid=159644)|[14th #4](https://zwiftpower.com/events.php?zid=163028)|[22nd #5](https://zwiftpower.com/events.php?zid=167034)|[27th #2](https://zwiftpower.com/events.php?zid=155997)
B|7|5826|[Nathan Kershner 🅥](https://zwiftpower\.com/profile\.php?z=920279)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[9th #1](https://zwiftpower.com/events.php?zid=152781)|[10th #4](https://zwiftpower.com/events.php?zid=163028)|[10th #7](https://zwiftpower.com/events.php?zid=173965)|[12th #2](https://zwiftpower.com/events.php?zid=155997)|[14th #8](https://zwiftpower.com/events.php?zid=176476)|[35th #6](https://zwiftpower.com/events.php?zid=170592)
B|8|5788|[Jim Horstmann Ⓥ](https://zwiftpower\.com/profile\.php?z=182374)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[11th #8](https://zwiftpower.com/events.php?zid=176476)|[12th #7](https://zwiftpower.com/events.php?zid=173965)|[20th #4](https://zwiftpower.com/events.php?zid=163028)|[21st #3](https://zwiftpower.com/events.php?zid=159644)|[22nd #1](https://zwiftpower.com/events.php?zid=152781)|[23rd #5](https://zwiftpower.com/events.php?zid=167034)
B|9|5770|[Florian Lungu](https://zwiftpower\.com/profile\.php?z=225897)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|[11th #1](https://zwiftpower.com/events.php?zid=152781)|[17th #7](https://zwiftpower.com/events.php?zid=173965)|[19th #8](https://zwiftpower.com/events.php?zid=176476)|[20th #6](https://zwiftpower.com/events.php?zid=170592)|[25th #3](https://zwiftpower.com/events.php?zid=159644)|[26th #2](https://zwiftpower.com/events.php?zid=155997)
B|9|5770|[David MacKusick](https://zwiftpower\.com/profile\.php?z=364871)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[12th #5](https://zwiftpower.com/events.php?zid=167034)|[18th #2](https://zwiftpower.com/events.php?zid=155997)|[20th #1](https://zwiftpower.com/events.php?zid=152781)|[21st #6](https://zwiftpower.com/events.php?zid=170592)|[23rd #4](https://zwiftpower.com/events.php?zid=163028)|[24th #3](https://zwiftpower.com/events.php?zid=159644)
B|10|5738|[Carl Desrosiers](https://zwiftpower\.com/profile\.php?z=413249)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[9th #5](https://zwiftpower.com/events.php?zid=167034)|[12th #8](https://zwiftpower.com/events.php?zid=176476)|[14th #7](https://zwiftpower.com/events.php?zid=173965)|[29th #3](https://zwiftpower.com/events.php?zid=159644)|[31st #4](https://zwiftpower.com/events.php?zid=163028)|[39th #2](https://zwiftpower.com/events.php?zid=155997)
B|11|5710|[Justin Terry Ⓥ](https://zwiftpower\.com/profile\.php?z=85080)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[19th #1](https://zwiftpower.com/events.php?zid=152781)|[20th #7](https://zwiftpower.com/events.php?zid=173965)|[24th #4](https://zwiftpower.com/events.php?zid=163028)|[25th #6](https://zwiftpower.com/events.php?zid=170592)|[26th #5](https://zwiftpower.com/events.php?zid=167034)|[34th #2](https://zwiftpower.com/events.php?zid=155997)
B|12|5706|[Eric Guimond](https://zwiftpower\.com/profile\.php?z=931823)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #7](https://zwiftpower.com/events.php?zid=173965)|[19th #3](https://zwiftpower.com/events.php?zid=159644)|[21st #8](https://zwiftpower.com/events.php?zid=176476)|[23rd #6](https://zwiftpower.com/events.php?zid=170592)|[29th #2](https://zwiftpower.com/events.php?zid=155997)|[43rd #4](https://zwiftpower.com/events.php?zid=163028)
B|13|5666|[Ben Nick\[CRYOGEN\]B](https://zwiftpower\.com/profile\.php?z=751441)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[15th #6](https://zwiftpower.com/events.php?zid=170592)|[17th #8](https://zwiftpower.com/events.php?zid=176476)|[20th #5](https://zwiftpower.com/events.php?zid=167034)|[33rd #3](https://zwiftpower.com/events.php?zid=159644)|[42nd #4](https://zwiftpower.com/events.php?zid=163028)|[43rd #2](https://zwiftpower.com/events.php?zid=155997)
B|14|5546|[LOUIS\-PIERRE DUPUIS\(pjcgrch\)\(JZQc\) B 62 years](https://zwiftpower\.com/profile\.php?z=261617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[26th #8](https://zwiftpower.com/events.php?zid=176476)|[29th #6](https://zwiftpower.com/events.php?zid=170592)|[32nd #5](https://zwiftpower.com/events.php?zid=167034)|[32nd #7](https://zwiftpower.com/events.php?zid=173965)|[53rd #3](https://zwiftpower.com/events.php?zid=159644)|[58th #4](https://zwiftpower.com/events.php?zid=163028)
B|15|4955|[David Partridge](https://zwiftpower\.com/profile\.php?z=238940)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥇 #5](https://zwiftpower.com/events.php?zid=167034)|[🥈 #6](https://zwiftpower.com/events.php?zid=170592)|[4th #4](https://zwiftpower.com/events.php?zid=163028)|[8th #1](https://zwiftpower.com/events.php?zid=152781)|[10th #2](https://zwiftpower.com/events.php?zid=155997)|—
B|16|4933|[Tom Schultz](https://zwiftpower\.com/profile\.php?z=214796)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉 #2](https://zwiftpower.com/events.php?zid=155997)|[🥉 #6](https://zwiftpower.com/events.php?zid=170592)|[4th #1](https://zwiftpower.com/events.php?zid=152781)|[10th #5](https://zwiftpower.com/events.php?zid=167034)|[16th #4](https://zwiftpower.com/events.php?zid=163028)|—
B|17|4823|[Chad Skinner](https://zwiftpower\.com/profile\.php?z=83631)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[16th #8](https://zwiftpower.com/events.php?zid=176476)|[18th #3](https://zwiftpower.com/events.php?zid=159644)|[19th #2](https://zwiftpower.com/events.php?zid=155997)|[19th #4](https://zwiftpower.com/events.php?zid=163028)|[19th #7](https://zwiftpower.com/events.php?zid=173965)|—
B|18|4803|[Simon Piniel](https://zwiftpower\.com/profile\.php?z=202301)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[11th #3](https://zwiftpower.com/events.php?zid=159644)|[15th #8](https://zwiftpower.com/events.php?zid=176476)|[18th #4](https://zwiftpower.com/events.php?zid=163028)|[27th #6](https://zwiftpower.com/events.php?zid=170592)|[30th #2](https://zwiftpower.com/events.php?zid=155997)|—
B|19|4691|[Jeff Arnier, Sr\. CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=322928)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[24th #8](https://zwiftpower.com/events.php?zid=176476)|[25th #7](https://zwiftpower.com/events.php?zid=173965)|[32nd #2](https://zwiftpower.com/events.php?zid=155997)|[32nd #3](https://zwiftpower.com/events.php?zid=159644)|[44th #4](https://zwiftpower.com/events.php?zid=163028)|—
B|20|4645|[Bruce MacDonald Cryo\-Gen](https://zwiftpower\.com/profile\.php?z=76296)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[18th #8](https://zwiftpower.com/events.php?zid=176476)|[24th #7](https://zwiftpower.com/events.php?zid=173965)|[34th #5](https://zwiftpower.com/events.php?zid=167034)|[48th #3](https://zwiftpower.com/events.php?zid=159644)|[56th #2](https://zwiftpower.com/events.php?zid=155997)|—
B|21|4591|[Jimmy Merckx](https://zwiftpower\.com/profile\.php?z=326614)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[31st #6](https://zwiftpower.com/events.php?zid=170592)|[38th #2](https://zwiftpower.com/events.php?zid=155997)|[41st #5](https://zwiftpower.com/events.php?zid=167034)|[45th #3](https://zwiftpower.com/events.php?zid=159644)|[52nd #4](https://zwiftpower.com/events.php?zid=163028)|—
B|22|3970|[Steven Drost](https://zwiftpower\.com/profile\.php?z=115162)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[🥉 #4](https://zwiftpower.com/events.php?zid=163028)|[4th #3](https://zwiftpower.com/events.php?zid=159644)|[5th #2](https://zwiftpower.com/events.php?zid=155997)|[5th #5](https://zwiftpower.com/events.php?zid=167034)|—|—
B|23|3938|[Jr Caron](https://zwiftpower\.com/profile\.php?z=489115)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #2](https://zwiftpower.com/events.php?zid=155997)|[6th #6](https://zwiftpower.com/events.php?zid=170592)|[9th #3](https://zwiftpower.com/events.php?zid=159644)|[12th #4](https://zwiftpower.com/events.php?zid=163028)|—|—
B|24|3808|[Mike Dooley](https://zwiftpower\.com/profile\.php?z=535701)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|[19th #6](https://zwiftpower.com/events.php?zid=170592)|[23rd #3](https://zwiftpower.com/events.php?zid=159644)|[25th #4](https://zwiftpower.com/events.php?zid=163028)|[31st #2](https://zwiftpower.com/events.php?zid=155997)|—|—
B|25|3804|[Jeff Allen](https://zwiftpower\.com/profile\.php?z=225142)|—|[8th #7](https://zwiftpower.com/events.php?zid=173965)|[10th #1](https://zwiftpower.com/events.php?zid=152781)|[27th #4](https://zwiftpower.com/events.php?zid=163028)|[55th #3](https://zwiftpower.com/events.php?zid=159644)|—|—
B|26|3798|[Francois Caron JZQc \(B\)](https://zwiftpower\.com/profile\.php?z=437205)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[16th #6](https://zwiftpower.com/events.php?zid=170592)|[22nd #7](https://zwiftpower.com/events.php?zid=173965)|[28th #3](https://zwiftpower.com/events.php?zid=159644)|[37th #2](https://zwiftpower.com/events.php?zid=155997)|—|—
B|27|3758|[Russell S](https://zwiftpower\.com/profile\.php?z=11426)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[24th #1](https://zwiftpower.com/events.php?zid=152781)|[28th #6](https://zwiftpower.com/events.php?zid=170592)|[35th #7](https://zwiftpower.com/events.php?zid=173965)|[36th #3](https://zwiftpower.com/events.php?zid=159644)|—|—
B|28|3728|[Don Morgan](https://zwiftpower\.com/profile\.php?z=16961)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[22nd #6](https://zwiftpower.com/events.php?zid=170592)|[31st #1](https://zwiftpower.com/events.php?zid=152781)|[35th #5](https://zwiftpower.com/events.php?zid=167034)|[50th #2](https://zwiftpower.com/events.php?zid=155997)|—|—
B|29|3682|[Jonathan Bergeron\#JZQc](https://zwiftpower\.com/profile\.php?z=418430)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[29th #1](https://zwiftpower.com/events.php?zid=152781)|[33rd #5](https://zwiftpower.com/events.php?zid=167034)|[41st #4](https://zwiftpower.com/events.php?zid=163028)|[58th #2](https://zwiftpower.com/events.php?zid=155997)|—|—
B|30|3662|[Keven Cormier \(JZQc\)](https://zwiftpower\.com/profile\.php?z=892751)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[32nd #6](https://zwiftpower.com/events.php?zid=170592)|[39th #5](https://zwiftpower.com/events.php?zid=167034)|[46th #3](https://zwiftpower.com/events.php?zid=159644)|[54th #4](https://zwiftpower.com/events.php?zid=163028)|—|—
B|31|2975|[John Morris](https://zwiftpower\.com/profile\.php?z=116770)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #4](https://zwiftpower.com/events.php?zid=163028)|[5th #3](https://zwiftpower.com/events.php?zid=159644)|[7th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
B|32|2951|[Bartek Ksok](https://zwiftpower\.com/profile\.php?z=226723)|—|[🥈 #3](https://zwiftpower.com/events.php?zid=159644)|[🥉 #1](https://zwiftpower.com/events.php?zid=152781)|[21st #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
B|33|2917|[Steve Klins \[JZQC\]](https://zwiftpower\.com/profile\.php?z=166167)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[13th #5](https://zwiftpower.com/events.php?zid=167034)|[14th #6](https://zwiftpower.com/events.php?zid=170592)|[16th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
B|34|2905|[JP Provost1936](https://zwiftpower\.com/profile\.php?z=846208)|—|[12th #6](https://zwiftpower.com/events.php?zid=170592)|[15th #1](https://zwiftpower.com/events.php?zid=152781)|[22nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
B|35|2895|[James Kupernik](https://zwiftpower\.com/profile\.php?z=166558)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|[10th #6](https://zwiftpower.com/events.php?zid=170592)|[20th #2](https://zwiftpower.com/events.php?zid=155997)|[24th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—
B|36|2891|[Adam Weddington 🔰](https://zwiftpower\.com/profile\.php?z=206248)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[14th #1](https://zwiftpower.com/events.php?zid=152781)|[14th #2](https://zwiftpower.com/events.php?zid=155997)|[28th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—
B|37|2887|[Guy M](https://zwiftpower\.com/profile\.php?z=161461)|—|[13th #1](https://zwiftpower.com/events.php?zid=152781)|[22nd #4](https://zwiftpower.com/events.php?zid=163028)|[23rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—
B|38|2841|[Jason Funderburk](https://zwiftpower\.com/profile\.php?z=114715)|—|[8th #8](https://zwiftpower.com/events.php?zid=176476)|[29th #4](https://zwiftpower.com/events.php?zid=163028)|[44th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
B|38|2841|[Sylvain Roux](https://zwiftpower\.com/profile\.php?z=167374)|[JFZ](https://zwiftpower\.com/team\.php?id=3974)|[17th #4](https://zwiftpower.com/events.php?zid=163028)|[29th #5](https://zwiftpower.com/events.php?zid=167034)|[35th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
B|39|2833|[Demian Bailey](https://zwiftpower\.com/profile\.php?z=413276)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[18th #6](https://zwiftpower.com/events.php?zid=170592)|[27th #5](https://zwiftpower.com/events.php?zid=167034)|[40th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—
B|40|2823|[J\-P Mercier](https://zwiftpower\.com/profile\.php?z=244856)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[21st #5](https://zwiftpower.com/events.php?zid=167034)|[22nd #2](https://zwiftpower.com/events.php?zid=155997)|[47th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—
B|41|2745|[Pascal Gauthier](https://zwiftpower\.com/profile\.php?z=995428)|—|[40th #6](https://zwiftpower.com/events.php?zid=170592)|[42nd #5](https://zwiftpower.com/events.php?zid=167034)|[47th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
B|42|1972|[Jeremie Fontanaud Scott/RackUltra](https://zwiftpower\.com/profile\.php?z=127235)|[Scott\-RackUltra](https://zwiftpower\.com/team\.php?id=3562)|[7th #4](https://zwiftpower.com/events.php?zid=163028)|[8th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|43|1968|[Enrico Magnani](https://zwiftpower\.com/profile\.php?z=492758)|—|[6th #5](https://zwiftpower.com/events.php?zid=167034)|[11th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
B|44|1950|[Ryan McGee \[DDC\]](https://zwiftpower\.com/profile\.php?z=545017)|[Dutch Diesel Cyc](https://zwiftpower\.com/team\.php?id=2649)|[10th #8](https://zwiftpower.com/events.php?zid=176476)|[16th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
B|45|1926|[Nicolas Fortin](https://zwiftpower\.com/profile\.php?z=490825)|—|[11th #5](https://zwiftpower.com/events.php?zid=167034)|[27th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|46|1912|[Ryosuke Baba](https://zwiftpower\.com/profile\.php?z=585484)|—|[10th #3](https://zwiftpower.com/events.php?zid=159644)|[35th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|47|1904|[Michael Rex Schumacher](https://zwiftpower\.com/profile\.php?z=381825)|—|[24th #6](https://zwiftpower.com/events.php?zid=170592)|[25th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
B|48|1896|[David Navarrete \[ODZ\]](https://zwiftpower\.com/profile\.php?z=194375)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[12th #3](https://zwiftpower.com/events.php?zid=159644)|[41st #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|49|1884|[Ryan Davies](https://zwiftpower\.com/profile\.php?z=412618)|—|[20th #8](https://zwiftpower.com/events.php?zid=176476)|[39th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|50|1882|[Michael Placidi Vegan Italiano/Australiano7558](https://zwiftpower\.com/profile\.php?z=82858)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[23rd #8](https://zwiftpower.com/events.php?zid=176476)|[37th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
B|51|1880|[PP V JZQC](https://zwiftpower\.com/profile\.php?z=788575)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[17th #1](https://zwiftpower.com/events.php?zid=152781)|[44th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|52|1866|[S\. Deslauriers Velozone JZQC](https://zwiftpower\.com/profile\.php?z=182546)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[20th #3](https://zwiftpower.com/events.php?zid=159644)|[48th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
B|52|1866|[Tim Duivesteyn](https://zwiftpower\.com/profile\.php?z=510856)|—|[31st #3](https://zwiftpower.com/events.php?zid=159644)|[37th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
B|53|1864|[Gene Fowler \(BCC\)](https://zwiftpower\.com/profile\.php?z=186111)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[31st #5](https://zwiftpower.com/events.php?zid=167034)|[38th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
B|54|1860|[Scott Farrell ZHR \(E\)](https://zwiftpower\.com/profile\.php?z=581147)|—|[35th #4](https://zwiftpower.com/events.php?zid=163028)|[36th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—
B|55|1858|[O prah Spinfrey](https://zwiftpower\.com/profile\.php?z=838061)|—|[34th #6](https://zwiftpower.com/events.php?zid=170592)|[38th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
B|56|1842|[Brian Griff Dirt](https://zwiftpower\.com/profile\.php?z=423430)|—|[30th #1](https://zwiftpower.com/events.php?zid=152781)|[50th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|57|1838|[Robert Jr Gagné \{DiRT\} \(B\)](https://zwiftpower\.com/profile\.php?z=678429)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[39th #4](https://zwiftpower.com/events.php?zid=163028)|[43rd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|58|1828|[Pierre Maurice Dufresne](https://zwiftpower\.com/profile\.php?z=431492)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[38th #3](https://zwiftpower.com/events.php?zid=159644)|[49th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|59|1822|[Jeff Grabinski \(BSR\)](https://zwiftpower\.com/profile\.php?z=59133)|—|[37th #3](https://zwiftpower.com/events.php?zid=159644)|[53rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|59|1822|[Jason Wright Ⓥ](https://zwiftpower\.com/profile\.php?z=71194)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[42nd #3](https://zwiftpower.com/events.php?zid=159644)|[48th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
B|60|1800|[Ryan La](https://zwiftpower\.com/profile\.php?z=63118)|[WUMS](https://zwiftpower\.com/team\.php?id=4067)|[50th #4](https://zwiftpower.com/events.php?zid=163028)|[51st #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|60|1800|[Chris Tommila](https://zwiftpower\.com/profile\.php?z=984241)|[WUMS](https://zwiftpower\.com/team\.php?id=4067)|[49th #4](https://zwiftpower.com/events.php?zid=163028)|[52nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
B|61|999|[Ben Beger](https://zwiftpower\.com/profile\.php?z=163892)|—|[🥇 #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|61|999|[Ryan Kettleson](https://zwiftpower\.com/profile\.php?z=188883)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥇 #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|61|999|[Jonathan Dee](https://zwiftpower\.com/profile\.php?z=449521)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|61|999|[Blair Drader ZHR B TeamWKG](https://zwiftpower\.com/profile\.php?z=520895)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[🥇 #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|61|999|[Jeffrey Werner](https://zwiftpower\.com/profile\.php?z=1037294)|—|[🥇 #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|62|997|[Gregg Magnus](https://zwiftpower\.com/profile\.php?z=60786)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈 #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|62|997|[kevin thompson](https://zwiftpower\.com/profile\.php?z=92113)|—|[🥈 #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|62|997|[Randy Reichardt \(Above & Beyond Cancer\)](https://zwiftpower\.com/profile\.php?z=235686)|—|[🥈 #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|63|995|[MarcAndre Doucet](https://zwiftpower\.com/profile\.php?z=703114)|[Stallion](https://zwiftpower\.com/team\.php?id=4154)|[🥉 #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|64|993|[Bryan Wenzel](https://zwiftpower\.com/profile\.php?z=90771)|[IOWA](https://zwiftpower\.com/team\.php?id=3870)|[4th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|65|991|[David Anderson](https://zwiftpower\.com/profile\.php?z=385760)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[5th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|66|989|[Neil Ragan 🐘](https://zwiftpower\.com/profile\.php?z=41844)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|[6th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|66|989|[Sergio Estrada](https://zwiftpower\.com/profile\.php?z=882916)|—|[6th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|67|987|[Pete Watson \- 1R](https://zwiftpower\.com/profile\.php?z=106825)|[1R](https://zwiftpower\.com/team\.php?id=387)|[7th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|68|985|[Jean\-François Lorange CCC](https://zwiftpower\.com/profile\.php?z=383096)|[CCC](https://zwiftpower\.com/team\.php?id=2975)|[8th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|68|985|[Rob Peters \(Team EN\)](https://zwiftpower\.com/profile\.php?z=536679)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[8th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|68|985|[Jeremy Pletka](https://zwiftpower\.com/profile\.php?z=1186202)|—|[8th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|69|979|[Jon Roobol](https://zwiftpower\.com/profile\.php?z=12131)|—|[11th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|69|979|[Derek D'Angelo 🐇](https://zwiftpower\.com/profile\.php?z=121836)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[11th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|69|979|[Chilam Balam](https://zwiftpower\.com/profile\.php?z=923642)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[11th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|70|975|[Alexandre Filiatrault](https://zwiftpower\.com/profile\.php?z=302498)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[13th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|70|975|[Ed Braz](https://zwiftpower\.com/profile\.php?z=1179624)|—|[13th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|71|971|[Nicolas Berger](https://zwiftpower\.com/profile\.php?z=193109)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|71|971|[Josh McBee \(Seattle\!\) 4751](https://zwiftpower\.com/profile\.php?z=258134)|—|[15th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|71|971|[Simon Emery3823](https://zwiftpower\.com/profile\.php?z=865804)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|72|969|[Tom Box](https://zwiftpower\.com/profile\.php?z=84)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[16th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|72|969|[Jason Johnson](https://zwiftpower\.com/profile\.php?z=129871)|—|[16th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|72|969|[Martin Faucher](https://zwiftpower\.com/profile\.php?z=762011)|[IRIS\-Bob Cyclo](https://zwiftpower\.com/team\.php?id=3601)|[16th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|73|967|[Frederic Wallman JZQc](https://zwiftpower\.com/profile\.php?z=8531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[17th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|73|967|[John Stoker](https://zwiftpower\.com/profile\.php?z=99918)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[17th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|73|967|[Poochiemoomoos 🐌](https://zwiftpower\.com/profile\.php?z=426785)|—|[17th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|73|967|[Mason Butler](https://zwiftpower\.com/profile\.php?z=481523)|[FlatLine](https://zwiftpower\.com/team\.php?id=2471)|[17th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|74|965|[Martin Roy](https://zwiftpower\.com/profile\.php?z=69421)|—|[18th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|74|965|[Anson Michael Brooks](https://zwiftpower\.com/profile\.php?z=155766)|—|[18th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|74|965|[Nam Lam TeamEN](https://zwiftpower\.com/profile\.php?z=594184)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[18th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|75|963|[Alain St\-Louis](https://zwiftpower\.com/profile\.php?z=449185)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[19th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|76|959|[Francois Menard\-Kilrane](https://zwiftpower\.com/profile\.php?z=195299)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[21st #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|76|959|[Joshua Ricks](https://zwiftpower\.com/profile\.php?z=823201)|—|[21st #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|76|959|[Andrew Zurn](https://zwiftpower\.com/profile\.php?z=985501)|—|[21st #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|77|957|[Cory Dams](https://zwiftpower\.com/profile\.php?z=460663)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[22nd #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|78|955|[Brian McInnes](https://zwiftpower\.com/profile\.php?z=171139)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[23rd #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|78|955|[Dieter Sticht](https://zwiftpower\.com/profile\.php?z=404754)|—|[23rd #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|79|953|[P\. Schaefer](https://zwiftpower\.com/profile\.php?z=1125195)|—|[24th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|80|951|[Jeff Geisler 4278](https://zwiftpower\.com/profile\.php?z=709348)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[25th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|80|951|[Franck FLP 6622](https://zwiftpower\.com/profile\.php?z=1051223)|—|[25th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|81|949|[Bjortmund Malmhagen](https://zwiftpower\.com/profile\.php?z=58622)|[HBC](https://zwiftpower\.com/team\.php?id=1819)|[26th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|81|949|[George York](https://zwiftpower\.com/profile\.php?z=98092)|[COS Racing](https://zwiftpower\.com/team\.php?id=86)|[26th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|81|949|[Nicolas Genest](https://zwiftpower\.com/profile\.php?z=182597)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[26th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|81|949|[Christof F](https://zwiftpower\.com/profile\.php?z=538691)|—|[26th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|81|949|[Hugo Gosselin](https://zwiftpower\.com/profile\.php?z=904218)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[26th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|82|947|[James Easden](https://zwiftpower\.com/profile\.php?z=45531)|—|[27th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
B|82|947|[Carlos Cancio](https://zwiftpower\.com/profile\.php?z=211428)|—|[27th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|82|947|[Jason Blanchette](https://zwiftpower\.com/profile\.php?z=1153335)|—|[27th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|83|945|[Will Peterson\- 9072](https://zwiftpower\.com/profile\.php?z=232287)|[GPVELOTEK](https://zwiftpower\.com/team\.php?id=3056)|[28th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|83|945|[Paul St\-Arneault JZQc](https://zwiftpower\.com/profile\.php?z=635764)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[28th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|83|945|[Myles Fennell](https://zwiftpower\.com/profile\.php?z=928234)|—|[28th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
B|83|945|[Addison Simis](https://zwiftpower\.com/profile\.php?z=1208309)|—|[28th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|84|943|[Lance Mitamura 5699](https://zwiftpower\.com/profile\.php?z=631643)|—|[29th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|85|941|[AJ Jessee](https://zwiftpower\.com/profile\.php?z=261444)|—|[30th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|85|941|[Josh Mahlandt BSC](https://zwiftpower\.com/profile\.php?z=324219)|[BikeSurgeon](https://zwiftpower\.com/team\.php?id=2179)|[30th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|85|941|[Francis Bissonnette3926](https://zwiftpower\.com/profile\.php?z=452415)|—|[30th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|85|941|[David Torrelio🐊🔰](https://zwiftpower\.com/profile\.php?z=490665)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[30th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|85|941|[\# OneKzBiker6813](https://zwiftpower\.com/profile\.php?z=523588)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[30th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|86|939|[Matias Oyola](https://zwiftpower\.com/profile\.php?z=242270)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[31st #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|87|937|[Randy Gemenden](https://zwiftpower\.com/profile\.php?z=780529)|—|[32nd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|88|935|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[33rd #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|88|935|[Sarah\-anne Beaupre](https://zwiftpower\.com/profile\.php?z=613766)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[33rd #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|88|935|[Adam Susong](https://zwiftpower\.com/profile\.php?z=978092)|—|[33rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|89|933|[Erick Kercheval](https://zwiftpower\.com/profile\.php?z=234962)|[NLC](https://zwiftpower\.com/team\.php?id=1957)|[34th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|89|933|[Samuel Moore BSC B1](https://zwiftpower\.com/profile\.php?z=290222)|[BikeSurgeon](https://zwiftpower\.com/team\.php?id=2179)|[34th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
B|90|929|[Cody McCullough](https://zwiftpower\.com/profile\.php?z=6044)|—|[36th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|90|929|[Nathan Tamayo 🐱](https://zwiftpower\.com/profile\.php?z=11921)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[36th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|90|929|[Y ISSY](https://zwiftpower\.com/profile\.php?z=208937)|—|[36th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|90|929|[J\. Caselli](https://zwiftpower\.com/profile\.php?z=934540)|—|[36th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|91|927|[Grant W](https://zwiftpower\.com/profile\.php?z=6014)|[Bat City Cyc](https://zwiftpower\.com/team\.php?id=171)|[37th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|92|925|[Dan Glowitz](https://zwiftpower\.com/profile\.php?z=612100)|—|[38th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|93|923|[Martin Levasseur](https://zwiftpower\.com/profile\.php?z=1009997)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[39th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
B|94|921|[Sylvain Roy](https://zwiftpower\.com/profile\.php?z=162971)|—|[40th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|94|921|[Mack Levine2951](https://zwiftpower\.com/profile\.php?z=726976)|—|[40th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|94|921|[Scott Izutsu3697](https://zwiftpower\.com/profile\.php?z=1074538)|—|[40th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|95|919|[David Coupland HERD](https://zwiftpower\.com/profile\.php?z=429495)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[41st #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|96|917|[Sébastien Hue](https://zwiftpower\.com/profile\.php?z=174461)|[CXS](https://zwiftpower\.com/team\.php?id=2321)|[42nd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|97|915|[Stephane BELLIVIER](https://zwiftpower\.com/profile\.php?z=568538)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[43rd #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
B|98|911|[Peter Walker](https://zwiftpower\.com/profile\.php?z=115121)|—|[45th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|98|911|[Derek McIntire Ⓥ](https://zwiftpower\.com/profile\.php?z=955953)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[45th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|99|909|[Cycling Jobin](https://zwiftpower\.com/profile\.php?z=18671)|—|[46th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|99|909|[Brian Kingston](https://zwiftpower\.com/profile\.php?z=651269)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[46th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|100|907|[Michelle Patton](https://zwiftpower\.com/profile\.php?z=98297)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[47th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|101|903|[Max Rapoff](https://zwiftpower\.com/profile\.php?z=179823)|[Warriors](https://zwiftpower\.com/team\.php?id=3941)|[49th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|102|899|[Alan Dempsey](https://zwiftpower\.com/profile\.php?z=6999)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[51st #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|102|899|[Mark Springer](https://zwiftpower\.com/profile\.php?z=513187)|—|[51st #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|103|897|[D Zeggelaar](https://zwiftpower\.com/profile\.php?z=812036)|—|[52nd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|104|895|[Chris Herlin](https://zwiftpower\.com/profile\.php?z=167348)|—|[53rd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|105|893|[JR Ritchie](https://zwiftpower\.com/profile\.php?z=877541)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[54th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|105|893|[Robert Jackson](https://zwiftpower\.com/profile\.php?z=1065723)|—|[54th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|106|891|[Mike Platt](https://zwiftpower\.com/profile\.php?z=307681)|—|[55th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|106|891|[David Calfat JZQC](https://zwiftpower\.com/profile\.php?z=1074349)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[55th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|107|889|[Michael Latsch](https://zwiftpower\.com/profile\.php?z=570992)|—|[56th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|107|889|[R oadieJodi \(Vegan\)](https://zwiftpower\.com/profile\.php?z=1004115)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[56th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|108|887|[Rick Borns \(WMCR\)](https://zwiftpower\.com/profile\.php?z=69308)|—|[57th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|108|887|[Terry Patton](https://zwiftpower\.com/profile\.php?z=96440)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[57th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|108|887|[Norman Romero](https://zwiftpower\.com/profile\.php?z=980877)|—|[57th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
B|109|883|[Doug Descheneaux](https://zwiftpower\.com/profile\.php?z=48160)|—|[59th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|109|883|[Michael Steinberg](https://zwiftpower\.com/profile\.php?z=238678)|—|[59th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
B|110|879|[Steve Laurin](https://zwiftpower\.com/profile\.php?z=172582)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|[61st #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|111|877|[Jorge Martinez \#E3TriCoaching7100](https://zwiftpower\.com/profile\.php?z=15031)|—|[62nd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|112|875|[Stu Schopp \(B\-\)](https://zwiftpower\.com/profile\.php?z=468681)|—|[63rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
B|113|873|[Juan Ascoli](https://zwiftpower\.com/profile\.php?z=46115)|—|[64th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|1|5956|[Mick Elliott](https://zwiftpower\.com/profile\.php?z=131471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥇 #1](https://zwiftpower.com/events.php?zid=152781)|[🥇 #6](https://zwiftpower.com/events.php?zid=170592)|[🥈 #8](https://zwiftpower.com/events.php?zid=176476)|[6th #3](https://zwiftpower.com/events.php?zid=159644)|[6th #7](https://zwiftpower.com/events.php?zid=173965)|[9th #4](https://zwiftpower.com/events.php?zid=163028)
C|2|5946|[Glynn Broner \(BCC\)KY](https://zwiftpower\.com/profile\.php?z=77670)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #4](https://zwiftpower.com/events.php?zid=163028)|[🥈 #6](https://zwiftpower.com/events.php?zid=170592)|[🥉 #8](https://zwiftpower.com/events.php?zid=176476)|[5th #2](https://zwiftpower.com/events.php?zid=155997)|[9th #3](https://zwiftpower.com/events.php?zid=159644)|[9th #5](https://zwiftpower.com/events.php?zid=167034)
C|3|5944|[Pete Greges](https://zwiftpower\.com/profile\.php?z=11230)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥇 #8](https://zwiftpower.com/events.php?zid=176476)|[4th #1](https://zwiftpower.com/events.php?zid=152781)|[4th #7](https://zwiftpower.com/events.php?zid=173965)|[5th #3](https://zwiftpower.com/events.php?zid=159644)|[8th #4](https://zwiftpower.com/events.php?zid=163028)|[9th #2](https://zwiftpower.com/events.php?zid=155997)
C|4|5910|[Jason Economou](https://zwiftpower\.com/profile\.php?z=290132)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[🥉 #7](https://zwiftpower.com/events.php?zid=173965)|[6th #6](https://zwiftpower.com/events.php?zid=170592)|[7th #5](https://zwiftpower.com/events.php?zid=167034)|[8th #3](https://zwiftpower.com/events.php?zid=159644)|[11th #8](https://zwiftpower.com/events.php?zid=176476)|[13th #4](https://zwiftpower.com/events.php?zid=163028)
C|5|5900|[Marko Hlibchuk Ⓥ](https://zwiftpower\.com/profile\.php?z=426765)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[🥉 #1](https://zwiftpower.com/events.php?zid=152781)|[9th #6](https://zwiftpower.com/events.php?zid=170592)|[10th #4](https://zwiftpower.com/events.php?zid=163028)|[10th #5](https://zwiftpower.com/events.php?zid=167034)|[10th #8](https://zwiftpower.com/events.php?zid=176476)|[11th #3](https://zwiftpower.com/events.php?zid=159644)
C|6|5856|[Kevin Mabey 🔰](https://zwiftpower\.com/profile\.php?z=835773)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th #4](https://zwiftpower.com/events.php?zid=163028)|[5th #5](https://zwiftpower.com/events.php?zid=167034)|[7th #7](https://zwiftpower.com/events.php?zid=173965)|[8th #8](https://zwiftpower.com/events.php?zid=176476)|[21st #6](https://zwiftpower.com/events.php?zid=170592)|[29th #3](https://zwiftpower.com/events.php?zid=159644)
C|7|5848|[Jean Grenier JZQC \(C\)](https://zwiftpower\.com/profile\.php?z=421295)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[7th #8](https://zwiftpower.com/events.php?zid=176476)|[11th #2](https://zwiftpower.com/events.php?zid=155997)|[11th #7](https://zwiftpower.com/events.php?zid=173965)|[15th #3](https://zwiftpower.com/events.php?zid=159644)|[16th #6](https://zwiftpower.com/events.php?zid=170592)|[19th #4](https://zwiftpower.com/events.php?zid=163028)
C|8|5832|[Mike Naessens Cryo\-Gen](https://zwiftpower\.com/profile\.php?z=132112)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[8th #6](https://zwiftpower.com/events.php?zid=170592)|[12th #7](https://zwiftpower.com/events.php?zid=173965)|[13th #3](https://zwiftpower.com/events.php?zid=159644)|[16th #2](https://zwiftpower.com/events.php?zid=155997)|[17th #4](https://zwiftpower.com/events.php?zid=163028)|[21st #8](https://zwiftpower.com/events.php?zid=176476)
C|9|5818|[Sebastien Bergeron](https://zwiftpower\.com/profile\.php?z=938286)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th #1](https://zwiftpower.com/events.php?zid=152781)|[14th #7](https://zwiftpower.com/events.php?zid=173965)|[15th #5](https://zwiftpower.com/events.php?zid=167034)|[19th #6](https://zwiftpower.com/events.php?zid=170592)|[19th #8](https://zwiftpower.com/events.php?zid=176476)|[22nd #3](https://zwiftpower.com/events.php?zid=159644)
C|10|5758|[Ayaz Asif Ⓥ](https://zwiftpower\.com/profile\.php?z=7759)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[12th #8](https://zwiftpower.com/events.php?zid=176476)|[18th #4](https://zwiftpower.com/events.php?zid=163028)|[18th #6](https://zwiftpower.com/events.php?zid=170592)|[24th #2](https://zwiftpower.com/events.php?zid=155997)|[25th #3](https://zwiftpower.com/events.php?zid=159644)|[27th #5](https://zwiftpower.com/events.php?zid=167034)
C|11|5740|[Manuel Rocha](https://zwiftpower\.com/profile\.php?z=70647)|[eRoberto](https://zwiftpower\.com/team\.php?id=3470)|[16th #8](https://zwiftpower.com/events.php?zid=176476)|[19th #7](https://zwiftpower.com/events.php?zid=173965)|[22nd #5](https://zwiftpower.com/events.php?zid=167034)|[23rd #6](https://zwiftpower.com/events.php?zid=170592)|[25th #4](https://zwiftpower.com/events.php?zid=163028)|[28th #2](https://zwiftpower.com/events.php?zid=155997)
C|12|5730|[Michael Siefritz Ⓥ](https://zwiftpower\.com/profile\.php?z=514647)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[13th #7](https://zwiftpower.com/events.php?zid=173965)|[21st #4](https://zwiftpower.com/events.php?zid=163028)|[21st #5](https://zwiftpower.com/events.php?zid=167034)|[24th #6](https://zwiftpower.com/events.php?zid=170592)|[24th #8](https://zwiftpower.com/events.php?zid=176476)|[35th #3](https://zwiftpower.com/events.php?zid=159644)
C|13|5702|[Jeff Cockerill CryoGen](https://zwiftpower\.com/profile\.php?z=204152)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[14th #8](https://zwiftpower.com/events.php?zid=176476)|[19th #3](https://zwiftpower.com/events.php?zid=159644)|[23rd #4](https://zwiftpower.com/events.php?zid=163028)|[31st #6](https://zwiftpower.com/events.php?zid=170592)|[32nd #2](https://zwiftpower.com/events.php?zid=155997)|[33rd #5](https://zwiftpower.com/events.php?zid=167034)
C|14|4857|[Percy S](https://zwiftpower\.com/profile\.php?z=166608)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[5th #6](https://zwiftpower.com/events.php?zid=170592)|[9th #8](https://zwiftpower.com/events.php?zid=176476)|[16th #5](https://zwiftpower.com/events.php?zid=167034)|[17th #3](https://zwiftpower.com/events.php?zid=159644)|[27th #4](https://zwiftpower.com/events.php?zid=163028)|—
C|15|4777|[Don Lycette CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=244209)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[7th #1](https://zwiftpower.com/events.php?zid=152781)|[18th #8](https://zwiftpower.com/events.php?zid=176476)|[23rd #3](https://zwiftpower.com/events.php?zid=159644)|[32nd #6](https://zwiftpower.com/events.php?zid=170592)|[34th #4](https://zwiftpower.com/events.php?zid=163028)|—
C|16|4759|[Adam Larrea \(5170\)](https://zwiftpower\.com/profile\.php?z=925229)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[16th #3](https://zwiftpower.com/events.php?zid=159644)|[22nd #8](https://zwiftpower.com/events.php?zid=176476)|[25th #6](https://zwiftpower.com/events.php?zid=170592)|[26th #4](https://zwiftpower.com/events.php?zid=163028)|[34th #2](https://zwiftpower.com/events.php?zid=155997)|—
C|17|4703|[P\. Routledge CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=526163)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[24th #4](https://zwiftpower.com/events.php?zid=163028)|[26th #6](https://zwiftpower.com/events.php?zid=170592)|[30th #5](https://zwiftpower.com/events.php?zid=167034)|[31st #2](https://zwiftpower.com/events.php?zid=155997)|[40th #3](https://zwiftpower.com/events.php?zid=159644)|—
C|18|3910|[Vince Kim🤖TFC](https://zwiftpower\.com/profile\.php?z=261873)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th #4](https://zwiftpower.com/events.php?zid=163028)|[10th #1](https://zwiftpower.com/events.php?zid=152781)|[12th #6](https://zwiftpower.com/events.php?zid=170592)|[21st #3](https://zwiftpower.com/events.php?zid=159644)|—|—
C|19|3860|[John P](https://zwiftpower\.com/profile\.php?z=164752)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[7th #4](https://zwiftpower.com/events.php?zid=163028)|[10th #6](https://zwiftpower.com/events.php?zid=170592)|[24th #5](https://zwiftpower.com/events.php?zid=167034)|[31st #3](https://zwiftpower.com/events.php?zid=159644)|—|—
C|20|3838|[Steve Thibodeau](https://zwiftpower\.com/profile\.php?z=897282)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[8th #7](https://zwiftpower.com/events.php?zid=173965)|[15th #6](https://zwiftpower.com/events.php?zid=170592)|[19th #2](https://zwiftpower.com/events.php?zid=155997)|[41st #3](https://zwiftpower.com/events.php?zid=159644)|—|—
C|21|3812|[Martin Brousseau \(JZQc\)](https://zwiftpower\.com/profile\.php?z=886279)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #8](https://zwiftpower.com/events.php?zid=176476)|[20th #2](https://zwiftpower.com/events.php?zid=155997)|[28th #6](https://zwiftpower.com/events.php?zid=170592)|[33rd #4](https://zwiftpower.com/events.php?zid=163028)|—|—
C|22|3786|[Michel Descoteaux JZQC](https://zwiftpower\.com/profile\.php?z=193845)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #1](https://zwiftpower.com/events.php?zid=152781)|[29th #6](https://zwiftpower.com/events.php?zid=170592)|[32nd #5](https://zwiftpower.com/events.php?zid=167034)|[42nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—
C|23|2975|[Justin Kitney](https://zwiftpower\.com/profile\.php?z=123889)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥇 #7](https://zwiftpower.com/events.php?zid=173965)|[6th #4](https://zwiftpower.com/events.php?zid=163028)|[7th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
C|24|2965|[John Culhane](https://zwiftpower\.com/profile\.php?z=156937)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥉 #2](https://zwiftpower.com/events.php?zid=155997)|[4th #5](https://zwiftpower.com/events.php?zid=167034)|[12th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—
C|25|2879|[Dale Bradbury](https://zwiftpower\.com/profile\.php?z=817234)|—|[20th #3](https://zwiftpower.com/events.php?zid=159644)|[20th #8](https://zwiftpower.com/events.php?zid=176476)|[22nd #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—
C|26|2797|[Danny Arana](https://zwiftpower\.com/profile\.php?z=204059)|—|[23rd #8](https://zwiftpower.com/events.php?zid=176476)|[36th #4](https://zwiftpower.com/events.php?zid=163028)|[44th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
C|27|1992|[Stefanie Hollmichel](https://zwiftpower\.com/profile\.php?z=51322)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥇 #3](https://zwiftpower.com/events.php?zid=159644)|[4th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
C|27|1992|[Ryan DeadPool Miller TFC \(C\)](https://zwiftpower\.com/profile\.php?z=331384)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #5](https://zwiftpower.com/events.php?zid=167034)|[4th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|28|1986|[Aaron Fields](https://zwiftpower\.com/profile\.php?z=640932)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[🥈 #3](https://zwiftpower.com/events.php?zid=159644)|[6th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
C|29|1984|[Jim Coker](https://zwiftpower\.com/profile\.php?z=589891)|—|[🥉 #4](https://zwiftpower.com/events.php?zid=163028)|[6th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—
C|30|1980|[Dany Bouchard4414](https://zwiftpower\.com/profile\.php?z=434253)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #8](https://zwiftpower.com/events.php?zid=176476)|[7th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|31|1978|[Rudolfo Poomp](https://zwiftpower\.com/profile\.php?z=64796)|[HBC](https://zwiftpower\.com/team\.php?id=1819)|[🥉 #6](https://zwiftpower.com/events.php?zid=170592)|[9th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—
C|32|1962|[Jeff Skinner CRYO\-GEN \(C\)](https://zwiftpower\.com/profile\.php?z=96418)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th #8](https://zwiftpower.com/events.php?zid=176476)|[15th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—
C|33|1960|[Keevin Griffith](https://zwiftpower\.com/profile\.php?z=538740)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[10th #7](https://zwiftpower.com/events.php?zid=173965)|[11th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|34|1950|[Jeremy hopwood](https://zwiftpower\.com/profile\.php?z=36654)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=163028)|[25th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
C|35|1944|[Nicolas Trépanier](https://zwiftpower\.com/profile\.php?z=183617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[14th #3](https://zwiftpower.com/events.php?zid=159644)|[15th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
C|35|1944|[Wynn Legon](https://zwiftpower\.com/profile\.php?z=560232)|—|[13th #6](https://zwiftpower.com/events.php?zid=170592)|[16th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
C|36|1942|[Ed Phinney <HERD>](https://zwiftpower\.com/profile\.php?z=769917)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[10th #3](https://zwiftpower.com/events.php?zid=159644)|[20th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|37|1932|[shii tod](https://zwiftpower\.com/profile\.php?z=365733)|—|[5th #7](https://zwiftpower.com/events.php?zid=173965)|[30th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|38|1920|[JF Marsan](https://zwiftpower\.com/profile\.php?z=411968)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[9th #1](https://zwiftpower.com/events.php?zid=152781)|[32nd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
C|39|1918|[Jason Thompson](https://zwiftpower\.com/profile\.php?z=838875)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[20th #5](https://zwiftpower.com/events.php?zid=167034)|[22nd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
C|40|1910|[Francois Pelletier JZQc](https://zwiftpower\.com/profile\.php?z=943511)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[13th #5](https://zwiftpower.com/events.php?zid=167034)|[33rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
C|41|1900|[Brian Reimer \[HERD\] 🍺🐝](https://zwiftpower\.com/profile\.php?z=147510)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[20th #7](https://zwiftpower.com/events.php?zid=173965)|[31st #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
C|42|1898|[Carlos Roldán](https://zwiftpower\.com/profile\.php?z=396592)|—|[16th #7](https://zwiftpower.com/events.php?zid=173965)|[36th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
C|43|1888|[Terio Sutherland \(BCC\)](https://zwiftpower\.com/profile\.php?z=912201)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[20th #4](https://zwiftpower.com/events.php?zid=163028)|[37th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—
C|44|1880|[Dave Pyle](https://zwiftpower\.com/profile\.php?z=430828)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[27th #6](https://zwiftpower.com/events.php?zid=170592)|[34th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
C|45|1876|[Martin Levasseur](https://zwiftpower\.com/profile\.php?z=1009997)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[18th #5](https://zwiftpower.com/events.php?zid=167034)|[45th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
C|46|1868|[Marc Sharer](https://zwiftpower\.com/profile\.php?z=901750)|—|[28th #4](https://zwiftpower.com/events.php?zid=163028)|[39th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
C|47|1858|[Darrell Kincer \(BCC\)](https://zwiftpower\.com/profile\.php?z=27344)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[34th #6](https://zwiftpower.com/events.php?zid=170592)|[38th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
C|48|1854|[Bryan Pearson \[CRYO\-GEN\]](https://zwiftpower\.com/profile\.php?z=649507)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[34th #5](https://zwiftpower.com/events.php?zid=167034)|[40th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—
C|49|999|[S\. Deslauriers Velozone JZQC](https://zwiftpower\.com/profile\.php?z=182546)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|50|997|[David M Renton III](https://zwiftpower\.com/profile\.php?z=83511)|[Zwift HQ](https://zwiftpower\.com/team\.php?id=1977)|[🥈 #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
C|50|997|[Gene Fowler \(BCC\)](https://zwiftpower\.com/profile\.php?z=186111)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|50|997|"[Charlie ""Dactah"" Miller TFC](https://zwiftpower\.com/profile\.php?z=276186)"|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥈 #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|50|997|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[🥈 #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|51|995|[Martin Savard](https://zwiftpower\.com/profile\.php?z=235515)|—|[🥉 #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|51|995|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥉 #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|52|993|[Bailey Hamilton](https://zwiftpower\.com/profile\.php?z=833877)|—|[4th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|53|989|[Pierre Arbez2134JZQc](https://zwiftpower\.com/profile\.php?z=568382)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|54|987|[Mike Dent](https://zwiftpower\.com/profile\.php?z=955715)|—|[7th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|55|985|[Will Botti \(C\)](https://zwiftpower\.com/profile\.php?z=211237)|—|[8th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
C|55|985|[David Sorg](https://zwiftpower\.com/profile\.php?z=243886)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|[8th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|55|985|[Zach Bennett \#EN](https://zwiftpower\.com/profile\.php?z=968668)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[8th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|56|981|[Olivier De Ladurantaye6755](https://zwiftpower\.com/profile\.php?z=840643)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[10th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|57|979|[Dennis Porter1754](https://zwiftpower\.com/profile\.php?z=68968)|—|[11th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|57|979|[Chris McCormick](https://zwiftpower\.com/profile\.php?z=380762)|—|[11th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|58|977|[Daniel Roy \- CCC 🚴](https://zwiftpower\.com/profile\.php?z=720162)|[CCC](https://zwiftpower\.com/team\.php?id=2975)|[12th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|58|977|[Luke Austen 7802](https://zwiftpower\.com/profile\.php?z=817734)|—|[12th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|59|975|[Doug Pape](https://zwiftpower\.com/profile\.php?z=5425)|—|[13th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|59|975|[Brian Kristof](https://zwiftpower\.com/profile\.php?z=878403)|—|[13th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
C|60|973|[Ray J4124](https://zwiftpower\.com/profile\.php?z=319615)|—|[14th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
C|60|973|[Laurent Huot JZQc](https://zwiftpower\.com/profile\.php?z=669891)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[14th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|60|973|[Kim Schaff](https://zwiftpower\.com/profile\.php?z=1053714)|—|[14th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|61|971|[Martin Wright Velozone\(JZQC\)](https://zwiftpower\.com/profile\.php?z=76715)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[15th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|62|967|[Ryan Johs](https://zwiftpower\.com/profile\.php?z=1953)|—|[17th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
C|62|967|[Pierre Higgins](https://zwiftpower\.com/profile\.php?z=114363)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[17th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|62|967|[Craig Gritzen ⓥ](https://zwiftpower\.com/profile\.php?z=527678)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[17th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|63|965|[Bob Friesen](https://zwiftpower\.com/profile\.php?z=110692)|—|[18th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|63|965|[Eric Olds \[BIKEMAN\.COM\]](https://zwiftpower\.com/profile\.php?z=873324)|[Bikeman\.com](https://zwiftpower\.com/team\.php?id=3969)|[18th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|63|965|[Christopher Mc Mahon JZQc](https://zwiftpower\.com/profile\.php?z=966195)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[18th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|64|959|[Steve Lidston Ⓥ](https://zwiftpower\.com/profile\.php?z=180464)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[21st #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|64|959|[R NII](https://zwiftpower\.com/profile\.php?z=516693)|—|[21st #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|65|957|[Josh Becker](https://zwiftpower\.com/profile\.php?z=203930)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[22nd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|65|957|[Me Bonkers](https://zwiftpower\.com/profile\.php?z=246764)|—|[22nd #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|66|955|[Paul Kelliher \(HERD\)](https://zwiftpower\.com/profile\.php?z=657014)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[23rd #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|66|955|[Jeff Brand](https://zwiftpower\.com/profile\.php?z=938363)|—|[23rd #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|66|955|[Doug Streeter](https://zwiftpower\.com/profile\.php?z=1032450)|—|[23rd #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|67|953|[Hugo Walker](https://zwiftpower\.com/profile\.php?z=113284)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[24th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|67|953|[William Hunt](https://zwiftpower\.com/profile\.php?z=724628)|—|[24th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
C|68|951|[AJ Carr CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=744194)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[25th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
C|68|951|[Joel Davidson](https://zwiftpower\.com/profile\.php?z=1103458)|—|[25th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|69|949|[Mike Louie](https://zwiftpower\.com/profile\.php?z=261230)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[26th #8](https://zwiftpower.com/events.php?zid=176476)|—|—|—|—|—
C|69|949|[Christian Dumaine \(C\)](https://zwiftpower\.com/profile\.php?z=340884)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[26th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|69|949|[Ed Lai \[DIRT\]](https://zwiftpower\.com/profile\.php?z=795742)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[26th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|69|949|[Winston Young](https://zwiftpower\.com/profile\.php?z=855584)|—|[26th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|70|947|[James Hall](https://zwiftpower\.com/profile\.php?z=382471)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[27th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|70|947|[Scott Haller \(HERD\)](https://zwiftpower\.com/profile\.php?z=1122925)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[27th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|71|945|[Scott Sugimoto](https://zwiftpower\.com/profile\.php?z=862965)|—|[28th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|72|943|[Chris Vachon](https://zwiftpower\.com/profile\.php?z=2190)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[29th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|72|943|[Tom Lokar 🐺](https://zwiftpower\.com/profile\.php?z=88976)|—|[29th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|72|943|[PC Mountain](https://zwiftpower\.com/profile\.php?z=150525)|—|[29th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|73|941|[Paul Wright](https://zwiftpower\.com/profile\.php?z=303501)|—|[30th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|73|941|[Burt Ayers](https://zwiftpower\.com/profile\.php?z=708161)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[30th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|73|941|[Francois Courteau 4770](https://zwiftpower\.com/profile\.php?z=788394)|—|[30th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|74|939|[Henry Holmes](https://zwiftpower\.com/profile\.php?z=8494)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|[31st #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|75|937|[joe payd](https://zwiftpower\.com/profile\.php?z=654223)|—|[32nd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|76|935|[Benoit Auger Forest KISS\(D\)](https://zwiftpower\.com/profile\.php?z=222531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[33rd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|76|935|[Martin Ross](https://zwiftpower\.com/profile\.php?z=388088)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[33rd #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
C|77|931|[J Hysler \(HERD\)](https://zwiftpower\.com/profile\.php?z=313837)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[35th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
C|77|931|[Ghislain Cormier 7660](https://zwiftpower\.com/profile\.php?z=460728)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[35th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|77|931|[Shug Wilson Ⓥ\[VEGAN\]](https://zwiftpower\.com/profile\.php?z=541203)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[35th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|77|931|[JC Smith](https://zwiftpower\.com/profile\.php?z=811004)|—|[35th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|78|929|[Brian Mull CRYO\-GEN \(C\)](https://zwiftpower\.com/profile\.php?z=671125)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[36th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
C|78|929|[Garron Amos](https://zwiftpower\.com/profile\.php?z=870571)|[ExpFay](https://zwiftpower\.com/team\.php?id=2260)|[36th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|78|929|[Stefan Van Kessel](https://zwiftpower\.com/profile\.php?z=901165)|—|[36th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
C|79|927|[Mark Jeans](https://zwiftpower\.com/profile\.php?z=428951)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[37th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|79|927|[Kevin Hyre CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=557094)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[37th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|80|925|[marian haiducJZQC](https://zwiftpower\.com/profile\.php?z=483193)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[38th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|80|925|[Chris Sweigard](https://zwiftpower\.com/profile\.php?z=951710)|—|[38th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|81|923|[Paige Haydon Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=311565)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[39th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|81|923|[Tim Norejko](https://zwiftpower\.com/profile\.php?z=1092297)|—|[39th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
C|82|919|[Brian Hester](https://zwiftpower\.com/profile\.php?z=33436)|—|[41st #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|83|917|[David KHUONG VietVelo](https://zwiftpower\.com/profile\.php?z=438132)|[V I E T V E L O](https://zwiftpower\.com/team\.php?id=2231)|[42nd #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
C|84|915|[John Walker](https://zwiftpower\.com/profile\.php?z=411864)|—|[43rd #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|85|909|[David LeBaron](https://zwiftpower\.com/profile\.php?z=414671)|—|[46th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|86|907|[Tee Rex](https://zwiftpower\.com/profile\.php?z=246952)|—|[47th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
C|87|905|[Wenzel Ablola](https://zwiftpower\.com/profile\.php?z=302851)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[48th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
D|1|5974|[Alain Lejeune](https://zwiftpower\.com/profile\.php?z=36413)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #2](https://zwiftpower.com/events.php?zid=155997)|[🥇 #3](https://zwiftpower.com/events.php?zid=159644)|[🥇 #8](https://zwiftpower.com/events.php?zid=176476)|[🥉 #1](https://zwiftpower.com/events.php?zid=152781)|[5th #5](https://zwiftpower.com/events.php?zid=167034)|[5th #7](https://zwiftpower.com/events.php?zid=173965)
D|2|5946|[Sylvain Normand JZQc](https://zwiftpower\.com/profile\.php?z=357055)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉 #4](https://zwiftpower.com/events.php?zid=163028)|[4th #6](https://zwiftpower.com/events.php?zid=170592)|[4th #8](https://zwiftpower.com/events.php?zid=176476)|[5th #1](https://zwiftpower.com/events.php?zid=152781)|[5th #3](https://zwiftpower.com/events.php?zid=159644)|[9th #2](https://zwiftpower.com/events.php?zid=155997)
D|3|5908|[Chuck Kozlowski](https://zwiftpower\.com/profile\.php?z=998)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #8](https://zwiftpower.com/events.php?zid=176476)|[5th #6](https://zwiftpower.com/events.php?zid=170592)|[9th #7](https://zwiftpower.com/events.php?zid=173965)|[10th #3](https://zwiftpower.com/events.php?zid=159644)|[11th #5](https://zwiftpower.com/events.php?zid=167034)|[12th #4](https://zwiftpower.com/events.php?zid=163028)
D|4|4973|[Xavier Laroche](https://zwiftpower\.com/profile\.php?z=427658)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥈 #1](https://zwiftpower.com/events.php?zid=152781)|[🥈 #5](https://zwiftpower.com/events.php?zid=167034)|[🥈 #7](https://zwiftpower.com/events.php?zid=173965)|[🥉 #2](https://zwiftpower.com/events.php?zid=155997)|[7th #3](https://zwiftpower.com/events.php?zid=159644)|—
D|5|4953|[Christopher Mc Mahon JZQc](https://zwiftpower\.com/profile\.php?z=966195)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #4](https://zwiftpower.com/events.php?zid=163028)|[🥇 #6](https://zwiftpower.com/events.php?zid=170592)|[5th #2](https://zwiftpower.com/events.php?zid=155997)|[7th #5](https://zwiftpower.com/events.php?zid=167034)|[12th #3](https://zwiftpower.com/events.php?zid=159644)|—
D|6|3972|[Ranji Carluen](https://zwiftpower\.com/profile\.php?z=552690)|[MGCC](https://zwiftpower\.com/team\.php?id=114)|[🥈 #4](https://zwiftpower.com/events.php?zid=163028)|[4th #1](https://zwiftpower.com/events.php?zid=152781)|[4th #5](https://zwiftpower.com/events.php?zid=167034)|[6th #3](https://zwiftpower.com/events.php?zid=159644)|—|—
D|7|3960|[AJ Carr CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=744194)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #6](https://zwiftpower.com/events.php?zid=170592)|[🥉 #5](https://zwiftpower.com/events.php?zid=167034)|[6th #7](https://zwiftpower.com/events.php?zid=173965)|[11th #4](https://zwiftpower.com/events.php?zid=163028)|—|—
D|8|3940|[Jacques Wiseman JZQC \(D\)](https://zwiftpower\.com/profile\.php?z=327823)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #1](https://zwiftpower.com/events.php?zid=152781)|[6th #4](https://zwiftpower.com/events.php?zid=163028)|[7th #2](https://zwiftpower.com/events.php?zid=155997)|[13th #3](https://zwiftpower.com/events.php?zid=159644)|—|—
D|9|2973|[Allen Madore1830](https://zwiftpower\.com/profile\.php?z=4134)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[🥉 #6](https://zwiftpower.com/events.php?zid=170592)|[🥉 #8](https://zwiftpower.com/events.php?zid=176476)|[9th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—
D|10|2955|[Joseph Padilla \[HERD\]](https://zwiftpower\.com/profile\.php?z=923771)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[5th #8](https://zwiftpower.com/events.php?zid=176476)|[8th #4](https://zwiftpower.com/events.php?zid=163028)|[11th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—
D|11|2943|[Didier Bonneville JZQC](https://zwiftpower\.com/profile\.php?z=824267)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[8th #1](https://zwiftpower.com/events.php?zid=152781)|[10th #2](https://zwiftpower.com/events.php?zid=155997)|[12th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—
D|12|1982|[Brian Mackinnon](https://zwiftpower\.com/profile\.php?z=39554)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #3](https://zwiftpower.com/events.php?zid=159644)|[8th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—
D|12|1982|[Marc Boily](https://zwiftpower\.com/profile\.php?z=175371)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #2](https://zwiftpower.com/events.php?zid=155997)|[6th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—
D|12|1982|[Monty McNeil](https://zwiftpower\.com/profile\.php?z=966252)|—|[🥉 #3](https://zwiftpower.com/events.php?zid=159644)|[7th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—
D|12|1982|[Doug Moore](https://zwiftpower\.com/profile\.php?z=972377)|—|[🥇 #5](https://zwiftpower.com/events.php?zid=167034)|[9th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—
D|13|999|[Benoit Auger Forest KISS\(D\)](https://zwiftpower\.com/profile\.php?z=222531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
D|13|999|[JAvello Temukano](https://zwiftpower\.com/profile\.php?z=479841)|—|[🥇 #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
D|14|997|[Jonathan Rivest](https://zwiftpower\.com/profile\.php?z=48031)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥈 #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
D|15|995|[Genevieve Dumais](https://zwiftpower\.com/profile\.php?z=955847)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉 #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
D|16|993|[Jeff May CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=194178)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[4th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
D|16|993|[Ian Tuck](https://zwiftpower\.com/profile\.php?z=622819)|—|[4th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
D|16|993|[Xenoatic K](https://zwiftpower\.com/profile\.php?z=920072)|—|[4th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
D|17|991|[Jeff Patterson](https://zwiftpower\.com/profile\.php?z=1151609)|—|[5th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
D|18|989|[Mark Mamrega](https://zwiftpower\.com/profile\.php?z=766185)|—|[6th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
D|18|989|[Jamie Overton](https://zwiftpower\.com/profile\.php?z=774249)|—|[6th #2](https://zwiftpower.com/events.php?zid=155997)|—|—|—|—|—
D|19|987|[Keri Hindman](https://zwiftpower\.com/profile\.php?z=429197)|—|[7th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
D|19|987|[David Hunt](https://zwiftpower\.com/profile\.php?z=777791)|—|[7th #1](https://zwiftpower.com/events.php?zid=152781)|—|—|—|—|—
D|19|987|[Anthony Gregoire](https://zwiftpower\.com/profile\.php?z=1024640)|—|[7th #6](https://zwiftpower.com/events.php?zid=170592)|—|—|—|—|—
D|20|985|[Brandon OHern](https://zwiftpower\.com/profile\.php?z=312554)|—|[8th #7](https://zwiftpower.com/events.php?zid=173965)|—|—|—|—|—
D|20|985|[Terio Sutherland \(BCC\)](https://zwiftpower\.com/profile\.php?z=912201)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[8th #3](https://zwiftpower.com/events.php?zid=159644)|—|—|—|—|—
D|20|985|[Brad Bunde](https://zwiftpower\.com/profile\.php?z=1151067)|—|[8th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
D|21|983|[Jeffry Archer \(Montreal\)](https://zwiftpower\.com/profile\.php?z=1005681)|—|[9th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—
D|22|981|[Richard Lanoue](https://zwiftpower\.com/profile\.php?z=178958)|—|[10th #5](https://zwiftpower.com/events.php?zid=167034)|—|—|—|—|—
D|22|981|[Jason Mcdonald \(anderson sc\)](https://zwiftpower\.com/profile\.php?z=227384)|—|[10th #4](https://zwiftpower.com/events.php?zid=163028)|—|—|—|—|—



## KISS Community League — Americas Women

Individual rider rankings by top 6 results.

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6
---|---|---|---|---|---|---|---|---|---|---
A|1|5972|[Catrina Weiss \(CIS/Ladies\)](https://zwiftpower\.com/profile\.php?z=4727)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|[🥇 #7](https://zwiftpower.com/events.php?zid=173966)|[🥈 #3](https://zwiftpower.com/events.php?zid=159645)|[🥉 #5](https://zwiftpower.com/events.php?zid=167035)|[🥉 #8](https://zwiftpower.com/events.php?zid=176369)|[4th #1](https://zwiftpower.com/events.php?zid=154311)|[4th #6](https://zwiftpower.com/events.php?zid=170591)
A|2|4991|[Ronna Akins](https://zwiftpower\.com/profile\.php?z=72495)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|[🥇 #4](https://zwiftpower.com/events.php?zid=163029)|[🥇 #5](https://zwiftpower.com/events.php?zid=167035)|[🥇 #6](https://zwiftpower.com/events.php?zid=170591)|[🥈 #2](https://zwiftpower.com/events.php?zid=155996)|[🥈 #8](https://zwiftpower.com/events.php?zid=176369)|—
A|3|4985|[Stephanie Ossenbrink](https://zwiftpower\.com/profile\.php?z=479580)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|[🥇 #1](https://zwiftpower.com/events.php?zid=154311)|[🥇 #8](https://zwiftpower.com/events.php?zid=176369)|[🥈 #5](https://zwiftpower.com/events.php?zid=167035)|[🥈 #6](https://zwiftpower.com/events.php?zid=170591)|[4th #4](https://zwiftpower.com/events.php?zid=163029)|—
A|4|2985|[Angela Webster](https://zwiftpower\.com/profile\.php?z=1366)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[🥉 #3](https://zwiftpower.com/events.php?zid=159645)|[🥉 #4](https://zwiftpower.com/events.php?zid=163029)|[🥉 #6](https://zwiftpower.com/events.php?zid=170591)|—|—|—
A|4|2985|[Jill Seeman](https://zwiftpower\.com/profile\.php?z=15327)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥈 #4](https://zwiftpower.com/events.php?zid=163029)|[🥉 #1](https://zwiftpower.com/events.php?zid=154311)|[4th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—
A|5|1998|[Janna Gillick](https://zwiftpower\.com/profile\.php?z=460700)|[GSZ](https://zwiftpower\.com/team\.php?id=3992)|[🥇 #2](https://zwiftpower.com/events.php?zid=155996)|[🥇 #3](https://zwiftpower.com/events.php?zid=159645)|—|—|—|—
A|6|1994|[Emily Joyner](https://zwiftpower\.com/profile\.php?z=394039)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[🥈 #1](https://zwiftpower.com/events.php?zid=154311)|[🥈 #7](https://zwiftpower.com/events.php?zid=173966)|—|—|—|—
A|7|995|[Isabelle Veilleux JZQC](https://zwiftpower\.com/profile\.php?z=932401)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉 #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
B|1|5988|[Debra Sluz Parker](https://zwiftpower\.com/profile\.php?z=260577)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #6](https://zwiftpower.com/events.php?zid=170591)|[🥇 #7](https://zwiftpower.com/events.php?zid=173966)|[🥇 #8](https://zwiftpower.com/events.php?zid=176369)|[🥈 #1](https://zwiftpower.com/events.php?zid=154311)|[🥈 #2](https://zwiftpower.com/events.php?zid=155996)|[🥈 #3](https://zwiftpower.com/events.php?zid=159645)
B|2|5968|[Kristina Pearson](https://zwiftpower\.com/profile\.php?z=43324)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #3](https://zwiftpower.com/events.php?zid=159645)|[🥈 #7](https://zwiftpower.com/events.php?zid=173966)|[🥉 #2](https://zwiftpower.com/events.php?zid=155996)|[4th #1](https://zwiftpower.com/events.php?zid=154311)|[4th #8](https://zwiftpower.com/events.php?zid=176369)|[5th #4](https://zwiftpower.com/events.php?zid=163029)
B|3|3966|[Annie Dulude JZQC](https://zwiftpower\.com/profile\.php?z=558272)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #5](https://zwiftpower.com/events.php?zid=167035)|[4th #4](https://zwiftpower.com/events.php?zid=163029)|[4th #6](https://zwiftpower.com/events.php?zid=170591)|[10th #2](https://zwiftpower.com/events.php?zid=155996)|—|—
B|4|3952|[Lorena Sandoval](https://zwiftpower\.com/profile\.php?z=328805)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥉 #5](https://zwiftpower.com/events.php?zid=167035)|[7th #3](https://zwiftpower.com/events.php?zid=159645)|[7th #4](https://zwiftpower.com/events.php?zid=163029)|[9th #2](https://zwiftpower.com/events.php?zid=155996)|—|—
B|5|2989|[tracy moore](https://zwiftpower\.com/profile\.php?z=654026)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|[🥈 #6](https://zwiftpower.com/events.php?zid=170591)|[🥈 #8](https://zwiftpower.com/events.php?zid=176369)|[🥉 #7](https://zwiftpower.com/events.php?zid=173966)|—|—|—
B|6|2981|[Lisa Hynes](https://zwiftpower\.com/profile\.php?z=405131)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉 #1](https://zwiftpower.com/events.php?zid=154311)|[4th #2](https://zwiftpower.com/events.php?zid=155996)|[4th #3](https://zwiftpower.com/events.php?zid=159645)|—|—|—
B|7|2979|[Ms Sakushi](https://zwiftpower\.com/profile\.php?z=179751)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥇 #1](https://zwiftpower.com/events.php?zid=154311)|[5th #2](https://zwiftpower.com/events.php?zid=155996)|[6th #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—
B|7|2979|[Elise Rowe](https://zwiftpower\.com/profile\.php?z=213683)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|[🥉 #3](https://zwiftpower.com/events.php?zid=159645)|[🥉 #4](https://zwiftpower.com/events.php?zid=163029)|[6th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—
B|8|2969|[Sheryl Mohwinkel](https://zwiftpower\.com/profile\.php?z=586407)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[5th #1](https://zwiftpower.com/events.php?zid=154311)|[5th #3](https://zwiftpower.com/events.php?zid=159645)|[7th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—
B|9|1998|[Morgan Cabot](https://zwiftpower\.com/profile\.php?z=967586)|[GSZ](https://zwiftpower\.com/team\.php?id=3992)|[🥇 #2](https://zwiftpower.com/events.php?zid=155996)|[🥇 #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—
B|10|1990|[Anna Rogers \[REVO\]](https://zwiftpower\.com/profile\.php?z=265859)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|[🥉 #6](https://zwiftpower.com/events.php?zid=170591)|[🥉 #8](https://zwiftpower.com/events.php?zid=176369)|—|—|—|—
B|11|1982|[Sarah\-anne Beaupre](https://zwiftpower\.com/profile\.php?z=613766)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥈 #5](https://zwiftpower.com/events.php?zid=167035)|[8th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—
B|12|1978|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|[6th #1](https://zwiftpower.com/events.php?zid=154311)|[6th #3](https://zwiftpower.com/events.php?zid=159645)|—|—|—|—
B|13|991|[Krista Kadatz](https://zwiftpower\.com/profile\.php?z=928030)|—|[5th #6](https://zwiftpower.com/events.php?zid=170591)|—|—|—|—|—
B|14|979|[Elizabeth Lawless8309](https://zwiftpower\.com/profile\.php?z=567443)|—|[11th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
B|15|977|[AM FANNING](https://zwiftpower\.com/profile\.php?z=1134093)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[12th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
C|1|5982|[Pamela Abbott \- CISCYCLING](https://zwiftpower\.com/profile\.php?z=146648)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|[🥇 #5](https://zwiftpower.com/events.php?zid=167035)|[🥇 #7](https://zwiftpower.com/events.php?zid=173966)|[🥇 #8](https://zwiftpower.com/events.php?zid=176369)|[🥉 #2](https://zwiftpower.com/events.php?zid=155996)|[🥉 #3](https://zwiftpower.com/events.php?zid=159645)|[🥉 #6](https://zwiftpower.com/events.php?zid=170591)
C|2|4987|[Sherie Masse](https://zwiftpower\.com/profile\.php?z=177731)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|[🥇 #3](https://zwiftpower.com/events.php?zid=159645)|[🥈 #1](https://zwiftpower.com/events.php?zid=154311)|[🥈 #4](https://zwiftpower.com/events.php?zid=163029)|[🥈 #6](https://zwiftpower.com/events.php?zid=170591)|[🥈 #8](https://zwiftpower.com/events.php?zid=176369)|—
C|3|2997|[Kelsey Tranel](https://zwiftpower\.com/profile\.php?z=214587)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #2](https://zwiftpower.com/events.php?zid=155996)|[🥇 #4](https://zwiftpower.com/events.php?zid=163029)|[🥇 #6](https://zwiftpower.com/events.php?zid=170591)|—|—|—
C|4|1996|[Janie Lapointe Vélozone JZQC](https://zwiftpower\.com/profile\.php?z=462469)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #1](https://zwiftpower.com/events.php?zid=154311)|[🥈 #3](https://zwiftpower.com/events.php?zid=159645)|—|—|—|—
C|5|1982|[Nathalie Van Acker JZQC 6326](https://zwiftpower\.com/profile\.php?z=329231)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th #2](https://zwiftpower.com/events.php?zid=155996)|[5th #6](https://zwiftpower.com/events.php?zid=170591)|—|—|—|—
C|5|1982|[Martin Ross](https://zwiftpower\.com/profile\.php?z=388088)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #3](https://zwiftpower.com/events.php?zid=159645)|[6th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—
C|6|1978|[Manon Lafrance JZQc](https://zwiftpower\.com/profile\.php?z=224868)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #6](https://zwiftpower.com/events.php?zid=170591)|[8th #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—
C|7|997|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|[🥈 #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
C|8|995|[Breanna Cahoon](https://zwiftpower\.com/profile\.php?z=726270)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥉 #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—|—
C|8|995|[YC\-Meao喵~ Tseng\[FEAR<\]](https://zwiftpower\.com/profile\.php?z=811326)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥉 #1](https://zwiftpower.com/events.php?zid=154311)|—|—|—|—|—
C|9|993|[Sheila Leard TeamEN KISS \(C\)](https://zwiftpower\.com/profile\.php?z=276466)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[4th #1](https://zwiftpower.com/events.php?zid=154311)|—|—|—|—|—
C|9|993|[Kath Ferland\[Laferté Bicyles\]](https://zwiftpower\.com/profile\.php?z=487646)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[4th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
C|10|991|[Kati Petry](https://zwiftpower\.com/profile\.php?z=698274)|—|[5th #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—|—
C|11|989|[Carole Sinou](https://zwiftpower\.com/profile\.php?z=97277)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—|—
C|11|989|[Isabelle\. Brault 9211](https://zwiftpower\.com/profile\.php?z=322684)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th #6](https://zwiftpower.com/events.php?zid=170591)|—|—|—|—|—
C|12|987|[Genevieve Dumais](https://zwiftpower\.com/profile\.php?z=955847)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[7th #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—|—
D|1|4967|[Isabelle Picard JZQC7179](https://zwiftpower\.com/profile\.php?z=869091)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥈 #5](https://zwiftpower.com/events.php?zid=167035)|[🥉 #3](https://zwiftpower.com/events.php?zid=159645)|[🥉 #6](https://zwiftpower.com/events.php?zid=170591)|[5th #4](https://zwiftpower.com/events.php?zid=163029)|[6th #2](https://zwiftpower.com/events.php?zid=155996)|—
D|2|3992|[J Langlois](https://zwiftpower\.com/profile\.php?z=430523)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #4](https://zwiftpower.com/events.php?zid=163029)|[🥇 #5](https://zwiftpower.com/events.php?zid=167035)|[🥈 #3](https://zwiftpower.com/events.php?zid=159645)|[🥈 #6](https://zwiftpower.com/events.php?zid=170591)|—|—
D|3|3990|[Isabelle\. Brault 9211](https://zwiftpower\.com/profile\.php?z=322684)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇 #1](https://zwiftpower.com/events.php?zid=154311)|[🥇 #2](https://zwiftpower.com/events.php?zid=155996)|[🥇 #3](https://zwiftpower.com/events.php?zid=159645)|[4th #4](https://zwiftpower.com/events.php?zid=163029)|—|—
D|4|3986|[L Conrad 6533](https://zwiftpower\.com/profile\.php?z=283014)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥇 #6](https://zwiftpower.com/events.php?zid=170591)|[🥈 #1](https://zwiftpower.com/events.php?zid=154311)|[🥈 #4](https://zwiftpower.com/events.php?zid=163029)|[4th #2](https://zwiftpower.com/events.php?zid=155996)|—|—
D|5|1982|[Lucie Cauchon](https://zwiftpower\.com/profile\.php?z=472347)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉 #5](https://zwiftpower.com/events.php?zid=167035)|[7th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—
D|6|997|[Andrea Thackeray](https://zwiftpower\.com/profile\.php?z=322738)|—|[🥈 #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
D|7|995|[Manon Lafrance JZQc](https://zwiftpower\.com/profile\.php?z=224868)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉 #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—
D|7|995|[Wendy Chow](https://zwiftpower\.com/profile\.php?z=379916)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|[🥉 #4](https://zwiftpower.com/events.php?zid=163029)|—|—|—|—|—
D|8|991|[Julie Arcand \[JZQC\] 4916](https://zwiftpower\.com/profile\.php?z=272362)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th #2](https://zwiftpower.com/events.php?zid=155996)|—|—|—|—|—



## KISS Community League — Europe

Individual rider rankings by top 6 results.

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6
---|---|---|---|---|---|---|---|---|---|---
A|1|5980|[Samuel Brännlund INC🔰](https://zwiftpower\.com/profile\.php?z=660836)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇 #3](https://zwiftpower.com/events.php?zid=159334)|[🥇 #5](https://zwiftpower.com/events.php?zid=166630)|[🥇 #7](https://zwiftpower.com/events.php?zid=173451)|[🥈 #6](https://zwiftpower.com/events.php?zid=171935)|[🥉 #4](https://zwiftpower.com/events.php?zid=162691)|[5th #8](https://zwiftpower.com/events.php?zid=176148)
A|2|5966|[Leandro Messineo 🐸](https://zwiftpower\.com/profile\.php?z=688785)|—|[🥇 #1](https://zwiftpower.com/events.php?zid=152778)|[🥇 #4](https://zwiftpower.com/events.php?zid=162691)|[🥇 #6](https://zwiftpower.com/events.php?zid=171935)|[4th #3](https://zwiftpower.com/events.php?zid=159334)|[4th #7](https://zwiftpower.com/events.php?zid=173451)|[9th #8](https://zwiftpower.com/events.php?zid=176148)
A|3|5956|[Alex West 🔰🦏](https://zwiftpower\.com/profile\.php?z=401735)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[🥈 #3](https://zwiftpower.com/events.php?zid=159334)|[🥈 #4](https://zwiftpower.com/events.php?zid=162691)|[🥈 #7](https://zwiftpower.com/events.php?zid=173451)|[4th #5](https://zwiftpower.com/events.php?zid=166630)|[4th #8](https://zwiftpower.com/events.php?zid=176148)|[11th #1](https://zwiftpower.com/events.php?zid=152778)
A|4|5920|[Lionel Vujasin](https://zwiftpower\.com/profile\.php?z=311597)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[5th #6](https://zwiftpower.com/events.php?zid=171935)|[6th #1](https://zwiftpower.com/events.php?zid=152778)|[6th #7](https://zwiftpower.com/events.php?zid=173451)|[7th #5](https://zwiftpower.com/events.php?zid=166630)|[8th #4](https://zwiftpower.com/events.php?zid=162691)|[11th #8](https://zwiftpower.com/events.php?zid=176148)
A|5|5912|[Stefan Kirchmair \(KC\)](https://zwiftpower\.com/profile\.php?z=595291)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥉 #7](https://zwiftpower.com/events.php?zid=173451)|[4th #4](https://zwiftpower.com/events.php?zid=162691)|[4th #6](https://zwiftpower.com/events.php?zid=171935)|[9th #5](https://zwiftpower.com/events.php?zid=166630)|[13th #1](https://zwiftpower.com/events.php?zid=152778)|[14th #3](https://zwiftpower.com/events.php?zid=159334)
A|6|5804|[Frank Sorbara 🔰](https://zwiftpower\.com/profile\.php?z=110603)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[11th #7](https://zwiftpower.com/events.php?zid=173451)|[14th #6](https://zwiftpower.com/events.php?zid=171935)|[15th #5](https://zwiftpower.com/events.php?zid=166630)|[16th #1](https://zwiftpower.com/events.php?zid=152778)|[16th #3](https://zwiftpower.com/events.php?zid=159334)|[29th #8](https://zwiftpower.com/events.php?zid=176148)
A|7|5796|[James Phillips](https://zwiftpower\.com/profile\.php?z=280522)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[🥈 #8](https://zwiftpower.com/events.php?zid=176148)|[10th #4](https://zwiftpower.com/events.php?zid=162691)|[20th #1](https://zwiftpower.com/events.php?zid=152778)|[20th #3](https://zwiftpower.com/events.php?zid=159334)|[25th #5](https://zwiftpower.com/events.php?zid=166630)|[28th #6](https://zwiftpower.com/events.php?zid=171935)
A|8|5782|[Stevie Young 🐍](https://zwiftpower\.com/profile\.php?z=482899)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[🥉 #8](https://zwiftpower.com/events.php?zid=176148)|[8th #3](https://zwiftpower.com/events.php?zid=159334)|[9th #1](https://zwiftpower.com/events.php?zid=152778)|[11th #4](https://zwiftpower.com/events.php?zid=162691)|[17th #6](https://zwiftpower.com/events.php?zid=171935)|[64th #5](https://zwiftpower.com/events.php?zid=166630)
A|9|5732|[Ingo Reichart ✪](https://zwiftpower\.com/profile\.php?z=94244)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥉 #6](https://zwiftpower.com/events.php?zid=171935)|[5th #4](https://zwiftpower.com/events.php?zid=162691)|[10th #7](https://zwiftpower.com/events.php?zid=173451)|[13th #3](https://zwiftpower.com/events.php?zid=159334)|[23rd #1](https://zwiftpower.com/events.php?zid=152778)|[83rd #5](https://zwiftpower.com/events.php?zid=166630)
A|10|5728|[Fabian Dé 🦍](https://zwiftpower\.com/profile\.php?z=4635)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[12th #6](https://zwiftpower.com/events.php?zid=171935)|[19th #1](https://zwiftpower.com/events.php?zid=152778)|[20th #7](https://zwiftpower.com/events.php?zid=173451)|[26th #8](https://zwiftpower.com/events.php?zid=176148)|[6th (📣30th) #2](https://zwiftpower.com/events.php?zid=155628)|[32nd #5](https://zwiftpower.com/events.php?zid=166630)
A|11|5704|[Marc Waldemar \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=448426)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[15th #8](https://zwiftpower.com/events.php?zid=176148)|[19th #4](https://zwiftpower.com/events.php?zid=162691)|[21st #6](https://zwiftpower.com/events.php?zid=171935)|[23rd #7](https://zwiftpower.com/events.php?zid=173451)|[29th #5](https://zwiftpower.com/events.php?zid=166630)|[20th (📣44th) #2](https://zwiftpower.com/events.php?zid=155628)
A|12|5634|[Birk Hansen 🔰](https://zwiftpower\.com/profile\.php?z=550613)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[7th #4](https://zwiftpower.com/events.php?zid=162691)|[24th #3](https://zwiftpower.com/events.php?zid=159334)|[🥈 (📣26th) #2](https://zwiftpower.com/events.php?zid=155628)|[33rd #6](https://zwiftpower.com/events.php?zid=171935)|[36th #7](https://zwiftpower.com/events.php?zid=173451)|[60th #5](https://zwiftpower.com/events.php?zid=166630)
A|13|5618|[Morten Vaeng](https://zwiftpower\.com/profile\.php?z=259131)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[19th #7](https://zwiftpower.com/events.php?zid=173451)|[20th #6](https://zwiftpower.com/events.php?zid=171935)|[23rd #5](https://zwiftpower.com/events.php?zid=166630)|[30th #3](https://zwiftpower.com/events.php?zid=159334)|[40th #8](https://zwiftpower.com/events.php?zid=176148)|[62nd #1](https://zwiftpower.com/events.php?zid=152778)
A|14|5572|[Mike Swart](https://zwiftpower\.com/profile\.php?z=503)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[28th #4](https://zwiftpower.com/events.php?zid=162691)|[28th #8](https://zwiftpower.com/events.php?zid=176148)|[35th #7](https://zwiftpower.com/events.php?zid=173451)|[38th #3](https://zwiftpower.com/events.php?zid=159334)|[39th #6](https://zwiftpower.com/events.php?zid=171935)|[25th (📣49th) #2](https://zwiftpower.com/events.php?zid=155628)
A|15|5548|[Stian Lersveen 🔰](https://zwiftpower\.com/profile\.php?z=381136)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[25th #7](https://zwiftpower.com/events.php?zid=173451)|[34th #6](https://zwiftpower.com/events.php?zid=171935)|[37th #8](https://zwiftpower.com/events.php?zid=176148)|[39th #1](https://zwiftpower.com/events.php?zid=152778)|[17th (📣41st) #2](https://zwiftpower.com/events.php?zid=155628)|[53rd #4](https://zwiftpower.com/events.php?zid=162691)
A|16|5540|"[Paul de Haan \| Zwift Dutch Racing](https://zwiftpower\.com/profile\.php?z=525478)"|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[6th #6](https://zwiftpower.com/events.php?zid=171935)|[14th #7](https://zwiftpower.com/events.php?zid=173451)|[17th #4](https://zwiftpower.com/events.php?zid=162691)|[23rd #3](https://zwiftpower.com/events.php?zid=159334)|[5th (📣29th) #2](https://zwiftpower.com/events.php?zid=155628)|[144th #1](https://zwiftpower.com/events.php?zid=152778)
A|17|5506|[Klaus Näher \(KC\)](https://zwiftpower\.com/profile\.php?z=518275)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[22nd #8](https://zwiftpower.com/events.php?zid=176148)|[36th #6](https://zwiftpower.com/events.php?zid=171935)|[39th #4](https://zwiftpower.com/events.php?zid=162691)|[23rd (📣47th) #2](https://zwiftpower.com/events.php?zid=155628)|[48th #7](https://zwiftpower.com/events.php?zid=173451)|[58th #3](https://zwiftpower.com/events.php?zid=159334)
A|18|5502|[Maxim Golev \[V\]](https://zwiftpower\.com/profile\.php?z=811341)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[18th #4](https://zwiftpower.com/events.php?zid=162691)|[22nd #1](https://zwiftpower.com/events.php?zid=152778)|[27th #3](https://zwiftpower.com/events.php?zid=159334)|[33rd #8](https://zwiftpower.com/events.php?zid=176148)|[45th #5](https://zwiftpower.com/events.php?zid=166630)|[107th #6](https://zwiftpower.com/events.php?zid=171935)
A|19|5448|[Dean Cunningham](https://zwiftpower\.com/profile\.php?z=78803)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[23rd #6](https://zwiftpower.com/events.php?zid=171935)|[27th #7](https://zwiftpower.com/events.php?zid=173451)|[36th #5](https://zwiftpower.com/events.php?zid=166630)|[59th #8](https://zwiftpower.com/events.php?zid=176148)|[37th (📣61st) #2](https://zwiftpower.com/events.php?zid=155628)|[73rd #3](https://zwiftpower.com/events.php?zid=159334)
A|20|5424|[Matthew Collins U17](https://zwiftpower\.com/profile\.php?z=541352)|—|[28th #7](https://zwiftpower.com/events.php?zid=173451)|[37th #3](https://zwiftpower.com/events.php?zid=159334)|[37th #6](https://zwiftpower.com/events.php?zid=171935)|[51st #4](https://zwiftpower.com/events.php?zid=162691)|[61st #8](https://zwiftpower.com/events.php?zid=176148)|[77th #5](https://zwiftpower.com/events.php?zid=166630)
A|21|5412|[Joshua Modert](https://zwiftpower\.com/profile\.php?z=584631)|—|[27th #6](https://zwiftpower.com/events.php?zid=171935)|[29th #7](https://zwiftpower.com/events.php?zid=173451)|[38th #8](https://zwiftpower.com/events.php?zid=176148)|[50th #4](https://zwiftpower.com/events.php?zid=162691)|[34th (📣58th) #2](https://zwiftpower.com/events.php?zid=155628)|[95th #3](https://zwiftpower.com/events.php?zid=159334)
A|22|5400|[Vinz van Teeseling 🦈](https://zwiftpower\.com/profile\.php?z=807046)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[22nd #6](https://zwiftpower.com/events.php?zid=171935)|[35th #8](https://zwiftpower.com/events.php?zid=176148)|[13th (📣37th) #2](https://zwiftpower.com/events.php?zid=155628)|[64th #1](https://zwiftpower.com/events.php?zid=152778)|[70th #5](https://zwiftpower.com/events.php?zid=166630)|[75th #7](https://zwiftpower.com/events.php?zid=173451)
A|23|5358|[Tom Ove Kaland](https://zwiftpower\.com/profile\.php?z=416)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[34th #7](https://zwiftpower.com/events.php?zid=173451)|[45th #6](https://zwiftpower.com/events.php?zid=171935)|[28th (📣52nd) #2](https://zwiftpower.com/events.php?zid=155628)|[58th #1](https://zwiftpower.com/events.php?zid=152778)|[64th #3](https://zwiftpower.com/events.php?zid=159334)|[71st #4](https://zwiftpower.com/events.php?zid=162691)
A|24|5296|[D\. Sputnik 🚀](https://zwiftpower\.com/profile\.php?z=579101)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[33rd #7](https://zwiftpower.com/events.php?zid=173451)|[51st #6](https://zwiftpower.com/events.php?zid=171935)|[58th #5](https://zwiftpower.com/events.php?zid=166630)|[39th (📣63rd) #2](https://zwiftpower.com/events.php?zid=155628)|[72nd #1](https://zwiftpower.com/events.php?zid=152778)|[78th #4](https://zwiftpower.com/events.php?zid=162691)
A|25|5262|[Josh Harrison](https://zwiftpower\.com/profile\.php?z=700044)|—|[37th #7](https://zwiftpower.com/events.php?zid=173451)|[41st #6](https://zwiftpower.com/events.php?zid=171935)|[46th #5](https://zwiftpower.com/events.php?zid=166630)|[65th #4](https://zwiftpower.com/events.php?zid=162691)|[87th #3](https://zwiftpower.com/events.php?zid=159334)|[96th #1](https://zwiftpower.com/events.php?zid=152778)
A|26|5260|[Dafydd Williams](https://zwiftpower\.com/profile\.php?z=123836)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[50th #7](https://zwiftpower.com/events.php?zid=173451)|[58th #8](https://zwiftpower.com/events.php?zid=176148)|[35th (📣59th) #2](https://zwiftpower.com/events.php?zid=155628)|[59th #3](https://zwiftpower.com/events.php?zid=159334)|[62nd #5](https://zwiftpower.com/events.php?zid=166630)|[85th #4](https://zwiftpower.com/events.php?zid=162691)
A|27|5246|[Marcin Krzywonos](https://zwiftpower\.com/profile\.php?z=308424)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[46th #3](https://zwiftpower.com/events.php?zid=159334)|[53rd #8](https://zwiftpower.com/events.php?zid=176148)|[60th #7](https://zwiftpower.com/events.php?zid=173451)|[67th #6](https://zwiftpower.com/events.php?zid=171935)|[52nd (📣76th) #2](https://zwiftpower.com/events.php?zid=155628)|[78th #1](https://zwiftpower.com/events.php?zid=152778)
A|28|5244|[Freddy van der Ploeg 💣](https://zwiftpower\.com/profile\.php?z=95453)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[20th #8](https://zwiftpower.com/events.php?zid=176148)|[53rd #6](https://zwiftpower.com/events.php?zid=171935)|[53rd #7](https://zwiftpower.com/events.php?zid=173451)|[66th #5](https://zwiftpower.com/events.php?zid=166630)|[88th #4](https://zwiftpower.com/events.php?zid=162691)|[101st #1](https://zwiftpower.com/events.php?zid=152778)
A|29|5216|[atsushi yamaguchi](https://zwiftpower\.com/profile\.php?z=870907)|[ZJRT](https://zwiftpower\.com/team\.php?id=2054)|[40th #5](https://zwiftpower.com/events.php?zid=166630)|[41st #7](https://zwiftpower.com/events.php?zid=173451)|[52nd #4](https://zwiftpower.com/events.php?zid=162691)|[43rd (📣67th) #2](https://zwiftpower.com/events.php?zid=155628)|[77th #1](https://zwiftpower.com/events.php?zid=152778)|[118th #3](https://zwiftpower.com/events.php?zid=159334)
A|30|5204|[Adam Chamberlin](https://zwiftpower\.com/profile\.php?z=9087)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[42nd #6](https://zwiftpower.com/events.php?zid=171935)|[42nd #7](https://zwiftpower.com/events.php?zid=173451)|[62nd #8](https://zwiftpower.com/events.php?zid=176148)|[74th #1](https://zwiftpower.com/events.php?zid=152778)|[59th (📣83rd) #2](https://zwiftpower.com/events.php?zid=155628)|[98th #3](https://zwiftpower.com/events.php?zid=159334)
A|31|5174|[Justinas Leveika](https://zwiftpower\.com/profile\.php?z=423707)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[26th (📣50th) #2](https://zwiftpower.com/events.php?zid=155628)|[56th #4](https://zwiftpower.com/events.php?zid=162691)|[64th #7](https://zwiftpower.com/events.php?zid=173451)|[69th #6](https://zwiftpower.com/events.php?zid=171935)|[71st #3](https://zwiftpower.com/events.php?zid=159334)|[106th #5](https://zwiftpower.com/events.php?zid=166630)
A|32|5166|[Keith Robertson <BRT> ⚡](https://zwiftpower\.com/profile\.php?z=894741)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[49th #7](https://zwiftpower.com/events.php?zid=173451)|[58th #6](https://zwiftpower.com/events.php?zid=171935)|[59th #5](https://zwiftpower.com/events.php?zid=166630)|[64th #8](https://zwiftpower.com/events.php?zid=176148)|[83rd #3](https://zwiftpower.com/events.php?zid=159334)|[107th #4](https://zwiftpower.com/events.php?zid=162691)
A|33|5120|[Wesley Ooms🔰Tacx Neo](https://zwiftpower\.com/profile\.php?z=561362)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[55th #7](https://zwiftpower.com/events.php?zid=173451)|[63rd #8](https://zwiftpower.com/events.php?zid=176148)|[69th #1](https://zwiftpower.com/events.php?zid=152778)|[72nd #6](https://zwiftpower.com/events.php?zid=171935)|[76th #5](https://zwiftpower.com/events.php?zid=166630)|[84th (📣108th) #2](https://zwiftpower.com/events.php?zid=155628)
A|34|5086|[Philippe Van Laethem](https://zwiftpower\.com/profile\.php?z=688960)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[41st #8](https://zwiftpower.com/events.php?zid=176148)|[68th #5](https://zwiftpower.com/events.php?zid=166630)|[72nd #3](https://zwiftpower.com/events.php?zid=159334)|[74th #6](https://zwiftpower.com/events.php?zid=171935)|[71st (📣95th) #2](https://zwiftpower.com/events.php?zid=155628)|[110th #7](https://zwiftpower.com/events.php?zid=173451)
A|35|5024|[Michael Durr \[BRT\]](https://zwiftpower\.com/profile\.php?z=74813)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[44th #7](https://zwiftpower.com/events.php?zid=173451)|[59th #6](https://zwiftpower.com/events.php?zid=171935)|[91st #5](https://zwiftpower.com/events.php?zid=166630)|[93rd #8](https://zwiftpower.com/events.php?zid=176148)|[99th #1](https://zwiftpower.com/events.php?zid=152778)|[105th #3](https://zwiftpower.com/events.php?zid=159334)
A|36|5022|[Hal Blackburn](https://zwiftpower\.com/profile\.php?z=904023)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[52nd #7](https://zwiftpower.com/events.php?zid=173451)|[77th #6](https://zwiftpower.com/events.php?zid=171935)|[54th (📣78th) #2](https://zwiftpower.com/events.php?zid=155628)|[79th #8](https://zwiftpower.com/events.php?zid=176148)|[94th #1](https://zwiftpower.com/events.php?zid=152778)|[112th #3](https://zwiftpower.com/events.php?zid=159334)
A|37|5010|[Balazs Galambos \(IRL\)](https://zwiftpower\.com/profile\.php?z=146672)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|[65th #6](https://zwiftpower.com/events.php?zid=171935)|[67th #7](https://zwiftpower.com/events.php?zid=173451)|[73rd #4](https://zwiftpower.com/events.php?zid=162691)|[84th #5](https://zwiftpower.com/events.php?zid=166630)|[90th #1](https://zwiftpower.com/events.php?zid=152778)|[119th #3](https://zwiftpower.com/events.php?zid=159334)
A|38|5002|[Vytautas Jokubonis](https://zwiftpower\.com/profile\.php?z=199043)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[65th #7](https://zwiftpower.com/events.php?zid=173451)|[78th #6](https://zwiftpower.com/events.php?zid=171935)|[81st #3](https://zwiftpower.com/events.php?zid=159334)|[83rd #8](https://zwiftpower.com/events.php?zid=176148)|[67th (📣91st) #2](https://zwiftpower.com/events.php?zid=155628)|[104th #4](https://zwiftpower.com/events.php?zid=162691)
A|39|4950|[Erling Strømsholm \[V\]](https://zwiftpower\.com/profile\.php?z=44022)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[70th #7](https://zwiftpower.com/events.php?zid=173451)|[71st #8](https://zwiftpower.com/events.php?zid=176148)|[81st #6](https://zwiftpower.com/events.php?zid=171935)|[86th #4](https://zwiftpower.com/events.php?zid=162691)|[86th (📣110th) #2](https://zwiftpower.com/events.php?zid=155628)|[110th #5](https://zwiftpower.com/events.php?zid=166630)
A|40|4896|[Karl W ⚡️](https://zwiftpower\.com/profile\.php?z=77662)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[61st #4](https://zwiftpower.com/events.php?zid=162691)|[80th #1](https://zwiftpower.com/events.php?zid=152778)|[94th #3](https://zwiftpower.com/events.php?zid=159334)|[96th #5](https://zwiftpower.com/events.php?zid=166630)|[87th (📣111th) #2](https://zwiftpower.com/events.php?zid=155628)|[113th #8](https://zwiftpower.com/events.php?zid=176148)
A|41|4865|[Maico Voets ✈](https://zwiftpower\.com/profile\.php?z=543349)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[7th #8](https://zwiftpower.com/events.php?zid=176148)|[12th #4](https://zwiftpower.com/events.php?zid=162691)|[12th #7](https://zwiftpower.com/events.php?zid=173451)|[13th #6](https://zwiftpower.com/events.php?zid=171935)|[26th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|42|4859|[Adam Zimmerman 🔰](https://zwiftpower\.com/profile\.php?z=7915)|[⚒](https://zwiftpower\.com/team\.php?id=3973)|[🥇 #8](https://zwiftpower.com/events.php?zid=176148)|[6th #3](https://zwiftpower.com/events.php?zid=159334)|[8th #1](https://zwiftpower.com/events.php?zid=152778)|[15th #6](https://zwiftpower.com/events.php?zid=171935)|[43rd #5](https://zwiftpower.com/events.php?zid=166630)|—
A|42|4859|[Piotr Truszczyński \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=175948)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[12th #8](https://zwiftpower.com/events.php?zid=176148)|[13th #7](https://zwiftpower.com/events.php?zid=173451)|[14th #1](https://zwiftpower.com/events.php?zid=152778)|[16th #6](https://zwiftpower.com/events.php?zid=171935)|[18th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|43|4832|[Kjetil Berntzen🔰 🐥](https://zwiftpower\.com/profile\.php?z=38911)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[89th #7](https://zwiftpower.com/events.php?zid=173451)|[91st #6](https://zwiftpower.com/events.php?zid=171935)|[92nd #4](https://zwiftpower.com/events.php?zid=162691)|[72nd (📣96th) #2](https://zwiftpower.com/events.php?zid=155628)|[107th #3](https://zwiftpower.com/events.php?zid=159334)|[112th #8](https://zwiftpower.com/events.php?zid=176148)
A|44|4808|[Mark Gardiner](https://zwiftpower\.com/profile\.php?z=691846)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[62nd #6](https://zwiftpower.com/events.php?zid=171935)|[80th #7](https://zwiftpower.com/events.php?zid=173451)|[82nd #8](https://zwiftpower.com/events.php?zid=176148)|[91st (📣115th) #2](https://zwiftpower.com/events.php?zid=155628)|[128th #3](https://zwiftpower.com/events.php?zid=159334)|[132nd #4](https://zwiftpower.com/events.php?zid=162691)
A|45|4790|[Mitchell Bell](https://zwiftpower\.com/profile\.php?z=445221)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[85th #7](https://zwiftpower.com/events.php?zid=173451)|[93rd #6](https://zwiftpower.com/events.php?zid=171935)|[94th #8](https://zwiftpower.com/events.php?zid=176148)|[83rd (📣107th) #2](https://zwiftpower.com/events.php?zid=155628)|[112th #5](https://zwiftpower.com/events.php?zid=166630)|[117th #1](https://zwiftpower.com/events.php?zid=152778)
A|46|4782|[Bartosz Nowaczek](https://zwiftpower\.com/profile\.php?z=358286)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[78th #8](https://zwiftpower.com/events.php?zid=176148)|[81st #7](https://zwiftpower.com/events.php?zid=173451)|[108th #5](https://zwiftpower.com/events.php?zid=166630)|[113th #4](https://zwiftpower.com/events.php?zid=162691)|[115th #1](https://zwiftpower.com/events.php?zid=152778)|[117th #3](https://zwiftpower.com/events.php?zid=159334)
A|47|4777|[Søren Bay 🔰](https://zwiftpower\.com/profile\.php?z=255659)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[10th #3](https://zwiftpower.com/events.php?zid=159334)|[15th #4](https://zwiftpower.com/events.php?zid=162691)|[21st #8](https://zwiftpower.com/events.php?zid=176148)|[7th (📣31st) #2](https://zwiftpower.com/events.php?zid=155628)|[37th #1](https://zwiftpower.com/events.php?zid=152778)|—
A|48|4730|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[86th #7](https://zwiftpower.com/events.php?zid=173451)|[87th #8](https://zwiftpower.com/events.php?zid=176148)|[98th #4](https://zwiftpower.com/events.php?zid=162691)|[121st #1](https://zwiftpower.com/events.php?zid=152778)|[97th (📣121st) #2](https://zwiftpower.com/events.php?zid=155628)|[125th #5](https://zwiftpower.com/events.php?zid=166630)
A|49|4718|[Nik Hall](https://zwiftpower\.com/profile\.php?z=297624)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[76th #7](https://zwiftpower.com/events.php?zid=173451)|[92nd #6](https://zwiftpower.com/events.php?zid=171935)|[97th #8](https://zwiftpower.com/events.php?zid=176148)|[99th (📣123rd) #2](https://zwiftpower.com/events.php?zid=155628)|[128th #1](https://zwiftpower.com/events.php?zid=152778)|[128th #5](https://zwiftpower.com/events.php?zid=166630)
A|50|4675|[Oliver Dowd 🔰🐆](https://zwiftpower\.com/profile\.php?z=244714)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[29th #6](https://zwiftpower.com/events.php?zid=171935)|[32nd #1](https://zwiftpower.com/events.php?zid=152778)|[8th (📣32nd) #2](https://zwiftpower.com/events.php?zid=155628)|[33rd #4](https://zwiftpower.com/events.php?zid=162691)|[39th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|51|4674|[Anders Broberg](https://zwiftpower\.com/profile\.php?z=407765)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[84th #7](https://zwiftpower.com/events.php?zid=173451)|[91st #8](https://zwiftpower.com/events.php?zid=176148)|[100th #6](https://zwiftpower.com/events.php?zid=171935)|[95th (📣119th) #2](https://zwiftpower.com/events.php?zid=155628)|[132nd #3](https://zwiftpower.com/events.php?zid=159334)|[140th #1](https://zwiftpower.com/events.php?zid=152778)
A|52|4661|[Mon Ritch](https://zwiftpower\.com/profile\.php?z=135635)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[16th #4](https://zwiftpower.com/events.php?zid=162691)|[4th (📣28th) #2](https://zwiftpower.com/events.php?zid=155628)|[34th #1](https://zwiftpower.com/events.php?zid=152778)|[40th #3](https://zwiftpower.com/events.php?zid=159334)|[54th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|53|4655|[Cameron Jeffers](https://zwiftpower\.com/profile\.php?z=9659)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[13th #5](https://zwiftpower.com/events.php?zid=166630)|[20th #4](https://zwiftpower.com/events.php?zid=162691)|[30th #8](https://zwiftpower.com/events.php?zid=176148)|[33rd #3](https://zwiftpower.com/events.php?zid=159334)|[79th #6](https://zwiftpower.com/events.php?zid=171935)|—
A|54|4653|[Simon Merritt \(CRYO\-GEN\) \(GSV\)](https://zwiftpower\.com/profile\.php?z=38493)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[8th #5](https://zwiftpower.com/events.php?zid=166630)|[24th #7](https://zwiftpower.com/events.php?zid=173451)|[25th #3](https://zwiftpower.com/events.php?zid=159334)|[30th #4](https://zwiftpower.com/events.php?zid=162691)|[89th #1](https://zwiftpower.com/events.php?zid=152778)|—
A|55|4633|[Dan Fleeman \- DigDeepCoaching\.Com](https://zwiftpower\.com/profile\.php?z=49551)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #5](https://zwiftpower.com/events.php?zid=166630)|[5th #1](https://zwiftpower.com/events.php?zid=152778)|[7th #3](https://zwiftpower.com/events.php?zid=159334)|[7th #7](https://zwiftpower.com/events.php?zid=173451)|[141st (📣165th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|56|4609|[Philipp Diegner 🔰](https://zwiftpower\.com/profile\.php?z=26829)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[🥉 #3](https://zwiftpower.com/events.php?zid=159334)|[7th #1](https://zwiftpower.com/events.php?zid=152778)|[8th #6](https://zwiftpower.com/events.php?zid=171935)|[19th #8](https://zwiftpower.com/events.php?zid=176148)|[137th (📣161st) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|57|4596|[Matthias Blümel \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=855399)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[100th #7](https://zwiftpower.com/events.php?zid=173451)|[100th #8](https://zwiftpower.com/events.php?zid=176148)|[105th #6](https://zwiftpower.com/events.php?zid=171935)|[123rd #4](https://zwiftpower.com/events.php?zid=162691)|[127th #3](https://zwiftpower.com/events.php?zid=159334)|[126th (📣150th) #2](https://zwiftpower.com/events.php?zid=155628)
A|58|4595|[David Pacheco 🔰](https://zwiftpower\.com/profile\.php?z=57523)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[14th #8](https://zwiftpower.com/events.php?zid=176148)|[30th #7](https://zwiftpower.com/events.php?zid=173451)|[38th #6](https://zwiftpower.com/events.php?zid=171935)|[53rd #5](https://zwiftpower.com/events.php?zid=166630)|[70th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|59|4585|[Jarno Vader](https://zwiftpower\.com/profile\.php?z=61708)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[27th #1](https://zwiftpower.com/events.php?zid=152778)|[27th #4](https://zwiftpower.com/events.php?zid=162691)|[33rd #5](https://zwiftpower.com/events.php?zid=166630)|[48th #6](https://zwiftpower.com/events.php?zid=171935)|[75th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|60|4559|[Max Powersimon Ⓥ](https://zwiftpower\.com/profile\.php?z=19498)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[24th #6](https://zwiftpower.com/events.php?zid=171935)|[41st #4](https://zwiftpower.com/events.php?zid=162691)|[43rd #8](https://zwiftpower.com/events.php?zid=176148)|[48th #5](https://zwiftpower.com/events.php?zid=166630)|[67th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|61|4521|[james gelsthorpe](https://zwiftpower\.com/profile\.php?z=965858)|[Velo One](https://zwiftpower\.com/team\.php?id=2078)|[17th #5](https://zwiftpower.com/events.php?zid=166630)|[27th #8](https://zwiftpower.com/events.php?zid=176148)|[57th #4](https://zwiftpower.com/events.php?zid=162691)|[40th (📣64th) #2](https://zwiftpower.com/events.php?zid=155628)|[77th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|62|4501|[Ryo Misaki](https://zwiftpower\.com/profile\.php?z=380080)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[10th #5](https://zwiftpower.com/events.php?zid=166630)|[17th #1](https://zwiftpower.com/events.php?zid=152778)|[26th #3](https://zwiftpower.com/events.php?zid=159334)|[36th #4](https://zwiftpower.com/events.php?zid=162691)|[139th (📣163rd) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|63|4413|[Kristian Nøhr 🔰](https://zwiftpower\.com/profile\.php?z=473748)|—|[50th #3](https://zwiftpower.com/events.php?zid=159334)|[50th #6](https://zwiftpower.com/events.php?zid=171935)|[52nd #1](https://zwiftpower.com/events.php?zid=152778)|[33rd (📣57th) #2](https://zwiftpower.com/events.php?zid=155628)|[87th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|64|4401|[Tom Gakes](https://zwiftpower\.com/profile\.php?z=387828)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[23rd #4](https://zwiftpower.com/events.php?zid=162691)|[30th #1](https://zwiftpower.com/events.php?zid=152778)|[53rd #3](https://zwiftpower.com/events.php?zid=159334)|[92nd #5](https://zwiftpower.com/events.php?zid=166630)|[80th (📣104th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|65|4387|[LUKASZ GRABARCZYK](https://zwiftpower\.com/profile\.php?z=506082)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[43rd #7](https://zwiftpower.com/events.php?zid=173451)|[48th #3](https://zwiftpower.com/events.php?zid=159334)|[63rd #1](https://zwiftpower.com/events.php?zid=152778)|[68th #4](https://zwiftpower.com/events.php?zid=162691)|[63rd (📣87th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|66|4353|[Rusty Devlin🔰Charged Drinks](https://zwiftpower\.com/profile\.php?z=27060)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[49th #8](https://zwiftpower.com/events.php?zid=176148)|[52nd #6](https://zwiftpower.com/events.php?zid=171935)|[65th #1](https://zwiftpower.com/events.php?zid=152778)|[73rd #5](https://zwiftpower.com/events.php?zid=166630)|[87th #4](https://zwiftpower.com/events.php?zid=162691)|—
A|67|4341|[Tomasz Ciepłucha \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=934433)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[42nd #3](https://zwiftpower.com/events.php?zid=159334)|[45th #8](https://zwiftpower.com/events.php?zid=176148)|[46th #7](https://zwiftpower.com/events.php?zid=173451)|[99th #6](https://zwiftpower.com/events.php?zid=171935)|[100th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|68|4323|[Eric Borchert // Track Mob Berlin](https://zwiftpower\.com/profile\.php?z=408117)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[57th #8](https://zwiftpower.com/events.php?zid=176148)|[38th (📣62nd) #2](https://zwiftpower.com/events.php?zid=155628)|[71st #1](https://zwiftpower.com/events.php?zid=152778)|[75th #4](https://zwiftpower.com/events.php?zid=162691)|[76th #3](https://zwiftpower.com/events.php?zid=159334)|—
A|69|4317|[Brage Bergeng \[V\]🐢](https://zwiftpower\.com/profile\.php?z=429749)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[23rd #8](https://zwiftpower.com/events.php?zid=176148)|[32nd #7](https://zwiftpower.com/events.php?zid=173451)|[40th #4](https://zwiftpower.com/events.php?zid=162691)|[79th #1](https://zwiftpower.com/events.php?zid=152778)|[146th (📣170th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|70|4273|[Guus Bouman🦆](https://zwiftpower\.com/profile\.php?z=408391)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[40th #1](https://zwiftpower.com/events.php?zid=152778)|[44th #6](https://zwiftpower.com/events.php?zid=171935)|[75th #5](https://zwiftpower.com/events.php?zid=166630)|[90th #4](https://zwiftpower.com/events.php?zid=162691)|[93rd (📣117th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|71|4253|[Ryan Gooderham](https://zwiftpower\.com/profile\.php?z=266973)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[54th #3](https://zwiftpower.com/events.php?zid=159334)|[55th #6](https://zwiftpower.com/events.php?zid=171935)|[73rd #8](https://zwiftpower.com/events.php?zid=176148)|[65th (📣89th) #2](https://zwiftpower.com/events.php?zid=155628)|[105th #7](https://zwiftpower.com/events.php?zid=173451)|—
A|72|4129|[Jürgen Schäfer\(KC\) Elbspitze 19](https://zwiftpower\.com/profile\.php?z=466660)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[51st #7](https://zwiftpower.com/events.php?zid=173451)|[73rd #1](https://zwiftpower.com/events.php?zid=152778)|[81st #5](https://zwiftpower.com/events.php?zid=166630)|[68th (📣92nd) #2](https://zwiftpower.com/events.php?zid=155628)|[141st #4](https://zwiftpower.com/events.php?zid=162691)|—
A|73|3967|[Jacek Tarach](https://zwiftpower\.com/profile\.php?z=62218)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[84th #6](https://zwiftpower.com/events.php?zid=171935)|[87th #7](https://zwiftpower.com/events.php?zid=173451)|[110th #3](https://zwiftpower.com/events.php?zid=159334)|[111th #4](https://zwiftpower.com/events.php?zid=162691)|[127th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|74|3872|[Wouter Sybrandy](https://zwiftpower\.com/profile\.php?z=2021)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[6th #8](https://zwiftpower.com/events.php?zid=176148)|[17th #3](https://zwiftpower.com/events.php?zid=159334)|[18th #7](https://zwiftpower.com/events.php?zid=173451)|[25th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
A|75|3861|[Miha Kosmač](https://zwiftpower\.com/profile\.php?z=484492)|[Top\-Fit](https://zwiftpower\.com/team\.php?id=2019)|[95th #5](https://zwiftpower.com/events.php?zid=166630)|[99th #4](https://zwiftpower.com/events.php?zid=162691)|[107th #1](https://zwiftpower.com/events.php?zid=152778)|[135th #3](https://zwiftpower.com/events.php?zid=159334)|[112th (📣136th) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|76|3834|[Gavin Dempster🦕🔰](https://zwiftpower\.com/profile\.php?z=6499)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[🥉 #1](https://zwiftpower.com/events.php?zid=152778)|[9th #6](https://zwiftpower.com/events.php?zid=171935)|[15th #3](https://zwiftpower.com/events.php?zid=159334)|[58th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
A|77|3831|[Marcel Bischof \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=170911)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[104th #3](https://zwiftpower.com/events.php?zid=159334)|[106th #4](https://zwiftpower.com/events.php?zid=162691)|[121st #5](https://zwiftpower.com/events.php?zid=166630)|[98th (📣122nd) #2](https://zwiftpower.com/events.php?zid=155628)|[134th #1](https://zwiftpower.com/events.php?zid=152778)|—
A|78|3827|[Shane Johnson CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=879319)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[98th #5](https://zwiftpower.com/events.php?zid=166630)|[98th #7](https://zwiftpower.com/events.php?zid=173451)|[108th #4](https://zwiftpower.com/events.php?zid=162691)|[133rd #3](https://zwiftpower.com/events.php?zid=159334)|[128th (📣152nd) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|79|3798|[Daniel Pettinger](https://zwiftpower\.com/profile\.php?z=90946)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[19th #5](https://zwiftpower.com/events.php?zid=166630)|[22nd #7](https://zwiftpower.com/events.php?zid=173451)|[26th #6](https://zwiftpower.com/events.php?zid=171935)|[36th #8](https://zwiftpower.com/events.php?zid=176148)|—|—
A|80|3771|[Manuel Ruhland \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=1047299)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[104th #7](https://zwiftpower.com/events.php?zid=173451)|[106th #6](https://zwiftpower.com/events.php?zid=171935)|[118th #5](https://zwiftpower.com/events.php?zid=166630)|[138th #3](https://zwiftpower.com/events.php?zid=159334)|[127th (📣151st) #2](https://zwiftpower.com/events.php?zid=155628)|—
A|81|3768|[Brian Hodges](https://zwiftpower\.com/profile\.php?z=399270)|—|[10th #8](https://zwiftpower.com/events.php?zid=176148)|[18th #3](https://zwiftpower.com/events.php?zid=159334)|[45th #4](https://zwiftpower.com/events.php?zid=162691)|[45th #7](https://zwiftpower.com/events.php?zid=173451)|—|—
A|82|3739|[Ingo Ruhmund \(BMPRO\)](https://zwiftpower\.com/profile\.php?z=452096)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[108th #6](https://zwiftpower.com/events.php?zid=171935)|[108th #7](https://zwiftpower.com/events.php?zid=173451)|[115th #8](https://zwiftpower.com/events.php?zid=176148)|[145th #4](https://zwiftpower.com/events.php?zid=162691)|[157th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|83|3701|[Leon van Bon](https://zwiftpower\.com/profile\.php?z=96777)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[110th #6](https://zwiftpower.com/events.php?zid=171935)|[117th #4](https://zwiftpower.com/events.php?zid=162691)|[100th (📣124th) #2](https://zwiftpower.com/events.php?zid=155628)|[147th #1](https://zwiftpower.com/events.php?zid=152778)|[154th #5](https://zwiftpower.com/events.php?zid=166630)|—
A|84|3636|[Ed Laverack](https://zwiftpower\.com/profile\.php?z=703388)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[11th #5](https://zwiftpower.com/events.php?zid=166630)|[26th #1](https://zwiftpower.com/events.php?zid=152778)|[63rd #3](https://zwiftpower.com/events.php?zid=159334)|[84th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
A|85|3628|[Reece Egner](https://zwiftpower\.com/profile\.php?z=243035)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[16th #8](https://zwiftpower.com/events.php?zid=176148)|[41st #1](https://zwiftpower.com/events.php?zid=152778)|[63rd #5](https://zwiftpower.com/events.php?zid=166630)|[44th (📣68th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—
A|86|3614|[Andreas Rois \(KC\)](https://zwiftpower\.com/profile\.php?z=991994)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[22nd #5](https://zwiftpower.com/events.php?zid=166630)|[46th #8](https://zwiftpower.com/events.php?zid=176148)|[53rd #1](https://zwiftpower.com/events.php?zid=152778)|[74th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
A|87|3600|[Ash Beech](https://zwiftpower\.com/profile\.php?z=644482)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[32nd #6](https://zwiftpower.com/events.php?zid=171935)|[34th #4](https://zwiftpower.com/events.php?zid=162691)|[45th #1](https://zwiftpower.com/events.php?zid=152778)|[91st #7](https://zwiftpower.com/events.php?zid=173451)|—|—
A|88|3594|[gavin arber RPR](https://zwiftpower\.com/profile\.php?z=606)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[15th (📣39th) #2](https://zwiftpower.com/events.php?zid=155628)|[47th #1](https://zwiftpower.com/events.php?zid=152778)|[48th #4](https://zwiftpower.com/events.php?zid=162691)|[71st #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|89|3532|[Matt Parsons](https://zwiftpower\.com/profile\.php?z=845430)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[38th #5](https://zwiftpower.com/events.php?zid=166630)|[29th (📣53rd) #2](https://zwiftpower.com/events.php?zid=155628)|[54th #6](https://zwiftpower.com/events.php?zid=171935)|[91st #4](https://zwiftpower.com/events.php?zid=162691)|—|—
A|90|3464|[Evan Bennewies 🔰](https://zwiftpower\.com/profile\.php?z=360816)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[27th (📣51st) #2](https://zwiftpower.com/events.php?zid=155628)|[61st #1](https://zwiftpower.com/events.php?zid=152778)|[76th #4](https://zwiftpower.com/events.php?zid=162691)|[82nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—
A|91|3444|[Simon Harrison](https://zwiftpower\.com/profile\.php?z=524175)|—|[47th #7](https://zwiftpower.com/events.php?zid=173451)|[56th #6](https://zwiftpower.com/events.php?zid=171935)|[74th #8](https://zwiftpower.com/events.php?zid=176148)|[103rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|92|3430|[Tim Sullivan 🔰](https://zwiftpower\.com/profile\.php?z=165253)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[43rd #4](https://zwiftpower.com/events.php?zid=162691)|[60th #6](https://zwiftpower.com/events.php?zid=171935)|[72nd #5](https://zwiftpower.com/events.php?zid=166630)|[88th (📣112th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—
A|93|3426|[Klaudiusz Debski \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=58823)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[30th #5](https://zwiftpower.com/events.php?zid=166630)|[43rd #6](https://zwiftpower.com/events.php?zid=171935)|[105th #4](https://zwiftpower.com/events.php?zid=162691)|[111th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
A|94|3398|[Espen Komissar Fossheim](https://zwiftpower\.com/profile\.php?z=54738)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[56th #5](https://zwiftpower.com/events.php?zid=166630)|[68th #6](https://zwiftpower.com/events.php?zid=171935)|[62nd (📣86th) #2](https://zwiftpower.com/events.php?zid=155628)|[93rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—
A|95|3388|[Alex Sankovic](https://zwiftpower\.com/profile\.php?z=506515)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[69th #8](https://zwiftpower.com/events.php?zid=176148)|[74th #4](https://zwiftpower.com/events.php?zid=162691)|[80th #6](https://zwiftpower.com/events.php?zid=171935)|[85th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|96|3384|[Peter Göransson](https://zwiftpower\.com/profile\.php?z=674292)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[61st #6](https://zwiftpower.com/events.php?zid=171935)|[66th #3](https://zwiftpower.com/events.php?zid=159334)|[66th (📣90th) #2](https://zwiftpower.com/events.php?zid=155628)|[93rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—
A|97|3260|[Sam Mills 🍌](https://zwiftpower\.com/profile\.php?z=171558)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[66th #8](https://zwiftpower.com/events.php?zid=176148)|[78th #7](https://zwiftpower.com/events.php?zid=173451)|[69th (📣93rd) #2](https://zwiftpower.com/events.php?zid=155628)|[135th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
A|98|3234|[Johannes Reining \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=629172)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[85th #8](https://zwiftpower.com/events.php?zid=176148)|[86th #6](https://zwiftpower.com/events.php?zid=171935)|[88th #7](https://zwiftpower.com/events.php?zid=173451)|[126th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|99|3208|[Morten Jul Pedersen 🐌](https://zwiftpower\.com/profile\.php?z=162474)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|[92nd #3](https://zwiftpower.com/events.php?zid=159334)|[96th #4](https://zwiftpower.com/events.php?zid=162691)|[75th (📣99th) #2](https://zwiftpower.com/events.php?zid=155628)|[111th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|100|3194|[Thomas Jendrusch 🔰 \[R2\]](https://zwiftpower\.com/profile\.php?z=28466)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[87th #6](https://zwiftpower.com/events.php?zid=171935)|[101st #4](https://zwiftpower.com/events.php?zid=162691)|[102nd #5](https://zwiftpower.com/events.php?zid=166630)|[115th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
A|101|3134|[Artūras K1212](https://zwiftpower\.com/profile\.php?z=900471)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[75th #6](https://zwiftpower.com/events.php?zid=171935)|[116th #3](https://zwiftpower.com/events.php?zid=159334)|[119th #1](https://zwiftpower.com/events.php?zid=152778)|[101st (📣125th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—
A|102|3108|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[98th #6](https://zwiftpower.com/events.php?zid=171935)|[102nd #7](https://zwiftpower.com/events.php?zid=173451)|[117th #8](https://zwiftpower.com/events.php?zid=176148)|[131st #4](https://zwiftpower.com/events.php?zid=162691)|—|—
A|103|3102|[Richard Moorhouse](https://zwiftpower\.com/profile\.php?z=8903)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[96th #6](https://zwiftpower.com/events.php?zid=171935)|[96th #7](https://zwiftpower.com/events.php?zid=173451)|[105th (📣129th) #2](https://zwiftpower.com/events.php?zid=155628)|[130th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
A|103|3102|[Pedro Errecart](https://zwiftpower\.com/profile\.php?z=745344)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[69th #4](https://zwiftpower.com/events.php?zid=162691)|[93rd #7](https://zwiftpower.com/events.php?zid=173451)|[138th #1](https://zwiftpower.com/events.php?zid=152778)|[151st #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|104|3080|[Michał Dobrzyński\[Team Poland\]](https://zwiftpower\.com/profile\.php?z=1021580)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[97th #6](https://zwiftpower.com/events.php?zid=171935)|[101st #7](https://zwiftpower.com/events.php?zid=173451)|[129th #4](https://zwiftpower.com/events.php?zid=162691)|[135th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|105|3016|[Sven Gerhardt INC 👽](https://zwiftpower\.com/profile\.php?z=982806)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[89th #8](https://zwiftpower.com/events.php?zid=176148)|[106th #7](https://zwiftpower.com/events.php?zid=173451)|[146th #1](https://zwiftpower.com/events.php?zid=152778)|[129th (📣153rd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—
A|106|2961|[Sam Lindsay](https://zwiftpower\.com/profile\.php?z=545920)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[5th #3](https://zwiftpower.com/events.php?zid=159334)|[5th #7](https://zwiftpower.com/events.php?zid=173451)|[11th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
A|107|2946|[Luke GJ Potter](https://zwiftpower\.com/profile\.php?z=55379)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|[111th #7](https://zwiftpower.com/events.php?zid=173451)|[112th #6](https://zwiftpower.com/events.php?zid=171935)|[147th #4](https://zwiftpower.com/events.php?zid=162691)|[159th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
A|108|2935|[Gavin Hughes](https://zwiftpower\.com/profile\.php?z=54447)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[8th #7](https://zwiftpower.com/events.php?zid=173451)|[10th #6](https://zwiftpower.com/events.php?zid=171935)|[16th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|109|2903|[Alan Macleod Ⓥ](https://zwiftpower\.com/profile\.php?z=429012)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[12th #3](https://zwiftpower.com/events.php?zid=159334)|[14th #5](https://zwiftpower.com/events.php?zid=166630)|[24th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|110|2894|[norman henderson Cryo Gen](https://zwiftpower\.com/profile\.php?z=99549)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[124th #4](https://zwiftpower.com/events.php?zid=162691)|[132nd #5](https://zwiftpower.com/events.php?zid=166630)|[140th #3](https://zwiftpower.com/events.php?zid=159334)|[135th (📣159th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—
A|111|2833|[Julian White](https://zwiftpower\.com/profile\.php?z=563359)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[17th #7](https://zwiftpower.com/events.php?zid=173451)|[22nd #3](https://zwiftpower.com/events.php?zid=159334)|[22nd (📣46th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|112|2815|[Tom Townsend](https://zwiftpower\.com/profile\.php?z=66453)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[28th #1](https://zwiftpower.com/events.php?zid=152778)|[31st #3](https://zwiftpower.com/events.php?zid=159334)|[11th (📣35th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|113|2813|[Mattias Lundqvist🔰 Aktivitus Testcenter](https://zwiftpower\.com/profile\.php?z=455884)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[16th #7](https://zwiftpower.com/events.php?zid=173451)|[25th #8](https://zwiftpower.com/events.php?zid=176148)|[30th (📣54th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|114|2801|[Johan Noraker Nossen 🔰](https://zwiftpower\.com/profile\.php?z=135469)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[28th #3](https://zwiftpower.com/events.php?zid=159334)|[29th #4](https://zwiftpower.com/events.php?zid=162691)|[44th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|115|2777|[Richard Unwin](https://zwiftpower\.com/profile\.php?z=49421)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[27th #5](https://zwiftpower.com/events.php?zid=166630)|[39th #8](https://zwiftpower.com/events.php?zid=176148)|[47th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|116|2765|[Łukasz Góralik](https://zwiftpower\.com/profile\.php?z=409658)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[31st #1](https://zwiftpower.com/events.php?zid=152778)|[12th (📣36th) #2](https://zwiftpower.com/events.php?zid=155628)|[52nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|116|2765|[jesse klassen5240](https://zwiftpower\.com/profile\.php?z=1001790)|—|[26th #7](https://zwiftpower.com/events.php?zid=173451)|[42nd #4](https://zwiftpower.com/events.php?zid=162691)|[51st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|117|2763|[Patrick Woelke \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=883585)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[8th #8](https://zwiftpower.com/events.php?zid=176148)|[13th #4](https://zwiftpower.com/events.php?zid=162691)|[99th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|118|2761|[Aleksej Calmann 🔰](https://zwiftpower\.com/profile\.php?z=297914)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[9th (📣33rd) #2](https://zwiftpower.com/events.php?zid=155628)|[38th #4](https://zwiftpower.com/events.php?zid=162691)|[50th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|119|2725|[Sam Greenhalgh](https://zwiftpower\.com/profile\.php?z=143679)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[34th #5](https://zwiftpower.com/events.php?zid=166630)|[43rd #3](https://zwiftpower.com/events.php?zid=159334)|[62nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|120|2717|[Phil Whiteley](https://zwiftpower\.com/profile\.php?z=161329)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[22nd #4](https://zwiftpower.com/events.php?zid=162691)|[56th #1](https://zwiftpower.com/events.php?zid=152778)|[65th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
A|121|2715|[Jesper Attermann 🔰](https://zwiftpower\.com/profile\.php?z=67632)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[14th (📣38th) #2](https://zwiftpower.com/events.php?zid=155628)|[44th #1](https://zwiftpower.com/events.php?zid=152778)|[62nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|122|2701|[Sandy Taylor](https://zwiftpower\.com/profile\.php?z=212882)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|[31st #4](https://zwiftpower.com/events.php?zid=162691)|[49th #1](https://zwiftpower.com/events.php?zid=152778)|[47th (📣71st) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|123|2651|[Steve Lampier\- Saint Piran](https://zwiftpower\.com/profile\.php?z=332529)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[44th #3](https://zwiftpower.com/events.php?zid=159334)|[44th #8](https://zwiftpower.com/events.php?zid=176148)|[88th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
A|124|2643|[Darren Reddick](https://zwiftpower\.com/profile\.php?z=843142)|—|[51st #5](https://zwiftpower.com/events.php?zid=166630)|[59th #7](https://zwiftpower.com/events.php?zid=173451)|[70th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
A|125|2597|[Guido De Vries](https://zwiftpower\.com/profile\.php?z=180278)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[24th #8](https://zwiftpower.com/events.php?zid=176148)|[84th #3](https://zwiftpower.com/events.php?zid=159334)|[95th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|125|2597|[Maciej Majewski](https://zwiftpower\.com/profile\.php?z=458986)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[65th #3](https://zwiftpower.com/events.php?zid=159334)|[66th #6](https://zwiftpower.com/events.php?zid=171935)|[72nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|126|2591|[Kristian Frøland INC](https://zwiftpower\.com/profile\.php?z=323707)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[55th #3](https://zwiftpower.com/events.php?zid=159334)|[48th (📣72nd) #2](https://zwiftpower.com/events.php?zid=155628)|[79th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—
A|127|2587|[Janek Czapliński](https://zwiftpower\.com/profile\.php?z=213569)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[61st #5](https://zwiftpower.com/events.php?zid=166630)|[66th #4](https://zwiftpower.com/events.php?zid=162691)|[81st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|128|2577|[Koen Van Geyt](https://zwiftpower\.com/profile\.php?z=115080)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[24th (📣48th) #2](https://zwiftpower.com/events.php?zid=155628)|[51st #1](https://zwiftpower.com/events.php?zid=152778)|[114th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
A|129|2573|[Jeroen Wellner](https://zwiftpower\.com/profile\.php?z=44582)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[57th #6](https://zwiftpower.com/events.php?zid=171935)|[65th #5](https://zwiftpower.com/events.php?zid=166630)|[93rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|130|2571|[Magnus Brinch Larsen 🔰](https://zwiftpower\.com/profile\.php?z=20421)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|[35th #5](https://zwiftpower.com/events.php?zid=166630)|[68th #1](https://zwiftpower.com/events.php?zid=152778)|[89th (📣113th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|131|2549|[Steven Pink \(FNet Velo\)](https://zwiftpower\.com/profile\.php?z=905145)|[FNet Velo](https://zwiftpower\.com/team\.php?id=3840)|[63rd #6](https://zwiftpower.com/events.php?zid=171935)|[76th #8](https://zwiftpower.com/events.php?zid=176148)|[88th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|132|2543|[Birger Rye](https://zwiftpower\.com/profile\.php?z=446438)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[59th #4](https://zwiftpower.com/events.php?zid=162691)|[66th #7](https://zwiftpower.com/events.php?zid=173451)|[105th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|133|2525|[Anders Jensen](https://zwiftpower\.com/profile\.php?z=863566)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[21st #1](https://zwiftpower.com/events.php?zid=152778)|[54th #4](https://zwiftpower.com/events.php?zid=162691)|[140th (📣164th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|134|2519|[Mathieu Decock](https://zwiftpower\.com/profile\.php?z=509565)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[46th #1](https://zwiftpower.com/events.php?zid=152778)|[48th #8](https://zwiftpower.com/events.php?zid=176148)|[124th (📣148th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|135|2513|[Jesper Olsen](https://zwiftpower\.com/profile\.php?z=41135)|—|[57th #7](https://zwiftpower.com/events.php?zid=173451)|[90th #5](https://zwiftpower.com/events.php?zid=166630)|[98th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
A|136|2507|[kris Spin King](https://zwiftpower\.com/profile\.php?z=292133)|—|[63rd #7](https://zwiftpower.com/events.php?zid=173451)|[72nd #8](https://zwiftpower.com/events.php?zid=176148)|[113th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|137|2491|[Jason Hegarty](https://zwiftpower\.com/profile\.php?z=45648)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[60th #8](https://zwiftpower.com/events.php?zid=176148)|[97th #5](https://zwiftpower.com/events.php?zid=166630)|[99th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|138|2485|[Morten Holt](https://zwiftpower\.com/profile\.php?z=508874)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[81st #4](https://zwiftpower.com/events.php?zid=162691)|[84th #1](https://zwiftpower.com/events.php?zid=152778)|[70th (📣94th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|139|2479|[Jarryd Else](https://zwiftpower\.com/profile\.php?z=169015)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[67th #1](https://zwiftpower.com/events.php?zid=152778)|[50th (📣74th) #2](https://zwiftpower.com/events.php?zid=155628)|[121st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|140|2461|[Luke Dix](https://zwiftpower\.com/profile\.php?z=214249)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[56th (📣80th) #2](https://zwiftpower.com/events.php?zid=155628)|[89th #3](https://zwiftpower.com/events.php?zid=159334)|[102nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|141|2459|[Bas Seldenrijk 👌](https://zwiftpower\.com/profile\.php?z=274131)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[68th #3](https://zwiftpower.com/events.php?zid=159334)|[76th (📣100th) #2](https://zwiftpower.com/events.php?zid=155628)|[104th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|142|2415|[Paweł Kalita](https://zwiftpower\.com/profile\.php?z=64694)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[82nd #6](https://zwiftpower.com/events.php?zid=171935)|[103rd #4](https://zwiftpower.com/events.php?zid=162691)|[109th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|143|2399|[Quentijn Mulleneers 🍟](https://zwiftpower\.com/profile\.php?z=161017)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[51st (📣75th) #2](https://zwiftpower.com/events.php?zid=155628)|[102nd #3](https://zwiftpower.com/events.php?zid=159334)|[125th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|144|2397|[Bernard Opedal](https://zwiftpower\.com/profile\.php?z=586093)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[49th (📣73rd) #2](https://zwiftpower.com/events.php?zid=155628)|[110th #1](https://zwiftpower.com/events.php?zid=152778)|[120th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|145|2371|[gavin ash 🔰](https://zwiftpower\.com/profile\.php?z=157370)|[WECC](https://zwiftpower\.com/team\.php?id=1992)|[94th #6](https://zwiftpower.com/events.php?zid=171935)|[110th #4](https://zwiftpower.com/events.php?zid=162691)|[112th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|146|2361|[Dustin Byers](https://zwiftpower\.com/profile\.php?z=399357)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[85th #6](https://zwiftpower.com/events.php?zid=171935)|[106th #8](https://zwiftpower.com/events.php?zid=176148)|[130th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|147|2357|[Russell Jolliffe](https://zwiftpower\.com/profile\.php?z=291602)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[74th #7](https://zwiftpower.com/events.php?zid=173451)|[99th #8](https://zwiftpower.com/events.php?zid=176148)|[150th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|148|2333|[J\. Williams 🐿](https://zwiftpower\.com/profile\.php?z=189343)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[88th #8](https://zwiftpower.com/events.php?zid=176148)|[111th #3](https://zwiftpower.com/events.php?zid=159334)|[136th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
A|149|2329|[Ruben Larsen 🔰](https://zwiftpower\.com/profile\.php?z=32290)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[86th #3](https://zwiftpower.com/events.php?zid=159334)|[114th #4](https://zwiftpower.com/events.php?zid=162691)|[113th (📣137th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|150|2315|[Anton Alex](https://zwiftpower\.com/profile\.php?z=601528)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[85th (📣109th) #2](https://zwiftpower.com/events.php?zid=155628)|[114th #1](https://zwiftpower.com/events.php?zid=152778)|[121st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
A|151|2297|[Jonas Salamonsen](https://zwiftpower\.com/profile\.php?z=441899)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[90th #8](https://zwiftpower.com/events.php?zid=176148)|[122nd #1](https://zwiftpower.com/events.php?zid=152778)|[141st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|152|2279|[Linas S\.](https://zwiftpower\.com/profile\.php?z=224318)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[56th #3](https://zwiftpower.com/events.php?zid=159334)|[150th #4](https://zwiftpower.com/events.php?zid=162691)|[132nd (📣156th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|153|2259|[Luis GO Ⓥ เจ ☁️](https://zwiftpower\.com/profile\.php?z=270869)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[107th #7](https://zwiftpower.com/events.php?zid=173451)|[131st #5](https://zwiftpower.com/events.php?zid=166630)|[134th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
A|154|2243|[Paul Hayward](https://zwiftpower\.com/profile\.php?z=6917)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[115th #4](https://zwiftpower.com/events.php?zid=162691)|[122nd #3](https://zwiftpower.com/events.php?zid=159334)|[143rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|155|2239|[Jan Jöckel \(KC\)](https://zwiftpower\.com/profile\.php?z=444506)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[109th #4](https://zwiftpower.com/events.php?zid=162691)|[136th #3](https://zwiftpower.com/events.php?zid=159334)|[137th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|156|2233|[Nuno Mendez](https://zwiftpower\.com/profile\.php?z=899397)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[104th #5](https://zwiftpower.com/events.php?zid=166630)|[137th #4](https://zwiftpower.com/events.php?zid=162691)|[120th (📣144th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|157|2171|[Grahame Mckenzie\[BRT\]](https://zwiftpower\.com/profile\.php?z=77990)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[134th #3](https://zwiftpower.com/events.php?zid=159334)|[114th (📣138th) #2](https://zwiftpower.com/events.php?zid=155628)|[144th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|158|2155|[Maokoto Smz](https://zwiftpower\.com/profile\.php?z=303116)|—|[130th #3](https://zwiftpower.com/events.php?zid=159334)|[117th (📣141st) #2](https://zwiftpower.com/events.php?zid=155628)|[153rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
A|159|2129|[makoto takahashi4774](https://zwiftpower\.com/profile\.php?z=979149)|—|[137th #3](https://zwiftpower.com/events.php?zid=159334)|[142nd #4](https://zwiftpower.com/events.php?zid=162691)|[134th (📣158th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
A|160|1976|[Niki H 🔰](https://zwiftpower\.com/profile\.php?z=174313)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[4th #1](https://zwiftpower.com/events.php?zid=152778)|[9th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
A|161|1960|[Max Bike INC](https://zwiftpower\.com/profile\.php?z=505474)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[9th #7](https://zwiftpower.com/events.php?zid=173451)|[12th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|162|1946|[Adam Pierzga](https://zwiftpower\.com/profile\.php?z=44142)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[7th #6](https://zwiftpower.com/events.php?zid=171935)|[21st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
A|163|1936|[Jo Karstensen 🔰](https://zwiftpower\.com/profile\.php?z=499890)|—|[15th #7](https://zwiftpower.com/events.php?zid=173451)|[18th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
A|164|1924|[Vidar Mehl 🐂](https://zwiftpower\.com/profile\.php?z=91105)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th #1](https://zwiftpower.com/events.php?zid=152778)|[🥉 (📣27th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|165|1916|[Damir Ejup](https://zwiftpower\.com/profile\.php?z=299036)|—|[14th #4](https://zwiftpower.com/events.php?zid=162691)|[29th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
A|166|1896|[Jake Simons 🔰](https://zwiftpower\.com/profile\.php?z=305016)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|[11th #3](https://zwiftpower.com/events.php?zid=159334)|[42nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|167|1892|[Conor White](https://zwiftpower\.com/profile\.php?z=310524)|—|[17th #8](https://zwiftpower.com/events.php?zid=176148)|[38th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|168|1890|[Igor Kopse](https://zwiftpower\.com/profile\.php?z=79239)|—|[20th #5](https://zwiftpower.com/events.php?zid=166630)|[36th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|169|1880|[Martin Schei Rye XP](https://zwiftpower\.com/profile\.php?z=295831)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[26th #4](https://zwiftpower.com/events.php?zid=162691)|[35th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
A|170|1878|[Yasuyuki Kobayashi](https://zwiftpower\.com/profile\.php?z=917068)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[31st #5](https://zwiftpower.com/events.php?zid=166630)|[31st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|171|1850|[Justin Brusche](https://zwiftpower\.com/profile\.php?z=1024428)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[21st #7](https://zwiftpower.com/events.php?zid=173451)|[55th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
A|172|1842|[Jan Vanaudenaerde](https://zwiftpower\.com/profile\.php?z=152302)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[31st #6](https://zwiftpower.com/events.php?zid=171935)|[49th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|173|1828|[Gavin Richardson](https://zwiftpower\.com/profile\.php?z=184210)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[40th #7](https://zwiftpower.com/events.php?zid=173451)|[47th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
A|174|1826|[Jokim Pluijmers](https://zwiftpower\.com/profile\.php?z=15354)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[43rd #1](https://zwiftpower.com/events.php?zid=152778)|[21st (📣45th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|175|1824|[Fordy💋](https://zwiftpower\.com/profile\.php?z=104271)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[34th #8](https://zwiftpower.com/events.php?zid=176148)|[55th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|176|1804|[Paul Cox](https://zwiftpower\.com/profile\.php?z=78847)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[29th #1](https://zwiftpower.com/events.php?zid=152778)|[46th (📣70th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|177|1798|[Sam Wadsley](https://zwiftpower\.com/profile\.php?z=237172)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[47th #3](https://zwiftpower.com/events.php?zid=159334)|[31st (📣55th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|178|1796|[Matthieu Daudé](https://zwiftpower\.com/profile\.php?z=919908)|[TC15](https://zwiftpower\.com/team\.php?id=3041)|[49th #6](https://zwiftpower.com/events.php?zid=171935)|[54th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|179|1778|[Frømling 🔰](https://zwiftpower\.com/profile\.php?z=105270)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[46th #4](https://zwiftpower.com/events.php?zid=162691)|[66th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|180|1770|[Nathan Killam 👨🏻 4 Hire](https://zwiftpower\.com/profile\.php?z=16044)|—|[37th #5](https://zwiftpower.com/events.php?zid=166630)|[79th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|180|1770|[Nicolmighty NSK](https://zwiftpower\.com/profile\.php?z=967384)|—|[56th #7](https://zwiftpower.com/events.php?zid=173451)|[60th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|181|1768|[Mark Hansom](https://zwiftpower\.com/profile\.php?z=14136)|[AIMS](https://zwiftpower\.com/team\.php?id=4072)|[40th #6](https://zwiftpower.com/events.php?zid=171935)|[53rd (📣77th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|182|1762|[Gavin Berry](https://zwiftpower\.com/profile\.php?z=153073)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[32nd #8](https://zwiftpower.com/events.php?zid=176148)|[88th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|183|1756|[James Vickers 🦜🔰](https://zwiftpower\.com/profile\.php?z=738971)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[36th #3](https://zwiftpower.com/events.php?zid=159334)|[87th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|184|1752|[Third Wheel 🚳](https://zwiftpower\.com/profile\.php?z=48068)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[55th #5](https://zwiftpower.com/events.php?zid=166630)|[70th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|185|1750|[John Blumenthal](https://zwiftpower\.com/profile\.php?z=10853)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[50th #5](https://zwiftpower.com/events.php?zid=166630)|[76th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|186|1738|[Jörgen Johansson](https://zwiftpower\.com/profile\.php?z=388931)|—|[31st #8](https://zwiftpower.com/events.php?zid=176148)|[101st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|187|1728|[Andrew Hale](https://zwiftpower\.com/profile\.php?z=571545)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[64th #6](https://zwiftpower.com/events.php?zid=171935)|[73rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|188|1720|[Michael Larsen](https://zwiftpower\.com/profile\.php?z=104458)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[61st #7](https://zwiftpower.com/events.php?zid=173451)|[80th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|189|1718|[Simon Oxenham](https://zwiftpower\.com/profile\.php?z=1011)|[VC10](https://zwiftpower\.com/team\.php?id=2888)|[61st #3](https://zwiftpower.com/events.php?zid=159334)|[57th (📣81st) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|189|1718|[Artur Kub \[Grupetto W\-wa\]](https://zwiftpower\.com/profile\.php?z=498366)|[GrupettoWarszawa](https://zwiftpower\.com/team\.php?id=3022)|[71st #6](https://zwiftpower.com/events.php?zid=171935)|[71st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|190|1714|[Mick Kemper 🚀](https://zwiftpower\.com/profile\.php?z=20932)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[42nd #1](https://zwiftpower.com/events.php?zid=152778)|[102nd #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
A|191|1696|[Staffan Hopstadius](https://zwiftpower\.com/profile\.php?z=182715)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[68th #7](https://zwiftpower.com/events.php?zid=173451)|[85th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
A|192|1690|[Tom Fitzpatrick](https://zwiftpower\.com/profile\.php?z=60739)|—|[70th #8](https://zwiftpower.com/events.php?zid=176148)|[86th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|192|1690|[Joakim Lisson](https://zwiftpower\.com/profile\.php?z=113401)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[41st (📣65th) #2](https://zwiftpower.com/events.php?zid=155628)|[91st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|193|1688|[David Kim Amado 🥐 🍰 🥖](https://zwiftpower\.com/profile\.php?z=690464)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[60th #3](https://zwiftpower.com/events.php?zid=159334)|[97th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|194|1676|[Gerrie Scholtes 🔰](https://zwiftpower\.com/profile\.php?z=22315)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[67th #5](https://zwiftpower.com/events.php?zid=166630)|[96th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
A|194|1676|[Thomas Maaløe 🔰](https://zwiftpower\.com/profile\.php?z=265502)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[57th #3](https://zwiftpower.com/events.php?zid=159334)|[106th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|195|1668|[Ross Hetherington 🐌](https://zwiftpower\.com/profile\.php?z=129517)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[79th #3](https://zwiftpower.com/events.php?zid=159334)|[64th (📣88th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|196|1666|[Ernie Austin](https://zwiftpower\.com/profile\.php?z=432019)|—|[73rd #6](https://zwiftpower.com/events.php?zid=171935)|[95th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
A|197|1664|[Yukio Bergholdt 🔰](https://zwiftpower\.com/profile\.php?z=49249)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[80th #4](https://zwiftpower.com/events.php?zid=162691)|[89th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|198|1662|[Goswinus Dijkstra 🔰](https://zwiftpower\.com/profile\.php?z=103446)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[68th #8](https://zwiftpower.com/events.php?zid=176148)|[102nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|199|1654|[Brennen Dwyer](https://zwiftpower\.com/profile\.php?z=13867)|—|[86th #8](https://zwiftpower.com/events.php?zid=176148)|[88th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|199|1654|[Ethan Browne](https://zwiftpower\.com/profile\.php?z=294744)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[85th #1](https://zwiftpower.com/events.php?zid=152778)|[89th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|200|1632|[Ben Oliver\-lewis7856](https://zwiftpower\.com/profile\.php?z=404182)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[77th #8](https://zwiftpower.com/events.php?zid=176148)|[108th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
A|201|1614|[Kaj\-André Døving \[X\]](https://zwiftpower\.com/profile\.php?z=69210)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[25th #1](https://zwiftpower.com/events.php?zid=152778)|[145th (📣169th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|202|1610|[James Donaldson](https://zwiftpower\.com/profile\.php?z=5081)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[91st #3](https://zwiftpower.com/events.php?zid=159334)|[105th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|202|1610|[Remco Beckers](https://zwiftpower\.com/profile\.php?z=506582)|—|[69th #5](https://zwiftpower.com/events.php?zid=166630)|[127th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|203|1600|[Andy Barrie](https://zwiftpower\.com/profile\.php?z=248557)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[97th #7](https://zwiftpower.com/events.php?zid=173451)|[104th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
A|204|1598|[Martin Sundelin](https://zwiftpower\.com/profile\.php?z=325750)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[97th #1](https://zwiftpower.com/events.php?zid=152778)|[81st (📣105th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|205|1584|[John Bergenske⚡️BRT A2](https://zwiftpower\.com/profile\.php?z=180408)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[101st #8](https://zwiftpower.com/events.php?zid=176148)|[108th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|206|1576|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[103rd #6](https://zwiftpower.com/events.php?zid=171935)|[110th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
A|207|1572|[Michał Woźniak](https://zwiftpower\.com/profile\.php?z=64147)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[92nd #7](https://zwiftpower.com/events.php?zid=173451)|[123rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|208|1560|[The Spocker \[X\] 🔰 🖖](https://zwiftpower\.com/profile\.php?z=125855)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[58th (📣82nd) #2](https://zwiftpower.com/events.php?zid=155628)|[139th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|209|1556|[Sonny Peterson](https://zwiftpower\.com/profile\.php?z=917866)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[83rd #7](https://zwiftpower.com/events.php?zid=173451)|[140th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|210|1552|[Adam Egner](https://zwiftpower\.com/profile\.php?z=247562)|[NRT](https://zwiftpower\.com/team\.php?id=2389)|[107th #5](https://zwiftpower.com/events.php?zid=166630)|[94th (📣118th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|211|1550|[Phil Donnelly \[BRT\]](https://zwiftpower\.com/profile\.php?z=514370)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[84th #8](https://zwiftpower.com/events.php?zid=176148)|[142nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|212|1544|[Jan Fiebekorn \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=462183)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[82nd (📣106th) #2](https://zwiftpower.com/events.php?zid=155628)|[123rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|213|1534|[Paul\-Philippe Nelles 🔰](https://zwiftpower\.com/profile\.php?z=401617)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[99th #7](https://zwiftpower.com/events.php?zid=173451)|[111th (📣135th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|214|1528|[Christoph Scheiber](https://zwiftpower\.com/profile\.php?z=457587)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[83rd #1](https://zwiftpower.com/events.php?zid=152778)|[130th (📣154th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|215|1500|[Sam Duffy](https://zwiftpower\.com/profile\.php?z=771005)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[124th #5](https://zwiftpower.com/events.php?zid=166630)|[103rd (📣127th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|216|1498|[a\_TRI\_ called\_quest](https://zwiftpower\.com/profile\.php?z=265783)|—|[109th #7](https://zwiftpower.com/events.php?zid=173451)|[119th (📣143rd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|217|1494|[James Kiff](https://zwiftpower\.com/profile\.php?z=230612)|—|[111th #6](https://zwiftpower.com/events.php?zid=171935)|[143rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|218|1490|[simon lloyd](https://zwiftpower\.com/profile\.php?z=367040)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[101st #3](https://zwiftpower.com/events.php?zid=159334)|[131st (📣155th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|219|1488|[MAS PPT](https://zwiftpower\.com/profile\.php?z=148298)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[111th #8](https://zwiftpower.com/events.php?zid=176148)|[146th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|220|1486|[michael Openshaw](https://zwiftpower\.com/profile\.php?z=1080848)|—|[116th #5](https://zwiftpower.com/events.php?zid=166630)|[118th (📣142nd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|221|1484|[Bart Keizer](https://zwiftpower\.com/profile\.php?z=41959)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[109th #3](https://zwiftpower.com/events.php?zid=159334)|[150th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|222|1456|[Marc Pierschel \[VEGAN\]](https://zwiftpower\.com/profile\.php?z=918949)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[131st #3](https://zwiftpower.com/events.php?zid=159334)|[142nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
A|223|1452|[Dicky @ The Gateway Cycles](https://zwiftpower\.com/profile\.php?z=89796)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[132nd #1](https://zwiftpower.com/events.php?zid=152778)|[143rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|224|1448|[Bart Van den Berghe](https://zwiftpower\.com/profile\.php?z=699029)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[138th #4](https://zwiftpower.com/events.php?zid=162691)|[139th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|225|1440|[Danijel Cancarevic](https://zwiftpower\.com/profile\.php?z=935351)|—|[129th #3](https://zwiftpower.com/events.php?zid=159334)|[152nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
A|226|1438|[A\.Guilhoto<<GARBIKE>>](https://zwiftpower\.com/profile\.php?z=51911)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[136th #5](https://zwiftpower.com/events.php?zid=166630)|[146th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
A|227|1432|[Christoph Kroenert \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=70131)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[138th #5](https://zwiftpower.com/events.php?zid=166630)|[123rd (📣147th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
A|228|997|[Dan Bigham](https://zwiftpower\.com/profile\.php?z=548979)|—|[🥈 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|229|995|[Sean Vintin 🔰](https://zwiftpower\.com/profile\.php?z=137577)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥉 #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|230|991|[M\.A\.](https://zwiftpower\.com/profile\.php?z=219167)|—|[5th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|231|989|[SCOTTIE WEISS](https://zwiftpower\.com/profile\.php?z=745)|—|[6th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|231|989|[Pierre Almeida](https://zwiftpower\.com/profile\.php?z=1028049)|—|[6th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|232|983|[Aaron Coles🔰](https://zwiftpower\.com/profile\.php?z=428001)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[9th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|233|981|[Ollie Peckover 3R 🔰](https://zwiftpower\.com/profile\.php?z=382994)|—|[10th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|234|975|[Luke Barfoot](https://zwiftpower\.com/profile\.php?z=30816)|—|[13th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|235|971|[Nico Wüstenfeld 🔰 \[R2\]](https://zwiftpower\.com/profile\.php?z=1130)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[15th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|236|965|[Ollie Jones 🔰🐝](https://zwiftpower\.com/profile\.php?z=137795)|—|[18th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|236|965|[Garry Beanland 7939](https://zwiftpower\.com/profile\.php?z=489072)|—|[18th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|237|963|[Valentin Cheval](https://zwiftpower\.com/profile\.php?z=134586)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[19th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|238|959|[Neil Richards ⚽](https://zwiftpower\.com/profile\.php?z=385769)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[21st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|238|959|[Kristian Oftedal](https://zwiftpower\.com/profile\.php?z=407425)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[21st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|239|953|[Ryan Bevis 🐉](https://zwiftpower\.com/profile\.php?z=291522)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[24th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|239|953|[Sean Pantellere 👖 🔰](https://zwiftpower\.com/profile\.php?z=486826)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[24th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|240|951|[Adam Connellan](https://zwiftpower\.com/profile\.php?z=122246)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[25th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|241|945|[Edwin Driessen](https://zwiftpower\.com/profile\.php?z=219489)|—|[28th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|242|941|[Ricardo Veras](https://zwiftpower\.com/profile\.php?z=790603)|—|[30th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|243|937|[Kasper Norden INC](https://zwiftpower\.com/profile\.php?z=48656)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[32nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|243|937|[Nick Glavac](https://zwiftpower\.com/profile\.php?z=602717)|—|[32nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|244|935|[Patrick BLAIR \[AFC\] 🔰](https://zwiftpower\.com/profile\.php?z=8149)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[33rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|245|933|[Pär Bäcker🔰](https://zwiftpower\.com/profile\.php?z=504853)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|[10th (📣34th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|246|931|[Andy Sheret ØVB](https://zwiftpower\.com/profile\.php?z=174184)|[ØVB](https://zwiftpower\.com/team\.php?id=1576)|[35th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|247|927|[Michael Podzaline](https://zwiftpower\.com/profile\.php?z=15522)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[37th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|248|925|[Matt Sinclair](https://zwiftpower\.com/profile\.php?z=115138)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[38th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|249|923|[Rob Britt](https://zwiftpower\.com/profile\.php?z=307476)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[39th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|249|923|[Bruce Zoom7214](https://zwiftpower\.com/profile\.php?z=1060589)|—|[39th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|250|921|[Jim McQuaide 🔰](https://zwiftpower\.com/profile\.php?z=7528)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[16th (📣40th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|251|919|[Jens Thoegersen](https://zwiftpower\.com/profile\.php?z=295550)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[41st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|251|919|[Teppo Laurio](https://zwiftpower\.com/profile\.php?z=486471)|[FZR](https://zwiftpower\.com/team\.php?id=2834)|[41st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|252|917|[Michael Madsen 🔰](https://zwiftpower\.com/profile\.php?z=104082)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[18th (📣42nd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|252|917|[Tadeusz Wlodarczyk \[Team PL\]](https://zwiftpower\.com/profile\.php?z=698392)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[42nd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|253|915|[Mariusz Cukierski \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=2984)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[19th (📣43rd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|254|913|[Quentin Goossens](https://zwiftpower\.com/profile\.php?z=246034)|—|[44th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|255|909|[Keith Gater WRT](https://zwiftpower\.com/profile\.php?z=288649)|—|[46th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|256|907|[Baptiste Boivin](https://zwiftpower\.com/profile\.php?z=87980)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|[47th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|256|907|[Benjamin Werner \(KC\)](https://zwiftpower\.com/profile\.php?z=178127)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[47th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|257|905|[Jeroen Blondeel](https://zwiftpower\.com/profile\.php?z=381944)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[48th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|258|903|[Arvils Vikstrems Ⓥ](https://zwiftpower\.com/profile\.php?z=48835)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[49th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|258|903|[Chris Gregory](https://zwiftpower\.com/profile\.php?z=149534)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[49th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|259|901|[Jeffrey van Norden](https://zwiftpower\.com/profile\.php?z=620461)|[JDM](https://zwiftpower\.com/team\.php?id=2317)|[50th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|260|899|[Bertus Turksma](https://zwiftpower\.com/profile\.php?z=197582)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[51st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|261|897|[Tristan Guillemette](https://zwiftpower\.com/profile\.php?z=578461)|—|[52nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|262|893|[Rune Jogert](https://zwiftpower\.com/profile\.php?z=271935)|—|[54th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|262|893|[Michael Damm](https://zwiftpower\.com/profile\.php?z=402565)|—|[54th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|263|891|[Shawn Gray \[X\]](https://zwiftpower\.com/profile\.php?z=48456)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[55th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|264|889|[Per\-Johnny Doving \[X\]](https://zwiftpower\.com/profile\.php?z=46603)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[32nd (📣56th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|264|889|[Espen Jogert 7498](https://zwiftpower\.com/profile\.php?z=541455)|—|[56th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|265|885|[H Jonkers](https://zwiftpower\.com/profile\.php?z=1021960)|—|[58th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|266|881|[Brett Wakefield](https://zwiftpower\.com/profile\.php?z=4310)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[60th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|266|881|[Ludovic Pollet](https://zwiftpower\.com/profile\.php?z=187160)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[36th (📣60th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|267|877|[Eoin McGovern](https://zwiftpower\.com/profile\.php?z=109461)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[62nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|268|875|[Patrick McCrann \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=328064)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[63rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|269|873|[Simon Nielsen 🔰](https://zwiftpower\.com/profile\.php?z=182972)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[64th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|270|869|[Cam Napper](https://zwiftpower\.com/profile\.php?z=11806)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|[42nd (📣66th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|271|867|[Gunter Gellaerts](https://zwiftpower\.com/profile\.php?z=4109)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[67th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|271|867|[Henrik Öijer www\.henrikoijer\.com](https://zwiftpower\.com/profile\.php?z=484770)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[67th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|272|863|[Jens Frede Rasmussen](https://zwiftpower\.com/profile\.php?z=42327)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[45th (📣69th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|272|863|[Tom Schyvens](https://zwiftpower\.com/profile\.php?z=426416)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[69th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|272|863|[Tom Lowe](https://zwiftpower\.com/profile\.php?z=832843)|—|[69th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|273|861|[Fredrik Steen](https://zwiftpower\.com/profile\.php?z=180411)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[70th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|274|857|[Hans Forhaug](https://zwiftpower\.com/profile\.php?z=982758)|—|[72nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|275|851|[Niels Grote Beverborg](https://zwiftpower\.com/profile\.php?z=44114)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[75th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|275|851|[Lyndon Thompson \(Colten Care/TFC\) 🏡](https://zwiftpower\.com/profile\.php?z=297988)|[Colten Care](https://zwiftpower\.com/team\.php?id=4150)|[75th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|276|849|[Richard Watson](https://zwiftpower\.com/profile\.php?z=265568)|[MOONGLU](https://zwiftpower\.com/team\.php?id=2863)|[76th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|277|847|[Jim O'Daniel](https://zwiftpower\.com/profile\.php?z=45462)|—|[77th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|277|847|"[Matt Bruss \| Brazen Dropouts](https://zwiftpower\.com/profile\.php?z=420808)"|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[77th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|278|845|[Chris Bartley](https://zwiftpower\.com/profile\.php?z=396062)|[Aerobyeq\-KRT](https://zwiftpower\.com/team\.php?id=3795)|[78th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|278|845|[Jorre Rentmeesters](https://zwiftpower\.com/profile\.php?z=596495)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[78th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|279|843|[Andrew Taylor](https://zwiftpower\.com/profile\.php?z=185391)|—|[79th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|279|843|[Roar Lindalen, Holand CK](https://zwiftpower\.com/profile\.php?z=212924)|[Holand SK](https://zwiftpower\.com/team\.php?id=293)|[55th (📣79th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|280|841|[Stuart Mcgibbon](https://zwiftpower\.com/profile\.php?z=21336)|—|[80th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|280|841|[Etienne Mille](https://zwiftpower\.com/profile\.php?z=492621)|—|[80th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|281|839|[Seth Stingley](https://zwiftpower\.com/profile\.php?z=942020)|[MarSting Story](https://zwiftpower\.com/team\.php?id=3253)|[81st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|282|837|[Jorge Sousa](https://zwiftpower\.com/profile\.php?z=159892)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[82nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|282|837|[Pieter Verhoeven](https://zwiftpower\.com/profile\.php?z=311684)|—|[82nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|283|835|[il pinguino](https://zwiftpower\.com/profile\.php?z=53921)|—|[83rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|283|835|[Ole\-Christian Ostreng](https://zwiftpower\.com/profile\.php?z=185955)|[Holand SK](https://zwiftpower\.com/team\.php?id=293)|[83rd #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|284|833|[Josh Taylor](https://zwiftpower\.com/profile\.php?z=720120)|—|[60th (📣84th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|285|831|[Stuart Crawford 🚑 \[KRT\]](https://zwiftpower\.com/profile\.php?z=58435)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[61st (📣85th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|286|823|[Bram Maassen3474](https://zwiftpower\.com/profile\.php?z=405176)|—|[89th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|287|821|[Chris Smith ZHR A](https://zwiftpower\.com/profile\.php?z=70787)|—|[90th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|287|821|[Gaute Vikøren](https://zwiftpower\.com/profile\.php?z=115865)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[90th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|288|817|[Tom Shaw](https://zwiftpower\.com/profile\.php?z=230819)|—|[92nd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|288|817|[N\.Zaki](https://zwiftpower\.com/profile\.php?z=349154)|—|[92nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|289|815|[Oliver W Bates](https://zwiftpower\.com/profile\.php?z=256868)|—|[93rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|290|813|[Miles Billings](https://zwiftpower\.com/profile\.php?z=8901)|[545Velo](https://zwiftpower\.com/team\.php?id=341)|[94th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|290|813|[Don Fucs](https://zwiftpower\.com/profile\.php?z=47334)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[94th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|290|813|[Ben Skinner](https://zwiftpower\.com/profile\.php?z=852057)|—|[94th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|291|811|[Morten Austestad](https://zwiftpower\.com/profile\.php?z=59075)|[Holmenkollen CK](https://zwiftpower\.com/team\.php?id=3469)|[95th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|291|811|[Gijs Lummen](https://zwiftpower\.com/profile\.php?z=164835)|—|[95th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|291|811|[Ben Jones](https://zwiftpower\.com/profile\.php?z=537347)|[Cycle Specific](https://zwiftpower\.com/team\.php?id=4135)|[95th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|292|809|[Piotr Kramarzyk \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=480688)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[96th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|293|807|[James Ward](https://zwiftpower\.com/profile\.php?z=57004)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[73rd (📣97th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|293|807|[Steve Fleetwood 🐀🔰](https://zwiftpower\.com/profile\.php?z=75421)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[97th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|294|805|[Tobias Müller](https://zwiftpower\.com/profile\.php?z=119944)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[98th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|294|805|[Odd Erlend Hansen Berg](https://zwiftpower\.com/profile\.php?z=183471)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[74th (📣98th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|295|801|[Bob McGlue🐉](https://zwiftpower\.com/profile\.php?z=129911)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[100th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|296|799|[A Friend \(DT\)](https://zwiftpower\.com/profile\.php?z=6925)|—|[101st #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|296|799|[Darren Jones](https://zwiftpower\.com/profile\.php?z=76507)|—|[77th (📣101st) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|297|797|[Joseph Bank 🔰](https://zwiftpower\.com/profile\.php?z=11837)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[78th (📣102nd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|297|797|[Christian Moxon \[BRT\]](https://zwiftpower\.com/profile\.php?z=35037)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[102nd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|298|795|[Richard Smith](https://zwiftpower\.com/profile\.php?z=111707)|—|[79th (📣103rd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|298|795|[Alf Erik Hagebakken](https://zwiftpower\.com/profile\.php?z=167476)|[GTD](https://zwiftpower\.com/team\.php?id=304)|[103rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|298|795|[Troels Vest 🔰](https://zwiftpower\.com/profile\.php?z=402078)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[103rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|298|795|[Pawel Kochanowski](https://zwiftpower\.com/profile\.php?z=659309)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[103rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
A|298|795|[Scott Main \[BRT\]](https://zwiftpower\.com/profile\.php?z=1002910)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[103rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|299|793|[Matthew Hanford 2792](https://zwiftpower\.com/profile\.php?z=223820)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[104th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|300|791|[Aaron Barry⚡️ \[BRT\]](https://zwiftpower\.com/profile\.php?z=743592)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[105th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|301|789|[Louis FL\- Team QC](https://zwiftpower\.com/profile\.php?z=5710)|[1R](https://zwiftpower\.com/team\.php?id=387)|[106th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|302|787|[Simon Spinczyk](https://zwiftpower\.com/profile\.php?z=991216)|[Tri Geckos](https://zwiftpower\.com/team\.php?id=4073)|[107th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|303|785|[James Farren](https://zwiftpower\.com/profile\.php?z=63968)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[108th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|304|783|[Rich Wallwork](https://zwiftpower\.com/profile\.php?z=35468)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[109th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|304|783|[Lars K Granlund](https://zwiftpower\.com/profile\.php?z=162758)|[TTK](https://zwiftpower\.com/team\.php?id=2304)|[109th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|304|783|[Kevin Colton4910](https://zwiftpower\.com/profile\.php?z=809355)|—|[109th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
A|305|777|[Mark Brock](https://zwiftpower\.com/profile\.php?z=148876)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[112th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|306|775|[David Howell7106](https://zwiftpower\.com/profile\.php?z=231994)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[113th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|307|773|[Martin van Laak](https://zwiftpower\.com/profile\.php?z=185716)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[90th (📣114th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|307|773|[Jeremy Brunello](https://zwiftpower\.com/profile\.php?z=524642)|—|[114th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|307|773|[Alex Harris](https://zwiftpower\.com/profile\.php?z=561748)|—|[114th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|308|771|[James Corlett](https://zwiftpower\.com/profile\.php?z=190807)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[115th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|309|769|[Jason Kenning](https://zwiftpower\.com/profile\.php?z=71129)|—|[116th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|309|769|[Jesper Nielsen](https://zwiftpower\.com/profile\.php?z=256495)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[92nd (📣116th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|309|769|[Welsh Wizardd](https://zwiftpower\.com/profile\.php?z=642562)|—|[116th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
A|309|769|[Franek GREGARIO](https://zwiftpower\.com/profile\.php?z=834315)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[116th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|310|767|[Poul Mærsk \- WKG](https://zwiftpower\.com/profile\.php?z=1015309)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[117th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|311|763|[Will Duffy](https://zwiftpower\.com/profile\.php?z=820305)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[119th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|312|761|[Andre Schmidt](https://zwiftpower\.com/profile\.php?z=126837)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[120th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|312|761|[G Gauthier \# CLAN KNOX](https://zwiftpower\.com/profile\.php?z=214653)|[CLAN KNOX](https://zwiftpower\.com/team\.php?id=3073)|[120th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|312|761|[Henrik Johannesson](https://zwiftpower\.com/profile\.php?z=490481)|—|[120th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|312|761|[Super\.Byke Viana](https://zwiftpower\.com/profile\.php?z=520670)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[96th (📣120th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|313|757|[David Billings](https://zwiftpower\.com/profile\.php?z=181148)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[122nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|313|757|[Jacek Rządca](https://zwiftpower\.com/profile\.php?z=472694)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[122nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|314|755|[Håvard Bjerke](https://zwiftpower\.com/profile\.php?z=668604)|—|[123rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|315|753|[alm0st\.fast \(KC\)](https://zwiftpower\.com/profile\.php?z=458147)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[124th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|316|751|[Benjamin Klammt](https://zwiftpower\.com/profile\.php?z=37125)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[125th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|316|751|[Andrew Kirby](https://zwiftpower\.com/profile\.php?z=933861)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[125th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|317|749|[Lloyd Wise](https://zwiftpower\.com/profile\.php?z=385139)|[Bikestrong\-KTM](https://zwiftpower\.com/team\.php?id=3878)|[126th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|317|749|[Jeroen Kniep](https://zwiftpower\.com/profile\.php?z=506438)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[126th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|317|749|[Joakim Parviainen🔰](https://zwiftpower\.com/profile\.php?z=828573)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|[102nd (📣126th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|318|747|[Adam Bacon](https://zwiftpower\.com/profile\.php?z=841389)|—|[127th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|319|745|[Mark Dabell](https://zwiftpower\.com/profile\.php?z=297114)|—|[104th (📣128th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|320|743|[Per\-Christian Barholm](https://zwiftpower\.com/profile\.php?z=64554)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[129th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|320|743|[Anders Moustgaard 🔰](https://zwiftpower\.com/profile\.php?z=771995)|—|[129th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|321|741|[Tomasz Woźniak](https://zwiftpower\.com/profile\.php?z=66572)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[130th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|321|741|[Andreas Wolf](https://zwiftpower\.com/profile\.php?z=855340)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[106th (📣130th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|322|739|[Alex McMillan KISS B](https://zwiftpower\.com/profile\.php?z=525583)|—|[131st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|322|739|[Oliver Attinger CP](https://zwiftpower\.com/profile\.php?z=659834)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[107th (📣131st) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|323|737|[Bret Wilks](https://zwiftpower\.com/profile\.php?z=328081)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[108th (📣132nd) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|324|735|[Tim Mason](https://zwiftpower\.com/profile\.php?z=132847)|[EBR](https://zwiftpower\.com/team\.php?id=3741)|[133rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|324|735|[Ronny Kambach \(KC\)](https://zwiftpower\.com/profile\.php?z=985936)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[133rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|325|733|[Sam Vijay 6303](https://zwiftpower\.com/profile\.php?z=707058)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[134th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|326|723|[Jason Siegle](https://zwiftpower\.com/profile\.php?z=33716)|—|[115th (📣139th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|326|723|[Niels Heldens](https://zwiftpower\.com/profile\.php?z=883266)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[139th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|326|723|[Poisson Lune](https://zwiftpower\.com/profile\.php?z=1011634)|—|[139th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
A|327|721|[Romain Malbreil](https://zwiftpower\.com/profile\.php?z=259570)|—|[116th (📣140th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|327|721|"[Alexander Chatzakis \[BRT\]⚡️\| 🏊🏽‍🚴🏽‍🏃🏽70\.3\| 🇳🇱🇬🇷](https://zwiftpower\.com/profile\.php?z=496345)"|[BRT](https://zwiftpower\.com/team\.php?id=4)|[140th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|328|719|[Craig Hodgson \(Muckle CC\)](https://zwiftpower\.com/profile\.php?z=1125198)|—|[141st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|329|711|[J CCL](https://zwiftpower\.com/profile\.php?z=32116)|—|[145th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|329|711|[Karl Tiselius](https://zwiftpower\.com/profile\.php?z=181548)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[145th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|329|711|[Daniel Colman](https://zwiftpower\.com/profile\.php?z=915207)|[P2R](https://zwiftpower\.com/team\.php?id=4025)|[121st (📣145th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|330|709|[Mark Warters](https://zwiftpower\.com/profile\.php?z=7164)|—|[122nd (📣146th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|331|705|[Kilian Hemart Ⓥ](https://zwiftpower\.com/profile\.php?z=488019)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[148th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|331|705|[Jonas Larsen](https://zwiftpower\.com/profile\.php?z=592962)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|[148th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|332|703|[Tom Wassink](https://zwiftpower\.com/profile\.php?z=3351)|—|[149th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
A|332|703|[Scott Mitchell](https://zwiftpower\.com/profile\.php?z=144703)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[149th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|332|703|[David Akesson](https://zwiftpower\.com/profile\.php?z=396670)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[125th (📣149th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|332|703|[Christiaan Stange](https://zwiftpower\.com/profile\.php?z=452001)|—|[149th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|333|697|[Tyler Pearce Ⓥ](https://zwiftpower\.com/profile\.php?z=290307)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[152nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
A|334|691|[Matteo Casadei](https://zwiftpower\.com/profile\.php?z=199374)|[VelociT](https://zwiftpower\.com/team\.php?id=1071)|[155th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|335|689|[Matthew Goss](https://zwiftpower\.com/profile\.php?z=874407)|—|[156th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
A|336|687|[Nathan Hey](https://zwiftpower\.com/profile\.php?z=20082)|—|[133rd (📣157th) #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
A|337|685|[Paul vdG CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=329191)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[158th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|1|5986|[Mark Worthington](https://zwiftpower\.com/profile\.php?z=71062)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥇 #3](https://zwiftpower.com/events.php?zid=159334)|[🥇 #4](https://zwiftpower.com/events.php?zid=162691)|[🥇 #8](https://zwiftpower.com/events.php?zid=176148)|[🥈 #2](https://zwiftpower.com/events.php?zid=155628)|[🥈 #5](https://zwiftpower.com/events.php?zid=166630)|[🥉 #7](https://zwiftpower.com/events.php?zid=173451)
B|2|5910|[Dave Brodmann](https://zwiftpower\.com/profile\.php?z=15543)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥉 #2](https://zwiftpower.com/events.php?zid=155628)|[6th #8](https://zwiftpower.com/events.php?zid=176148)|[7th #3](https://zwiftpower.com/events.php?zid=159334)|[7th #5](https://zwiftpower.com/events.php?zid=166630)|[9th #7](https://zwiftpower.com/events.php?zid=173451)|[16th #6](https://zwiftpower.com/events.php?zid=171935)
B|3|5856|[Andrew Robinson TFC Racing](https://zwiftpower\.com/profile\.php?z=100901)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th #8](https://zwiftpower.com/events.php?zid=176148)|[9th #3](https://zwiftpower.com/events.php?zid=159334)|[10th #7](https://zwiftpower.com/events.php?zid=173451)|[11th #5](https://zwiftpower.com/events.php?zid=166630)|[19th #4](https://zwiftpower.com/events.php?zid=162691)|[21st #6](https://zwiftpower.com/events.php?zid=171935)
B|4|5834|[Jonathan Davey](https://zwiftpower\.com/profile\.php?z=1474)|—|[4th #3](https://zwiftpower.com/events.php?zid=159334)|[12th #6](https://zwiftpower.com/events.php?zid=171935)|[17th #1](https://zwiftpower.com/events.php?zid=152778)|[17th #7](https://zwiftpower.com/events.php?zid=173451)|[17th #8](https://zwiftpower.com/events.php?zid=176148)|[19th #2](https://zwiftpower.com/events.php?zid=155628)
B|5|5762|[Sonic The Hedgehog](https://zwiftpower\.com/profile\.php?z=49294)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[13th #5](https://zwiftpower.com/events.php?zid=166630)|[14th #7](https://zwiftpower.com/events.php?zid=173451)|[18th #8](https://zwiftpower.com/events.php?zid=176148)|[22nd #6](https://zwiftpower.com/events.php?zid=171935)|[27th #2](https://zwiftpower.com/events.php?zid=155628)|[28th #1](https://zwiftpower.com/events.php?zid=152778)
B|6|5744|[Peter Campbell](https://zwiftpower\.com/profile\.php?z=40108)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[12th #2](https://zwiftpower.com/events.php?zid=155628)|[15th #7](https://zwiftpower.com/events.php?zid=173451)|[19th #8](https://zwiftpower.com/events.php?zid=176148)|[25th #6](https://zwiftpower.com/events.php?zid=171935)|[27th #3](https://zwiftpower.com/events.php?zid=159334)|[33rd #1](https://zwiftpower.com/events.php?zid=152778)
B|7|5732|[James Ayton](https://zwiftpower\.com/profile\.php?z=164395)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[12th #4](https://zwiftpower.com/events.php?zid=162691)|[22nd #8](https://zwiftpower.com/events.php?zid=176148)|[23rd #3](https://zwiftpower.com/events.php?zid=159334)|[25th #5](https://zwiftpower.com/events.php?zid=166630)|[27th #6](https://zwiftpower.com/events.php?zid=171935)|[28th #7](https://zwiftpower.com/events.php?zid=173451)
B|8|5726|[lee saunders \(BRT\)](https://zwiftpower\.com/profile\.php?z=177623)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[13th #8](https://zwiftpower.com/events.php?zid=176148)|[16th #7](https://zwiftpower.com/events.php?zid=173451)|[23rd #5](https://zwiftpower.com/events.php?zid=166630)|[26th #3](https://zwiftpower.com/events.php?zid=159334)|[29th #4](https://zwiftpower.com/events.php?zid=162691)|[33rd #2](https://zwiftpower.com/events.php?zid=155628)
B|9|5714|[Arthur Araújo](https://zwiftpower\.com/profile\.php?z=330442)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[5th #5](https://zwiftpower.com/events.php?zid=166630)|[19th #7](https://zwiftpower.com/events.php?zid=173451)|[24th #6](https://zwiftpower.com/events.php?zid=171935)|[29th #8](https://zwiftpower.com/events.php?zid=176148)|[33rd #4](https://zwiftpower.com/events.php?zid=162691)|[36th #2](https://zwiftpower.com/events.php?zid=155628)
B|10|5710|[Nick Nellis 🔰](https://zwiftpower\.com/profile\.php?z=460177)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th #7](https://zwiftpower.com/events.php?zid=173451)|[10th #5](https://zwiftpower.com/events.php?zid=166630)|[11th #8](https://zwiftpower.com/events.php?zid=176148)|[30th #3](https://zwiftpower.com/events.php?zid=159334)|[39th #4](https://zwiftpower.com/events.php?zid=162691)|[53rd #2](https://zwiftpower.com/events.php?zid=155628)
B|11|5684|[Ole Bergtun \(BCK\)](https://zwiftpower\.com/profile\.php?z=67220)|—|[16th #2](https://zwiftpower.com/events.php?zid=155628)|[20th #3](https://zwiftpower.com/events.php?zid=159334)|[24th #5](https://zwiftpower.com/events.php?zid=166630)|[31st #7](https://zwiftpower.com/events.php?zid=173451)|[32nd #6](https://zwiftpower.com/events.php?zid=171935)|[38th #1](https://zwiftpower.com/events.php?zid=152778)
B|12|5644|[Stephan Maertens Ⓥ 🐇🥕](https://zwiftpower\.com/profile\.php?z=37710)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[17th #5](https://zwiftpower.com/events.php?zid=166630)|[18th #2](https://zwiftpower.com/events.php?zid=155628)|[28th #8](https://zwiftpower.com/events.php?zid=176148)|[36th #3](https://zwiftpower.com/events.php?zid=159334)|[41st #1](https://zwiftpower.com/events.php?zid=152778)|[41st #4](https://zwiftpower.com/events.php?zid=162691)
B|12|5644|[Jon Izagirre PTz](https://zwiftpower\.com/profile\.php?z=170606)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[21st #7](https://zwiftpower.com/events.php?zid=173451)|[25th #8](https://zwiftpower.com/events.php?zid=176148)|[27th #4](https://zwiftpower.com/events.php?zid=162691)|[28th #3](https://zwiftpower.com/events.php?zid=159334)|[33rd #6](https://zwiftpower.com/events.php?zid=171935)|[47th #5](https://zwiftpower.com/events.php?zid=166630)
B|13|5602|[Oliver Steffen](https://zwiftpower\.com/profile\.php?z=196547)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[23rd #2](https://zwiftpower.com/events.php?zid=155628)|[29th #5](https://zwiftpower.com/events.php?zid=166630)|[35th #7](https://zwiftpower.com/events.php?zid=173451)|[36th #8](https://zwiftpower.com/events.php?zid=176148)|[37th #3](https://zwiftpower.com/events.php?zid=159334)|[42nd #4](https://zwiftpower.com/events.php?zid=162691)
B|14|5558|[Ricardo Gama 💣](https://zwiftpower\.com/profile\.php?z=108059)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[26th #6](https://zwiftpower.com/events.php?zid=171935)|[27th #7](https://zwiftpower.com/events.php?zid=173451)|[39th #5](https://zwiftpower.com/events.php?zid=166630)|[40th #2](https://zwiftpower.com/events.php?zid=155628)|[45th #8](https://zwiftpower.com/events.php?zid=176148)|[47th #4](https://zwiftpower.com/events.php?zid=162691)
B|15|5498|[Daniel Revo \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=427562)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[31st #3](https://zwiftpower.com/events.php?zid=159334)|[34th #7](https://zwiftpower.com/events.php?zid=173451)|[36th #6](https://zwiftpower.com/events.php?zid=171935)|[45th #1](https://zwiftpower.com/events.php?zid=152778)|[51st #2](https://zwiftpower.com/events.php?zid=155628)|[57th #4](https://zwiftpower.com/events.php?zid=162691)
B|16|5402|[Andrew Keedle](https://zwiftpower\.com/profile\.php?z=117018)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[25th #3](https://zwiftpower.com/events.php?zid=159334)|[38th #7](https://zwiftpower.com/events.php?zid=173451)|[41st #8](https://zwiftpower.com/events.php?zid=176148)|[43rd #6](https://zwiftpower.com/events.php?zid=171935)|[52nd #2](https://zwiftpower.com/events.php?zid=155628)|[103rd #1](https://zwiftpower.com/events.php?zid=152778)
B|17|5400|[S Ssmith \(V\) Ⓥ ^\.\.^ 🐣🌱🐾](https://zwiftpower\.com/profile\.php?z=175002)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[25th #7](https://zwiftpower.com/events.php?zid=173451)|[32nd #8](https://zwiftpower.com/events.php?zid=176148)|[45th #5](https://zwiftpower.com/events.php?zid=166630)|[66th #1](https://zwiftpower.com/events.php?zid=152778)|[67th #2](https://zwiftpower.com/events.php?zid=155628)|[68th #3](https://zwiftpower.com/events.php?zid=159334)
B|18|5324|[Tom Weiler \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=157775)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[38th #6](https://zwiftpower.com/events.php?zid=171935)|[51st #8](https://zwiftpower.com/events.php?zid=176148)|[52nd #5](https://zwiftpower.com/events.php?zid=166630)|[61st #3](https://zwiftpower.com/events.php?zid=159334)|[64th #2](https://zwiftpower.com/events.php?zid=155628)|[75th #1](https://zwiftpower.com/events.php?zid=152778)
B|19|4975|[Neil Hughes](https://zwiftpower\.com/profile\.php?z=160845)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[🥇 #2](https://zwiftpower.com/events.php?zid=155628)|[🥇 #7](https://zwiftpower.com/events.php?zid=173451)|[🥈 #4](https://zwiftpower.com/events.php?zid=162691)|[🥉 #3](https://zwiftpower.com/events.php?zid=159334)|[8th #1](https://zwiftpower.com/events.php?zid=152778)|—
B|20|4847|[M agnus \(KC\)](https://zwiftpower\.com/profile\.php?z=348200)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[8th #4](https://zwiftpower.com/events.php?zid=162691)|[10th #3](https://zwiftpower.com/events.php?zid=159334)|[14th #2](https://zwiftpower.com/events.php?zid=155628)|[18th #5](https://zwiftpower.com/events.php?zid=166630)|[29th #1](https://zwiftpower.com/events.php?zid=152778)|—
B|21|4787|[Ned Bowen 🏹](https://zwiftpower\.com/profile\.php?z=833859)|[Flèche](https://zwiftpower\.com/team\.php?id=3497)|[11th #3](https://zwiftpower.com/events.php?zid=159334)|[11th #4](https://zwiftpower.com/events.php?zid=162691)|[25th #2](https://zwiftpower.com/events.php?zid=155628)|[27th #1](https://zwiftpower.com/events.php?zid=152778)|[35th #5](https://zwiftpower.com/events.php?zid=166630)|—
B|22|4591|[Rob Bate \[TPH\]](https://zwiftpower\.com/profile\.php?z=73789)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[24th #8](https://zwiftpower.com/events.php?zid=176148)|[29th #7](https://zwiftpower.com/events.php?zid=173451)|[41st #6](https://zwiftpower.com/events.php?zid=171935)|[51st #3](https://zwiftpower.com/events.php?zid=159334)|[62nd #1](https://zwiftpower.com/events.php?zid=152778)|—
B|23|4499|[Quentin Lafaye \[Vision\]](https://zwiftpower\.com/profile\.php?z=96812)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[35th #8](https://zwiftpower.com/events.php?zid=176148)|[45th #4](https://zwiftpower.com/events.php?zid=162691)|[48th #2](https://zwiftpower.com/events.php?zid=155628)|[48th #3](https://zwiftpower.com/events.php?zid=159334)|[77th #1](https://zwiftpower.com/events.php?zid=152778)|—
B|24|4493|[Andreas Kasperek \(KC\)](https://zwiftpower\.com/profile\.php?z=86467)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[46th #8](https://zwiftpower.com/events.php?zid=176148)|[47th #3](https://zwiftpower.com/events.php?zid=159334)|[52nd #1](https://zwiftpower.com/events.php?zid=152778)|[53rd #7](https://zwiftpower.com/events.php?zid=173451)|[58th #6](https://zwiftpower.com/events.php?zid=171935)|—
B|25|4457|[Trond Sannes Marthinsen](https://zwiftpower\.com/profile\.php?z=124057)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[48th #8](https://zwiftpower.com/events.php?zid=176148)|[50th #3](https://zwiftpower.com/events.php?zid=159334)|[50th #6](https://zwiftpower.com/events.php?zid=171935)|[54th #5](https://zwiftpower.com/events.php?zid=166630)|[72nd #4](https://zwiftpower.com/events.php?zid=162691)|—
B|26|3966|[Chris Pritchard](https://zwiftpower\.com/profile\.php?z=30255)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[🥈 #3](https://zwiftpower.com/events.php?zid=159334)|[🥉 #4](https://zwiftpower.com/events.php?zid=162691)|[6th #2](https://zwiftpower.com/events.php?zid=155628)|[8th #6](https://zwiftpower.com/events.php?zid=171935)|—|—
B|27|3902|[Johannes Reining \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=629172)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[5th #3](https://zwiftpower.com/events.php?zid=159334)|[10th #4](https://zwiftpower.com/events.php?zid=162691)|[17th #2](https://zwiftpower.com/events.php?zid=155628)|[19th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
B|28|3878|[Nick Bunyard1980](https://zwiftpower\.com/profile\.php?z=612430)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[6th #7](https://zwiftpower.com/events.php?zid=173451)|[8th #5](https://zwiftpower.com/events.php?zid=166630)|[23rd #6](https://zwiftpower.com/events.php?zid=171935)|[26th #8](https://zwiftpower.com/events.php?zid=176148)|—|—
B|29|3840|[Mateusz Kucharski \[TeamPL\] B1](https://zwiftpower\.com/profile\.php?z=95410)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[5th #4](https://zwiftpower.com/events.php?zid=162691)|[20th #7](https://zwiftpower.com/events.php?zid=173451)|[22nd #5](https://zwiftpower.com/events.php?zid=166630)|[35th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
B|30|3776|[Dave Gould CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=600137)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[24th #2](https://zwiftpower.com/events.php?zid=155628)|[29th #3](https://zwiftpower.com/events.php?zid=159334)|[30th #5](https://zwiftpower.com/events.php?zid=166630)|[31st #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|31|3724|[Michael Haar \(KC\)](https://zwiftpower\.com/profile\.php?z=955094)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[28th #6](https://zwiftpower.com/events.php?zid=171935)|[33rd #7](https://zwiftpower.com/events.php?zid=173451)|[37th #8](https://zwiftpower.com/events.php?zid=176148)|[42nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—
B|32|3720|[Mark Anthony Jones](https://zwiftpower\.com/profile\.php?z=75787)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[15th #1](https://zwiftpower.com/events.php?zid=152778)|[15th #2](https://zwiftpower.com/events.php?zid=155628)|[23rd #7](https://zwiftpower.com/events.php?zid=173451)|[89th #8](https://zwiftpower.com/events.php?zid=176148)|—|—
B|33|3690|[Thomas Fischer 🦁 \(KC\)](https://zwiftpower\.com/profile\.php?z=160790)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[26th #2](https://zwiftpower.com/events.php?zid=155628)|[39th #7](https://zwiftpower.com/events.php?zid=173451)|[44th #4](https://zwiftpower.com/events.php?zid=162691)|[48th #1](https://zwiftpower.com/events.php?zid=152778)|—|—
B|34|3636|[Daniel Hunt OCC](https://zwiftpower\.com/profile\.php?z=182368)|[OCC](https://zwiftpower\.com/team\.php?id=2620)|[32nd #7](https://zwiftpower.com/events.php?zid=173451)|[42nd #6](https://zwiftpower.com/events.php?zid=171935)|[47th #8](https://zwiftpower.com/events.php?zid=176148)|[63rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|35|3612|[Sarel Boshoff Ⓥ](https://zwiftpower\.com/profile\.php?z=1128952)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[17th #6](https://zwiftpower.com/events.php?zid=171935)|[34th #5](https://zwiftpower.com/events.php?zid=166630)|[44th #3](https://zwiftpower.com/events.php?zid=159334)|[101st #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|36|3600|[Stuart Lynne](https://zwiftpower\.com/profile\.php?z=78201)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[34th #2](https://zwiftpower.com/events.php?zid=155628)|[43rd #4](https://zwiftpower.com/events.php?zid=162691)|[45th #3](https://zwiftpower.com/events.php?zid=159334)|[80th #8](https://zwiftpower.com/events.php?zid=176148)|—|—
B|37|3554|[Rene Meijer](https://zwiftpower\.com/profile\.php?z=803509)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[30th #7](https://zwiftpower.com/events.php?zid=173451)|[49th #8](https://zwiftpower.com/events.php?zid=176148)|[63rd #3](https://zwiftpower.com/events.php?zid=159334)|[83rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—
B|38|3542|[marinko barac\(KC\)](https://zwiftpower\.com/profile\.php?z=823018)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[52nd #8](https://zwiftpower.com/events.php?zid=176148)|[56th #6](https://zwiftpower.com/events.php?zid=171935)|[57th #1](https://zwiftpower.com/events.php?zid=152778)|[66th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
B|39|3502|[Glebster Carr \[BRT\]](https://zwiftpower\.com/profile\.php?z=572357)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[57th #3](https://zwiftpower.com/events.php?zid=159334)|[59th #7](https://zwiftpower.com/events.php?zid=173451)|[60th #8](https://zwiftpower.com/events.php?zid=176148)|[75th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|40|3488|[Pedro Correia](https://zwiftpower\.com/profile\.php?z=718875)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[10th #2](https://zwiftpower.com/events.php?zid=155628)|[75th #3](https://zwiftpower.com/events.php?zid=159334)|[80th #4](https://zwiftpower.com/events.php?zid=162691)|[93rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—
B|41|3462|[Ian Bottomore](https://zwiftpower\.com/profile\.php?z=163377)|—|[48th #7](https://zwiftpower.com/events.php?zid=173451)|[55th #6](https://zwiftpower.com/events.php?zid=171935)|[83rd #4](https://zwiftpower.com/events.php?zid=162691)|[85th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
B|42|3452|[Stu Gardiner ⓥ](https://zwiftpower\.com/profile\.php?z=446939)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[56th #7](https://zwiftpower.com/events.php?zid=173451)|[69th #6](https://zwiftpower.com/events.php?zid=171935)|[74th #5](https://zwiftpower.com/events.php?zid=166630)|[77th #8](https://zwiftpower.com/events.php?zid=176148)|—|—
B|43|3390|[Florian Meyer \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=45416)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[71st #8](https://zwiftpower.com/events.php?zid=176148)|[76th #5](https://zwiftpower.com/events.php?zid=166630)|[78th #1](https://zwiftpower.com/events.php?zid=152778)|[82nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—
B|44|3384|[Christian Kumpfe \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=49743)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[67th #1](https://zwiftpower.com/events.php?zid=152778)|[72nd #3](https://zwiftpower.com/events.php?zid=159334)|[74th #6](https://zwiftpower.com/events.php?zid=171935)|[97th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|44|3384|[Ben Harris](https://zwiftpower\.com/profile\.php?z=774028)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[59th #3](https://zwiftpower.com/events.php?zid=159334)|[65th #6](https://zwiftpower.com/events.php?zid=171935)|[88th #8](https://zwiftpower.com/events.php?zid=176148)|[98th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
B|45|3382|[Vítor Santos](https://zwiftpower\.com/profile\.php?z=281363)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[68th #7](https://zwiftpower.com/events.php?zid=173451)|[76th #8](https://zwiftpower.com/events.php?zid=176148)|[82nd #5](https://zwiftpower.com/events.php?zid=166630)|[85th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
B|46|3326|[Sam Bird](https://zwiftpower\.com/profile\.php?z=447439)|—|[74th #7](https://zwiftpower.com/events.php?zid=173451)|[79th #4](https://zwiftpower.com/events.php?zid=162691)|[87th #8](https://zwiftpower.com/events.php?zid=176148)|[99th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
B|47|3274|[Michael Birkenmaier\[BMPRO\]](https://zwiftpower\.com/profile\.php?z=470237)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[79th #6](https://zwiftpower.com/events.php?zid=171935)|[90th #2](https://zwiftpower.com/events.php?zid=155628)|[95th #1](https://zwiftpower.com/events.php?zid=152778)|[101st #5](https://zwiftpower.com/events.php?zid=166630)|—|—
B|48|2969|[Michi Seipold \(KC\)](https://zwiftpower\.com/profile\.php?z=415204)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥉 #5](https://zwiftpower.com/events.php?zid=166630)|[4th #8](https://zwiftpower.com/events.php?zid=176148)|[10th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
B|49|2933|[Dan Wong](https://zwiftpower\.com/profile\.php?z=541321)|—|[9th #8](https://zwiftpower.com/events.php?zid=176148)|[13th #6](https://zwiftpower.com/events.php?zid=171935)|[13th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—
B|50|2913|[MarcAndre Doucet](https://zwiftpower\.com/profile\.php?z=703114)|[Stallion](https://zwiftpower\.com/team\.php?id=4154)|[7th #7](https://zwiftpower.com/events.php?zid=173451)|[14th #6](https://zwiftpower.com/events.php?zid=171935)|[24th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|51|2871|[Mathias Kemnitz](https://zwiftpower\.com/profile\.php?z=116780)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[11th #7](https://zwiftpower.com/events.php?zid=173451)|[18th #6](https://zwiftpower.com/events.php?zid=171935)|[37th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|52|2853|[Toshiya Kondo](https://zwiftpower\.com/profile\.php?z=376815)|—|[13th #3](https://zwiftpower.com/events.php?zid=159334)|[26th #1](https://zwiftpower.com/events.php?zid=152778)|[36th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|53|2847|[Bogdan SITO \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=55688)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[20th #2](https://zwiftpower.com/events.php?zid=155628)|[23rd #1](https://zwiftpower.com/events.php?zid=152778)|[35th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
B|54|2817|[Stuart Hardy⚡️\[BRT\]🔰](https://zwiftpower\.com/profile\.php?z=591535)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[24th #7](https://zwiftpower.com/events.php?zid=173451)|[30th #6](https://zwiftpower.com/events.php?zid=171935)|[39th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|55|2805|[Paul Yoder](https://zwiftpower\.com/profile\.php?z=941353)|—|[15th #6](https://zwiftpower.com/events.php?zid=171935)|[26th #4](https://zwiftpower.com/events.php?zid=162691)|[58th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
B|56|2803|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[17th #3](https://zwiftpower.com/events.php?zid=159334)|[39th #8](https://zwiftpower.com/events.php?zid=176148)|[44th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|57|2759|[Darren Phillips](https://zwiftpower\.com/profile\.php?z=48345)|—|[21st #5](https://zwiftpower.com/events.php?zid=166630)|[43rd #1](https://zwiftpower.com/events.php?zid=152778)|[58th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|58|2741|[Espen Børsting](https://zwiftpower\.com/profile\.php?z=50855)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[23rd #8](https://zwiftpower.com/events.php?zid=176148)|[37th #7](https://zwiftpower.com/events.php?zid=173451)|[71st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|59|2733|[William price \-\-\-\- Chester tri 2087](https://zwiftpower\.com/profile\.php?z=320581)|—|[18th #7](https://zwiftpower.com/events.php?zid=173451)|[39th #3](https://zwiftpower.com/events.php?zid=159334)|[78th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
B|60|2729|[Sven Fessler \(KC\)](https://zwiftpower\.com/profile\.php?z=48283)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[20th #5](https://zwiftpower.com/events.php?zid=166630)|[44th #6](https://zwiftpower.com/events.php?zid=171935)|[73rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|61|2719|[James Fowler](https://zwiftpower\.com/profile\.php?z=237877)|[FinishStrong](https://zwiftpower\.com/team\.php?id=1740)|[43rd #8](https://zwiftpower.com/events.php?zid=176148)|[46th #7](https://zwiftpower.com/events.php?zid=173451)|[53rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|62|2715|[Matt Hilbert](https://zwiftpower\.com/profile\.php?z=946001)|—|[38th #8](https://zwiftpower.com/events.php?zid=176148)|[40th #6](https://zwiftpower.com/events.php?zid=171935)|[66th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|63|2705|[Oskar Thofte](https://zwiftpower\.com/profile\.php?z=334279)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[24th #3](https://zwiftpower.com/events.php?zid=159334)|[55th #4](https://zwiftpower.com/events.php?zid=162691)|[70th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
B|64|2693|[Tom Box](https://zwiftpower\.com/profile\.php?z=84)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[48th #4](https://zwiftpower.com/events.php?zid=162691)|[48th #5](https://zwiftpower.com/events.php?zid=166630)|[59th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
B|65|2687|[Daniel Potter ⚡](https://zwiftpower\.com/profile\.php?z=83111)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[33rd #8](https://zwiftpower.com/events.php?zid=176148)|[54th #6](https://zwiftpower.com/events.php?zid=171935)|[71st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|66|2673|[D Todd CRYO\-GEN\(B2\)](https://zwiftpower\.com/profile\.php?z=595717)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[42nd #8](https://zwiftpower.com/events.php?zid=176148)|[51st #7](https://zwiftpower.com/events.php?zid=173451)|[72nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|67|2659|[Dean Hardie](https://zwiftpower\.com/profile\.php?z=96086)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[54th #1](https://zwiftpower.com/events.php?zid=152778)|[54th #8](https://zwiftpower.com/events.php?zid=176148)|[64th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
B|68|2657|[Matt Hill](https://zwiftpower\.com/profile\.php?z=503800)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[47th #2](https://zwiftpower.com/events.php?zid=155628)|[63rd #1](https://zwiftpower.com/events.php?zid=152778)|[63rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
B|69|2643|[Carl Foster](https://zwiftpower\.com/profile\.php?z=879014)|—|[49th #7](https://zwiftpower.com/events.php?zid=173451)|[51st #6](https://zwiftpower.com/events.php?zid=171935)|[80th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
B|70|2639|[Trevor King](https://zwiftpower\.com/profile\.php?z=908872)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[41st #7](https://zwiftpower.com/events.php?zid=173451)|[63rd #6](https://zwiftpower.com/events.php?zid=171935)|[78th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|71|2615|[ⓥ Tim Jones \(Cowley Road Condors CC\)](https://zwiftpower\.com/profile\.php?z=850532)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[53rd #8](https://zwiftpower.com/events.php?zid=176148)|[57th #6](https://zwiftpower.com/events.php?zid=171935)|[84th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
B|72|2601|[Henrik Andlem Borresen](https://zwiftpower\.com/profile\.php?z=653639)|[HSIL](https://zwiftpower\.com/team\.php?id=3049)|[64th #6](https://zwiftpower.com/events.php?zid=171935)|[67th #7](https://zwiftpower.com/events.php?zid=173451)|[70th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—
B|73|2591|[Basti Löcken \[R2\]](https://zwiftpower\.com/profile\.php?z=591300)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[52nd #6](https://zwiftpower.com/events.php?zid=171935)|[72nd #7](https://zwiftpower.com/events.php?zid=173451)|[82nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|74|2585|[Richard Jackson](https://zwiftpower\.com/profile\.php?z=556224)|—|[58th #7](https://zwiftpower.com/events.php?zid=173451)|[72nd #8](https://zwiftpower.com/events.php?zid=176148)|[79th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
B|75|2571|[Rodney Young](https://zwiftpower\.com/profile\.php?z=113175)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[59th #4](https://zwiftpower.com/events.php?zid=162691)|[73rd #3](https://zwiftpower.com/events.php?zid=159334)|[84th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|76|2545|[Terje Dahl](https://zwiftpower\.com/profile\.php?z=282852)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[67th #3](https://zwiftpower.com/events.php?zid=159334)|[73rd #1](https://zwiftpower.com/events.php?zid=152778)|[89th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
B|77|2461|[Michael Placidi Vegan Italiano/Australiano7558](https://zwiftpower\.com/profile\.php?z=82858)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[84th #4](https://zwiftpower.com/events.php?zid=162691)|[87th #2](https://zwiftpower.com/events.php?zid=155628)|[100th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|78|2443|[Chris Forster](https://zwiftpower\.com/profile\.php?z=858816)|—|[89th #5](https://zwiftpower.com/events.php?zid=166630)|[92nd #2](https://zwiftpower.com/events.php?zid=155628)|[99th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
B|79|1988|[Andy Barrie](https://zwiftpower\.com/profile\.php?z=248557)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥇 #5](https://zwiftpower.com/events.php?zid=166630)|[6th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|80|1982|[Jon Arnar Sigurjonsson TFC](https://zwiftpower\.com/profile\.php?z=230021)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥉 #6](https://zwiftpower.com/events.php?zid=171935)|[7th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|80|1982|[Paul Smith 1145](https://zwiftpower\.com/profile\.php?z=467100)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[4th #2](https://zwiftpower.com/events.php?zid=155628)|[6th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|81|1980|[Michal Jablonka](https://zwiftpower\.com/profile\.php?z=44878)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[🥉 #8](https://zwiftpower.com/events.php?zid=176148)|[8th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|81|1980|[Eddie Kelly](https://zwiftpower\.com/profile\.php?z=821829)|—|[🥈 #6](https://zwiftpower.com/events.php?zid=171935)|[9th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|82|1974|[Jan Thrun](https://zwiftpower\.com/profile\.php?z=249691)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[5th #2](https://zwiftpower.com/events.php?zid=155628)|[9th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|83|1972|[Jacques La Cock](https://zwiftpower\.com/profile\.php?z=69795)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[7th #1](https://zwiftpower.com/events.php?zid=152778)|[8th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|84|1970|[Craig Nicholls](https://zwiftpower\.com/profile\.php?z=242281)|—|[🥇 #6](https://zwiftpower.com/events.php?zid=171935)|[15th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|85|1966|[Ruben Guis](https://zwiftpower\.com/profile\.php?z=52559)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[7th #2](https://zwiftpower.com/events.php?zid=155628)|[11th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|86|1952|[Paul Hayward](https://zwiftpower\.com/profile\.php?z=6917)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[11th #2](https://zwiftpower.com/events.php?zid=155628)|[14th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|87|1944|[TCaplen \- Toby McTobeface](https://zwiftpower\.com/profile\.php?z=14123)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th #5](https://zwiftpower.com/events.php?zid=166630)|[17th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|88|1936|[Przemo Wicio](https://zwiftpower\.com/profile\.php?z=403051)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[14th #4](https://zwiftpower.com/events.php?zid=162691)|[19th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|89|1924|[Jonathan Reeves TFC Racing](https://zwiftpower\.com/profile\.php?z=72122)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[12th #3](https://zwiftpower.com/events.php?zid=159334)|[27th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|90|1922|[Paul Lewis](https://zwiftpower\.com/profile\.php?z=544654)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[19th #3](https://zwiftpower.com/events.php?zid=159334)|[21st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|91|1918|[corey ring Bourne Wheelers CC](https://zwiftpower\.com/profile\.php?z=242439)|—|[6th #5](https://zwiftpower.com/events.php?zid=166630)|[36th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|92|1908|"[Alexander Chatzakis \[BRT\]⚡️\| 🏊🏽‍🚴🏽‍🏃🏽70\.3\| 🇳🇱🇬🇷](https://zwiftpower\.com/profile\.php?z=496345)"|[BRT](https://zwiftpower\.com/team\.php?id=4)|[22nd #3](https://zwiftpower.com/events.php?zid=159334)|[25th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|93|1904|[Daniel Valverde](https://zwiftpower\.com/profile\.php?z=59136)|[Team Bergziegen](https://zwiftpower\.com/team\.php?id=3486)|[21st #3](https://zwiftpower.com/events.php?zid=159334)|[28th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|94|1892|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[24th #1](https://zwiftpower.com/events.php?zid=152778)|[31st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|94|1892|[Jamie Woodward](https://zwiftpower\.com/profile\.php?z=685302)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[15th #5](https://zwiftpower.com/events.php?zid=166630)|[40th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|95|1890|[Dan Fox \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=268249)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[16th #4](https://zwiftpower.com/events.php?zid=162691)|[40th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|95|1890|[Trond G\. Amundsen](https://zwiftpower\.com/profile\.php?z=412981)|[Holmenkollen CK](https://zwiftpower\.com/team\.php?id=3469)|[26th #7](https://zwiftpower.com/events.php?zid=173451)|[30th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|96|1868|[Anders Ueland Skeie](https://zwiftpower\.com/profile\.php?z=44666)|—|[31st #5](https://zwiftpower.com/events.php?zid=166630)|[36th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|97|1866|[Peter van Gellekom](https://zwiftpower\.com/profile\.php?z=152483)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[34th #4](https://zwiftpower.com/events.php?zid=162691)|[34th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
B|98|1862|[Andrei Benea](https://zwiftpower\.com/profile\.php?z=218066)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|[33rd #3](https://zwiftpower.com/events.php?zid=159334)|[37th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|99|1860|[Zuko Inc7](https://zwiftpower\.com/profile\.php?z=285309)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[22nd #4](https://zwiftpower.com/events.php?zid=162691)|[49th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|100|1852|[Brian Hanson \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=675708)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[37th #4](https://zwiftpower.com/events.php?zid=162691)|[38th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|101|1850|[Jorge Sousa](https://zwiftpower\.com/profile\.php?z=159892)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[16th #5](https://zwiftpower.com/events.php?zid=166630)|[60th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|102|1844|[John Morton](https://zwiftpower\.com/profile\.php?z=186724)|—|[38th #2](https://zwiftpower.com/events.php?zid=155628)|[41st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|102|1844|[Bouke Lubach](https://zwiftpower\.com/profile\.php?z=851007)|—|[30th #4](https://zwiftpower.com/events.php?zid=162691)|[49th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|103|1830|[Ta abaluga1204](https://zwiftpower\.com/profile\.php?z=1216297)|—|[42nd #7](https://zwiftpower.com/events.php?zid=173451)|[44th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
B|104|1816|[Pedro Prego](https://zwiftpower\.com/profile\.php?z=867117)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[33rd #5](https://zwiftpower.com/events.php?zid=166630)|[60th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|105|1812|[Pete Tizzard](https://zwiftpower\.com/profile\.php?z=70038)|—|[14th #5](https://zwiftpower.com/events.php?zid=166630)|[81st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|105|1812|[Gerhard Reich \(KC\)](https://zwiftpower\.com/profile\.php?z=501754)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[35th #2](https://zwiftpower.com/events.php?zid=155628)|[60th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|106|1810|[Johan Öberg](https://zwiftpower\.com/profile\.php?z=96206)|[ART](https://zwiftpower\.com/team\.php?id=2271)|[43rd #2](https://zwiftpower.com/events.php?zid=155628)|[53rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|107|1808|[Mark Munro](https://zwiftpower\.com/profile\.php?z=133572)|—|[46th #4](https://zwiftpower.com/events.php?zid=162691)|[51st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|108|1798|[Stephen Balls KISS RT](https://zwiftpower\.com/profile\.php?z=122150)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[26th #5](https://zwiftpower.com/events.php?zid=166630)|[76th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|109|1792|[David Greenwood](https://zwiftpower\.com/profile\.php?z=60661)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[40th #8](https://zwiftpower.com/events.php?zid=176148)|[65th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|110|1788|[Grega Gregoric](https://zwiftpower\.com/profile\.php?z=509176)|[Top\-Fit](https://zwiftpower\.com/team\.php?id=2019)|[31st #6](https://zwiftpower.com/events.php?zid=171935)|[76th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|111|1784|[Roar Mathisen](https://zwiftpower\.com/profile\.php?z=481344)|—|[54th #4](https://zwiftpower.com/events.php?zid=162691)|[55th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|112|1782|[Diederik Westerhof \[X\]](https://zwiftpower\.com/profile\.php?z=56853)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[45th #2](https://zwiftpower.com/events.php?zid=155628)|[65th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|113|1776|[Stefan Seebauer \(KC\)](https://zwiftpower\.com/profile\.php?z=200900)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[45th #7](https://zwiftpower.com/events.php?zid=173451)|[68th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|114|1772|[Håvard Thomassen](https://zwiftpower\.com/profile\.php?z=46032)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[54th #7](https://zwiftpower.com/events.php?zid=173451)|[61st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|115|1762|[Guy Vdsmis TFC ZZRC](https://zwiftpower\.com/profile\.php?z=980133)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[56th #8](https://zwiftpower.com/events.php?zid=176148)|[64th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|116|1752|[Jorge Teixeira](https://zwiftpower\.com/profile\.php?z=296051)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[61st #4](https://zwiftpower.com/events.php?zid=162691)|[64th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|117|1750|[Brandon Zuckerberg](https://zwiftpower\.com/profile\.php?z=445447)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[57th #7](https://zwiftpower.com/events.php?zid=173451)|[69th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
B|118|1748|[Daniel Gassmann](https://zwiftpower\.com/profile\.php?z=57897)|—|[63rd #7](https://zwiftpower.com/events.php?zid=173451)|[64th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
B|119|1742|[Jonathan Graciano](https://zwiftpower\.com/profile\.php?z=57316)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[55th #8](https://zwiftpower.com/events.php?zid=176148)|[75th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|120|1738|[Don Morgan](https://zwiftpower\.com/profile\.php?z=16961)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[62nd #7](https://zwiftpower.com/events.php?zid=173451)|[70th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|120|1738|[Kris De Swert](https://zwiftpower\.com/profile\.php?z=282903)|[\-AG\-](https://zwiftpower\.com/team\.php?id=2897)|[57th #5](https://zwiftpower.com/events.php?zid=166630)|[75th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
B|121|1736|[Ben Gillett](https://zwiftpower\.com/profile\.php?z=315040)|—|[62nd #8](https://zwiftpower.com/events.php?zid=176148)|[71st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
B|121|1736|[Reino Kleband Ⓥ](https://zwiftpower\.com/profile\.php?z=803761)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[60th #6](https://zwiftpower.com/events.php?zid=171935)|[73rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|122|1734|[Paul Morrison](https://zwiftpower\.com/profile\.php?z=597478)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[54th #2](https://zwiftpower.com/events.php?zid=155628)|[80th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|122|1734|[Stuart Weatherley \[TPH\]](https://zwiftpower\.com/profile\.php?z=966125)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[52nd #3](https://zwiftpower.com/events.php?zid=159334)|[82nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|123|1730|[Ger van Mossel](https://zwiftpower\.com/profile\.php?z=347435)|—|[56th #3](https://zwiftpower.com/events.php?zid=159334)|[80th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|124|1722|[GORDON JACKSON](https://zwiftpower\.com/profile\.php?z=19671)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[69th #2](https://zwiftpower.com/events.php?zid=155628)|[71st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|125|1720|[Theun Bouius](https://zwiftpower\.com/profile\.php?z=700655)|—|[66th #7](https://zwiftpower.com/events.php?zid=173451)|[75th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
B|126|1712|[Magnus Andersson](https://zwiftpower\.com/profile\.php?z=891264)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[55th #2](https://zwiftpower.com/events.php?zid=155628)|[90th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|127|1710|[Jens Kroepelien 3399](https://zwiftpower\.com/profile\.php?z=197170)|—|[63rd #5](https://zwiftpower.com/events.php?zid=166630)|[83rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|128|1708|[John van der Griend](https://zwiftpower\.com/profile\.php?z=735211)|—|[70th #2](https://zwiftpower.com/events.php?zid=155628)|[77th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|129|1704|[James Wilkinson](https://zwiftpower\.com/profile\.php?z=448922)|[Ride WLF](https://zwiftpower\.com/team\.php?id=2257)|[61st #6](https://zwiftpower.com/events.php?zid=171935)|[88th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|130|1694|[Chris Sanders](https://zwiftpower\.com/profile\.php?z=893943)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[62nd #6](https://zwiftpower.com/events.php?zid=171935)|[92nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|131|1690|[Kenneth Nymoen 9631](https://zwiftpower\.com/profile\.php?z=611482)|—|[71st #4](https://zwiftpower.com/events.php?zid=162691)|[85th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
B|132|1688|[Ben Willson🦎\[Vision\]🔰](https://zwiftpower\.com/profile\.php?z=995424)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[76th #1](https://zwiftpower.com/events.php?zid=152778)|[81st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|133|1678|[Markus Krupp \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=441576)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[70th #1](https://zwiftpower.com/events.php?zid=152778)|[92nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
B|134|1676|[Alexandre Bernardo\[TugaZ\]](https://zwiftpower\.com/profile\.php?z=443530)|—|[77th #6](https://zwiftpower.com/events.php?zid=171935)|[86th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|135|1668|[Hauke Westenberg](https://zwiftpower\.com/profile\.php?z=449960)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[72nd #6](https://zwiftpower.com/events.php?zid=171935)|[95th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|136|1654|[Patrick Flickert \(KC\)](https://zwiftpower\.com/profile\.php?z=508975)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[86th #1](https://zwiftpower.com/events.php?zid=152778)|[88th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|137|1650|[Richard Skinstad\-Jansen](https://zwiftpower\.com/profile\.php?z=61248)|[Tunge Gir](https://zwiftpower\.com/team\.php?id=3418)|[85th #4](https://zwiftpower.com/events.php?zid=162691)|[91st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
B|138|1636|[Scott Moore \(RCS\)](https://zwiftpower\.com/profile\.php?z=86866)|—|[91st #2](https://zwiftpower.com/events.php?zid=155628)|[92nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
B|139|1632|[keith palmer](https://zwiftpower\.com/profile\.php?z=57284)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[89th #3](https://zwiftpower.com/events.php?zid=159334)|[96th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
B|140|999|[Paul\-Philippe Nelles 🔰](https://zwiftpower\.com/profile\.php?z=401617)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[🥇 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|141|997|[Simon Oxenham](https://zwiftpower\.com/profile\.php?z=1011)|[VC10](https://zwiftpower\.com/team\.php?id=2888)|[🥈 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|141|997|[Alex Gleim](https://zwiftpower\.com/profile\.php?z=4838)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥈 #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|141|997|[Mikel Aristegi](https://zwiftpower\.com/profile\.php?z=274849)|—|[🥈 #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|142|995|[Mark Gardiner](https://zwiftpower\.com/profile\.php?z=691846)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥉 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|143|993|[Tom Hargreaves 🦆](https://zwiftpower\.com/profile\.php?z=26709)|[NUUN SIGMA](https://zwiftpower\.com/team\.php?id=2330)|[4th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|143|993|[Gregg O'Malley](https://zwiftpower\.com/profile\.php?z=162054)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|[4th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|143|993|[Charles\-Antoine Marcotte](https://zwiftpower\.com/profile\.php?z=197250)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|143|993|[Stijn Derycke](https://zwiftpower\.com/profile\.php?z=335088)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[4th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|143|993|[Marcin Dabrowski](https://zwiftpower\.com/profile\.php?z=949300)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[4th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|144|991|[Brian Kelly](https://zwiftpower\.com/profile\.php?z=5147)|—|[5th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|144|991|[Hulk Smash](https://zwiftpower\.com/profile\.php?z=33385)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[5th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|145|989|[Dustin Byers](https://zwiftpower\.com/profile\.php?z=399357)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[6th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|146|987|[Richard Sparkes](https://zwiftpower\.com/profile\.php?z=50676)|—|[7th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|146|987|[Leon Bond](https://zwiftpower\.com/profile\.php?z=266642)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[7th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|147|985|[Rory McAdams](https://zwiftpower\.com/profile\.php?z=102983)|—|[8th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|147|985|[Rich B](https://zwiftpower\.com/profile\.php?z=247393)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[8th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|148|983|[Pawel Kochanowski](https://zwiftpower\.com/profile\.php?z=659309)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[9th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|148|983|[Horst Grünbichler\(KC\),radrooTeam](https://zwiftpower\.com/profile\.php?z=842176)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[9th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|148|983|[Adam Cole5011](https://zwiftpower\.com/profile\.php?z=988367)|—|[9th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|149|981|[Per\-Christian Barholm](https://zwiftpower\.com/profile\.php?z=64554)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[10th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|149|981|[Chris Whalley](https://zwiftpower\.com/profile\.php?z=73302)|—|[10th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|150|979|[C Sampley\-DIRT](https://zwiftpower\.com/profile\.php?z=61402)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[11th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|151|977|[Andy Hadfield](https://zwiftpower\.com/profile\.php?z=309897)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[12th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|151|977|[Jan Jöckel \(KC\)](https://zwiftpower\.com/profile\.php?z=444506)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[12th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|151|977|[Andre Bakken](https://zwiftpower\.com/profile\.php?z=474758)|—|[12th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|152|975|[Louis FL\- Team QC](https://zwiftpower\.com/profile\.php?z=5710)|[1R](https://zwiftpower\.com/team\.php?id=387)|[13th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|152|975|[Alexander Koop](https://zwiftpower\.com/profile\.php?z=168192)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[13th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|152|975|[Stian Gylder\-Haugland\[V\]](https://zwiftpower\.com/profile\.php?z=430101)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[13th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|153|973|[\. Sigurgrimur](https://zwiftpower\.com/profile\.php?z=82570)|[SCK](https://zwiftpower\.com/team\.php?id=3783)|[14th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|153|973|[Anders Tveit](https://zwiftpower\.com/profile\.php?z=596576)|[TCK](https://zwiftpower\.com/team\.php?id=2422)|[14th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|154|971|[Anton Kriel](https://zwiftpower\.com/profile\.php?z=14216)|—|[15th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|154|971|[Larry Parker🔰](https://zwiftpower\.com/profile\.php?z=229626)|[VR2209](https://zwiftpower\.com/team\.php?id=3121)|[15th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|155|969|[Andy Davies](https://zwiftpower\.com/profile\.php?z=6720)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[16th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|155|969|[Mark Tiffany ⚡️\[BRT\]](https://zwiftpower\.com/profile\.php?z=91996)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[16th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|155|969|[Tsuyoshi Shirasaki\(BALBA\-F\)](https://zwiftpower\.com/profile\.php?z=677784)|—|[16th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|156|965|[Tomasz Skalski](https://zwiftpower\.com/profile\.php?z=197372)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[18th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|156|965|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[18th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|156|965|[Oliver Attinger CP](https://zwiftpower\.com/profile\.php?z=659834)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[18th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|157|963|[P K](https://zwiftpower\.com/profile\.php?z=484)|—|[19th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|158|961|[Arne Danielsen \[V\]](https://zwiftpower\.com/profile\.php?z=65053)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[20th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|158|961|[Tom Palfrey](https://zwiftpower\.com/profile\.php?z=876356)|—|[20th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|158|961|[Lewis Wake](https://zwiftpower\.com/profile\.php?z=900521)|[KCRT](https://zwiftpower\.com/team\.php?id=4040)|[20th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|159|959|[Pete Hicks](https://zwiftpower\.com/profile\.php?z=62449)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[21st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|159|959|[Stephen Cann](https://zwiftpower\.com/profile\.php?z=128516)|—|[21st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|160|957|[Ørjan K Sandnes](https://zwiftpower\.com/profile\.php?z=181173)|—|[22nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|160|957|[Loris van de Kassteele \(ITA\)](https://zwiftpower\.com/profile\.php?z=810115)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[22nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|161|955|[B\. S Johansen](https://zwiftpower\.com/profile\.php?z=155898)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[23rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|162|951|[James Hay 👽](https://zwiftpower\.com/profile\.php?z=548008)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[25th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|163|947|[clement palozzi](https://zwiftpower\.com/profile\.php?z=88907)|—|[27th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|164|945|[Adrian Zvara](https://zwiftpower\.com/profile\.php?z=453256)|—|[28th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|164|945|[Eric Larson](https://zwiftpower\.com/profile\.php?z=981211)|—|[28th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|165|943|[Pete Butler](https://zwiftpower\.com/profile\.php?z=11957)|—|[29th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|166|941|[Marcin P](https://zwiftpower\.com/profile\.php?z=50059)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[30th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|166|941|[Alex Solomides](https://zwiftpower\.com/profile\.php?z=56178)|[NGCA](https://zwiftpower\.com/team\.php?id=2023)|[30th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|167|939|[Jan Becher](https://zwiftpower\.com/profile\.php?z=19250)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[31st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|167|939|[David Cardis](https://zwiftpower\.com/profile\.php?z=878863)|[CyclingGames](https://zwiftpower\.com/team\.php?id=4088)|[31st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|168|937|[Matthew Page \- Bikes Etc Magazine](https://zwiftpower\.com/profile\.php?z=472521)|—|[32nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|168|937|[Marcus Geßner](https://zwiftpower\.com/profile\.php?z=948975)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[32nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|169|933|[Paul Skipper \[BRT\]](https://zwiftpower\.com/profile\.php?z=12026)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[34th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|169|933|[Jon Poole](https://zwiftpower\.com/profile\.php?z=513566)|—|[34th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|169|933|[konsuke kondo](https://zwiftpower\.com/profile\.php?z=954147)|—|[34th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|170|931|[Harald Sætermo](https://zwiftpower\.com/profile\.php?z=516214)|—|[35th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|170|931|[Conor Hanna](https://zwiftpower\.com/profile\.php?z=589521)|—|[35th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|171|929|[CELLE Cédric](https://zwiftpower\.com/profile\.php?z=925935)|—|[36th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|172|927|[Mikkel Staunsholm 👾](https://zwiftpower\.com/profile\.php?z=19963)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[37th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|172|927|[David Mulholland](https://zwiftpower\.com/profile\.php?z=571602)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[37th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|173|925|[Heiko Witt](https://zwiftpower\.com/profile\.php?z=325159)|—|[38th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|173|925|[Andy Wylde](https://zwiftpower\.com/profile\.php?z=737856)|—|[38th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|174|923|[Brian Rhoden](https://zwiftpower\.com/profile\.php?z=53663)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[39th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|174|923|[Jim Horstmann Ⓥ](https://zwiftpower\.com/profile\.php?z=182374)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[39th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|175|921|[james barlow](https://zwiftpower\.com/profile\.php?z=168223)|—|[40th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|175|921|[Isaac Hulse](https://zwiftpower\.com/profile\.php?z=729330)|[CYH](https://zwiftpower\.com/team\.php?id=3812)|[40th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|175|921|[Fabian Frinzel \(KC\)](https://zwiftpower\.com/profile\.php?z=1075763)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[40th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|176|919|[Dave Hyne\-Jones](https://zwiftpower\.com/profile\.php?z=78280)|—|[41st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|176|919|[Chris Orgill CRC LFCC](https://zwiftpower\.com/profile\.php?z=181867)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[41st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|177|917|[Tom Wassink](https://zwiftpower\.com/profile\.php?z=3351)|—|[42nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|177|917|[Juanjo Ferres](https://zwiftpower\.com/profile\.php?z=69023)|[ZESP](https://zwiftpower\.com/team\.php?id=2146)|[42nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|177|917|[Paul Bridgman](https://zwiftpower\.com/profile\.php?z=330950)|[CycleChat](https://zwiftpower\.com/team\.php?id=1458)|[42nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|178|915|[Magnus Strahlert](https://zwiftpower\.com/profile\.php?z=227517)|[FSK](https://zwiftpower\.com/team\.php?id=3382)|[43rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|178|915|[Dominik Gielas \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=737147)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[43rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|178|915|[R Ansolo](https://zwiftpower\.com/profile\.php?z=920115)|—|[43rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|179|913|[David Szentesi6476](https://zwiftpower\.com/profile\.php?z=374304)|—|[44th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|179|913|[Daniel Hind9210](https://zwiftpower\.com/profile\.php?z=702156)|—|[44th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|179|913|[Henryk Tokarski](https://zwiftpower\.com/profile\.php?z=834023)|[PLN](https://zwiftpower\.com/team\.php?id=3124)|[44th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|180|909|[Martin Rezac](https://zwiftpower\.com/profile\.php?z=38051)|—|[46th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|180|909|[George Olive Ⓥ](https://zwiftpower\.com/profile\.php?z=80614)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|[46th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|180|909|[Mark Howlett](https://zwiftpower\.com/profile\.php?z=172455)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|[46th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|180|909|[Bobby Riley](https://zwiftpower\.com/profile\.php?z=245435)|—|[46th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|180|909|[Brandon Reinholt](https://zwiftpower\.com/profile\.php?z=963378)|—|[46th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|181|907|[Marc Schulz\-Bacon \(KC\)](https://zwiftpower\.com/profile\.php?z=177541)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[47th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|181|907|[Francis Lafleche](https://zwiftpower\.com/profile\.php?z=469512)|—|[47th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|181|907|[Kevin Vander Ende \[Splunk>\]](https://zwiftpower\.com/profile\.php?z=966423)|[Splunk>](https://zwiftpower\.com/team\.php?id=3933)|[47th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|182|905|[Paul Hornby](https://zwiftpower\.com/profile\.php?z=66046)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[48th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|183|903|[Harry Roberts](https://zwiftpower\.com/profile\.php?z=320)|—|[49th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|183|903|[Alex Mitchell\-Touche](https://zwiftpower\.com/profile\.php?z=46172)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[49th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|183|903|[William Warthon](https://zwiftpower\.com/profile\.php?z=95164)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[49th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|184|901|[Bob Chew](https://zwiftpower\.com/profile\.php?z=4859)|—|[50th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|184|901|[Dirk Jan Los](https://zwiftpower\.com/profile\.php?z=41646)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[50th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|184|901|[Alexis Lavoie](https://zwiftpower\.com/profile\.php?z=156078)|—|[50th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|184|901|[Pawel Szczygielski](https://zwiftpower\.com/profile\.php?z=170523)|—|[50th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|184|901|[Shane O'Neill](https://zwiftpower\.com/profile\.php?z=956534)|—|[50th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|185|899|[James Bentley](https://zwiftpower\.com/profile\.php?z=64145)|—|[51st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|185|899|[Blasius K\.](https://zwiftpower\.com/profile\.php?z=902957)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[51st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|186|897|[Brian Monaghan](https://zwiftpower\.com/profile\.php?z=308807)|—|[52nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|186|897|[Stewart Box](https://zwiftpower\.com/profile\.php?z=342042)|—|[52nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|187|895|[James Gray](https://zwiftpower\.com/profile\.php?z=313395)|—|[53rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|187|895|[Ole Koch](https://zwiftpower\.com/profile\.php?z=415815)|—|[53rd #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|187|895|[Maarten Clochet \(GuideOn Flanders\)](https://zwiftpower\.com/profile\.php?z=729901)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[53rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|188|893|[T obi\.Scheller 5944](https://zwiftpower\.com/profile\.php?z=257264)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[54th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|189|891|[Augustin Bossart](https://zwiftpower\.com/profile\.php?z=68390)|—|[55th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|189|891|[Koen van Wieren](https://zwiftpower\.com/profile\.php?z=466388)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[55th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|189|891|[giuseppe durante](https://zwiftpower\.com/profile\.php?z=904546)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[55th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|190|889|[Matthew James](https://zwiftpower\.com/profile\.php?z=23924)|[RCC](https://zwiftpower\.com/team\.php?id=265)|[56th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|190|889|[gabriel montesi](https://zwiftpower\.com/profile\.php?z=118491)|—|[56th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|190|889|[Mighty Mariner \(06:30CC\)1897](https://zwiftpower\.com/profile\.php?z=876037)|—|[56th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|191|887|[Ricardo Alarcon](https://zwiftpower\.com/profile\.php?z=421155)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[57th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|191|887|[Peter Juel Jensen](https://zwiftpower\.com/profile\.php?z=1021739)|—|[57th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|192|885|[Joel hawks / Windsor Door Racing p/b Bicycles Plus](https://zwiftpower\.com/profile\.php?z=197)|[Windsor Door Rac](https://zwiftpower\.com/team\.php?id=2037)|[58th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|192|885|[chris cooksey](https://zwiftpower\.com/profile\.php?z=219752)|—|[58th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|192|885|[Roland Schleyer](https://zwiftpower\.com/profile\.php?z=421721)|[Ruhr\-Sued](https://zwiftpower\.com/team\.php?id=1912)|[58th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|192|885|[André Kachlmeier \[Vegan\]](https://zwiftpower\.com/profile\.php?z=891003)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[58th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|193|883|[Travis Pollock](https://zwiftpower\.com/profile\.php?z=64205)|—|[59th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|193|883|[Tristan Young](https://zwiftpower\.com/profile\.php?z=625599)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[59th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|193|883|[Dirk Koning](https://zwiftpower\.com/profile\.php?z=856651)|—|[59th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|193|883|[Simon Spinczyk](https://zwiftpower\.com/profile\.php?z=991216)|[Tri Geckos](https://zwiftpower\.com/team\.php?id=4073)|[59th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|194|881|[George York](https://zwiftpower\.com/profile\.php?z=98092)|[COS Racing](https://zwiftpower\.com/team\.php?id=86)|[60th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|194|881|[Craig McIntosh](https://zwiftpower\.com/profile\.php?z=210353)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[60th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|194|881|[Hideaki Hara](https://zwiftpower\.com/profile\.php?z=404558)|—|[60th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|195|879|[James Richardson](https://zwiftpower\.com/profile\.php?z=441885)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[61st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|195|879|[Simeon Wishlade](https://zwiftpower\.com/profile\.php?z=521363)|—|[61st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|195|879|[Alvin Martin1768](https://zwiftpower\.com/profile\.php?z=592927)|—|[61st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|195|879|[Mark Boyle \(ECC\)](https://zwiftpower\.com/profile\.php?z=918500)|—|[61st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|196|877|[Rob Lyons TFC B2](https://zwiftpower\.com/profile\.php?z=100630)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[62nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|196|877|[L\. Saga](https://zwiftpower\.com/profile\.php?z=388475)|—|[62nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|196|877|[Arild Myklebust](https://zwiftpower\.com/profile\.php?z=478415)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[62nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|196|877|[Calle Mellergård](https://zwiftpower\.com/profile\.php?z=868092)|—|[62nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|197|875|[Thomas Schröter \[R2\]](https://zwiftpower\.com/profile\.php?z=271773)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[63rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|198|873|[Derek Brauch](https://zwiftpower\.com/profile\.php?z=275844)|—|[64th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|198|873|[Scott Main \[BRT\]](https://zwiftpower\.com/profile\.php?z=1002910)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[64th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|199|871|[Derek Atkins](https://zwiftpower\.com/profile\.php?z=11615)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[65th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|199|871|[Jonah Arnold](https://zwiftpower\.com/profile\.php?z=172379)|—|[65th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|199|871|[A\.sugi](https://zwiftpower\.com/profile\.php?z=278071)|—|[65th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|199|871|[Marc Rutar](https://zwiftpower\.com/profile\.php?z=420915)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[65th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|199|871|[Nelson Guia](https://zwiftpower\.com/profile\.php?z=1091621)|—|[65th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|200|869|[Sven De Schryver](https://zwiftpower\.com/profile\.php?z=177491)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[66th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|200|869|[Frank Wöhrleee](https://zwiftpower\.com/profile\.php?z=639232)|—|[66th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|200|869|[Trond Bjoerklund](https://zwiftpower\.com/profile\.php?z=763978)|—|[66th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|200|869|[John Davidson](https://zwiftpower\.com/profile\.php?z=878188)|—|[66th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|201|867|[Jon Carpenter](https://zwiftpower\.com/profile\.php?z=371910)|[OB's](https://zwiftpower\.com/team\.php?id=3104)|[67th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|201|867|[David Santos](https://zwiftpower\.com/profile\.php?z=437626)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|[67th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|201|867|[Christian Opedal](https://zwiftpower\.com/profile\.php?z=462694)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[67th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|201|867|[Matt Kurton](https://zwiftpower\.com/profile\.php?z=1062872)|—|[67th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|202|865|[Simon Piniel](https://zwiftpower\.com/profile\.php?z=202301)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[68th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|202|865|[Stefan Ihlen](https://zwiftpower\.com/profile\.php?z=251644)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[68th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|202|865|[Matthias Dietrich](https://zwiftpower\.com/profile\.php?z=414645)|—|[68th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|202|865|[Jeff Nary](https://zwiftpower\.com/profile\.php?z=415598)|—|[68th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|202|865|[Jeff is a Banana 1193](https://zwiftpower\.com/profile\.php?z=1096837)|—|[68th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|203|863|[Jan\-Willem Scheerder](https://zwiftpower\.com/profile\.php?z=66397)|—|[69th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|203|863|[The Real Sam Woodhouse \- RideBiker/RNR/Velosport](https://zwiftpower\.com/profile\.php?z=117531)|—|[69th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|203|863|[Matthew Thomas](https://zwiftpower\.com/profile\.php?z=452396)|—|[69th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|203|863|[Lee Perkins](https://zwiftpower\.com/profile\.php?z=484667)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[69th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|203|863|[Paul Wisbey](https://zwiftpower\.com/profile\.php?z=542812)|—|[69th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|204|861|[Martin Leduc](https://zwiftpower\.com/profile\.php?z=4236)|—|[70th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|204|861|[Glenn Seljåsen](https://zwiftpower\.com/profile\.php?z=67228)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[70th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|204|861|[Christoph Dahmen](https://zwiftpower\.com/profile\.php?z=701233)|—|[70th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|205|859|[Craig Gardner](https://zwiftpower\.com/profile\.php?z=151938)|—|[71st #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|205|859|[Raitis Nackalns](https://zwiftpower\.com/profile\.php?z=964958)|—|[71st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|206|857|[Nick Dickinson](https://zwiftpower\.com/profile\.php?z=161092)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[72nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|206|857|[Christophe Moeri1173](https://zwiftpower\.com/profile\.php?z=1137726)|—|[72nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|207|855|[huw Williams](https://zwiftpower\.com/profile\.php?z=350605)|[WECC](https://zwiftpower\.com/team\.php?id=1992)|[73rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|207|855|[Julien Guyon](https://zwiftpower\.com/profile\.php?z=578354)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|[73rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
B|207|855|[Leo Bandi](https://zwiftpower\.com/profile\.php?z=959474)|—|[73rd #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|207|855|[Nuno Felicio](https://zwiftpower\.com/profile\.php?z=1112258)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|[73rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|208|853|[Tim Hoogstrate](https://zwiftpower\.com/profile\.php?z=68716)|[Digglers \(Doped\)](https://zwiftpower\.com/team\.php?id=4060)|[74th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|208|853|[Richard Lewey](https://zwiftpower\.com/profile\.php?z=122702)|—|[74th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|208|853|[Nick Lloyd Kiss B](https://zwiftpower\.com/profile\.php?z=137357)|—|[74th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|208|853|[wim van bouwel](https://zwiftpower\.com/profile\.php?z=332102)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[74th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|209|851|[Martin Diserens](https://zwiftpower\.com/profile\.php?z=116708)|[Team SUI](https://zwiftpower\.com/team\.php?id=391)|[75th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|209|851|[Felix Graf](https://zwiftpower\.com/profile\.php?z=977242)|—|[75th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|210|849|[Jim Downing6518](https://zwiftpower\.com/profile\.php?z=961226)|—|[76th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|211|847|[David Farrington](https://zwiftpower\.com/profile\.php?z=5534)|[CCC](https://zwiftpower\.com/team\.php?id=2930)|[77th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|211|847|[Andreas Spieß](https://zwiftpower\.com/profile\.php?z=371455)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[77th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|211|847|[Tom Clough](https://zwiftpower\.com/profile\.php?z=394805)|—|[77th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|212|845|[Tako Bruinsma](https://zwiftpower\.com/profile\.php?z=21126)|—|[78th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|212|845|[Thorsten Neugebauer \[R2\]](https://zwiftpower\.com/profile\.php?z=172830)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[78th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|212|845|[Marcin Kowalczyk](https://zwiftpower\.com/profile\.php?z=530478)|—|[78th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|212|845|[Jarosław Lapawa9777](https://zwiftpower\.com/profile\.php?z=908424)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[78th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|213|843|[Ben Cox](https://zwiftpower\.com/profile\.php?z=15232)|—|[79th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|213|843|[Robert Oroloko](https://zwiftpower\.com/profile\.php?z=211178)|[Oroloko Team](https://zwiftpower\.com/team\.php?id=1892)|[79th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|213|843|[Murray](https://zwiftpower\.com/profile\.php?z=487003)|—|[79th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|213|843|[Florian Felderer](https://zwiftpower\.com/profile\.php?z=912929)|—|[79th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|214|841|[Ryan Olszewski](https://zwiftpower\.com/profile\.php?z=95407)|—|[80th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
B|214|841|[Gary Fawcett](https://zwiftpower\.com/profile\.php?z=307228)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[80th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|215|839|[Barry Dickson](https://zwiftpower\.com/profile\.php?z=3309)|—|[81st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|215|839|[Jim Bullock](https://zwiftpower\.com/profile\.php?z=69883)|—|[81st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|215|839|[Jon DeLacey](https://zwiftpower\.com/profile\.php?z=133933)|—|[81st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|215|839|[Peer Schweder5020](https://zwiftpower\.com/profile\.php?z=911594)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[81st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|216|837|[Robbie Thorpe](https://zwiftpower\.com/profile\.php?z=543451)|—|[82nd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|216|837|[Colin Sweeney](https://zwiftpower\.com/profile\.php?z=597696)|—|[82nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|217|835|[Rob Keates TFC Racing](https://zwiftpower\.com/profile\.php?z=98197)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[83rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|217|835|[Oliver Peacock](https://zwiftpower\.com/profile\.php?z=404761)|—|[83rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|217|835|[Benedikt Mankel](https://zwiftpower\.com/profile\.php?z=968253)|—|[83rd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|218|833|[Tom Incaking](https://zwiftpower\.com/profile\.php?z=6386)|—|[84th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|218|833|[Lance Mitamura 5699](https://zwiftpower\.com/profile\.php?z=631643)|—|[84th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|218|833|[Chris Kaye](https://zwiftpower\.com/profile\.php?z=1092286)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[84th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|219|831|[Dave Crowther](https://zwiftpower\.com/profile\.php?z=527640)|—|[85th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|219|831|[Marek Grzesiecki](https://zwiftpower\.com/profile\.php?z=616717)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[85th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|220|829|[Justin G 3905](https://zwiftpower\.com/profile\.php?z=531409)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[86th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|220|829|[matthew lebelinski icc](https://zwiftpower\.com/profile\.php?z=791971)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[86th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|220|829|[Luca Cathomen1082105](https://zwiftpower\.com/profile\.php?z=1082105)|—|[86th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|220|829|[Miha Rojko \(Canyon\)](https://zwiftpower\.com/profile\.php?z=1129382)|—|[86th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
B|221|827|[David Darley](https://zwiftpower\.com/profile\.php?z=25658)|—|[87th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|221|827|[Martin Browne](https://zwiftpower\.com/profile\.php?z=285984)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[87th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|221|827|[Sebastian Lopens](https://zwiftpower\.com/profile\.php?z=675173)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[87th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|221|827|[Steve Tuck TFC](https://zwiftpower\.com/profile\.php?z=988894)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[87th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|222|825|[DAVID PARRINGTON](https://zwiftpower\.com/profile\.php?z=95706)|—|[88th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|222|825|[j perry \(B\)](https://zwiftpower\.com/profile\.php?z=377081)|—|[88th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|222|825|[Max Olesen \[TMF\]](https://zwiftpower\.com/profile\.php?z=1126914)|[TMF](https://zwiftpower\.com/team\.php?id=3815)|[88th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|223|823|[Daren Chandisingh 🌱🔰](https://zwiftpower\.com/profile\.php?z=40771)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[89th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|223|823|[Stephen Williams](https://zwiftpower\.com/profile\.php?z=759123)|—|[89th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|224|821|[Chris Duffy](https://zwiftpower\.com/profile\.php?z=476846)|—|[90th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|224|821|[Rafał Karolewicz](https://zwiftpower\.com/profile\.php?z=1064292)|—|[90th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|224|821|[Mike T2015](https://zwiftpower\.com/profile\.php?z=1147478)|—|[90th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|225|819|[Yi Zhu](https://zwiftpower\.com/profile\.php?z=99960)|—|[91st #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|225|819|[Edwin Croucher TeamEN](https://zwiftpower\.com/profile\.php?z=348947)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[91st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|225|819|[Vince Van Gestel](https://zwiftpower\.com/profile\.php?z=457942)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[91st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|226|817|[Lars Hoffmann \(KC\)](https://zwiftpower\.com/profile\.php?z=456254)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[92nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|227|815|[Andreas Wolf](https://zwiftpower\.com/profile\.php?z=36808)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[93rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|227|815|[David Farrer](https://zwiftpower\.com/profile\.php?z=173551)|—|[93rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|227|815|[Conor Dalby](https://zwiftpower\.com/profile\.php?z=396581)|—|[93rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|227|815|[Macario Chen](https://zwiftpower\.com/profile\.php?z=830497)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[93rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
B|228|813|[Joe Allen \[BRT\]](https://zwiftpower\.com/profile\.php?z=162388)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[94th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|228|813|[Jon Steadman](https://zwiftpower\.com/profile\.php?z=366020)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[94th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|228|813|[Adrian Kopf](https://zwiftpower\.com/profile\.php?z=559115)|—|[94th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|228|813|[Tim Kniep 8737](https://zwiftpower\.com/profile\.php?z=694829)|—|[94th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|229|811|[Simon Crerie TFC](https://zwiftpower\.com/profile\.php?z=137267)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[95th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|229|811|[Step\_ Mey](https://zwiftpower\.com/profile\.php?z=518912)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|[95th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|230|809|[Ace Thunder](https://zwiftpower\.com/profile\.php?z=56559)|—|[96th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|230|809|[C\.FightMS](https://zwiftpower\.com/profile\.php?z=58651)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[96th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|230|809|[Robert Zatorski](https://zwiftpower\.com/profile\.php?z=593924)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[96th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|231|807|[ben caine](https://zwiftpower\.com/profile\.php?z=44421)|—|[97th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|231|807|[Thomas Gerstmann \(KC\)](https://zwiftpower\.com/profile\.php?z=189206)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[97th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|231|807|[Steve Trout](https://zwiftpower\.com/profile\.php?z=231683)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[97th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
B|232|805|[LOUIS\-PIERRE DUPUIS\(pjcgrch\)\(JZQc\) B 62 years](https://zwiftpower\.com/profile\.php?z=261617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[98th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|232|805|[Christian Lüddeckens](https://zwiftpower\.com/profile\.php?z=269482)|[338](https://zwiftpower\.com/team\.php?id=3784)|[98th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|233|803|[Mike Herle](https://zwiftpower\.com/profile\.php?z=292576)|—|[99th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|234|801|[Nuno Madeira](https://zwiftpower\.com/profile\.php?z=122420)|—|[100th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|234|801|[Jeff Allen](https://zwiftpower\.com/profile\.php?z=225142)|—|[100th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
B|235|799|[Anders Grauballe](https://zwiftpower\.com/profile\.php?z=1148907)|—|[101st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|236|797|[Rüdiger Grosch](https://zwiftpower\.com/profile\.php?z=14064)|—|[102nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
B|236|797|[Joost Rusche 3893](https://zwiftpower\.com/profile\.php?z=954385)|—|[102nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
B|237|795|[Nathan Weselake](https://zwiftpower\.com/profile\.php?z=951720)|—|[103rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|1|5926|[Jorge Caldeira 🐹](https://zwiftpower\.com/profile\.php?z=181509)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥈 #8](https://zwiftpower.com/events.php?zid=176148)|[🥉 #2](https://zwiftpower.com/events.php?zid=155628)|[5th #5](https://zwiftpower.com/events.php?zid=166630)|[9th #3](https://zwiftpower.com/events.php?zid=159334)|[10th #6](https://zwiftpower.com/events.php?zid=171935)|[11th #7](https://zwiftpower.com/events.php?zid=173451)
C|2|5890|[Janosch Zbick](https://zwiftpower\.com/profile\.php?z=180178)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥉 #8](https://zwiftpower.com/events.php?zid=176148)|[6th #3](https://zwiftpower.com/events.php?zid=159334)|[6th #6](https://zwiftpower.com/events.php?zid=171935)|[10th #2](https://zwiftpower.com/events.php?zid=155628)|[14th #5](https://zwiftpower.com/events.php?zid=166630)|[19th #1](https://zwiftpower.com/events.php?zid=152778)
C|3|5848|[Dave Clarkson CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=393818)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[6th #5](https://zwiftpower.com/events.php?zid=166630)|[6th #7](https://zwiftpower.com/events.php?zid=173451)|[9th #8](https://zwiftpower.com/events.php?zid=176148)|[12th #3](https://zwiftpower.com/events.php?zid=159334)|[22nd #2](https://zwiftpower.com/events.php?zid=155628)|[24th #6](https://zwiftpower.com/events.php?zid=171935)
C|4|5836|[Robin Strong](https://zwiftpower\.com/profile\.php?z=174273)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[5th #1](https://zwiftpower.com/events.php?zid=152778)|[8th #7](https://zwiftpower.com/events.php?zid=173451)|[8th #8](https://zwiftpower.com/events.php?zid=176148)|[14th #3](https://zwiftpower.com/events.php?zid=159334)|[20th #5](https://zwiftpower.com/events.php?zid=166630)|[30th #4](https://zwiftpower.com/events.php?zid=162691)
C|5|5792|[Victor Klinkvis \(vision C1\)](https://zwiftpower\.com/profile\.php?z=383195)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[11th #1](https://zwiftpower.com/events.php?zid=152778)|[16th #2](https://zwiftpower.com/events.php?zid=155628)|[16th #4](https://zwiftpower.com/events.php?zid=162691)|[16th #6](https://zwiftpower.com/events.php?zid=171935)|[22nd #7](https://zwiftpower.com/events.php?zid=173451)|[26th #3](https://zwiftpower.com/events.php?zid=159334)
C|6|5788|[Miguel Rola Costa](https://zwiftpower\.com/profile\.php?z=94972)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥈 #2](https://zwiftpower.com/events.php?zid=155628)|[7th #1](https://zwiftpower.com/events.php?zid=152778)|[10th #7](https://zwiftpower.com/events.php?zid=173451)|[28th #3](https://zwiftpower.com/events.php?zid=159334)|[28th #4](https://zwiftpower.com/events.php?zid=162691)|[34th #5](https://zwiftpower.com/events.php?zid=166630)
C|7|5764|[Simon Crerie TFC](https://zwiftpower\.com/profile\.php?z=137267)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[11th #4](https://zwiftpower.com/events.php?zid=162691)|[13th #6](https://zwiftpower.com/events.php?zid=171935)|[13th #7](https://zwiftpower.com/events.php?zid=173451)|[22nd #8](https://zwiftpower.com/events.php?zid=176148)|[30th #3](https://zwiftpower.com/events.php?zid=159334)|[32nd #5](https://zwiftpower.com/events.php?zid=166630)
C|8|5664|[Frank Zitzelsberger \(KC\)](https://zwiftpower\.com/profile\.php?z=466517)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[5th #7](https://zwiftpower.com/events.php?zid=173451)|[10th #5](https://zwiftpower.com/events.php?zid=166630)|[23rd #3](https://zwiftpower.com/events.php?zid=159334)|[23rd #4](https://zwiftpower.com/events.php?zid=162691)|[50th #1](https://zwiftpower.com/events.php?zid=152778)|[60th #2](https://zwiftpower.com/events.php?zid=155628)
C|9|5650|[Igor Molino \-ITA\-\(C\)](https://zwiftpower\.com/profile\.php?z=35914)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[18th #5](https://zwiftpower.com/events.php?zid=166630)|[20th #7](https://zwiftpower.com/events.php?zid=173451)|[26th #6](https://zwiftpower.com/events.php?zid=171935)|[28th #8](https://zwiftpower.com/events.php?zid=176148)|[36th #1](https://zwiftpower.com/events.php?zid=152778)|[50th #4](https://zwiftpower.com/events.php?zid=162691)
C|10|5640|[Peter O'Donoghue 5083](https://zwiftpower\.com/profile\.php?z=107547)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[12th #7](https://zwiftpower.com/events.php?zid=173451)|[23rd #5](https://zwiftpower.com/events.php?zid=166630)|[29th #8](https://zwiftpower.com/events.php?zid=176148)|[31st #1](https://zwiftpower.com/events.php?zid=152778)|[41st #3](https://zwiftpower.com/events.php?zid=159334)|[47th #2](https://zwiftpower.com/events.php?zid=155628)
C|11|5584|[Peter Enserer\(KC\)](https://zwiftpower\.com/profile\.php?z=887372)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[27th #7](https://zwiftpower.com/events.php?zid=173451)|[31st #3](https://zwiftpower.com/events.php?zid=159334)|[33rd #1](https://zwiftpower.com/events.php?zid=152778)|[33rd #6](https://zwiftpower.com/events.php?zid=171935)|[38th #2](https://zwiftpower.com/events.php?zid=155628)|[49th #5](https://zwiftpower.com/events.php?zid=166630)
C|12|5534|[Paulo Monteiro](https://zwiftpower\.com/profile\.php?z=631207)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[32nd #6](https://zwiftpower.com/events.php?zid=171935)|[34th #7](https://zwiftpower.com/events.php?zid=173451)|[37th #5](https://zwiftpower.com/events.php?zid=166630)|[39th #8](https://zwiftpower.com/events.php?zid=176148)|[45th #2](https://zwiftpower.com/events.php?zid=155628)|[49th #3](https://zwiftpower.com/events.php?zid=159334)
C|13|5450|[Sandy McMillan](https://zwiftpower\.com/profile\.php?z=987913)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[33rd #8](https://zwiftpower.com/events.php?zid=176148)|[44th #7](https://zwiftpower.com/events.php?zid=173451)|[46th #2](https://zwiftpower.com/events.php?zid=155628)|[46th #3](https://zwiftpower.com/events.php?zid=159334)|[50th #6](https://zwiftpower.com/events.php?zid=171935)|[59th #4](https://zwiftpower.com/events.php?zid=162691)
C|14|4817|[Mark Mortimer](https://zwiftpower\.com/profile\.php?z=73851)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[7th #7](https://zwiftpower.com/events.php?zid=173451)|[12th #8](https://zwiftpower.com/events.php?zid=176148)|[21st #3](https://zwiftpower.com/events.php?zid=159334)|[24th #4](https://zwiftpower.com/events.php?zid=162691)|[30th #5](https://zwiftpower.com/events.php?zid=166630)|—
C|15|4787|[Jarek Slezak](https://zwiftpower\.com/profile\.php?z=466194)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[16th #8](https://zwiftpower.com/events.php?zid=176148)|[17th #6](https://zwiftpower.com/events.php?zid=171935)|[18th #7](https://zwiftpower.com/events.php?zid=173451)|[21st #5](https://zwiftpower.com/events.php?zid=166630)|[37th #4](https://zwiftpower.com/events.php?zid=162691)|—
C|16|4627|[Joe Reynolds](https://zwiftpower\.com/profile\.php?z=273556)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[25th #8](https://zwiftpower.com/events.php?zid=176148)|[31st #2](https://zwiftpower.com/events.php?zid=155628)|[35th #1](https://zwiftpower.com/events.php?zid=152778)|[48th #4](https://zwiftpower.com/events.php?zid=162691)|[50th #3](https://zwiftpower.com/events.php?zid=159334)|—
C|17|4613|[Shug Wilson Ⓥ\[VEGAN\]](https://zwiftpower\.com/profile\.php?z=541203)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[35th #2](https://zwiftpower.com/events.php?zid=155628)|[36th #7](https://zwiftpower.com/events.php?zid=173451)|[39th #5](https://zwiftpower.com/events.php?zid=166630)|[39th #6](https://zwiftpower.com/events.php?zid=171935)|[47th #3](https://zwiftpower.com/events.php?zid=159334)|—
C|18|4463|[Dave Kerr TFC](https://zwiftpower\.com/profile\.php?z=617959)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[41st #1](https://zwiftpower.com/events.php?zid=152778)|[48th #7](https://zwiftpower.com/events.php?zid=173451)|[58th #2](https://zwiftpower.com/events.php?zid=155628)|[60th #4](https://zwiftpower.com/events.php?zid=162691)|[64th #5](https://zwiftpower.com/events.php?zid=166630)|—
C|19|3880|[Steve Fry](https://zwiftpower\.com/profile\.php?z=159322)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[12th #4](https://zwiftpower.com/events.php?zid=162691)|[13th #3](https://zwiftpower.com/events.php?zid=159334)|[15th #5](https://zwiftpower.com/events.php?zid=166630)|[22nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—
C|20|3834|[Akers Harake](https://zwiftpower\.com/profile\.php?z=585048)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[6th #8](https://zwiftpower.com/events.php?zid=176148)|[15th #7](https://zwiftpower.com/events.php?zid=173451)|[22nd #6](https://zwiftpower.com/events.php?zid=171935)|[42nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—
C|21|3824|[Andrew Lea Watkins](https://zwiftpower\.com/profile\.php?z=255851)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[14th #6](https://zwiftpower.com/events.php?zid=171935)|[20th #8](https://zwiftpower.com/events.php?zid=176148)|[22nd #4](https://zwiftpower.com/events.php?zid=162691)|[34th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
C|22|3768|[Jamie Fox](https://zwiftpower\.com/profile\.php?z=454725)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[24th #8](https://zwiftpower.com/events.php?zid=176148)|[30th #2](https://zwiftpower.com/events.php?zid=155628)|[32nd #1](https://zwiftpower.com/events.php?zid=152778)|[32nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—
C|23|3728|[paul williams](https://zwiftpower\.com/profile\.php?z=122512)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[24th #7](https://zwiftpower.com/events.php?zid=173451)|[27th #8](https://zwiftpower.com/events.php?zid=176148)|[43rd #5](https://zwiftpower.com/events.php?zid=166630)|[44th #4](https://zwiftpower.com/events.php?zid=162691)|—|—
C|24|3678|[Ole Petter Lødøen](https://zwiftpower\.com/profile\.php?z=554963)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[26th #1](https://zwiftpower.com/events.php?zid=152778)|[35th #6](https://zwiftpower.com/events.php?zid=171935)|[47th #7](https://zwiftpower.com/events.php?zid=173451)|[55th #3](https://zwiftpower.com/events.php?zid=159334)|—|—
C|25|3612|[Fabian Rothmeier](https://zwiftpower\.com/profile\.php?z=887482)|—|[36th #6](https://zwiftpower.com/events.php?zid=171935)|[41st #8](https://zwiftpower.com/events.php?zid=176148)|[58th #5](https://zwiftpower.com/events.php?zid=166630)|[61st #3](https://zwiftpower.com/events.php?zid=159334)|—|—
C|26|3588|[Tam Burns CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=17442)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[45th #3](https://zwiftpower.com/events.php?zid=159334)|[45th #7](https://zwiftpower.com/events.php?zid=173451)|[57th #4](https://zwiftpower.com/events.php?zid=162691)|[61st #5](https://zwiftpower.com/events.php?zid=166630)|—|—
C|27|2975|[Ian Bottomore](https://zwiftpower\.com/profile\.php?z=163377)|—|[🥉 #3](https://zwiftpower.com/events.php?zid=159334)|[5th #2](https://zwiftpower.com/events.php?zid=155628)|[6th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
C|28|2933|[Tulipunane Radiaator](https://zwiftpower\.com/profile\.php?z=984823)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥇 #5](https://zwiftpower.com/events.php?zid=166630)|[🥉 #6](https://zwiftpower.com/events.php?zid=171935)|[31st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
C|29|2929|[Michael Willer](https://zwiftpower\.com/profile\.php?z=436965)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[4th #8](https://zwiftpower.com/events.php?zid=176148)|[11th #6](https://zwiftpower.com/events.php?zid=171935)|[22nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
C|30|2921|[Theun Bouius](https://zwiftpower\.com/profile\.php?z=700655)|—|[7th #6](https://zwiftpower.com/events.php?zid=171935)|[9th #4](https://zwiftpower.com/events.php?zid=162691)|[25th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
C|31|2911|[Martin Ashton CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=319164)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[4th #3](https://zwiftpower.com/events.php?zid=159334)|[17th #2](https://zwiftpower.com/events.php?zid=155628)|[25th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
C|32|2903|[Tristan Young](https://zwiftpower\.com/profile\.php?z=625599)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[🥉 #7](https://zwiftpower.com/events.php?zid=173451)|[23rd #6](https://zwiftpower.com/events.php?zid=171935)|[24th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—
C|33|2897|[Matt Hilbert](https://zwiftpower\.com/profile\.php?z=946001)|—|[14th #4](https://zwiftpower.com/events.php?zid=162691)|[18th #3](https://zwiftpower.com/events.php?zid=159334)|[21st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
C|34|2895|[Matty Hermann](https://zwiftpower\.com/profile\.php?z=246215)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th #4](https://zwiftpower.com/events.php?zid=162691)|[19th #8](https://zwiftpower.com/events.php?zid=176148)|[30th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—
C|35|2885|[Ben Gillett](https://zwiftpower\.com/profile\.php?z=315040)|—|[11th #2](https://zwiftpower.com/events.php?zid=155628)|[13th #4](https://zwiftpower.com/events.php?zid=162691)|[35th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
C|36|2873|[Sven Fahrenwald](https://zwiftpower\.com/profile\.php?z=434649)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[14th #8](https://zwiftpower.com/events.php?zid=176148)|[23rd #7](https://zwiftpower.com/events.php?zid=173451)|[28th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
C|37|2835|[Jurian Onderdijk](https://zwiftpower\.com/profile\.php?z=679250)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[16th #3](https://zwiftpower.com/events.php?zid=159334)|[19th #4](https://zwiftpower.com/events.php?zid=162691)|[49th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
C|38|2807|[Paulo Gago](https://zwiftpower\.com/profile\.php?z=418284)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[27th #4](https://zwiftpower.com/events.php?zid=162691)|[32nd #8](https://zwiftpower.com/events.php?zid=176148)|[39th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—
C|39|2797|[Rene Fuchs \(KC\)](https://zwiftpower\.com/profile\.php?z=220518)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[23rd #1](https://zwiftpower.com/events.php?zid=152778)|[37th #2](https://zwiftpower.com/events.php?zid=155628)|[43rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
C|39|2797|[Greg Agran 5235](https://zwiftpower\.com/profile\.php?z=466810)|—|[27th #6](https://zwiftpower.com/events.php?zid=171935)|[32nd #2](https://zwiftpower.com/events.php?zid=155628)|[44th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—
C|40|2795|[Lars Schrödel\(KC\)](https://zwiftpower\.com/profile\.php?z=946113)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[26th #7](https://zwiftpower.com/events.php?zid=173451)|[31st #8](https://zwiftpower.com/events.php?zid=176148)|[47th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
C|41|2745|[Bill Manard](https://zwiftpower\.com/profile\.php?z=7678)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[37th #3](https://zwiftpower.com/events.php?zid=159334)|[38th #1](https://zwiftpower.com/events.php?zid=152778)|[54th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
C|42|2727|[Eric P\. Flanderhijn](https://zwiftpower\.com/profile\.php?z=723344)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[41st #6](https://zwiftpower.com/events.php?zid=171935)|[43rd #1](https://zwiftpower.com/events.php?zid=152778)|[54th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—
C|43|1994|[Lar Hayden TFC \(C\)](https://zwiftpower\.com/profile\.php?z=763657)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #2](https://zwiftpower.com/events.php?zid=155628)|[🥉 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|44|1992|[Ryan DeadPool Miller TFC \(C\)](https://zwiftpower\.com/profile\.php?z=331384)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #8](https://zwiftpower.com/events.php?zid=176148)|[4th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|45|1980|[Chris Moore](https://zwiftpower\.com/profile\.php?z=51627)|—|[🥉 #5](https://zwiftpower.com/events.php?zid=166630)|[8th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
C|46|1974|[Tom Lincoln](https://zwiftpower\.com/profile\.php?z=665383)|—|[🥈 #3](https://zwiftpower.com/events.php?zid=159334)|[12th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|47|1968|[Mark Leeming \[KRT\]](https://zwiftpower\.com/profile\.php?z=325878)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[5th #3](https://zwiftpower.com/events.php?zid=159334)|[12th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
C|48|1966|[Kamil Szelag](https://zwiftpower\.com/profile\.php?z=610990)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[7th #4](https://zwiftpower.com/events.php?zid=162691)|[11th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|49|1962|[Oliver Ennig \(KC\)](https://zwiftpower\.com/profile\.php?z=419731)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[7th #2](https://zwiftpower.com/events.php?zid=155628)|[13th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|50|1960|[Hauke Westenberg](https://zwiftpower\.com/profile\.php?z=449960)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[4th #7](https://zwiftpower.com/events.php?zid=173451)|[17th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
C|50|1960|[Micha Roth \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=800842)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[7th #5](https://zwiftpower.com/events.php?zid=166630)|[14th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|51|1956|[Gav Holzman](https://zwiftpower\.com/profile\.php?z=767825)|—|[7th #8](https://zwiftpower.com/events.php?zid=176148)|[16th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|52|1952|[René Rolighed](https://zwiftpower\.com/profile\.php?z=42595)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[4th #2](https://zwiftpower.com/events.php?zid=155628)|[21st #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|53|1946|[Brendan Worley CRYO\-GEN C](https://zwiftpower\.com/profile\.php?z=300554)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[13th #8](https://zwiftpower.com/events.php?zid=176148)|[15th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|53|1946|"[Ralph Hoffman \| Gruppo C](https://zwiftpower\.com/profile\.php?z=967255)"|—|[🥈 #5](https://zwiftpower.com/events.php?zid=166630)|[26th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|54|1944|[Harald Saurer \(KC\)](https://zwiftpower\.com/profile\.php?z=1048852)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[4th #5](https://zwiftpower.com/events.php?zid=166630)|[25th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|55|1942|[F\. Afonso](https://zwiftpower\.com/profile\.php?z=125584)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[15th #4](https://zwiftpower.com/events.php?zid=162691)|[15th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
C|56|1938|[Andrew Block Ⓥ](https://zwiftpower\.com/profile\.php?z=19300)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[15th #2](https://zwiftpower.com/events.php?zid=155628)|[17th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|56|1938|[Fabrix Greg](https://zwiftpower\.com/profile\.php?z=821450)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[13th #5](https://zwiftpower.com/events.php?zid=166630)|[19th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
C|57|1928|[Fabrice Lccq Ⓥ](https://zwiftpower\.com/profile\.php?z=650081)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[14th #7](https://zwiftpower.com/events.php?zid=173451)|[23rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
C|58|1926|[Darren Green \(BRT\)](https://zwiftpower\.com/profile\.php?z=30856)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[18th #2](https://zwiftpower.com/events.php?zid=155628)|[20th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|59|1924|[Paulo Próspero](https://zwiftpower\.com/profile\.php?z=91765)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[12th #5](https://zwiftpower.com/events.php?zid=166630)|[27th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|60|1922|[GedR](https://zwiftpower\.com/profile\.php?z=298972)|[PTT](https://zwiftpower\.com/team\.php?id=3757)|[8th #5](https://zwiftpower.com/events.php?zid=166630)|[32nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|60|1922|[Jon Wakefield](https://zwiftpower\.com/profile\.php?z=703866)|—|[18th #1](https://zwiftpower.com/events.php?zid=152778)|[22nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|60|1922|[Tobias Hindemitt](https://zwiftpower\.com/profile\.php?z=1066527)|—|[19th #3](https://zwiftpower.com/events.php?zid=159334)|[21st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
C|61|1894|[Nuno Felicio](https://zwiftpower\.com/profile\.php?z=1112258)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|[9th #7](https://zwiftpower.com/events.php?zid=173451)|[45th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|62|1892|[Thomas Engel \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=520425)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[26th #2](https://zwiftpower.com/events.php?zid=155628)|[29th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
C|63|1878|[Waldemar Kacprzyk](https://zwiftpower\.com/profile\.php?z=169976)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[26th #8](https://zwiftpower.com/events.php?zid=176148)|[36th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|64|1876|[Gonçalo Afonso](https://zwiftpower\.com/profile\.php?z=355653)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[30th #6](https://zwiftpower.com/events.php?zid=171935)|[33rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
C|65|1874|[John Tetreault5620](https://zwiftpower\.com/profile\.php?z=999091)|—|[29th #1](https://zwiftpower.com/events.php?zid=152778)|[35th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
C|66|1854|[Mick Fisher](https://zwiftpower\.com/profile\.php?z=38412)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[37th #7](https://zwiftpower.com/events.php?zid=173451)|[37th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
C|66|1854|[Tyler Bradshaw2470](https://zwiftpower\.com/profile\.php?z=491267)|—|[20th #6](https://zwiftpower.com/events.php?zid=171935)|[54th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|67|1848|[Tomek Spyra](https://zwiftpower\.com/profile\.php?z=954752)|—|[38th #3](https://zwiftpower.com/events.php?zid=159334)|[39th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—
C|68|1846|[shii tod](https://zwiftpower\.com/profile\.php?z=365733)|—|[31st #5](https://zwiftpower.com/events.php?zid=166630)|[47th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|69|1844|[Philippe Galland](https://zwiftpower\.com/profile\.php?z=65991)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[38th #4](https://zwiftpower.com/events.php?zid=162691)|[41st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|70|1842|[Darren Ridley](https://zwiftpower\.com/profile\.php?z=178403)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[29th #7](https://zwiftpower.com/events.php?zid=173451)|[51st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|71|1838|[Jamie Baker](https://zwiftpower\.com/profile\.php?z=43788)|[None](https://zwiftpower\.com/team\.php?id=30)|[36th #8](https://zwiftpower.com/events.php?zid=176148)|[46th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—
C|72|1834|[Alexander Jehle \(KC\)](https://zwiftpower\.com/profile\.php?z=1093931)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[38th #8](https://zwiftpower.com/events.php?zid=176148)|[46th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
C|73|1826|[Jon Gaudio](https://zwiftpower\.com/profile\.php?z=153683)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|[39th #3](https://zwiftpower.com/events.php?zid=159334)|[49th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
C|74|1824|[Jon Hume](https://zwiftpower\.com/profile\.php?z=465110)|—|[40th #2](https://zwiftpower.com/events.php?zid=155628)|[49th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
C|75|1820|[Zorza Von K](https://zwiftpower\.com/profile\.php?z=231903)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[40th #7](https://zwiftpower.com/events.php?zid=173451)|[51st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
C|76|1802|[T schuckNorris](https://zwiftpower\.com/profile\.php?z=967999)|—|[43rd #2](https://zwiftpower.com/events.php?zid=155628)|[57th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|77|1792|[Ann\-Katrin Kuzmenko \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=871472)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[49th #1](https://zwiftpower.com/events.php?zid=152778)|[56th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—
C|78|1790|[josef haider \( KC \)](https://zwiftpower\.com/profile\.php?z=1094195)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[53rd #2](https://zwiftpower.com/events.php?zid=155628)|[53rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
C|79|1786|[Tom Scott](https://zwiftpower\.com/profile\.php?z=36868)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[53rd #4](https://zwiftpower.com/events.php?zid=162691)|[55th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
C|80|999|[Dan Lucey TFC \(C1\)](https://zwiftpower\.com/profile\.php?z=14693)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|80|999|[Simon Russell](https://zwiftpower\.com/profile\.php?z=68077)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|80|999|[Dennis Braun](https://zwiftpower\.com/profile\.php?z=220647)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥇 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|80|999|[Matt Anderson](https://zwiftpower\.com/profile\.php?z=478039)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|80|999|[Chris Morton](https://zwiftpower\.com/profile\.php?z=1046991)|—|[🥇 #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|81|997|[Kenny Peterson](https://zwiftpower\.com/profile\.php?z=40492)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥈 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|81|997|[Jimmy Tee](https://zwiftpower\.com/profile\.php?z=59364)|—|[🥈 #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|81|997|[jody wainwright](https://zwiftpower\.com/profile\.php?z=480927)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[🥈 #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|81|997|[Todd Hoover](https://zwiftpower\.com/profile\.php?z=496158)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|[🥈 #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|82|995|[Henrik Andlem Borresen](https://zwiftpower\.com/profile\.php?z=653639)|[HSIL](https://zwiftpower\.com/team\.php?id=3049)|[🥉 #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|83|993|[Daniel Mould](https://zwiftpower\.com/profile\.php?z=885901)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|83|993|[Allen Jones](https://zwiftpower\.com/profile\.php?z=962108)|—|[4th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|84|991|[Reg Bestwick TFC](https://zwiftpower\.com/profile\.php?z=45213)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|84|991|[Charlie Ryan NTR](https://zwiftpower\.com/profile\.php?z=185957)|[NTR](https://zwiftpower\.com/team\.php?id=2860)|[5th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|85|989|[Olav Harkjerr Owesen](https://zwiftpower\.com/profile\.php?z=69270)|[Tunge Gir](https://zwiftpower\.com/team\.php?id=3418)|[6th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|85|989|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[6th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|86|987|[Alasdair MacMillan](https://zwiftpower\.com/profile\.php?z=522143)|—|[7th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|87|985|[Stian Horne](https://zwiftpower\.com/profile\.php?z=5322)|[RCC](https://zwiftpower\.com/team\.php?id=265)|[8th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|87|985|[philip jackson](https://zwiftpower\.com/profile\.php?z=133743)|—|[8th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|87|985|[Brandon Zuckerberg](https://zwiftpower\.com/profile\.php?z=445447)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[8th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|87|985|[David Cooper](https://zwiftpower\.com/profile\.php?z=449415)|[GNF](https://zwiftpower\.com/team\.php?id=2324)|[8th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|88|983|[keith palmer](https://zwiftpower\.com/profile\.php?z=57284)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[9th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|88|983|[John Spanner](https://zwiftpower\.com/profile\.php?z=227827)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[9th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|88|983|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[9th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|88|983|[Udo Möller \(KC\)](https://zwiftpower\.com/profile\.php?z=903326)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[9th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|89|981|[Pål Eddy Stokvik](https://zwiftpower\.com/profile\.php?z=45387)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[10th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|89|981|[Aaron Cupp](https://zwiftpower\.com/profile\.php?z=402580)|—|[10th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|89|981|[J T](https://zwiftpower\.com/profile\.php?z=909782)|—|[10th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|89|981|[Sarel Boshoff Ⓥ](https://zwiftpower\.com/profile\.php?z=1128952)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[10th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|90|979|[kenneth skoglund](https://zwiftpower\.com/profile\.php?z=174307)|—|[11th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|90|979|[Brant Law3193](https://zwiftpower\.com/profile\.php?z=897994)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|[11th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|91|977|[Thomas Reinecke](https://zwiftpower\.com/profile\.php?z=209977)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[12th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|92|975|[Ted Kupsick6136](https://zwiftpower\.com/profile\.php?z=214086)|[USMES](https://zwiftpower\.com/team\.php?id=79)|[13th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|93|973|[Stuart Davidson](https://zwiftpower\.com/profile\.php?z=197270)|[Myerscough](https://zwiftpower\.com/team\.php?id=2401)|[14th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|94|971|[Iain Morrison](https://zwiftpower\.com/profile\.php?z=320606)|—|[15th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|94|971|[Michael Fischbach \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=859636)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[15th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|95|969|[Andreas Phillipsen](https://zwiftpower\.com/profile\.php?z=151939)|—|[16th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|95|969|[83Graus](https://zwiftpower\.com/profile\.php?z=1117159)|—|[16th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|96|967|[Daniel Gassmann](https://zwiftpower\.com/profile\.php?z=57897)|—|[17th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|96|967|[Rui Inácio](https://zwiftpower\.com/profile\.php?z=196516)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[17th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|96|967|[Ben Hrycej](https://zwiftpower\.com/profile\.php?z=578319)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[17th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|96|967|[Luke Huebschen 2173](https://zwiftpower\.com/profile\.php?z=1033903)|—|[17th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|97|965|[Chris Johnson Mapperley CC](https://zwiftpower\.com/profile\.php?z=35931)|—|[18th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|97|965|[Vitor Pires \(Carecas Do Pedal/ATX CYCLINGSTORE\)](https://zwiftpower\.com/profile\.php?z=76624)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[18th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|98|963|[Laurens Van Meeteren](https://zwiftpower\.com/profile\.php?z=298701)|[HIK](https://zwiftpower\.com/team\.php?id=719)|[19th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|98|963|[Lee Millward](https://zwiftpower\.com/profile\.php?z=299923)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[19th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|98|963|[Simon Morgan](https://zwiftpower\.com/profile\.php?z=865457)|—|[19th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|99|961|[Skyler Street](https://zwiftpower\.com/profile\.php?z=120373)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[20th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|99|961|[Ove Wadman SZR](https://zwiftpower\.com/profile\.php?z=840538)|—|[20th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|99|961|[Stuart Martin](https://zwiftpower\.com/profile\.php?z=1046694)|—|[20th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|100|959|[Ken Olav Jenssen](https://zwiftpower\.com/profile\.php?z=87153)|—|[21st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|100|959|[Fredrik Larsen](https://zwiftpower\.com/profile\.php?z=253013)|—|[21st #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|100|959|[Michiel Kersten](https://zwiftpower\.com/profile\.php?z=425370)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[21st #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|101|955|[Mathew Jones](https://zwiftpower\.com/profile\.php?z=635949)|—|[23rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|102|953|[James Fowler](https://zwiftpower\.com/profile\.php?z=237877)|[FinishStrong](https://zwiftpower\.com/team\.php?id=1740)|[24th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|102|953|[Sebastian Slotwinski \[DIRT\]](https://zwiftpower\.com/profile\.php?z=251729)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[24th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|102|953|[Nathan Beauchamp1422](https://zwiftpower\.com/profile\.php?z=1105977)|—|[24th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|103|951|[Mike Taylor](https://zwiftpower\.com/profile\.php?z=369058)|—|[25th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|103|951|[Ryan Hawkins](https://zwiftpower\.com/profile\.php?z=581316)|—|[25th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|103|951|[Anders Nilsen](https://zwiftpower\.com/profile\.php?z=836490)|—|[25th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|103|951|[Michael walsh](https://zwiftpower\.com/profile\.php?z=1063692)|—|[25th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|104|947|[David George](https://zwiftpower\.com/profile\.php?z=215470)|[Slipstreamers](https://zwiftpower\.com/team\.php?id=2360)|[27th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|104|947|[Ben Pentland](https://zwiftpower\.com/profile\.php?z=216403)|—|[27th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|104|947|[Reiner Heming ZRG\-R](https://zwiftpower\.com/profile\.php?z=800245)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[27th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|105|945|[Edwin M](https://zwiftpower\.com/profile\.php?z=254175)|—|[28th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|105|945|[Marc\-Olivier Jacques](https://zwiftpower\.com/profile\.php?z=450022)|—|[28th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|105|945|[Anthony Walstow](https://zwiftpower\.com/profile\.php?z=464979)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[28th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|105|945|[William Turner7233](https://zwiftpower\.com/profile\.php?z=727174)|—|[28th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|106|943|[Alan Clarke Kiss C](https://zwiftpower\.com/profile\.php?z=77260)|—|[29th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|106|943|[Jim Rauser TFC \(C\)](https://zwiftpower\.com/profile\.php?z=533103)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[29th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|106|943|[Andy Webb 5012](https://zwiftpower\.com/profile\.php?z=849616)|—|[29th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|106|943|[Piotr Kaliski](https://zwiftpower\.com/profile\.php?z=850113)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[29th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|107|941|[New Scot](https://zwiftpower\.com/profile\.php?z=126700)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[30th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|107|941|[pato briggs](https://zwiftpower\.com/profile\.php?z=360770)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[30th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|108|939|[James Wyatt](https://zwiftpower\.com/profile\.php?z=525033)|—|[31st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|108|939|[Aart Valkhof](https://zwiftpower\.com/profile\.php?z=1063943)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[31st #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|109|937|[mark stringfellow](https://zwiftpower\.com/profile\.php?z=232215)|—|[32nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|110|935|[Perrie Higgs](https://zwiftpower\.com/profile\.php?z=43047)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[33rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|110|935|[David 8656](https://zwiftpower\.com/profile\.php?z=86146)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[33rd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|110|935|[ben brook](https://zwiftpower\.com/profile\.php?z=412366)|—|[33rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|110|935|[Martijn Visser](https://zwiftpower\.com/profile\.php?z=947859)|—|[33rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|111|933|[Nuno Madeira](https://zwiftpower\.com/profile\.php?z=122420)|—|[34th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|111|933|[Roy Kämpe 7552](https://zwiftpower\.com/profile\.php?z=175788)|—|[34th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|111|933|[Jason Thompson](https://zwiftpower\.com/profile\.php?z=838875)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[34th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|111|933|[Steve Tuck TFC](https://zwiftpower\.com/profile\.php?z=988894)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[34th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|111|933|[Alan Aslin](https://zwiftpower\.com/profile\.php?z=1105559)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[34th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|112|931|[Ro Ka](https://zwiftpower\.com/profile\.php?z=74037)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[35th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|112|931|[Paul Paetsch](https://zwiftpower\.com/profile\.php?z=911884)|—|[35th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|112|931|[Ricardo Prazeres](https://zwiftpower\.com/profile\.php?z=1094852)|—|[35th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|113|929|[Jørn Peder Jørgensen DBR](https://zwiftpower\.com/profile\.php?z=59677)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[36th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|113|929|[M Barker](https://zwiftpower\.com/profile\.php?z=1144733)|[Ride WLF](https://zwiftpower\.com/team\.php?id=2257)|[36th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|114|927|[Manuel Rocha](https://zwiftpower\.com/profile\.php?z=70647)|[eRoberto](https://zwiftpower\.com/team\.php?id=3470)|[37th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|114|927|[Maj C](https://zwiftpower\.com/profile\.php?z=918062)|[TRON](https://zwiftpower\.com/team\.php?id=3861)|[37th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|115|925|[Jim Nelson 9806](https://zwiftpower\.com/profile\.php?z=372064)|—|[38th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|115|925|[Josselin Girault](https://zwiftpower\.com/profile\.php?z=378566)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[38th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|115|925|[Victor Moldoveanu](https://zwiftpower\.com/profile\.php?z=824632)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[38th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|116|923|[Oliver Loehlein](https://zwiftpower\.com/profile\.php?z=185005)|[MTCT](https://zwiftpower\.com/team\.php?id=120)|[39th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|116|923|[BJ Alcantara](https://zwiftpower\.com/profile\.php?z=458847)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[39th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|117|921|"[Erik van Kempen \| Trailrunners Brabant](https://zwiftpower\.com/profile\.php?z=829565)"|—|[40th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|117|921|[carrie hennessy](https://zwiftpower\.com/profile\.php?z=947459)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[40th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|117|921|[Lars Magnussen](https://zwiftpower\.com/profile\.php?z=976559)|—|[40th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|117|921|[Ian Jervis5254](https://zwiftpower\.com/profile\.php?z=980837)|—|[40th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|117|921|[Joel Honeycutt](https://zwiftpower\.com/profile\.php?z=1042921)|—|[40th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|117|921|[William Kletzien](https://zwiftpower\.com/profile\.php?z=1055895)|—|[40th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|118|919|[Simon Jackson](https://zwiftpower\.com/profile\.php?z=11130)|—|[41st #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|118|919|[David Robinson](https://zwiftpower\.com/profile\.php?z=230951)|—|[41st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|118|919|[Mr Marcin](https://zwiftpower\.com/profile\.php?z=378083)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[41st #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|119|917|[Suvir Venkataraman](https://zwiftpower\.com/profile\.php?z=487232)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[42nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|119|917|[Jonathan Wyszo](https://zwiftpower\.com/profile\.php?z=496663)|—|[42nd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|119|917|[Ben DeCoste](https://zwiftpower\.com/profile\.php?z=644101)|—|[42nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|119|917|[Krzysztof Kalek](https://zwiftpower\.com/profile\.php?z=894350)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[42nd #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|119|917|[Joey McGuire](https://zwiftpower\.com/profile\.php?z=980538)|—|[42nd #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|119|917|[Eknab Rabbit](https://zwiftpower\.com/profile\.php?z=1148807)|—|[42nd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|120|915|[Ryan Blackmon](https://zwiftpower\.com/profile\.php?z=140272)|—|[43rd #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|120|915|[Christian Pirker \(KC\)](https://zwiftpower\.com/profile\.php?z=304918)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[43rd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|120|915|[Andreas Eriksson](https://zwiftpower\.com/profile\.php?z=874457)|—|[43rd #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|120|915|[Tomasz Chojnowski](https://zwiftpower\.com/profile\.php?z=933278)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[43rd #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|121|913|[Alex Martin\-Verdinos](https://zwiftpower\.com/profile\.php?z=86125)|—|[44th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|121|913|[David Ashton \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=446846)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[44th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|121|913|[Dominic Pridham](https://zwiftpower\.com/profile\.php?z=855322)|—|[44th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|121|913|[Petr Proschl4603](https://zwiftpower\.com/profile\.php?z=975076)|—|[44th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|121|913|[Nic Laughton](https://zwiftpower\.com/profile\.php?z=1038438)|—|[44th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|122|911|[Dominique Louis G\-T](https://zwiftpower\.com/profile\.php?z=15213)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[45th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|122|911|[Steve Holmes 2202](https://zwiftpower\.com/profile\.php?z=101524)|—|[45th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|122|911|[Nate Leader](https://zwiftpower\.com/profile\.php?z=836977)|—|[45th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|122|911|[Lars Amelang](https://zwiftpower\.com/profile\.php?z=1080643)|—|[45th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|123|909|[B Graham](https://zwiftpower\.com/profile\.php?z=58369)|—|[46th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|123|909|[Scott Fenwick \[G\_T\] KISS \(C\)](https://zwiftpower\.com/profile\.php?z=777834)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[46th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|123|909|[Richard Ham](https://zwiftpower\.com/profile\.php?z=948394)|—|[46th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|123|909|[Barry Mark](https://zwiftpower\.com/profile\.php?z=971776)|—|[46th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|124|907|[Andrew BREEZE \(C\)](https://zwiftpower\.com/profile\.php?z=549813)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[47th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|124|907|[John Swaine](https://zwiftpower\.com/profile\.php?z=556486)|[MRSA](https://zwiftpower\.com/team\.php?id=2204)|[47th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|124|907|[Steve Stahl](https://zwiftpower\.com/profile\.php?z=729848)|—|[47th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|125|905|[Ditlev Friis](https://zwiftpower\.com/profile\.php?z=12422)|—|[48th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|125|905|[Joao Cravo](https://zwiftpower\.com/profile\.php?z=160528)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[48th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|125|905|[Adam Lewis \(C\) 5075](https://zwiftpower\.com/profile\.php?z=288749)|—|[48th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
C|125|905|[Keith Mitchell](https://zwiftpower\.com/profile\.php?z=414864)|[The Saints](https://zwiftpower\.com/team\.php?id=3086)|[48th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|125|905|[Matt Packer](https://zwiftpower\.com/profile\.php?z=471752)|—|[48th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|125|905|[Brooke Gibby](https://zwiftpower\.com/profile\.php?z=494306)|—|[48th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|126|903|[Erik van Beusekom HIK](https://zwiftpower\.com/profile\.php?z=254265)|[HIK](https://zwiftpower\.com/team\.php?id=719)|[49th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
C|126|903|[victor choi](https://zwiftpower\.com/profile\.php?z=1102973)|—|[49th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
C|127|901|[Maria Gydesen](https://zwiftpower\.com/profile\.php?z=176130)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[50th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
C|127|901|[Nicky Moir9054](https://zwiftpower\.com/profile\.php?z=896045)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[50th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|127|901|[Achim Neumann \(KC\)](https://zwiftpower\.com/profile\.php?z=1055544)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[50th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|128|899|[Yves Gagnon 8604](https://zwiftpower\.com/profile\.php?z=468035)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[51st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|129|897|[Jezz Ellis](https://zwiftpower\.com/profile\.php?z=10560)|—|[52nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|129|897|[Bruno Gonz Jones](https://zwiftpower\.com/profile\.php?z=1056617)|—|[52nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|130|895|[Matt Deans](https://zwiftpower\.com/profile\.php?z=507330)|—|[53rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|131|893|[Daniel Qr](https://zwiftpower\.com/profile\.php?z=984997)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[54th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|132|891|[Ronald Couture](https://zwiftpower\.com/profile\.php?z=932591)|—|[55th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|132|891|[Tom Lishman\-Walker \[DONNYONTWITCH\]](https://zwiftpower\.com/profile\.php?z=1167472)|—|[55th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|133|889|[john evans](https://zwiftpower\.com/profile\.php?z=243273)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|[56th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|133|889|[Ruben Rocha](https://zwiftpower\.com/profile\.php?z=325023)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[56th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|133|889|[Tom Freitag](https://zwiftpower\.com/profile\.php?z=425174)|—|[56th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|134|887|[Farrell Kilbane \(GKV\)](https://zwiftpower\.com/profile\.php?z=275412)|[WATTBIKE](https://zwiftpower\.com/team\.php?id=2738)|[57th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|134|887|[Brendan Howard](https://zwiftpower\.com/profile\.php?z=972696)|—|[57th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|135|885|[Vin L ⚡](https://zwiftpower\.com/profile\.php?z=868022)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[58th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|135|885|[David Macpherson](https://zwiftpower\.com/profile\.php?z=1161611)|—|[58th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|136|883|[Wade Wilson](https://zwiftpower\.com/profile\.php?z=84455)|—|[59th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
C|136|883|[Lukasz Zielinski](https://zwiftpower\.com/profile\.php?z=86725)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[59th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|136|883|[David LE GONIDEC](https://zwiftpower\.com/profile\.php?z=807025)|[Team France](https://zwiftpower\.com/team\.php?id=574)|[59th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|137|881|[Francois Sammut](https://zwiftpower\.com/profile\.php?z=108090)|—|[60th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|137|881|[raspin raspin](https://zwiftpower\.com/profile\.php?z=879744)|—|[60th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|138|879|[Fredrik Söderbäck](https://zwiftpower\.com/profile\.php?z=1127377)|—|[61st #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|139|877|[Oliver Kraft \(ZRG\)](https://zwiftpower\.com/profile\.php?z=593461)|—|[62nd #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|139|877|[Krzysiek Cierniak](https://zwiftpower\.com/profile\.php?z=627395)|—|[62nd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|139|877|[Joshua Gross](https://zwiftpower\.com/profile\.php?z=1107507)|—|[62nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|140|875|[Paige Haydon Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=311565)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[63rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
C|140|875|[Frank Schaudig](https://zwiftpower\.com/profile\.php?z=447596)|—|[63rd #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|141|873|[Ralf D\.](https://zwiftpower\.com/profile\.php?z=315260)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[64th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
C|141|873|[Frank Göldner](https://zwiftpower\.com/profile\.php?z=535460)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[64th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
C|142|871|[Hans VEGAN](https://zwiftpower\.com/profile\.php?z=422779)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[65th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
D|1|5966|[Marcus Pfurtscheller](https://zwiftpower\.com/profile\.php?z=1156098)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥇 #4](https://zwiftpower.com/events.php?zid=162691)|[🥈 #2](https://zwiftpower.com/events.php?zid=155628)|[🥉 #6](https://zwiftpower.com/events.php?zid=171935)|[🥉 #8](https://zwiftpower.com/events.php?zid=176148)|[5th #3](https://zwiftpower.com/events.php?zid=159334)|[6th #7](https://zwiftpower.com/events.php?zid=173451)
D|2|4923|[Jim Peyton](https://zwiftpower\.com/profile\.php?z=485205)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th #6](https://zwiftpower.com/events.php?zid=171935)|[7th #4](https://zwiftpower.com/events.php?zid=162691)|[8th #7](https://zwiftpower.com/events.php?zid=173451)|[8th #8](https://zwiftpower.com/events.php?zid=176148)|[14th #5](https://zwiftpower.com/events.php?zid=166630)|—
D|3|3938|[David Ashton \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=446846)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[6th #2](https://zwiftpower.com/events.php?zid=155628)|[6th #6](https://zwiftpower.com/events.php?zid=171935)|[10th #4](https://zwiftpower.com/events.php?zid=162691)|[11th #7](https://zwiftpower.com/events.php?zid=173451)|—|—
D|4|3934|[🛴BJ AFONSO🚲](https://zwiftpower\.com/profile\.php?z=11775)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[4th #8](https://zwiftpower.com/events.php?zid=176148)|[6th #4](https://zwiftpower.com/events.php?zid=162691)|[10th #7](https://zwiftpower.com/events.php?zid=173451)|[15th #5](https://zwiftpower.com/events.php?zid=166630)|—|—
D|5|3912|[Alex Cutmore \[TPH\]](https://zwiftpower\.com/profile\.php?z=1155359)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[9th #1](https://zwiftpower.com/events.php?zid=152778)|[10th #8](https://zwiftpower.com/events.php?zid=176148)|[12th #4](https://zwiftpower.com/events.php?zid=162691)|[15th #6](https://zwiftpower.com/events.php?zid=171935)|—|—
D|6|2993|[Victor Moldoveanu](https://zwiftpower\.com/profile\.php?z=824632)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥇 #2](https://zwiftpower.com/events.php?zid=155628)|[🥈 #3](https://zwiftpower.com/events.php?zid=159334)|[🥈 #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—
D|7|2985|[Joao Cravo](https://zwiftpower\.com/profile\.php?z=160528)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥇 #7](https://zwiftpower.com/events.php?zid=173451)|[🥉 #2](https://zwiftpower.com/events.php?zid=155628)|[5th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
D|8|2965|[Oldstuntmonkey MTB Some Thieving Bastards Stole My Bikes](https://zwiftpower\.com/profile\.php?z=292646)|—|[4th #3](https://zwiftpower.com/events.php?zid=159334)|[4th #7](https://zwiftpower.com/events.php?zid=173451)|[11th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—
D|9|2905|[Alex Diaz](https://zwiftpower\.com/profile\.php?z=408631)|—|[10th #3](https://zwiftpower.com/events.php?zid=159334)|[16th #6](https://zwiftpower.com/events.php?zid=171935)|[23rd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—
D|10|1996|[Andy Beveridge TFC](https://zwiftpower\.com/profile\.php?z=51283)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #8](https://zwiftpower.com/events.php?zid=176148)|[🥈 #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—
D|11|1990|[Gonçalo Afonso](https://zwiftpower\.com/profile\.php?z=355653)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥈 #5](https://zwiftpower.com/events.php?zid=166630)|[4th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
D|12|1986|[dave boudreau](https://zwiftpower\.com/profile\.php?z=14667)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥉 #7](https://zwiftpower.com/events.php?zid=173451)|[5th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
D|13|1984|[Oli Masters](https://zwiftpower\.com/profile\.php?z=948112)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #6](https://zwiftpower.com/events.php?zid=171935)|[7th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|14|1980|[Jeff\.Edwards\.](https://zwiftpower\.com/profile\.php?z=1450)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th #1](https://zwiftpower.com/events.php?zid=152778)|[6th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|15|1976|[Lee Barber](https://zwiftpower\.com/profile\.php?z=250163)|—|[6th #1](https://zwiftpower.com/events.php?zid=152778)|[7th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—
D|15|1976|[Mike Louie](https://zwiftpower\.com/profile\.php?z=261230)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[5th #2](https://zwiftpower.com/events.php?zid=155628)|[8th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—
D|16|1972|[Rob Jessen CRYO\-GEN \(D1\)](https://zwiftpower\.com/profile\.php?z=332069)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[7th #7](https://zwiftpower.com/events.php?zid=173451)|[8th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|17|1970|[Al Mac GTC](https://zwiftpower\.com/profile\.php?z=973293)|[GTC](https://zwiftpower\.com/team\.php?id=3736)|[7th #6](https://zwiftpower.com/events.php?zid=171935)|[9th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—
D|18|1964|[Damon McKay](https://zwiftpower\.com/profile\.php?z=3508)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #6](https://zwiftpower.com/events.php?zid=171935)|[18th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|19|1962|[Ann\-Katrin Kuzmenko \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=871472)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[8th #6](https://zwiftpower.com/events.php?zid=171935)|[12th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|20|1948|[Rich Collins](https://zwiftpower\.com/profile\.php?z=558894)|—|[7th #1](https://zwiftpower.com/events.php?zid=152778)|[20th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|21|1946|[Wolfgang Becker KC](https://zwiftpower\.com/profile\.php?z=379505)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[11th #4](https://zwiftpower.com/events.php?zid=162691)|[17th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—
D|22|999|[Zorza Von K](https://zwiftpower\.com/profile\.php?z=231903)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥇 #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|22|999|[Paulo Monteiro](https://zwiftpower\.com/profile\.php?z=631207)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥇 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
D|22|999|[Stephen Hunt](https://zwiftpower\.com/profile\.php?z=863229)|—|[🥇 #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|23|997|[Lee Robinson TFC](https://zwiftpower\.com/profile\.php?z=130504)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥈 #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
D|23|997|[Petr Proschl4603](https://zwiftpower\.com/profile\.php?z=975076)|—|[🥈 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
D|24|995|[Martin Björgvik 🐌](https://zwiftpower\.com/profile\.php?z=15197)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[🥉 #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|24|995|[Tom Atkinson](https://zwiftpower\.com/profile\.php?z=337225)|—|[🥉 #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
D|24|995|[Ole\-Martin Andersen](https://zwiftpower\.com/profile\.php?z=560581)|—|[🥉 #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|24|995|[Marek Milkamanowicz](https://zwiftpower\.com/profile\.php?z=889003)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[🥉 #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
D|25|993|[Ralf D\.](https://zwiftpower\.com/profile\.php?z=315260)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[4th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|25|993|[Achim Neumann \(KC\)](https://zwiftpower\.com/profile\.php?z=1055544)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[4th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
D|25|993|[Jonas Bo Madsen \[DBR\]](https://zwiftpower\.com/profile\.php?z=1109805)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[4th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|26|991|[Peter Walsh](https://zwiftpower\.com/profile\.php?z=136151)|—|[5th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
D|26|991|[David Jones](https://zwiftpower\.com/profile\.php?z=858171)|—|[5th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|26|991|[Marty Anfinsen](https://zwiftpower\.com/profile\.php?z=1137448)|—|[5th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
D|27|989|[Richard McCulloch](https://zwiftpower\.com/profile\.php?z=50300)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[6th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
D|27|989|[$nail W](https://zwiftpower\.com/profile\.php?z=250028)|—|[6th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|28|987|[Aaron Burns](https://zwiftpower\.com/profile\.php?z=738877)|[The Saints](https://zwiftpower\.com/team\.php?id=3086)|[7th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|28|987|[Eric Nadin](https://zwiftpower\.com/profile\.php?z=928144)|—|[7th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|29|985|[Xavier Laroche](https://zwiftpower\.com/profile\.php?z=427658)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[8th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
D|29|985|[Chris Blount](https://zwiftpower\.com/profile\.php?z=583514)|—|[8th #1](https://zwiftpower.com/events.php?zid=152778)|—|—|—|—|—
D|29|985|[CJ Spencer 2511](https://zwiftpower\.com/profile\.php?z=886960)|—|[8th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|30|983|[David Lopez](https://zwiftpower\.com/profile\.php?z=402705)|—|[9th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
D|30|983|[Magnus O AHLSEN Ahlsé](https://zwiftpower\.com/profile\.php?z=417135)|—|[9th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|30|983|[Joseph Padilla \[HERD\]](https://zwiftpower\.com/profile\.php?z=923771)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|[9th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|30|983|[John Brice](https://zwiftpower\.com/profile\.php?z=995905)|[PACK](https://zwiftpower\.com/team\.php?id=2928)|[9th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
D|30|983|[Marlene Ott \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=1006971)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[9th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|31|981|[Chris Taylor](https://zwiftpower\.com/profile\.php?z=407450)|—|[10th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|31|981|[Nigel Savage](https://zwiftpower\.com/profile\.php?z=953423)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[10th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|31|981|[L Young](https://zwiftpower\.com/profile\.php?z=973891)|—|[10th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|32|979|[Zoltán Fazekas](https://zwiftpower\.com/profile\.php?z=450622)|—|[11th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|32|979|[Anthony Darragh](https://zwiftpower\.com/profile\.php?z=933793)|—|[11th #8](https://zwiftpower.com/events.php?zid=176148)|—|—|—|—|—
D|32|979|[John Qertz 7842](https://zwiftpower\.com/profile\.php?z=1023771)|—|[11th #3](https://zwiftpower.com/events.php?zid=159334)|—|—|—|—|—
D|32|979|[Adam Herring](https://zwiftpower\.com/profile\.php?z=1095145)|—|[11th #2](https://zwiftpower.com/events.php?zid=155628)|—|—|—|—|—
D|33|977|[steve davis 8027](https://zwiftpower\.com/profile\.php?z=237249)|[SGCC](https://zwiftpower\.com/team\.php?id=3149)|[12th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|33|977|[Blair Cottreau](https://zwiftpower\.com/profile\.php?z=1044214)|—|[12th #7](https://zwiftpower.com/events.php?zid=173451)|—|—|—|—|—
D|34|975|[Adam Schwartz2600](https://zwiftpower\.com/profile\.php?z=2398)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[13th #4](https://zwiftpower.com/events.php?zid=162691)|—|—|—|—|—
D|34|975|[Dário Ferreira \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=910139)|—|[13th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|34|975|[Marty Doscher](https://zwiftpower\.com/profile\.php?z=927833)|—|[13th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|35|973|[Achim Butschek \(KC\)](https://zwiftpower\.com/profile\.php?z=80942)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[14th #6](https://zwiftpower.com/events.php?zid=171935)|—|—|—|—|—
D|36|969|[Hein Wick](https://zwiftpower\.com/profile\.php?z=51696)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[16th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|37|963|[David Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=410620)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[19th #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|38|959|[Miguel Remízio \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=1119738)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[21st #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—
D|39|957|[Patrick D\.](https://zwiftpower\.com/profile\.php?z=372550)|—|[22nd #5](https://zwiftpower.com/events.php?zid=166630)|—|—|—|—|—



## KISS Community League — Europe Women

Individual rider rankings by top 6 results.

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6
---|---|---|---|---|---|---|---|---|---|---
A|1|5980|[Cecilia Hansen👽🔰](https://zwiftpower\.com/profile\.php?z=6306)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇 #4](https://zwiftpower.com/events.php?zid=162692)|[🥈 #2](https://zwiftpower.com/events.php?zid=155629)|[🥈 #6](https://zwiftpower.com/events.php?zid=171936)|[🥈 #8](https://zwiftpower.com/events.php?zid=176149)|[🥉 #1](https://zwiftpower.com/events.php?zid=154308)|[🥉 #3](https://zwiftpower.com/events.php?zid=159335)
A|2|5972|[Vicki Whitelaw](https://zwiftpower\.com/profile\.php?z=372588)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇 #5](https://zwiftpower.com/events.php?zid=166629)|[🥈 #1](https://zwiftpower.com/events.php?zid=154308)|[🥈 #3](https://zwiftpower.com/events.php?zid=159335)|[🥉 #6](https://zwiftpower.com/events.php?zid=171936)|[4th #8](https://zwiftpower.com/events.php?zid=176149)|[5th #2](https://zwiftpower.com/events.php?zid=155629)
A|3|5946|[Kristin Falck](https://zwiftpower\.com/profile\.php?z=56138)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[4th #2](https://zwiftpower.com/events.php?zid=155629)|[4th #6](https://zwiftpower.com/events.php?zid=171936)|[5th #4](https://zwiftpower.com/events.php?zid=162692)|[5th #7](https://zwiftpower.com/events.php?zid=173447)|[5th #8](https://zwiftpower.com/events.php?zid=176149)|[7th #1](https://zwiftpower.com/events.php?zid=154308)
A|4|5886|[Laura van Regenmortel](https://zwiftpower\.com/profile\.php?z=376739)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[7th #7](https://zwiftpower.com/events.php?zid=173447)|[9th #6](https://zwiftpower.com/events.php?zid=171936)|[10th #1](https://zwiftpower.com/events.php?zid=154308)|[10th #3](https://zwiftpower.com/events.php?zid=159335)|[11th #8](https://zwiftpower.com/events.php?zid=176149)|[13th #2](https://zwiftpower.com/events.php?zid=155629)
A|5|5876|[Siri Hildonen \[CANYON\]](https://zwiftpower\.com/profile\.php?z=12501)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[6th #6](https://zwiftpower.com/events.php?zid=171936)|[8th #7](https://zwiftpower.com/events.php?zid=173447)|[9th #8](https://zwiftpower.com/events.php?zid=176149)|[11th #1](https://zwiftpower.com/events.php?zid=154308)|[15th #5](https://zwiftpower.com/events.php?zid=166629)|[16th #3](https://zwiftpower.com/events.php?zid=159335)
A|6|5866|[Lea Milleu](https://zwiftpower\.com/profile\.php?z=470626)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[7th #4](https://zwiftpower.com/events.php?zid=162692)|[8th #8](https://zwiftpower.com/events.php?zid=176149)|[10th #7](https://zwiftpower.com/events.php?zid=173447)|[13th #5](https://zwiftpower.com/events.php?zid=166629)|[13th #6](https://zwiftpower.com/events.php?zid=171936)|[19th #2](https://zwiftpower.com/events.php?zid=155629)
A|7|5856|[Leah Brewer](https://zwiftpower\.com/profile\.php?z=317804)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[9th #4](https://zwiftpower.com/events.php?zid=162692)|[10th #6](https://zwiftpower.com/events.php?zid=171936)|[12th #5](https://zwiftpower.com/events.php?zid=166629)|[14th #2](https://zwiftpower.com/events.php?zid=155629)|[15th #1](https://zwiftpower.com/events.php?zid=154308)|[15th #8](https://zwiftpower.com/events.php?zid=176149)
A|8|5848|[Lois Brewer](https://zwiftpower\.com/profile\.php?z=317074)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[10th #5](https://zwiftpower.com/events.php?zid=166629)|[11th #6](https://zwiftpower.com/events.php?zid=171936)|[13th #4](https://zwiftpower.com/events.php?zid=162692)|[14th #3](https://zwiftpower.com/events.php?zid=159335)|[14th #8](https://zwiftpower.com/events.php?zid=176149)|[17th #1](https://zwiftpower.com/events.php?zid=154308)
A|9|4991|[Louise Houbak](https://zwiftpower\.com/profile\.php?z=625825)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇 #1](https://zwiftpower.com/events.php?zid=154308)|[🥇 #2](https://zwiftpower.com/events.php?zid=155629)|[🥇 #3](https://zwiftpower.com/events.php?zid=159335)|[🥇 #7](https://zwiftpower.com/events.php?zid=173447)|[🥉 #4](https://zwiftpower.com/events.php?zid=162692)|—
A|10|4959|[Nathalie Eklund](https://zwiftpower\.com/profile\.php?z=185030)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[🥈 #4](https://zwiftpower.com/events.php?zid=162692)|[4th #3](https://zwiftpower.com/events.php?zid=159335)|[5th #1](https://zwiftpower.com/events.php?zid=154308)|[6th #2](https://zwiftpower.com/events.php?zid=155629)|[6th #8](https://zwiftpower.com/events.php?zid=176149)|—
A|11|4919|[Polly Mason](https://zwiftpower\.com/profile\.php?z=136220)|—|[5th #6](https://zwiftpower.com/events.php?zid=171936)|[7th #5](https://zwiftpower.com/events.php?zid=166629)|[9th #7](https://zwiftpower.com/events.php?zid=173447)|[10th #8](https://zwiftpower.com/events.php?zid=176149)|[12th #4](https://zwiftpower.com/events.php?zid=162692)|—
A|12|4881|[Clare Rhead](https://zwiftpower\.com/profile\.php?z=301757)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[9th #1](https://zwiftpower.com/events.php?zid=154308)|[10th #4](https://zwiftpower.com/events.php?zid=162692)|[13th #3](https://zwiftpower.com/events.php?zid=159335)|[14th #6](https://zwiftpower.com/events.php?zid=171936)|[16th #2](https://zwiftpower.com/events.php?zid=155629)|—
A|13|4847|[Caroline Selin](https://zwiftpower\.com/profile\.php?z=166443)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[13th #7](https://zwiftpower.com/events.php?zid=173447)|[15th #6](https://zwiftpower.com/events.php?zid=171936)|[16th #4](https://zwiftpower.com/events.php?zid=162692)|[17th #5](https://zwiftpower.com/events.php?zid=166629)|[18th #8](https://zwiftpower.com/events.php?zid=176149)|—
A|14|3952|[Moin Moin](https://zwiftpower\.com/profile\.php?z=417656)|—|[4th #4](https://zwiftpower.com/events.php?zid=162692)|[6th #1](https://zwiftpower.com/events.php?zid=154308)|[6th #3](https://zwiftpower.com/events.php?zid=159335)|[10th #2](https://zwiftpower.com/events.php?zid=155629)|—|—
A|15|3934|[Jennifer Real](https://zwiftpower\.com/profile\.php?z=360293)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[7th #8](https://zwiftpower.com/events.php?zid=176149)|[8th #1](https://zwiftpower.com/events.php?zid=154308)|[8th #2](https://zwiftpower.com/events.php?zid=155629)|[12th #7](https://zwiftpower.com/events.php?zid=173447)|—|—
A|16|3902|[Catherine Allen](https://zwiftpower\.com/profile\.php?z=281872)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[9th #5](https://zwiftpower.com/events.php?zid=166629)|[12th #8](https://zwiftpower.com/events.php?zid=176149)|[14th #7](https://zwiftpower.com/events.php?zid=173447)|[16th #6](https://zwiftpower.com/events.php?zid=171936)|—|—
A|17|3880|[Inge Jansen](https://zwiftpower\.com/profile\.php?z=126110)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[15th #3](https://zwiftpower.com/events.php?zid=159335)|[15th #4](https://zwiftpower.com/events.php?zid=162692)|[16th #1](https://zwiftpower.com/events.php?zid=154308)|[16th #5](https://zwiftpower.com/events.php?zid=166629)|—|—
A|18|2995|[D aisy B](https://zwiftpower\.com/profile\.php?z=1105834)|—|[🥇 #6](https://zwiftpower.com/events.php?zid=171936)|[🥇 #8](https://zwiftpower.com/events.php?zid=176149)|[🥈 #7](https://zwiftpower.com/events.php?zid=173447)|—|—|—
A|19|2947|[Eva\-Cathérine Buchholz 🦅](https://zwiftpower\.com/profile\.php?z=438555)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[8th #6](https://zwiftpower.com/events.php?zid=171936)|[9th #3](https://zwiftpower.com/events.php?zid=159335)|[11th #7](https://zwiftpower.com/events.php?zid=173447)|—|—|—
A|20|2939|[Kjersti Hoel](https://zwiftpower\.com/profile\.php?z=169175)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[8th #4](https://zwiftpower.com/events.php?zid=162692)|[11th #3](https://zwiftpower.com/events.php?zid=159335)|[13th #1](https://zwiftpower.com/events.php?zid=154308)|—|—|—
A|21|2935|[Monica Dew \[WAHOOCH\]](https://zwiftpower\.com/profile\.php?z=1180352)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[7th #6](https://zwiftpower.com/events.php?zid=171936)|[13th #8](https://zwiftpower.com/events.php?zid=176149)|[14th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—
A|22|2923|[Kerstin Pöhl](https://zwiftpower\.com/profile\.php?z=458121)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[11th #4](https://zwiftpower.com/events.php?zid=162692)|[12th #3](https://zwiftpower.com/events.php?zid=159335)|[17th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—
A|23|2905|[Katie Kookaburra](https://zwiftpower\.com/profile\.php?z=405642)|—|[16th #7](https://zwiftpower.com/events.php?zid=173447)|[16th #8](https://zwiftpower.com/events.php?zid=176149)|[17th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—
A|24|2891|[Haddi Conant](https://zwiftpower\.com/profile\.php?z=1141836)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[18th #6](https://zwiftpower.com/events.php?zid=171936)|[19th #5](https://zwiftpower.com/events.php?zid=166629)|[19th #8](https://zwiftpower.com/events.php?zid=176149)|—|—|—
A|25|1984|[Istrate Daniela](https://zwiftpower\.com/profile\.php?z=1056287)|—|[🥉 #8](https://zwiftpower.com/events.php?zid=176149)|[6th #7](https://zwiftpower.com/events.php?zid=173447)|—|—|—|—
A|26|1982|[Cassie Baldi](https://zwiftpower\.com/profile\.php?z=13266)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[🥉 #7](https://zwiftpower.com/events.php?zid=173447)|[7th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—
A|27|1980|[sylvie boermans \[X\]](https://zwiftpower\.com/profile\.php?z=100763)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[🥈 #5](https://zwiftpower.com/events.php?zid=166629)|[9th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—
A|28|1978|[Rachael Elliott](https://zwiftpower\.com/profile\.php?z=80103)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[4th #1](https://zwiftpower.com/events.php?zid=154308)|[8th #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—
A|29|1956|[Superwomen Sonia](https://zwiftpower\.com/profile\.php?z=626757)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[11th #5](https://zwiftpower.com/events.php?zid=166629)|[12th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—
A|30|1936|[Emily Slavin](https://zwiftpower\.com/profile\.php?z=385268)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[15th #7](https://zwiftpower.com/events.php?zid=173447)|[18th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—
A|31|1930|[Maddy Lee\-Smith](https://zwiftpower\.com/profile\.php?z=609300)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[14th #1](https://zwiftpower.com/events.php?zid=154308)|[22nd #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—
A|32|1924|[Kimberly Hoekstra](https://zwiftpower\.com/profile\.php?z=1122394)|—|[18th #1](https://zwiftpower.com/events.php?zid=154308)|[21st #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—
A|33|995|[Mary Wilkinson](https://zwiftpower\.com/profile\.php?z=150208)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉 #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
A|34|993|[Kerry Speers](https://zwiftpower\.com/profile\.php?z=20533)|—|[4th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
A|35|991|[Claudia Behring 🐻🔰](https://zwiftpower\.com/profile\.php?z=251988)|—|[5th #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—|—
A|35|991|[Caroline Murray](https://zwiftpower\.com/profile\.php?z=566158)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[5th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
A|36|989|[Leah Dixon](https://zwiftpower\.com/profile\.php?z=686696)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[6th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
A|37|979|[Illi Gardner](https://zwiftpower\.com/profile\.php?z=427078)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[11th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
A|38|977|[Aysegul Gokalp \[FEAR<\]](https://zwiftpower\.com/profile\.php?z=489939)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[12th #1](https://zwiftpower.com/events.php?zid=154308)|—|—|—|—|—
A|38|977|[Maria Ossiansson](https://zwiftpower\.com/profile\.php?z=1046923)|—|[12th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—|—|—
A|39|973|[Penny Priest ☁](https://zwiftpower\.com/profile\.php?z=199305)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[14th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—|—
A|40|971|[Emma Cockcroft](https://zwiftpower\.com/profile\.php?z=49469)|—|[15th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
A|41|967|[Manon Lannergan](https://zwiftpower\.com/profile\.php?z=70362)|—|[17th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—|—
A|41|967|[Laura Epelbaum](https://zwiftpower\.com/profile\.php?z=377170)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[17th #8](https://zwiftpower.com/events.php?zid=176149)|—|—|—|—|—
A|42|965|[Nina Gilljam](https://zwiftpower\.com/profile\.php?z=189774)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[18th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
A|42|965|[Stephanie Ossenbrink](https://zwiftpower\.com/profile\.php?z=479580)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|[18th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
A|43|963|[Sil Cal \[REVO\]](https://zwiftpower\.com/profile\.php?z=447517)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|[19th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—|—|—
B|1|5968|[Debbie Stockley \(Race3R\)](https://zwiftpower\.com/profile\.php?z=320052)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[🥇 #6](https://zwiftpower.com/events.php?zid=171936)|[🥈 #8](https://zwiftpower.com/events.php?zid=176149)|[🥉 #5](https://zwiftpower.com/events.php?zid=166629)|[🥉 #7](https://zwiftpower.com/events.php?zid=173447)|[4th #4](https://zwiftpower.com/events.php?zid=162692)|[6th #3](https://zwiftpower.com/events.php?zid=159335)
B|2|5964|[sonia insley](https://zwiftpower\.com/profile\.php?z=70830)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[🥇 #7](https://zwiftpower.com/events.php?zid=173447)|[🥈 #2](https://zwiftpower.com/events.php?zid=155629)|[🥉 #1](https://zwiftpower.com/events.php?zid=154308)|[🥉 #6](https://zwiftpower.com/events.php?zid=171936)|[5th #5](https://zwiftpower.com/events.php?zid=166629)|[7th #4](https://zwiftpower.com/events.php?zid=162692)
B|3|5962|[Frederique Hate](https://zwiftpower\.com/profile\.php?z=94641)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥈 #5](https://zwiftpower.com/events.php?zid=166629)|[🥈 #6](https://zwiftpower.com/events.php?zid=171936)|[🥉 #2](https://zwiftpower.com/events.php?zid=155629)|[🥉 #8](https://zwiftpower.com/events.php?zid=176149)|[4th #1](https://zwiftpower.com/events.php?zid=154308)|[8th #3](https://zwiftpower.com/events.php?zid=159335)
B|4|5954|[C Scott](https://zwiftpower\.com/profile\.php?z=340007)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[🥇 #1](https://zwiftpower.com/events.php?zid=154308)|[🥉 #4](https://zwiftpower.com/events.php?zid=162692)|[4th #7](https://zwiftpower.com/events.php?zid=173447)|[5th #3](https://zwiftpower.com/events.php?zid=159335)|[6th #5](https://zwiftpower.com/events.php?zid=166629)|[7th #8](https://zwiftpower.com/events.php?zid=176149)
B|5|5922|[Cheryl Muldoon 🥦](https://zwiftpower\.com/profile\.php?z=113057)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[5th #7](https://zwiftpower.com/events.php?zid=173447)|[6th #4](https://zwiftpower.com/events.php?zid=162692)|[7th #1](https://zwiftpower.com/events.php?zid=154308)|[7th #5](https://zwiftpower.com/events.php?zid=166629)|[8th #2](https://zwiftpower.com/events.php?zid=155629)|[9th #3](https://zwiftpower.com/events.php?zid=159335)
B|6|4917|[Martha Kuntz](https://zwiftpower\.com/profile\.php?z=448610)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[6th #6](https://zwiftpower.com/events.php?zid=171936)|[7th #2](https://zwiftpower.com/events.php?zid=155629)|[7th #7](https://zwiftpower.com/events.php?zid=173447)|[12th #3](https://zwiftpower.com/events.php?zid=159335)|[12th #4](https://zwiftpower.com/events.php?zid=162692)|—
B|7|3968|[Haddi Conant](https://zwiftpower\.com/profile\.php?z=1141836)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[🥉 #3](https://zwiftpower.com/events.php?zid=159335)|[5th #1](https://zwiftpower.com/events.php?zid=154308)|[5th #2](https://zwiftpower.com/events.php?zid=155629)|[5th #4](https://zwiftpower.com/events.php?zid=162692)|—|—
B|8|3966|[Siv Engmo](https://zwiftpower\.com/profile\.php?z=217993)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[🥈 #1](https://zwiftpower.com/events.php?zid=154308)|[🥈 #7](https://zwiftpower.com/events.php?zid=173447)|[7th #3](https://zwiftpower.com/events.php?zid=159335)|[8th #4](https://zwiftpower.com/events.php?zid=162692)|—|—
B|9|2991|[Line Heerup Madsen](https://zwiftpower\.com/profile\.php?z=927830)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=162692)|[🥇 #8](https://zwiftpower.com/events.php?zid=176149)|[4th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—
B|10|2989|[C\. Harris Momsen](https://zwiftpower\.com/profile\.php?z=719263)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥇 #5](https://zwiftpower.com/events.php?zid=166629)|[🥈 #4](https://zwiftpower.com/events.php?zid=162692)|[4th #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—
B|11|2965|[Charlotte Watts](https://zwiftpower\.com/profile\.php?z=617677)|—|[4th #6](https://zwiftpower.com/events.php?zid=171936)|[6th #7](https://zwiftpower.com/events.php?zid=173447)|[9th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—
B|12|1984|[Fran Bambust](https://zwiftpower\.com/profile\.php?z=427114)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|[4th #8](https://zwiftpower.com/events.php?zid=176149)|[5th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—|—
B|13|1974|[Silje Holmsen \[REVO\] B](https://zwiftpower\.com/profile\.php?z=188434)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|[6th #8](https://zwiftpower.com/events.php?zid=176149)|[8th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—
B|14|1970|[Janis Denning KoS](https://zwiftpower\.com/profile\.php?z=152749)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[6th #2](https://zwiftpower.com/events.php?zid=155629)|[10th #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—
B|15|1968|[Sylvie Holmes ZSUN](https://zwiftpower\.com/profile\.php?z=87284)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[6th #1](https://zwiftpower.com/events.php?zid=154308)|[11th #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—
B|16|1956|[Tina Berger\-Schauer](https://zwiftpower\.com/profile\.php?z=471560)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[10th #5](https://zwiftpower.com/events.php?zid=166629)|[13th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—
B|17|999|[Anja Leuendorff](https://zwiftpower\.com/profile\.php?z=170027)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
B|17|999|[Jessica Hamilton](https://zwiftpower\.com/profile\.php?z=653395)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥇 #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—|—
B|18|997|[Leah Brewer](https://zwiftpower\.com/profile\.php?z=317804)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥈 #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—|—|—
B|19|993|[Stephanie Wendel \(KC\)](https://zwiftpower\.com/profile\.php?z=189986)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[4th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
B|20|991|[Liwi Feuerzahn](https://zwiftpower\.com/profile\.php?z=1212166)|—|[5th #8](https://zwiftpower.com/events.php?zid=176149)|—|—|—|—|—
B|21|985|[Nicola Hogan](https://zwiftpower\.com/profile\.php?z=635868)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[8th #1](https://zwiftpower.com/events.php?zid=154308)|—|—|—|—|—
B|22|983|[Gillian Leech](https://zwiftpower\.com/profile\.php?z=59724)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[9th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—|—
B|22|983|[Becky Loberg Youngberg](https://zwiftpower\.com/profile\.php?z=421396)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[9th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
B|23|981|[Katarina Augustin INC](https://zwiftpower\.com/profile\.php?z=383329)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[10th #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—|—
C|1|5990|[Karissa Minn](https://zwiftpower\.com/profile\.php?z=39406)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #1](https://zwiftpower.com/events.php?zid=154308)|[🥇 #4](https://zwiftpower.com/events.php?zid=162692)|[🥇 #6](https://zwiftpower.com/events.php?zid=171936)|[🥇 #7](https://zwiftpower.com/events.php?zid=173447)|[🥈 #5](https://zwiftpower.com/events.php?zid=166629)|[🥈 #8](https://zwiftpower.com/events.php?zid=176149)
C|2|5972|[Tina Boine\-Frankenheim](https://zwiftpower\.com/profile\.php?z=107059)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥈 #2](https://zwiftpower.com/events.php?zid=155629)|[🥈 #3](https://zwiftpower.com/events.php?zid=159335)|[🥈 #6](https://zwiftpower.com/events.php?zid=171936)|[🥉 #7](https://zwiftpower.com/events.php?zid=173447)|[4th #5](https://zwiftpower.com/events.php?zid=166629)|[4th #8](https://zwiftpower.com/events.php?zid=176149)
C|3|4963|[Kimberley Gray](https://zwiftpower\.com/profile\.php?z=327961)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #3](https://zwiftpower.com/events.php?zid=159335)|[🥉 #2](https://zwiftpower.com/events.php?zid=155629)|[4th #4](https://zwiftpower.com/events.php?zid=162692)|[6th #6](https://zwiftpower.com/events.php?zid=171936)|[7th #8](https://zwiftpower.com/events.php?zid=176149)|—
C|4|3988|[Vicky Steadman](https://zwiftpower\.com/profile\.php?z=506303)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #8](https://zwiftpower.com/events.php?zid=176149)|[🥈 #4](https://zwiftpower.com/events.php?zid=162692)|[🥈 #7](https://zwiftpower.com/events.php?zid=173447)|[🥉 #5](https://zwiftpower.com/events.php?zid=166629)|—|—
C|5|3974|[Lucy Jay \[TPH\]](https://zwiftpower\.com/profile\.php?z=986803)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[🥉 #6](https://zwiftpower.com/events.php?zid=171936)|[🥉 #8](https://zwiftpower.com/events.php?zid=176149)|[4th #7](https://zwiftpower.com/events.php?zid=173447)|[5th #4](https://zwiftpower.com/events.php?zid=162692)|—|—
C|6|3970|[Julia Heckmann \(KC & tritime women\)](https://zwiftpower\.com/profile\.php?z=917510)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥉 #3](https://zwiftpower.com/events.php?zid=159335)|[4th #2](https://zwiftpower.com/events.php?zid=155629)|[5th #5](https://zwiftpower.com/events.php?zid=166629)|[5th #8](https://zwiftpower.com/events.php?zid=176149)|—|—
C|7|2977|[Ruth Stapleton SNCC](https://zwiftpower\.com/profile\.php?z=572213)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th #1](https://zwiftpower.com/events.php?zid=154308)|[4th #3](https://zwiftpower.com/events.php?zid=159335)|[5th #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—
C|8|2969|[Mandy McCoy](https://zwiftpower\.com/profile\.php?z=334424)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th #1](https://zwiftpower.com/events.php?zid=154308)|[6th #4](https://zwiftpower.com/events.php?zid=162692)|[6th #8](https://zwiftpower.com/events.php?zid=176149)|—|—|—
C|9|999|[tracking tracey](https://zwiftpower\.com/profile\.php?z=79911)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇 #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
C|9|999|[Nicole Ternel](https://zwiftpower\.com/profile\.php?z=135833)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
C|10|997|[Gunhild Westergaard](https://zwiftpower\.com/profile\.php?z=1120212)|—|[🥈 #1](https://zwiftpower.com/events.php?zid=154308)|—|—|—|—|—
C|11|995|[Ann\-Kathrin Bach \(ZRG\)](https://zwiftpower\.com/profile\.php?z=944139)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[🥉 #1](https://zwiftpower.com/events.php?zid=154308)|—|—|—|—|—
C|11|995|[Anne Loh](https://zwiftpower\.com/profile\.php?z=944358)|—|[🥉 #4](https://zwiftpower.com/events.php?zid=162692)|—|—|—|—|—
C|12|993|[a kent](https://zwiftpower\.com/profile\.php?z=693532)|—|[4th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—|—|—
C|13|991|[I Hall](https://zwiftpower\.com/profile\.php?z=944133)|—|[5th #7](https://zwiftpower.com/events.php?zid=173447)|—|—|—|—|—
C|13|991|[Heather Walke7128](https://zwiftpower\.com/profile\.php?z=1080715)|—|[5th #6](https://zwiftpower.com/events.php?zid=171936)|—|—|—|—|—
C|14|989|[A ZSUN Goat](https://zwiftpower\.com/profile\.php?z=346294)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[6th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
C|15|987|[Mariet Feltkamp](https://zwiftpower\.com/profile\.php?z=565539)|—|[7th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
C|16|985|[Kate Cronin](https://zwiftpower\.com/profile\.php?z=340509)|—|[8th #8](https://zwiftpower.com/events.php?zid=176149)|—|—|—|—|—
C|16|985|[jude woodcock BRT Hellcatz](https://zwiftpower\.com/profile\.php?z=659665)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[8th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—
D|1|4991|[Emma Lambert](https://zwiftpower\.com/profile\.php?z=19596)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #4](https://zwiftpower.com/events.php?zid=162692)|[🥇 #6](https://zwiftpower.com/events.php?zid=171936)|[🥇 #8](https://zwiftpower.com/events.php?zid=176149)|[🥈 #5](https://zwiftpower.com/events.php?zid=166629)|[🥈 #7](https://zwiftpower.com/events.php?zid=173447)|—
D|2|2997|[Lucy Jay \[TPH\]](https://zwiftpower\.com/profile\.php?z=986803)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[🥇 #1](https://zwiftpower.com/events.php?zid=154308)|[🥇 #2](https://zwiftpower.com/events.php?zid=155629)|[🥇 #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—
D|3|2995|[Molly Cutmore \(TPH\) 6698](https://zwiftpower\.com/profile\.php?z=1164747)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[🥇 #5](https://zwiftpower.com/events.php?zid=166629)|[🥇 #7](https://zwiftpower.com/events.php?zid=173447)|[🥈 #3](https://zwiftpower.com/events.php?zid=159335)|—|—|—
D|4|2987|[Stefanie Held](https://zwiftpower\.com/profile\.php?z=536821)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥈 #4](https://zwiftpower.com/events.php?zid=162692)|[🥈 #6](https://zwiftpower.com/events.php?zid=171936)|[4th #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—
D|5|997|[Jo Brown CRYO\-GEN\(D\)](https://zwiftpower\.com/profile\.php?z=439963)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥈 #2](https://zwiftpower.com/events.php?zid=155629)|—|—|—|—|—
D|6|995|[Sarah McNair](https://zwiftpower\.com/profile\.php?z=1032310)|—|[🥉 #5](https://zwiftpower.com/events.php?zid=166629)|—|—|—|—|—



## KISS Community League — Pacific-Asia

Individual rider rankings by top 6 results.

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6
---|---|---|---|---|---|---|---|---|---|---
A|1|5952|[IAM Faraone🍦](https://zwiftpower\.com/profile\.php?z=464375)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[🥈 #2](https://zwiftpower.com/events.php?zid=155565)|[🥉 #3](https://zwiftpower.com/events.php?zid=159271)|[🥉 #5](https://zwiftpower.com/events.php?zid=166628)|[5th #6](https://zwiftpower.com/events.php?zid=171934)|[6th #7](https://zwiftpower.com/events.php?zid=173434)|[8th #4](https://zwiftpower.com/events.php?zid=162625)
A|2|5934|[Martin Firestone III](https://zwiftpower\.com/profile\.php?z=99475)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈 #1](https://zwiftpower.com/events.php?zid=151390)|[🥈 #6](https://zwiftpower.com/events.php?zid=171934)|[4th #2](https://zwiftpower.com/events.php?zid=155565)|[5th #7](https://zwiftpower.com/events.php?zid=173434)|[10th #8](https://zwiftpower.com/events.php?zid=176147)|[13th #4](https://zwiftpower.com/events.php?zid=162625)
A|3|5930|[Holden Comeau 🔰](https://zwiftpower\.com/profile\.php?z=103766)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[🥇 #4](https://zwiftpower.com/events.php?zid=162625)|[🥇 #8](https://zwiftpower.com/events.php?zid=176147)|[6th #2](https://zwiftpower.com/events.php?zid=155565)|[6th #6](https://zwiftpower.com/events.php?zid=171934)|[12th #3](https://zwiftpower.com/events.php?zid=159271)|[12th #5](https://zwiftpower.com/events.php?zid=166628)
A|4|5928|[Robert Lyons 🛴](https://zwiftpower\.com/profile\.php?z=248)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[4th #3](https://zwiftpower.com/events.php?zid=159271)|[4th #4](https://zwiftpower.com/events.php?zid=162625)|[5th #1](https://zwiftpower.com/events.php?zid=151390)|[7th #2](https://zwiftpower.com/events.php?zid=155565)|[9th #6](https://zwiftpower.com/events.php?zid=171934)|[10th #5](https://zwiftpower.com/events.php?zid=166628)
A|5|5904|[Matt Johnson](https://zwiftpower\.com/profile\.php?z=16607)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th #6](https://zwiftpower.com/events.php?zid=171934)|[6th #3](https://zwiftpower.com/events.php?zid=159271)|[7th #1](https://zwiftpower.com/events.php?zid=151390)|[9th #5](https://zwiftpower.com/events.php?zid=166628)|[12th #4](https://zwiftpower.com/events.php?zid=162625)|[13th #7](https://zwiftpower.com/events.php?zid=173434)
A|6|5880|[Chris Beck 🔰](https://zwiftpower\.com/profile\.php?z=177885)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[4th #5](https://zwiftpower.com/events.php?zid=166628)|[7th #7](https://zwiftpower.com/events.php?zid=173434)|[8th #6](https://zwiftpower.com/events.php?zid=171934)|[11th #1](https://zwiftpower.com/events.php?zid=151390)|[14th #8](https://zwiftpower.com/events.php?zid=176147)|[19th #2](https://zwiftpower.com/events.php?zid=155565)
A|7|5872|[Matt Gardiner 🍩 🔰](https://zwiftpower\.com/profile\.php?z=58638)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[🥉 #1](https://zwiftpower.com/events.php?zid=151390)|[🥉 #8](https://zwiftpower.com/events.php?zid=176147)|[14th #5](https://zwiftpower.com/events.php?zid=166628)|[14th #6](https://zwiftpower.com/events.php?zid=171934)|[16th #2](https://zwiftpower.com/events.php?zid=155565)|[17th #3](https://zwiftpower.com/events.php?zid=159271)
A|8|5860|[Justin Wagner](https://zwiftpower\.com/profile\.php?z=49001)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[8th #1](https://zwiftpower.com/events.php?zid=151390)|[11th #5](https://zwiftpower.com/events.php?zid=166628)|[12th #2](https://zwiftpower.com/events.php?zid=155565)|[13th #6](https://zwiftpower.com/events.php?zid=171934)|[14th #3](https://zwiftpower.com/events.php?zid=159271)|[15th #7](https://zwiftpower.com/events.php?zid=173434)
A|9|5818|[Robin Zünd 🍺🔰](https://zwiftpower\.com/profile\.php?z=31080)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[7th #3](https://zwiftpower.com/events.php?zid=159271)|[7th #4](https://zwiftpower.com/events.php?zid=162625)|[17th #1](https://zwiftpower.com/events.php?zid=151390)|[17th #6](https://zwiftpower.com/events.php?zid=171934)|[21st #5](https://zwiftpower.com/events.php?zid=166628)|[25th #7](https://zwiftpower.com/events.php?zid=173434)
A|10|5804|[Jerome Mariaud 🐌](https://zwiftpower\.com/profile\.php?z=173884)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[10th #4](https://zwiftpower.com/events.php?zid=162625)|[13th #1](https://zwiftpower.com/events.php?zid=151390)|[13th #3](https://zwiftpower.com/events.php?zid=159271)|[16th #7](https://zwiftpower.com/events.php?zid=173434)|[24th #2](https://zwiftpower.com/events.php?zid=155565)|[25th #8](https://zwiftpower.com/events.php?zid=176147)
A|11|5742|[Lukman CyclingTraining\.cc](https://zwiftpower\.com/profile\.php?z=12512)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[15th #8](https://zwiftpower.com/events.php?zid=176147)|[17th #7](https://zwiftpower.com/events.php?zid=173434)|[23rd #1](https://zwiftpower.com/events.php?zid=151390)|[23rd #2](https://zwiftpower.com/events.php?zid=155565)|[26th #6](https://zwiftpower.com/events.php?zid=171934)|[28th #3](https://zwiftpower.com/events.php?zid=159271)
A|12|5738|[Adam Boyko](https://zwiftpower\.com/profile\.php?z=91519)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[16th #8](https://zwiftpower.com/events.php?zid=176147)|[19th #6](https://zwiftpower.com/events.php?zid=171934)|[21st #3](https://zwiftpower.com/events.php?zid=159271)|[24th #7](https://zwiftpower.com/events.php?zid=173434)|[26th #1](https://zwiftpower.com/events.php?zid=151390)|[28th #4](https://zwiftpower.com/events.php?zid=162625)
A|13|5728|[Mash Potato](https://zwiftpower\.com/profile\.php?z=309860)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[11th #4](https://zwiftpower.com/events.php?zid=162625)|[17th #8](https://zwiftpower.com/events.php?zid=176147)|[20th #6](https://zwiftpower.com/events.php?zid=171934)|[22nd #1](https://zwiftpower.com/events.php?zid=151390)|[30th #7](https://zwiftpower.com/events.php?zid=173434)|[39th #3](https://zwiftpower.com/events.php?zid=159271)
A|14|5724|[Tim Sullivan 🔰](https://zwiftpower\.com/profile\.php?z=165253)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[15th #6](https://zwiftpower.com/events.php?zid=171934)|[19th #1](https://zwiftpower.com/events.php?zid=151390)|[23rd #4](https://zwiftpower.com/events.php?zid=162625)|[24th #8](https://zwiftpower.com/events.php?zid=176147)|[29th #3](https://zwiftpower.com/events.php?zid=159271)|[31st #2](https://zwiftpower.com/events.php?zid=155565)
A|15|5718|[Patrick McCrann \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=328064)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[19th #8](https://zwiftpower.com/events.php?zid=176147)|[20th #3](https://zwiftpower.com/events.php?zid=159271)|[21st #1](https://zwiftpower.com/events.php?zid=151390)|[23rd #6](https://zwiftpower.com/events.php?zid=171934)|[29th #4](https://zwiftpower.com/events.php?zid=162625)|[32nd #5](https://zwiftpower.com/events.php?zid=166628)
A|16|5666|[Yuri Onofrei](https://zwiftpower\.com/profile\.php?z=194825)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[21st #6](https://zwiftpower.com/events.php?zid=171934)|[22nd #7](https://zwiftpower.com/events.php?zid=173434)|[30th #3](https://zwiftpower.com/events.php?zid=159271)|[30th #5](https://zwiftpower.com/events.php?zid=166628)|[33rd #1](https://zwiftpower.com/events.php?zid=151390)|[34th #4](https://zwiftpower.com/events.php?zid=162625)
A|17|5630|[Nick Bevilacqua](https://zwiftpower\.com/profile\.php?z=9876)|—|[23rd #8](https://zwiftpower.com/events.php?zid=176147)|[27th #6](https://zwiftpower.com/events.php?zid=171934)|[32nd #4](https://zwiftpower.com/events.php?zid=162625)|[33rd #2](https://zwiftpower.com/events.php?zid=155565)|[33rd #7](https://zwiftpower.com/events.php?zid=173434)|[40th #5](https://zwiftpower.com/events.php?zid=166628)
A|18|5590|[Donnacha Holmes 🦄](https://zwiftpower\.com/profile\.php?z=64274)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[28th #1](https://zwiftpower.com/events.php?zid=151390)|[31st #8](https://zwiftpower.com/events.php?zid=176147)|[34th #6](https://zwiftpower.com/events.php?zid=171934)|[37th #3](https://zwiftpower.com/events.php?zid=159271)|[38th #2](https://zwiftpower.com/events.php?zid=155565)|[40th #7](https://zwiftpower.com/events.php?zid=173434)
A|19|5480|[John Withrow \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=498960)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[30th #8](https://zwiftpower.com/events.php?zid=176147)|[40th #3](https://zwiftpower.com/events.php?zid=159271)|[46th #6](https://zwiftpower.com/events.php?zid=171934)|[48th #2](https://zwiftpower.com/events.php?zid=155565)|[48th #4](https://zwiftpower.com/events.php?zid=162625)|[51st #5](https://zwiftpower.com/events.php?zid=166628)
A|20|4869|[Tyler West 🔰 ☕️🍦](https://zwiftpower\.com/profile\.php?z=162405)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[7th #5](https://zwiftpower.com/events.php?zid=166628)|[10th #7](https://zwiftpower.com/events.php?zid=173434)|[12th #8](https://zwiftpower.com/events.php?zid=176147)|[17th #4](https://zwiftpower.com/events.php?zid=162625)|[22nd #3](https://zwiftpower.com/events.php?zid=159271)|—
A|21|4787|[Matthew Yankow](https://zwiftpower\.com/profile\.php?z=159402)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th #1](https://zwiftpower.com/events.php?zid=151390)|[17th #2](https://zwiftpower.com/events.php?zid=155565)|[18th #3](https://zwiftpower.com/events.php?zid=159271)|[25th #4](https://zwiftpower.com/events.php?zid=162625)|[37th #7](https://zwiftpower.com/events.php?zid=173434)|—
A|22|4777|[D CHEAMDW KISS\(A\)](https://zwiftpower\.com/profile\.php?z=44184)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[19th #3](https://zwiftpower.com/events.php?zid=159271)|[19th #4](https://zwiftpower.com/events.php?zid=162625)|[20th #1](https://zwiftpower.com/events.php?zid=151390)|[28th #2](https://zwiftpower.com/events.php?zid=155565)|[28th #8](https://zwiftpower.com/events.php?zid=176147)|—
A|23|4727|[James Hodges](https://zwiftpower\.com/profile\.php?z=16708)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[21st #8](https://zwiftpower.com/events.php?zid=176147)|[24th #5](https://zwiftpower.com/events.php?zid=166628)|[25th #6](https://zwiftpower.com/events.php?zid=171934)|[34th #3](https://zwiftpower.com/events.php?zid=159271)|[35th #4](https://zwiftpower.com/events.php?zid=162625)|—
A|24|4679|[George Thivaios](https://zwiftpower\.com/profile\.php?z=250316)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[29th #6](https://zwiftpower.com/events.php?zid=171934)|[30th #1](https://zwiftpower.com/events.php?zid=151390)|[31st #7](https://zwiftpower.com/events.php?zid=173434)|[36th #5](https://zwiftpower.com/events.php?zid=166628)|[37th #4](https://zwiftpower.com/events.php?zid=162625)|—
A|25|3980|[Huang Xiao Yu🔰](https://zwiftpower\.com/profile\.php?z=75898)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[🥇 #2](https://zwiftpower.com/events.php?zid=155565)|[🥈 #3](https://zwiftpower.com/events.php?zid=159271)|[🥉 #4](https://zwiftpower.com/events.php?zid=162625)|[6th #1](https://zwiftpower.com/events.php?zid=151390)|—|—
A|26|3954|[Bernie Lacourciere](https://zwiftpower\.com/profile\.php?z=220957)|[GET\-SET](https://zwiftpower\.com/team\.php?id=2644)|[4th #1](https://zwiftpower.com/events.php?zid=151390)|[5th #3](https://zwiftpower.com/events.php?zid=159271)|[7th #8](https://zwiftpower.com/events.php?zid=176147)|[9th #4](https://zwiftpower.com/events.php?zid=162625)|—|—
A|27|3938|[H\-G Becker \(V\)](https://zwiftpower\.com/profile\.php?z=1765)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[🥉 #7](https://zwiftpower.com/events.php?zid=173434)|[6th #5](https://zwiftpower.com/events.php?zid=166628)|[10th #2](https://zwiftpower.com/events.php?zid=155565)|[14th #4](https://zwiftpower.com/events.php?zid=162625)|—|—
A|28|3936|[Ulf Bäcker 🔰](https://zwiftpower\.com/profile\.php?z=884451)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|[4th #8](https://zwiftpower.com/events.php?zid=176147)|[5th #4](https://zwiftpower.com/events.php?zid=162625)|[9th #2](https://zwiftpower.com/events.php?zid=155565)|[16th #6](https://zwiftpower.com/events.php?zid=171934)|—|—
A|29|3904|[Sticky 🤡](https://zwiftpower\.com/profile\.php?z=196246)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[8th #3](https://zwiftpower.com/events.php?zid=159271)|[9th #8](https://zwiftpower.com/events.php?zid=176147)|[11th #6](https://zwiftpower.com/events.php?zid=171934)|[22nd #2](https://zwiftpower.com/events.php?zid=155565)|—|—
A|30|3882|[Matt Brandt🔰🐺](https://zwiftpower\.com/profile\.php?z=78451)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[6th #8](https://zwiftpower.com/events.php?zid=176147)|[8th #2](https://zwiftpower.com/events.php?zid=155565)|[16th #5](https://zwiftpower.com/events.php?zid=166628)|[31st #3](https://zwiftpower.com/events.php?zid=159271)|—|—
A|31|3876|[FinK Andi](https://zwiftpower\.com/profile\.php?z=495)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[9th #1](https://zwiftpower.com/events.php?zid=151390)|[13th #2](https://zwiftpower.com/events.php?zid=155565)|[16th #3](https://zwiftpower.com/events.php?zid=159271)|[26th #4](https://zwiftpower.com/events.php?zid=162625)|—|—
A|32|3814|[Colin Mitchell](https://zwiftpower\.com/profile\.php?z=55166)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[18th #1](https://zwiftpower.com/events.php?zid=151390)|[21st #2](https://zwiftpower.com/events.php?zid=155565)|[24th #6](https://zwiftpower.com/events.php?zid=171934)|[32nd #3](https://zwiftpower.com/events.php?zid=159271)|—|—
A|33|3764|"[Matt Bruss \| Brazen Dropouts](https://zwiftpower\.com/profile\.php?z=420808)"|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[20th #8](https://zwiftpower.com/events.php?zid=176147)|[28th #6](https://zwiftpower.com/events.php?zid=171934)|[33rd #3](https://zwiftpower.com/events.php?zid=159271)|[39th #2](https://zwiftpower.com/events.php?zid=155565)|—|—
A|34|3744|[THE Nighttrain ODZ](https://zwiftpower\.com/profile\.php?z=7134)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[26th #8](https://zwiftpower.com/events.php?zid=176147)|[33rd #6](https://zwiftpower.com/events.php?zid=171934)|[35th #7](https://zwiftpower.com/events.php?zid=173434)|[36th #3](https://zwiftpower.com/events.php?zid=159271)|—|—
A|35|3708|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[27th #8](https://zwiftpower.com/events.php?zid=176147)|[39th #5](https://zwiftpower.com/events.php?zid=166628)|[40th #6](https://zwiftpower.com/events.php?zid=171934)|[42nd #7](https://zwiftpower.com/events.php?zid=173434)|—|—
A|36|3656|[Marc Pierschel \[VEGAN\]](https://zwiftpower\.com/profile\.php?z=918949)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[41st #6](https://zwiftpower.com/events.php?zid=171934)|[43rd #4](https://zwiftpower.com/events.php?zid=162625)|[43rd #5](https://zwiftpower.com/events.php?zid=166628)|[47th #2](https://zwiftpower.com/events.php?zid=155565)|—|—
A|37|2989|[Aidar Zakarin 🔰](https://zwiftpower\.com/profile\.php?z=623306)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[🥇 #6](https://zwiftpower.com/events.php?zid=171934)|[🥇 #7](https://zwiftpower.com/events.php?zid=173434)|[5th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—
A|38|2949|[LaowaiのQindie👨🏻](https://zwiftpower\.com/profile\.php?z=467523)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[🥇 #1](https://zwiftpower.com/events.php?zid=151390)|[🥉 #2](https://zwiftpower.com/events.php?zid=155565)|[23rd #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—
A|39|2905|[Luiz Lanfredi](https://zwiftpower\.com/profile\.php?z=8481)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[9th #3](https://zwiftpower.com/events.php?zid=159271)|[18th #2](https://zwiftpower.com/events.php?zid=155565)|[22nd #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—
A|40|2871|[Richard Port](https://zwiftpower\.com/profile\.php?z=324062)|—|[19th #5](https://zwiftpower.com/events.php?zid=166628)|[20th #4](https://zwiftpower.com/events.php?zid=162625)|[27th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—
A|41|2861|[Watt Monkey 🐵🔰](https://zwiftpower\.com/profile\.php?z=44150)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[15th #1](https://zwiftpower.com/events.php?zid=151390)|[27th #7](https://zwiftpower.com/events.php?zid=173434)|[29th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—
A|42|2845|[Karl\-Axel Zander Persson](https://zwiftpower\.com/profile\.php?z=5994)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[24th #1](https://zwiftpower.com/events.php?zid=151390)|[24th #4](https://zwiftpower.com/events.php?zid=162625)|[31st #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—
A|43|2833|[Russell Crowder 🔰](https://zwiftpower\.com/profile\.php?z=22662)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[25th #3](https://zwiftpower.com/events.php?zid=159271)|[29th #2](https://zwiftpower.com/events.php?zid=155565)|[31st #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—
A|44|2801|[Michael Danish](https://zwiftpower\.com/profile\.php?z=257206)|—|[28th #7](https://zwiftpower.com/events.php?zid=173434)|[33rd #5](https://zwiftpower.com/events.php?zid=166628)|[40th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—
A|44|2801|"[Chris ""Awesomely Average"" Wilson](https://zwiftpower\.com/profile\.php?z=794637)"|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[31st #5](https://zwiftpower.com/events.php?zid=166628)|[34th #2](https://zwiftpower.com/events.php?zid=155565)|[36th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—
A|45|2783|[Jerome Ferson](https://zwiftpower\.com/profile\.php?z=213439)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[33rd #4](https://zwiftpower.com/events.php?zid=162625)|[38th #6](https://zwiftpower.com/events.php?zid=171934)|[39th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
A|46|1998|[Samuel Brännlund INC🔰](https://zwiftpower\.com/profile\.php?z=660836)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇 #3](https://zwiftpower.com/events.php?zid=159271)|[🥇 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
A|47|1978|[Mon Ritch](https://zwiftpower\.com/profile\.php?z=135635)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[4th #7](https://zwiftpower.com/events.php?zid=173434)|[8th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—
A|48|1960|[Abeki Kanji \[ZWC\]](https://zwiftpower\.com/profile\.php?z=335605)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[10th #3](https://zwiftpower.com/events.php?zid=159271)|[11th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
A|48|1960|[Pär Bäcker🔰](https://zwiftpower\.com/profile\.php?z=504853)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|[10th #6](https://zwiftpower.com/events.php?zid=171934)|[11th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—
A|49|1948|[Andreas Rois \(KC\)](https://zwiftpower\.com/profile\.php?z=991994)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[9th #7](https://zwiftpower.com/events.php?zid=173434)|[18th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
A|50|1912|[Third Wheel 🚳](https://zwiftpower\.com/profile\.php?z=48068)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[18th #6](https://zwiftpower.com/events.php?zid=171934)|[27th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
A|51|1894|[Andy Jenner 🦑](https://zwiftpower\.com/profile\.php?z=24974)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[22nd #8](https://zwiftpower.com/events.php?zid=176147)|[32nd #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
A|52|1890|[Glen Gollrad 🦊 \[VFOX\]](https://zwiftpower\.com/profile\.php?z=495351)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[14th #2](https://zwiftpower.com/events.php?zid=155565)|[42nd #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
A|53|1878|[Daan Laros](https://zwiftpower\.com/profile\.php?z=332952)|—|[28th #5](https://zwiftpower.com/events.php?zid=166628)|[34th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
A|54|1844|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[34th #5](https://zwiftpower.com/events.php?zid=166628)|[45th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
A|55|1842|[rob brunken](https://zwiftpower\.com/profile\.php?z=137488)|—|[35th #3](https://zwiftpower.com/events.php?zid=159271)|[45th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
A|56|1836|[Anatoly Solodov](https://zwiftpower\.com/profile\.php?z=1057970)|—|[34th #1](https://zwiftpower.com/events.php?zid=151390)|[49th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
A|57|1832|[Adam Chevin Ⓥ🌱](https://zwiftpower\.com/profile\.php?z=70177)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[41st #7](https://zwiftpower.com/events.php?zid=173434)|[44th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
A|58|1830|[Matthew Crawford 🌵](https://zwiftpower\.com/profile\.php?z=11265)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[40th #4](https://zwiftpower.com/events.php?zid=162625)|[46th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
A|59|1828|[Jean\-Yan Perron](https://zwiftpower\.com/profile\.php?z=390308)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|[42nd #2](https://zwiftpower.com/events.php?zid=155565)|[45th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
A|59|1828|[Poisson Lune](https://zwiftpower\.com/profile\.php?z=1011634)|—|[43rd #7](https://zwiftpower.com/events.php?zid=173434)|[44th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
A|60|1816|[yu guantou](https://zwiftpower\.com/profile\.php?z=640047)|—|[43rd #2](https://zwiftpower.com/events.php?zid=155565)|[50th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
A|61|997|[Alessandro Barra](https://zwiftpower\.com/profile\.php?z=12495)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[🥈 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|61|997|[Andreas Serop Larsen](https://zwiftpower\.com/profile\.php?z=20358)|—|[🥈 #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|61|997|[Yuki Tanaka](https://zwiftpower\.com/profile\.php?z=35498)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[🥈 #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|61|997|[Niki H 🔰](https://zwiftpower\.com/profile\.php?z=174313)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥈 #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|62|995|[Mattias Lundqvist🔰 Aktivitus Testcenter](https://zwiftpower\.com/profile\.php?z=455884)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[🥉 #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|63|991|[Yunfei Wu \+1S🍓](https://zwiftpower\.com/profile\.php?z=22572)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[5th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|63|991|[Simon Nielsen 🔰](https://zwiftpower\.com/profile\.php?z=182972)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[5th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|64|989|[Hefin Evans](https://zwiftpower\.com/profile\.php?z=1123557)|—|[6th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|65|987|[FUZHOU HANCHEN \+1S A](https://zwiftpower\.com/profile\.php?z=388541)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[7th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|66|985|[Chris Smith \[BRT\]](https://zwiftpower\.com/profile\.php?z=15080)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[8th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|66|985|[Bas Pater](https://zwiftpower\.com/profile\.php?z=859868)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[8th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|67|981|[David Talbott](https://zwiftpower\.com/profile\.php?z=270280)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[10th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|68|979|[Kim Little \[X\]](https://zwiftpower\.com/profile\.php?z=6724)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|[11th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|68|979|[Ryan Mckay](https://zwiftpower\.com/profile\.php?z=245426)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[11th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|69|977|[Matt Curbeau 🔰](https://zwiftpower\.com/profile\.php?z=11677)|[Velocio](https://zwiftpower\.com/team\.php?id=2582)|[12th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|69|977|[Norm Hughes](https://zwiftpower\.com/profile\.php?z=190665)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|70|975|[Timmy Bauer 🔰](https://zwiftpower\.com/profile\.php?z=861542)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[13th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|70|975|[Kirill Krasavin 🔰](https://zwiftpower\.com/profile\.php?z=960217)|[KLD](https://zwiftpower\.com/team\.php?id=2163)|[13th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|71|973|[Rob Wood](https://zwiftpower\.com/profile\.php?z=95589)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[14th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|71|973|[Kappa Ebisen](https://zwiftpower\.com/profile\.php?z=368295)|—|[14th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|72|971|[Daniel Pettinger](https://zwiftpower\.com/profile\.php?z=90946)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[15th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|72|971|[Chris Sherriffs](https://zwiftpower\.com/profile\.php?z=95241)|[Ribble Pro Cycling](https://zwiftpower\.com/team\.php?id=3686)|[15th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|72|971|[Dafydd Williams](https://zwiftpower\.com/profile\.php?z=123836)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[15th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
A|72|971|[Mathieu Decock](https://zwiftpower\.com/profile\.php?z=509565)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[15th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|73|969|[Dave Reed \(Bioracer Moriarty Bikes\)](https://zwiftpower\.com/profile\.php?z=558707)|[BMB](https://zwiftpower\.com/team\.php?id=2385)|[16th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|73|969|[K KONNO](https://zwiftpower\.com/profile\.php?z=563135)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[16th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|74|967|[Koen Van Geyt](https://zwiftpower\.com/profile\.php?z=115080)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[17th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|75|965|[Ludovic Pollet](https://zwiftpower\.com/profile\.php?z=187160)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[18th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|75|965|[Dariusz Zaremba \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=564515)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[18th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|75|965|[Maxim Golev \[V\]](https://zwiftpower\.com/profile\.php?z=811341)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[18th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|76|963|[A\-Ruei ☢️](https://zwiftpower\.com/profile\.php?z=364142)|[TTR](https://zwiftpower\.com/team\.php?id=3750)|[19th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|77|961|[Justin Purificati](https://zwiftpower\.com/profile\.php?z=52048)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|[20th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|77|961|[Tomoaki Takahashi 🍦🍨🍰🎂](https://zwiftpower\.com/profile\.php?z=76844)|[ZJRT](https://zwiftpower\.com/team\.php?id=2054)|[20th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|77|961|[Sir Wang \(SYB\) 💩](https://zwiftpower\.com/profile\.php?z=698447)|[TTR](https://zwiftpower\.com/team\.php?id=3750)|[20th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|78|959|[Alex Sankovic](https://zwiftpower\.com/profile\.php?z=506515)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[21st #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|78|959|[Peter Göransson](https://zwiftpower\.com/profile\.php?z=674292)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[21st #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|79|957|[Adrian Timmis 🔰 CadencESport](https://zwiftpower\.com/profile\.php?z=64680)|[CadencESport](https://zwiftpower\.com/team\.php?id=2839)|[22nd #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|80|955|[Matt Sinclair](https://zwiftpower\.com/profile\.php?z=115138)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[23rd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|80|955|[Miguel Indoorain \[P//A\]](https://zwiftpower\.com/profile\.php?z=325557)|[P//A](https://zwiftpower\.com/team\.php?id=3898)|[23rd #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|81|951|[Rod Leach](https://zwiftpower\.com/profile\.php?z=44795)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[25th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|81|951|[Michaël Duyck](https://zwiftpower\.com/profile\.php?z=274955)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[25th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|82|949|[Mathieu Hould](https://zwiftpower\.com/profile\.php?z=71850)|—|[26th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|82|949|[Tom Schyvens](https://zwiftpower\.com/profile\.php?z=426416)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[26th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|82|949|[Liu ZL \+1S](https://zwiftpower\.com/profile\.php?z=1155785)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[26th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
A|83|947|[Daniel Sutherland](https://zwiftpower\.com/profile\.php?z=54066)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|[27th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|83|947|[Brian H](https://zwiftpower\.com/profile\.php?z=128548)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[27th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|84|943|[Johnathan Freter 🚂 🔰](https://zwiftpower\.com/profile\.php?z=77064)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[29th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
A|84|943|[Jone Gravdal 🔰](https://zwiftpower\.com/profile\.php?z=121873)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[29th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|84|943|[Michael Totz](https://zwiftpower\.com/profile\.php?z=250857)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[29th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|85|941|[Derek Sawyer \- Rippers](https://zwiftpower\.com/profile\.php?z=168787)|—|[30th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|85|941|[Jon Duke \[DIRT\]](https://zwiftpower\.com/profile\.php?z=462877)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[30th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|86|939|[Okanomizu Masaru Oka](https://zwiftpower\.com/profile\.php?z=42834)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[31st #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|87|937|[Il Pirata](https://zwiftpower\.com/profile\.php?z=29496)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[32nd #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|87|937|[Syo Yone\(FRAT\-GSJ\)](https://zwiftpower\.com/profile\.php?z=31146)|—|[32nd #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
A|88|931|[Jarod Garrington](https://zwiftpower\.com/profile\.php?z=63002)|—|[35th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|88|931|[Goran Salavec Race3R](https://zwiftpower\.com/profile\.php?z=92474)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|[35th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|88|931|[Kilian Hemart Ⓥ](https://zwiftpower\.com/profile\.php?z=488019)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[35th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|89|929|[Dave Higham](https://zwiftpower\.com/profile\.php?z=113857)|—|[36th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|89|929|[Anton Alex](https://zwiftpower\.com/profile\.php?z=601528)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[36th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|90|927|[Mathieu Drujon 🎩🔰](https://zwiftpower\.com/profile\.php?z=75218)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|[37th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|90|927|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[37th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|90|927|[David Pell](https://zwiftpower\.com/profile\.php?z=772768)|—|[37th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|91|925|[Samuel Leguevaques](https://zwiftpower\.com/profile\.php?z=44125)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[38th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
A|91|925|[Jakob Grønbæk, LAMONT](https://zwiftpower\.com/profile\.php?z=79657)|[LAMONT](https://zwiftpower\.com/team\.php?id=302)|[38th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|91|925|[takaaki kawai \(a\)8839](https://zwiftpower\.com/profile\.php?z=166760)|—|[38th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|92|923|[T Sato](https://zwiftpower\.com/profile\.php?z=136374)|—|[39th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|92|923|[Edwin Driessen](https://zwiftpower\.com/profile\.php?z=219489)|—|[39th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|93|919|[Lucas Van Drunen](https://zwiftpower\.com/profile\.php?z=6184)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[41st #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|93|919|[Dan Swenson9257](https://zwiftpower\.com/profile\.php?z=89267)|—|[41st #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|93|919|[Steve Chadburn \[VFOX\]](https://zwiftpower\.com/profile\.php?z=822255)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[41st #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|94|917|[Marjan Dolamic Jr\.](https://zwiftpower\.com/profile\.php?z=69480)|[RT WINAX\-food artists](https://zwiftpower\.com/team\.php?id=3928)|[42nd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|94|917|[Hiroshi Kikuchi](https://zwiftpower\.com/profile\.php?z=155607)|—|[42nd #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|95|915|[Sven Gerhardt INC 👽](https://zwiftpower\.com/profile\.php?z=982806)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[43rd #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|96|913|[yuta otani](https://zwiftpower\.com/profile\.php?z=247629)|—|[44th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|96|913|[Yasuyuki Kobayashi](https://zwiftpower\.com/profile\.php?z=917068)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[44th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|96|913|[Jerry Olsson](https://zwiftpower\.com/profile\.php?z=996335)|—|[44th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|97|911|[Jonathan Wass](https://zwiftpower\.com/profile\.php?z=158486)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[45th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
A|98|909|[Elizabeth Ferguson 8217](https://zwiftpower\.com/profile\.php?z=991475)|—|[46th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
A|99|907|[Ghisallo Totsuka](https://zwiftpower\.com/profile\.php?z=90444)|—|[47th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
A|99|907|[Michi Kangaroo](https://zwiftpower\.com/profile\.php?z=218825)|—|[47th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|99|907|[Nicky Manson](https://zwiftpower\.com/profile\.php?z=237508)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[47th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|100|903|[alm0st\.fast \(KC\)](https://zwiftpower\.com/profile\.php?z=458147)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[49th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|101|901|[Dave Shacks](https://zwiftpower\.com/profile\.php?z=47745)|[Hale Velo](https://zwiftpower\.com/team\.php?id=1981)|[50th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
A|102|899|[HIPSTER p�p�9113�](https://zwiftpower\.com/profile\.php?z=460423)|[CYC](https://zwiftpower\.com/team\.php?id=2957)|[51st #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
A|103|897|[Dennis Lock](https://zwiftpower\.com/profile\.php?z=979320)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[52nd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|1|5972|[Rick Barbera](https://zwiftpower\.com/profile\.php?z=2946)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #4](https://zwiftpower.com/events.php?zid=162625)|[🥇 #7](https://zwiftpower.com/events.php?zid=173434)|[🥇 #8](https://zwiftpower.com/events.php?zid=176147)|[🥉 #3](https://zwiftpower.com/events.php?zid=159271)|[5th #6](https://zwiftpower.com/events.php?zid=171934)|[6th #2](https://zwiftpower.com/events.php?zid=155565)
B|2|5944|[Just A Rollerblader Ⓥ 🌱](https://zwiftpower\.com/profile\.php?z=114584)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥈 #2](https://zwiftpower.com/events.php?zid=155565)|[🥈 #3](https://zwiftpower.com/events.php?zid=159271)|[🥉 #1](https://zwiftpower.com/events.php?zid=151390)|[6th #7](https://zwiftpower.com/events.php?zid=173434)|[7th #5](https://zwiftpower.com/events.php?zid=166628)|[11th #6](https://zwiftpower.com/events.php?zid=171934)
B|3|5922|[Florin Vasilescu](https://zwiftpower\.com/profile\.php?z=167)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[🥈 #8](https://zwiftpower.com/events.php?zid=176147)|[4th #5](https://zwiftpower.com/events.php?zid=166628)|[5th #7](https://zwiftpower.com/events.php?zid=173434)|[6th #3](https://zwiftpower.com/events.php?zid=159271)|[11th #2](https://zwiftpower.com/events.php?zid=155565)|[14th #4](https://zwiftpower.com/events.php?zid=162625)
B|4|5910|[Matt Montgomery](https://zwiftpower\.com/profile\.php?z=240716)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[5th #3](https://zwiftpower.com/events.php?zid=159271)|[6th #1](https://zwiftpower.com/events.php?zid=151390)|[6th #6](https://zwiftpower.com/events.php?zid=171934)|[6th #8](https://zwiftpower.com/events.php?zid=176147)|[12th #7](https://zwiftpower.com/events.php?zid=173434)|[13th #2](https://zwiftpower.com/events.php?zid=155565)
B|5|5886|[Bryan Montgomery](https://zwiftpower\.com/profile\.php?z=199436)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉 #8](https://zwiftpower.com/events.php?zid=176147)|[4th #3](https://zwiftpower.com/events.php?zid=159271)|[12th #1](https://zwiftpower.com/events.php?zid=151390)|[12th #4](https://zwiftpower.com/events.php?zid=162625)|[13th #6](https://zwiftpower.com/events.php?zid=171934)|[16th #2](https://zwiftpower.com/events.php?zid=155565)
B|6|5880|[Jenn Edwards\(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=183549)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[🥉 #5](https://zwiftpower.com/events.php?zid=166628)|[7th #6](https://zwiftpower.com/events.php?zid=171934)|[10th #4](https://zwiftpower.com/events.php?zid=162625)|[13th #8](https://zwiftpower.com/events.php?zid=176147)|[14th #3](https://zwiftpower.com/events.php?zid=159271)|[16th #1](https://zwiftpower.com/events.php?zid=151390)
B|7|5848|[Nick Ashworth](https://zwiftpower\.com/profile\.php?z=260349)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[8th #6](https://zwiftpower.com/events.php?zid=171934)|[9th #3](https://zwiftpower.com/events.php?zid=159271)|[9th #4](https://zwiftpower.com/events.php?zid=162625)|[11th #8](https://zwiftpower.com/events.php?zid=176147)|[20th #1](https://zwiftpower.com/events.php?zid=151390)|[22nd #5](https://zwiftpower.com/events.php?zid=166628)
B|8|5732|[Mark Stahlkopf](https://zwiftpower\.com/profile\.php?z=329928)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[17th #8](https://zwiftpower.com/events.php?zid=176147)|[21st #1](https://zwiftpower.com/events.php?zid=151390)|[21st #4](https://zwiftpower.com/events.php?zid=162625)|[22nd #3](https://zwiftpower.com/events.php?zid=159271)|[24th #5](https://zwiftpower.com/events.php?zid=166628)|[32nd #2](https://zwiftpower.com/events.php?zid=155565)
B|9|5696|[Brian Hagan](https://zwiftpower\.com/profile\.php?z=102945)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[20th #4](https://zwiftpower.com/events.php?zid=162625)|[23rd #1](https://zwiftpower.com/events.php?zid=151390)|[23rd #6](https://zwiftpower.com/events.php?zid=171934)|[27th #5](https://zwiftpower.com/events.php?zid=166628)|[27th #7](https://zwiftpower.com/events.php?zid=173434)|[35th #2](https://zwiftpower.com/events.php?zid=155565)
B|10|5664|[Scott Giljum \(endurancenation\.us\)](https://zwiftpower\.com/profile\.php?z=341113)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[22nd #1](https://zwiftpower.com/events.php?zid=151390)|[24th #3](https://zwiftpower.com/events.php?zid=159271)|[25th #6](https://zwiftpower.com/events.php?zid=171934)|[32nd #5](https://zwiftpower.com/events.php?zid=166628)|[34th #2](https://zwiftpower.com/events.php?zid=155565)|[34th #7](https://zwiftpower.com/events.php?zid=173434)
B|11|4937|[Timothy Busick](https://zwiftpower\.com/profile\.php?z=6158)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥉 #7](https://zwiftpower.com/events.php?zid=173434)|[4th #4](https://zwiftpower.com/events.php?zid=162625)|[5th #5](https://zwiftpower.com/events.php?zid=166628)|[8th #1](https://zwiftpower.com/events.php?zid=151390)|[14th #2](https://zwiftpower.com/events.php?zid=155565)|—
B|12|4889|[JD Reich](https://zwiftpower\.com/profile\.php?z=832405)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[5th #8](https://zwiftpower.com/events.php?zid=176147)|[10th #1](https://zwiftpower.com/events.php?zid=151390)|[10th #2](https://zwiftpower.com/events.php?zid=155565)|[16th #6](https://zwiftpower.com/events.php?zid=171934)|[17th #7](https://zwiftpower.com/events.php?zid=173434)|—
B|13|4839|[Shaun Solhjou](https://zwiftpower\.com/profile\.php?z=213130)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[4th #8](https://zwiftpower.com/events.php?zid=176147)|[16th #7](https://zwiftpower.com/events.php?zid=173434)|[19th #4](https://zwiftpower.com/events.php?zid=162625)|[21st #5](https://zwiftpower.com/events.php?zid=166628)|[23rd #2](https://zwiftpower.com/events.php?zid=155565)|—
B|14|3924|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[5th #1](https://zwiftpower.com/events.php?zid=151390)|[8th #4](https://zwiftpower.com/events.php?zid=162625)|[12th #2](https://zwiftpower.com/events.php?zid=155565)|[15th #3](https://zwiftpower.com/events.php?zid=159271)|—|—
B|15|3884|[Tex Divine](https://zwiftpower\.com/profile\.php?z=82152)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈 #4](https://zwiftpower.com/events.php?zid=162625)|[🥉 #2](https://zwiftpower.com/events.php?zid=155565)|[18th #8](https://zwiftpower.com/events.php?zid=176147)|[37th #7](https://zwiftpower.com/events.php?zid=173434)|—|—
B|16|3846|[S Ssmith \(V\) Ⓥ ^\.\.^ 🐣🌱🐾](https://zwiftpower\.com/profile\.php?z=175002)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[12th #5](https://zwiftpower.com/events.php?zid=166628)|[20th #6](https://zwiftpower.com/events.php?zid=171934)|[22nd #4](https://zwiftpower.com/events.php?zid=162625)|[25th #7](https://zwiftpower.com/events.php?zid=173434)|—|—
B|17|2955|[scott prosser 4206](https://zwiftpower\.com/profile\.php?z=119089)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|[🥉 #6](https://zwiftpower.com/events.php?zid=171934)|[10th #3](https://zwiftpower.com/events.php?zid=159271)|[11th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
B|17|2955|[Jason Sterny](https://zwiftpower\.com/profile\.php?z=413819)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥉 #4](https://zwiftpower.com/events.php?zid=162625)|[10th #6](https://zwiftpower.com/events.php?zid=171934)|[11th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—
B|18|2921|[Shigeru Maehara](https://zwiftpower\.com/profile\.php?z=193217)|[TECHNOPOLIS](https://zwiftpower\.com/team\.php?id=3906)|[7th #8](https://zwiftpower.com/events.php?zid=176147)|[12th #3](https://zwiftpower.com/events.php?zid=159271)|[22nd #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—
B|19|2907|[Graham Partain](https://zwiftpower\.com/profile\.php?z=64048)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[14th #6](https://zwiftpower.com/events.php?zid=171934)|[15th #2](https://zwiftpower.com/events.php?zid=155565)|[19th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
B|20|2893|[Jouni Malmivaara](https://zwiftpower\.com/profile\.php?z=176062)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[18th #4](https://zwiftpower.com/events.php?zid=162625)|[18th #6](https://zwiftpower.com/events.php?zid=171934)|[19th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—
B|21|2877|[jakob wilson](https://zwiftpower\.com/profile\.php?z=64351)|[OMAHA VELO](https://zwiftpower\.com/team\.php?id=3984)|[11th #3](https://zwiftpower.com/events.php?zid=159271)|[14th #7](https://zwiftpower.com/events.php?zid=173434)|[38th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—
B|22|2841|[Ian Murray](https://zwiftpower\.com/profile\.php?z=20)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[23rd #4](https://zwiftpower.com/events.php?zid=162625)|[28th #6](https://zwiftpower.com/events.php?zid=171934)|[30th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
B|23|1986|[Franco Battiato](https://zwiftpower\.com/profile\.php?z=14632)|[Team Z TAZ](https://zwiftpower\.com/team\.php?id=496)|[🥈 #1](https://zwiftpower.com/events.php?zid=151390)|[6th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
B|24|1984|[Danny Baker Morvelo Basso RT](https://zwiftpower\.com/profile\.php?z=341967)|—|[🥈 #6](https://zwiftpower.com/events.php?zid=171934)|[7th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
B|25|1980|[Aleksandr Simashin 🐻](https://zwiftpower\.com/profile\.php?z=45225)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥈 #7](https://zwiftpower.com/events.php?zid=173434)|[9th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
B|25|1980|[Ryan Kettleson](https://zwiftpower\.com/profile\.php?z=188883)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th #1](https://zwiftpower.com/events.php?zid=151390)|[7th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
B|26|1962|[jhonny luke](https://zwiftpower\.com/profile\.php?z=388191)|—|[5th #4](https://zwiftpower.com/events.php?zid=162625)|[15th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|27|1960|[kosuke matsuura](https://zwiftpower\.com/profile\.php?z=1015507)|—|[9th #5](https://zwiftpower.com/events.php?zid=166628)|[12th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—
B|28|1958|[Donald Ritger](https://zwiftpower\.com/profile\.php?z=228063)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[9th #2](https://zwiftpower.com/events.php?zid=155565)|[13th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|29|1952|[Rob Peters \(Team EN\)](https://zwiftpower\.com/profile\.php?z=536679)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[8th #7](https://zwiftpower.com/events.php?zid=173434)|[17th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
B|30|1946|[simone sighinolfi Ⓥ](https://zwiftpower\.com/profile\.php?z=1085650)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[8th #8](https://zwiftpower.com/events.php?zid=176147)|[20th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|31|1936|[Wataru Yamada](https://zwiftpower\.com/profile\.php?z=626778)|—|[16th #8](https://zwiftpower.com/events.php?zid=176147)|[17th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—
B|32|1934|[Francesco Roberto Bell 🔔](https://zwiftpower\.com/profile\.php?z=328472)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[13th #1](https://zwiftpower.com/events.php?zid=151390)|[21st #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
B|33|1910|[Rene Meijer](https://zwiftpower\.com/profile\.php?z=803509)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[17th #1](https://zwiftpower.com/events.php?zid=151390)|[29th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
B|34|1906|[Martin Heintzelman](https://zwiftpower\.com/profile\.php?z=236040)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[23rd #7](https://zwiftpower.com/events.php?zid=173434)|[25th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
B|35|1902|[Greg Low](https://zwiftpower\.com/profile\.php?z=162837)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[21st #8](https://zwiftpower.com/events.php?zid=176147)|[29th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|36|1894|[Tatsuya Kitamura](https://zwiftpower\.com/profile\.php?z=178483)|—|[24th #4](https://zwiftpower.com/events.php?zid=162625)|[30th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
B|37|1890|[kainan](https://zwiftpower\.com/profile\.php?z=368108)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[26th #6](https://zwiftpower.com/events.php?zid=171934)|[30th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
B|37|1890|[Yannick Lagadec](https://zwiftpower\.com/profile\.php?z=885274)|—|[27th #4](https://zwiftpower.com/events.php?zid=162625)|[29th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—
B|38|1882|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|[19th #8](https://zwiftpower.com/events.php?zid=176147)|[41st #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|39|1868|[Matt Partridge \[DIRT\]](https://zwiftpower\.com/profile\.php?z=458388)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[24th #6](https://zwiftpower.com/events.php?zid=171934)|[43rd #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—
B|40|1854|[Bret Elder](https://zwiftpower\.com/profile\.php?z=128929)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[36th #5](https://zwiftpower.com/events.php?zid=166628)|[38th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
B|41|999|[Matthew Crawford 🌵](https://zwiftpower\.com/profile\.php?z=11265)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|41|999|[adrian vizzari](https://zwiftpower\.com/profile\.php?z=17189)|—|[🥇 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|41|999|[Peter Mosbaek 🔰](https://zwiftpower\.com/profile\.php?z=43225)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[🥇 #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|41|999|[Gregg O'Malley](https://zwiftpower\.com/profile\.php?z=162054)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|[🥇 #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|41|999|[Joergen Yksnoey Rakvaag](https://zwiftpower\.com/profile\.php?z=448140)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[🥇 #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|42|997|[Rik van IJzendoorn](https://zwiftpower\.com/profile\.php?z=34365)|—|[🥈 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|43|993|[Maxime F](https://zwiftpower\.com/profile\.php?z=177040)|—|[4th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|43|993|[Thomas Stemmet](https://zwiftpower\.com/profile\.php?z=732876)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[4th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|43|993|[jongmoon chae5931](https://zwiftpower\.com/profile\.php?z=1021389)|—|[4th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|44|991|[Clive King](https://zwiftpower\.com/profile\.php?z=1978)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[5th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|45|987|[Marlene Bjärehed \[VVixen\] 🔰](https://zwiftpower\.com/profile\.php?z=289822)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[7th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|45|987|[Nathan Kershner 🅥](https://zwiftpower\.com/profile\.php?z=920279)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[7th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|46|985|[Adam Chevin Ⓥ🌱](https://zwiftpower\.com/profile\.php?z=70177)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[8th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|46|985|[So Gyuseok/Team Trek\-Hwashin](https://zwiftpower\.com/profile\.php?z=178293)|[SilentK](https://zwiftpower\.com/team\.php?id=4129)|[8th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|46|985|[Janne Jakobsson](https://zwiftpower\.com/profile\.php?z=331176)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[8th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|47|983|[Stephan Maertens Ⓥ 🐇🥕](https://zwiftpower\.com/profile\.php?z=37710)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[9th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|47|983|[M BeenSlacking](https://zwiftpower\.com/profile\.php?z=88308)|—|[9th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
B|47|983|[Pierre Boorg](https://zwiftpower\.com/profile\.php?z=311040)|—|[9th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|48|981|[Ben Koh](https://zwiftpower\.com/profile\.php?z=16733)|[\[AsC\]](https://zwiftpower\.com/team\.php?id=3249)|[10th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
B|48|981|[Laura Epelbaum](https://zwiftpower\.com/profile\.php?z=377170)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[10th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|48|981|[Craig Wood](https://zwiftpower\.com/profile\.php?z=1115894)|[WATTBIKE](https://zwiftpower\.com/team\.php?id=2738)|[10th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|49|979|[Ian Anderson 🍊🚅](https://zwiftpower\.com/profile\.php?z=46121)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[11th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|49|979|[cyril TOURMENTE](https://zwiftpower\.com/profile\.php?z=855845)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|[11th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|50|977|[Jasper Vinten \[DBR\]\-RB](https://zwiftpower\.com/profile\.php?z=417880)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[12th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|51|975|[John Shin Ⓥ](https://zwiftpower\.com/profile\.php?z=24810)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[13th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|52|973|[Jono Finan \(IRL\)](https://zwiftpower\.com/profile\.php?z=210648)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|[14th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|52|973|[Kazuki Nishimori](https://zwiftpower\.com/profile\.php?z=220282)|—|[14th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|52|973|[David Szentesi6476](https://zwiftpower\.com/profile\.php?z=374304)|—|[14th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
B|53|971|[Korea Tan](https://zwiftpower\.com/profile\.php?z=18034)|[P2K](https://zwiftpower\.com/team\.php?id=1366)|[15th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
B|53|971|[Piotr Ciurzyński](https://zwiftpower\.com/profile\.php?z=161070)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[15th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|53|971|[Sergio Palacio](https://zwiftpower\.com/profile\.php?z=394487)|—|[15th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|53|971|[Sveinung Pedersen \[V\]](https://zwiftpower\.com/profile\.php?z=461978)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[15th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|53|971|[Joe Randall](https://zwiftpower\.com/profile\.php?z=887096)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[15th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|54|969|[Sakawa Agano](https://zwiftpower\.com/profile\.php?z=206174)|—|[16th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|54|969|[Mike Lister \(DACE\)](https://zwiftpower\.com/profile\.php?z=368678)|[DACE](https://zwiftpower\.com/team\.php?id=2177)|[16th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|54|969|[Igor Balakirev](https://zwiftpower\.com/profile\.php?z=441927)|[KLD](https://zwiftpower\.com/team\.php?id=2163)|[16th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|55|967|[Maxwell Smith](https://zwiftpower\.com/profile\.php?z=860816)|—|[17th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|56|965|[Dean Hardie](https://zwiftpower\.com/profile\.php?z=96086)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[18th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
B|56|965|[TAKASHI MAYAMA](https://zwiftpower\.com/profile\.php?z=156051)|—|[18th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|56|965|[Eldon Pyburn](https://zwiftpower\.com/profile\.php?z=250404)|—|[18th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|56|965|[Tsutomu Ohno](https://zwiftpower\.com/profile\.php?z=1157477)|—|[18th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|57|963|[Erick Kercheval](https://zwiftpower\.com/profile\.php?z=234962)|[NLC](https://zwiftpower\.com/team\.php?id=1957)|[19th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|57|963|[Heung Jini \[MAFIA\]](https://zwiftpower\.com/profile\.php?z=501891)|—|[19th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|57|963|[Trevor Waggoner7729](https://zwiftpower\.com/profile\.php?z=922739)|—|[19th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|58|961|[Dave Mehl TeamEN \(C\) 7824](https://zwiftpower\.com/profile\.php?z=8539)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[20th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
B|58|961|[Alex Hatziyannis](https://zwiftpower\.com/profile\.php?z=103835)|[CRUZBIKE](https://zwiftpower\.com/team\.php?id=1922)|[20th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|58|961|[Christophe Moeri1173](https://zwiftpower\.com/profile\.php?z=1137726)|—|[20th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|59|959|[생수 \[PipeLine CyClub\]9039](https://zwiftpower\.com/profile\.php?z=220373)|—|[21st #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|59|959|[Terenzo Downey](https://zwiftpower\.com/profile\.php?z=400481)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[21st #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|60|957|[Daniel Hunt OCC](https://zwiftpower\.com/profile\.php?z=182368)|[OCC](https://zwiftpower\.com/team\.php?id=2620)|[22nd #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|60|957|[ni qb](https://zwiftpower\.com/profile\.php?z=993849)|—|[22nd #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|61|955|[Masayasu Saitou\(panda\)](https://zwiftpower\.com/profile\.php?z=96502)|—|[23rd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|61|955|[Lewis Wake](https://zwiftpower\.com/profile\.php?z=900521)|[KCRT](https://zwiftpower\.com/team\.php?id=4040)|[23rd #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
B|62|951|[Fred Grafe](https://zwiftpower\.com/profile\.php?z=7074)|—|[25th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|62|951|[Claude Richard](https://zwiftpower\.com/profile\.php?z=63288)|—|[25th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|63|949|[Scott Adams](https://zwiftpower\.com/profile\.php?z=395445)|—|[26th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|63|949|[Akio Yamauchi \#EN](https://zwiftpower\.com/profile\.php?z=458632)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[26th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|63|949|[Claus D Christiansen \[DBR\]](https://zwiftpower\.com/profile\.php?z=833347)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|[26th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|63|949|[Sjors Jasperse](https://zwiftpower\.com/profile\.php?z=1026776)|—|[26th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|64|947|[Jennifer Crehan\. \(CLS\)](https://zwiftpower\.com/profile\.php?z=452471)|—|[27th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|64|947|[MHA\_ KOH\!\! 9595](https://zwiftpower\.com/profile\.php?z=587530)|—|[27th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|65|945|[Armando Mastracci XERT](https://zwiftpower\.com/profile\.php?z=93176)|—|[28th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|65|945|[Mattias Persson](https://zwiftpower\.com/profile\.php?z=172970)|—|[28th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|65|945|[Conor Dalby](https://zwiftpower\.com/profile\.php?z=396581)|—|[28th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|65|945|[Benedict Read](https://zwiftpower\.com/profile\.php?z=758452)|—|[28th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|66|943|[Scott Olsen](https://zwiftpower\.com/profile\.php?z=5027)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[29th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|66|943|[박 성필4699](https://zwiftpower\.com/profile\.php?z=528613)|[CaM](https://zwiftpower\.com/team\.php?id=4120)|[29th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|67|941|[Jonas Spinoy Ⓥ 🇧🇪](https://zwiftpower\.com/profile\.php?z=347291)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[30th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
B|67|941|[Christoph Schmidt](https://zwiftpower\.com/profile\.php?z=1101955)|—|[30th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|68|939|[Adam Blake \[DIRT\]](https://zwiftpower\.com/profile\.php?z=344281)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[31st #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|68|939|[Emily Slavin](https://zwiftpower\.com/profile\.php?z=385268)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[31st #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|68|939|[ALEXANDER IVAKHIN](https://zwiftpower\.com/profile\.php?z=928303)|—|[31st #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
B|69|935|[Mark Creaby](https://zwiftpower\.com/profile\.php?z=55551)|—|[33rd #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|69|935|[Brian Monaghan](https://zwiftpower\.com/profile\.php?z=308807)|—|[33rd #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|69|935|[David Santos](https://zwiftpower\.com/profile\.php?z=437626)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|[33rd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|70|933|[Jesper Alberts 3760](https://zwiftpower\.com/profile\.php?z=413981)|—|[34th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|71|931|[Steven Rittenberg](https://zwiftpower\.com/profile\.php?z=558133)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[35th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|71|931|[Les \- carlos](https://zwiftpower\.com/profile\.php?z=1057711)|—|[35th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
B|72|929|[FM 2k7](https://zwiftpower\.com/profile\.php?z=555098)|—|[36th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|72|929|[Paul Brehm](https://zwiftpower\.com/profile\.php?z=668328)|—|[36th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|73|927|[Vince Rowbottom](https://zwiftpower\.com/profile\.php?z=135040)|[WPTAZ](https://zwiftpower\.com/team\.php?id=23)|[37th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|74|923|[Richard Borget 6722](https://zwiftpower\.com/profile\.php?z=241408)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[39th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|74|923|[Don Bly](https://zwiftpower\.com/profile\.php?z=1134071)|—|[39th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|75|921|[Brian Martin](https://zwiftpower\.com/profile\.php?z=326628)|—|[40th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|75|921|[Rashidah](https://zwiftpower\.com/profile\.php?z=912236)|—|[40th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|76|919|[Tom Haug](https://zwiftpower\.com/profile\.php?z=967068)|—|[41st #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|77|917|[Minero Ⓥ](https://zwiftpower\.com/profile\.php?z=144549)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[42nd #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
B|77|917|[Cofitan Yeondoo/WATS](https://zwiftpower\.com/profile\.php?z=444658)|—|[42nd #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|78|915|[Kevin Robson5647](https://zwiftpower\.com/profile\.php?z=1099532)|[STOLEN GOAT](https://zwiftpower\.com/team\.php?id=2451)|[43rd #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
B|79|913|[Min Hong](https://zwiftpower\.com/profile\.php?z=68539)|[CaM](https://zwiftpower\.com/team\.php?id=4120)|[44th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|1|5972|[Tim dODZ](https://zwiftpower\.com/profile\.php?z=68374)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈 #2](https://zwiftpower.com/events.php?zid=155565)|[🥈 #7](https://zwiftpower.com/events.php?zid=173434)|[🥉 #5](https://zwiftpower.com/events.php?zid=166628)|[🥉 #6](https://zwiftpower.com/events.php?zid=171934)|[🥉 #8](https://zwiftpower.com/events.php?zid=176147)|[4th #1](https://zwiftpower.com/events.php?zid=151390)
C|2|5936|[Eric Olds \[BIKEMAN\.COM\]](https://zwiftpower\.com/profile\.php?z=873324)|[Bikeman\.com](https://zwiftpower\.com/team\.php?id=3969)|[4th #6](https://zwiftpower.com/events.php?zid=171934)|[5th #8](https://zwiftpower.com/events.php?zid=176147)|[6th #3](https://zwiftpower.com/events.php?zid=159271)|[6th #5](https://zwiftpower.com/events.php?zid=166628)|[6th #7](https://zwiftpower.com/events.php?zid=173434)|[8th #4](https://zwiftpower.com/events.php?zid=162625)
C|3|5890|[Morten Wernersen](https://zwiftpower\.com/profile\.php?z=727275)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[🥇 #5](https://zwiftpower.com/events.php?zid=166628)|[🥉 #2](https://zwiftpower.com/events.php?zid=155565)|[5th #1](https://zwiftpower.com/events.php?zid=151390)|[12th #3](https://zwiftpower.com/events.php?zid=159271)|[13th #6](https://zwiftpower.com/events.php?zid=171934)|[24th #4](https://zwiftpower.com/events.php?zid=162625)
C|4|5884|[Gerrie Delport](https://zwiftpower\.com/profile\.php?z=23452)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥉 #7](https://zwiftpower.com/events.php?zid=173434)|[7th #8](https://zwiftpower.com/events.php?zid=176147)|[8th #2](https://zwiftpower.com/events.php?zid=155565)|[14th #3](https://zwiftpower.com/events.php?zid=159271)|[14th #6](https://zwiftpower.com/events.php?zid=171934)|[15th #4](https://zwiftpower.com/events.php?zid=162625)
C|5|5844|[Ralph Moore \(endurancenation\.us\)](https://zwiftpower\.com/profile\.php?z=119074)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[5th #2](https://zwiftpower.com/events.php?zid=155565)|[12th #8](https://zwiftpower.com/events.php?zid=176147)|[13th #3](https://zwiftpower.com/events.php?zid=159271)|[16th #6](https://zwiftpower.com/events.php?zid=171934)|[17th #4](https://zwiftpower.com/events.php?zid=162625)|[18th #5](https://zwiftpower.com/events.php?zid=166628)
C|6|4977|[Bastian Zwack](https://zwiftpower\.com/profile\.php?z=116546)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[🥇 #1](https://zwiftpower.com/events.php?zid=151390)|[🥇 #2](https://zwiftpower.com/events.php?zid=155565)|[🥇 #7](https://zwiftpower.com/events.php?zid=173434)|[🥈 #8](https://zwiftpower.com/events.php?zid=176147)|[9th #4](https://zwiftpower.com/events.php?zid=162625)|—
C|7|4931|[Mitch Dangremond \(Trisuppo\)](https://zwiftpower\.com/profile\.php?z=15790)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[6th #4](https://zwiftpower.com/events.php?zid=162625)|[7th #3](https://zwiftpower.com/events.php?zid=159271)|[7th #6](https://zwiftpower.com/events.php?zid=171934)|[7th #7](https://zwiftpower.com/events.php?zid=173434)|[10th #5](https://zwiftpower.com/events.php?zid=166628)|—
C|8|4871|[R oryG \#TeamEN](https://zwiftpower\.com/profile\.php?z=391588)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[8th #8](https://zwiftpower.com/events.php?zid=176147)|[10th #2](https://zwiftpower.com/events.php?zid=155565)|[14th #4](https://zwiftpower.com/events.php?zid=162625)|[16th #7](https://zwiftpower.com/events.php?zid=173434)|[19th #6](https://zwiftpower.com/events.php?zid=171934)|—
C|9|3972|[Sabra Gonzalez Team EN](https://zwiftpower\.com/profile\.php?z=329464)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[🥇 #3](https://zwiftpower.com/events.php?zid=159271)|[5th #4](https://zwiftpower.com/events.php?zid=162625)|[5th #6](https://zwiftpower.com/events.php?zid=171934)|[5th #7](https://zwiftpower.com/events.php?zid=173434)|—|—
C|10|3920|[Derrek Sanks TeamEN](https://zwiftpower\.com/profile\.php?z=11111)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[6th #1](https://zwiftpower.com/events.php?zid=151390)|[11th #3](https://zwiftpower.com/events.php?zid=159271)|[11th #7](https://zwiftpower.com/events.php?zid=173434)|[14th #5](https://zwiftpower.com/events.php?zid=166628)|—|—
C|11|2963|[Skyler Street](https://zwiftpower\.com/profile\.php?z=120373)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉 #3](https://zwiftpower.com/events.php?zid=159271)|[🥉 #4](https://zwiftpower.com/events.php?zid=162625)|[14th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—
C|12|2957|[David Bussey](https://zwiftpower\.com/profile\.php?z=4725)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th #8](https://zwiftpower.com/events.php?zid=176147)|[9th #6](https://zwiftpower.com/events.php?zid=171934)|[10th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
C|13|2955|[Rich Stanbaugh \#EN](https://zwiftpower\.com/profile\.php?z=416912)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[6th #6](https://zwiftpower.com/events.php?zid=171934)|[6th #8](https://zwiftpower.com/events.php?zid=176147)|[12th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—
C|14|2951|[Ben Greenhalgh Cryogen C](https://zwiftpower\.com/profile\.php?z=145025)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th #3](https://zwiftpower.com/events.php?zid=159271)|[9th #5](https://zwiftpower.com/events.php?zid=166628)|[12th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—
C|14|2951|[Brad Wagz ODZ](https://zwiftpower\.com/profile\.php?z=1114373)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th #4](https://zwiftpower.com/events.php?zid=162625)|[8th #6](https://zwiftpower.com/events.php?zid=171934)|[14th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—
C|15|2923|[Hans VEGAN](https://zwiftpower\.com/profile\.php?z=422779)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[9th #2](https://zwiftpower.com/events.php?zid=155565)|[15th #8](https://zwiftpower.com/events.php?zid=176147)|[16th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—
C|16|1998|[Anina Blankenship 5603](https://zwiftpower\.com/profile\.php?z=29249)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥇 #4](https://zwiftpower.com/events.php?zid=162625)|[🥇 #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
C|17|1994|[Greg Low](https://zwiftpower\.com/profile\.php?z=162837)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈 #5](https://zwiftpower.com/events.php?zid=166628)|[🥈 #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
C|18|1964|[Kamil Szelag](https://zwiftpower\.com/profile\.php?z=610990)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[4th #5](https://zwiftpower.com/events.php?zid=166628)|[15th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
C|19|1962|[Megan Herzog](https://zwiftpower\.com/profile\.php?z=120109)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[9th #7](https://zwiftpower.com/events.php?zid=173434)|[11th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
C|20|1944|[Joseph Pittman](https://zwiftpower\.com/profile\.php?z=522522)|—|[12th #1](https://zwiftpower.com/events.php?zid=151390)|[17th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—
C|21|1942|[Donavon Beals4723](https://zwiftpower\.com/profile\.php?z=1123750)|—|[14th #2](https://zwiftpower.com/events.php?zid=155565)|[16th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—
C|22|1936|[George Rodrigues](https://zwiftpower\.com/profile\.php?z=277743)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[13th #4](https://zwiftpower.com/events.php?zid=162625)|[20th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—
C|23|1932|[LeRoy Nahay](https://zwiftpower\.com/profile\.php?z=286502)|—|[15th #5](https://zwiftpower.com/events.php?zid=166628)|[20th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
C|24|1930|[Scott Phillips](https://zwiftpower\.com/profile\.php?z=83123)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[17th #6](https://zwiftpower.com/events.php?zid=171934)|[19th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
C|25|1900|[ØYVIND Terjesen](https://zwiftpower\.com/profile\.php?z=867771)|—|[22nd #5](https://zwiftpower.com/events.php?zid=166628)|[29th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—
C|26|999|[Gaetan Duperron6179](https://zwiftpower\.com/profile\.php?z=513196)|—|[🥇 #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|27|997|[Alex Callejo](https://zwiftpower\.com/profile\.php?z=55568)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥈 #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|27|997|[Kerry Smith](https://zwiftpower\.com/profile\.php?z=164504)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥈 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|27|997|[Jim Williams ZZRC TIS](https://zwiftpower\.com/profile\.php?z=894423)|—|[🥈 #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|28|995|[Matt Partridge \[DIRT\]](https://zwiftpower\.com/profile\.php?z=458388)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥉 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|29|993|[Roger Rider](https://zwiftpower\.com/profile\.php?z=20137)|—|[4th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|29|993|[Ian Bundy](https://zwiftpower\.com/profile\.php?z=318161)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[4th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|29|993|[Alasdair MacMillan](https://zwiftpower\.com/profile\.php?z=522143)|—|[4th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|30|991|[Jonas Spinoy Ⓥ 🇧🇪](https://zwiftpower\.com/profile\.php?z=347291)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[5th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|31|989|[天蝎座 GUO🦂️⛵️](https://zwiftpower\.com/profile\.php?z=290402)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[6th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|32|987|[Dave Mehl TeamEN \(C\) 7824](https://zwiftpower\.com/profile\.php?z=8539)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[7th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|32|987|[Dawson Hardiman 4923](https://zwiftpower\.com/profile\.php?z=53909)|—|[7th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|32|987|[simone sighinolfi Ⓥ](https://zwiftpower\.com/profile\.php?z=1085650)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[7th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|33|985|[Sjoerd Keizers](https://zwiftpower\.com/profile\.php?z=233688)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[8th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|33|985|[Al Price \(C\)](https://zwiftpower\.com/profile\.php?z=336603)|—|[8th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|33|985|[Sulize Kriel](https://zwiftpower\.com/profile\.php?z=429670)|—|[8th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|34|983|[Danny Noble](https://zwiftpower\.com/profile\.php?z=92884)|—|[9th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|34|983|[TITO](https://zwiftpower\.com/profile\.php?z=177990)|—|[9th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|34|983|[Elise Yanover](https://zwiftpower\.com/profile\.php?z=195469)|—|[9th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|35|981|[Mark Middleton \(Kos\)](https://zwiftpower\.com/profile\.php?z=5666)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[10th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
C|35|981|[Bruno Barbosa](https://zwiftpower\.com/profile\.php?z=104049)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[10th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|35|981|[Jennifer Crehan\. \(CLS\)](https://zwiftpower\.com/profile\.php?z=452471)|—|[10th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|35|981|[Coffee Umanetz](https://zwiftpower\.com/profile\.php?z=945838)|—|[10th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|35|981|[John Maranto \(Riding realtor\) 7355 Dirt](https://zwiftpower\.com/profile\.php?z=965481)|—|[10th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|36|979|[Tim Parker](https://zwiftpower\.com/profile\.php?z=1496)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[11th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|36|979|[Gary Morris](https://zwiftpower\.com/profile\.php?z=198799)|[MGCC](https://zwiftpower\.com/team\.php?id=114)|[11th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|36|979|[David Ayton](https://zwiftpower\.com/profile\.php?z=225652)|—|[11th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|36|979|[Jay Chapin](https://zwiftpower\.com/profile\.php?z=830907)|—|[11th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|36|979|[Brandon Walls](https://zwiftpower\.com/profile\.php?z=1153231)|—|[11th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|37|977|[Mark Mortimer](https://zwiftpower\.com/profile\.php?z=73851)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[12th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
C|37|977|[Claude Anderson](https://zwiftpower\.com/profile\.php?z=179098)|—|[12th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|37|977|[Allan Howard 5857](https://zwiftpower\.com/profile\.php?z=987925)|—|[12th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|38|975|[Alex Bennett](https://zwiftpower\.com/profile\.php?z=14715)|—|[13th #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
C|38|975|[Steve RAWLINGS 6491](https://zwiftpower\.com/profile\.php?z=238826)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|[13th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|38|975|[Jason Strandberg](https://zwiftpower\.com/profile\.php?z=597788)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[13th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|38|975|[Matt Wilson](https://zwiftpower\.com/profile\.php?z=611819)|—|[13th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|38|975|[Mariusz Lembowicz](https://zwiftpower\.com/profile\.php?z=904184)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[13th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|39|973|[Alfred Lazelberger](https://zwiftpower\.com/profile\.php?z=935777)|—|[14th #8](https://zwiftpower.com/events.php?zid=176147)|—|—|—|—|—
C|40|971|[Jon Hume](https://zwiftpower\.com/profile\.php?z=465110)|—|[15th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|40|971|[P i\.G\.e\. Wang](https://zwiftpower\.com/profile\.php?z=1072304)|—|[15th #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
C|40|971|[mark aylett](https://zwiftpower\.com/profile\.php?z=1125451)|—|[15th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|41|969|[Sean Michael Barnardo](https://zwiftpower\.com/profile\.php?z=12290)|—|[16th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|41|969|[Frank Soltiz](https://zwiftpower\.com/profile\.php?z=64140)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[16th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|42|967|[Jeppe Sørensen](https://zwiftpower\.com/profile\.php?z=410451)|—|[17th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|43|965|[Josh Roark](https://zwiftpower\.com/profile\.php?z=14493)|—|[18th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|43|965|[Tam Burns CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=17442)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[18th #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
C|43|965|[G Pollack \#EN 9591](https://zwiftpower\.com/profile\.php?z=454924)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[18th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|43|965|[Les \- carlos](https://zwiftpower\.com/profile\.php?z=1057711)|—|[18th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|44|963|[Kim O'Neill](https://zwiftpower\.com/profile\.php?z=83416)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[19th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|44|963|[Paul Wakefield](https://zwiftpower\.com/profile\.php?z=714712)|—|[19th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|44|963|[Shinichiro Murakami](https://zwiftpower\.com/profile\.php?z=1160795)|—|[19th #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|45|961|[Ricardo Moreno](https://zwiftpower\.com/profile\.php?z=93510)|—|[20th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|45|961|[John Frame KISS \(C\)](https://zwiftpower\.com/profile\.php?z=446376)|—|[20th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|45|961|[Zaid Siddiqui](https://zwiftpower\.com/profile\.php?z=1073243)|—|[20th #3](https://zwiftpower.com/events.php?zid=159271)|—|—|—|—|—
C|46|959|[David Creelman](https://zwiftpower\.com/profile\.php?z=403843)|—|[21st #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|46|959|"[Laurent ""Froome"" France](https://zwiftpower\.com/profile\.php?z=604431)"|—|[21st #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|46|959|[Mike Graffeo TeamEN](https://zwiftpower\.com/profile\.php?z=811941)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[21st #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—|—
C|47|957|[Li Kevin](https://zwiftpower\.com/profile\.php?z=24862)|—|[22nd #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|48|955|[y terra](https://zwiftpower\.com/profile\.php?z=102591)|—|[23rd #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|48|955|[shirley partridge](https://zwiftpower\.com/profile\.php?z=465464)|—|[23rd #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|49|953|[Dicky P9NRY 9117](https://zwiftpower\.com/profile\.php?z=1132959)|—|[24th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|50|951|[Clare Pollard](https://zwiftpower\.com/profile\.php?z=115478)|—|[25th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|50|951|[Katy Toms Cryo\-gen\(C\)](https://zwiftpower\.com/profile\.php?z=197276)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[25th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|51|949|[Fook AuYeung](https://zwiftpower\.com/profile\.php?z=325897)|—|[26th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|51|949|[Robert Aston](https://zwiftpower\.com/profile\.php?z=1073407)|—|[26th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
C|52|947|[john Melia 3188](https://zwiftpower\.com/profile\.php?z=403960)|—|[27th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
C|53|945|[heaven KIM&LEE](https://zwiftpower\.com/profile\.php?z=893693)|—|[28th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
D|1|5978|[Mark Edwards \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=387173)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[🥇 #3](https://zwiftpower.com/events.php?zid=159271)|[🥈 #2](https://zwiftpower.com/events.php?zid=155565)|[🥈 #4](https://zwiftpower.com/events.php?zid=162625)|[🥈 #7](https://zwiftpower.com/events.php?zid=173434)|[🥉 #6](https://zwiftpower.com/events.php?zid=171934)|[4th #1](https://zwiftpower.com/events.php?zid=151390)
D|2|1998|[Alfred Lazelberger](https://zwiftpower\.com/profile\.php?z=935777)|—|[🥇 #6](https://zwiftpower.com/events.php?zid=171934)|[🥇 #7](https://zwiftpower.com/events.php?zid=173434)|—|—|—|—
D|3|999|[Jonathan Bridge](https://zwiftpower\.com/profile\.php?z=126867)|—|[🥇 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
D|3|999|[Jason Jones](https://zwiftpower\.com/profile\.php?z=443113)|—|[🥇 #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
D|3|999|[Gareth Walsh 6151](https://zwiftpower\.com/profile\.php?z=532857)|—|[🥇 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
D|3|999|[Erik Dickes](https://zwiftpower\.com/profile\.php?z=1112157)|[OMAHA VELO](https://zwiftpower\.com/team\.php?id=3984)|[🥇 #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
D|4|997|[Allan Card](https://zwiftpower\.com/profile\.php?z=88370)|—|[🥈 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
D|4|997|[IM IVO \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=630117)|—|[🥈 #6](https://zwiftpower.com/events.php?zid=171934)|—|—|—|—|—
D|4|997|[Stewart Latham](https://zwiftpower\.com/profile\.php?z=852002)|—|[🥈 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
D|5|995|[Ricardo Moreno](https://zwiftpower\.com/profile\.php?z=93510)|—|[🥉 #1](https://zwiftpower.com/events.php?zid=151390)|—|—|—|—|—
D|5|995|[Andrea Thackeray](https://zwiftpower\.com/profile\.php?z=322738)|—|[🥉 #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
D|5|995|[Rebecca Bourque BSR](https://zwiftpower\.com/profile\.php?z=906199)|—|[🥉 #2](https://zwiftpower.com/events.php?zid=155565)|—|—|—|—|—
D|5|995|[\. Popey\.](https://zwiftpower\.com/profile\.php?z=1198279)|—|[🥉 #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—
D|6|993|[Nicky Moir9054](https://zwiftpower\.com/profile\.php?z=896045)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[4th #4](https://zwiftpower.com/events.php?zid=162625)|—|—|—|—|—
D|6|993|[berit hougaard jensen](https://zwiftpower\.com/profile\.php?z=1147442)|—|[4th #5](https://zwiftpower.com/events.php?zid=166628)|—|—|—|—|—

